In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import joblib
import os
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split

In [2]:
gbdt_win0 = joblib.load('./gbdt_win0.pkl')
gbdt_b_win1 = joblib.load('./gbdt_b_win1.pkl')
gbdt_b_win2 = joblib.load('./gbdt_b_win2.pkl')
gbdt_b_win3 = joblib.load('./gbdt_b_win3.pkl')

In [3]:
# 数据加载
file_list = []
for root_dir,sub_dir,files in os.walk('./data'):
    file_list.append(files)
file_list = file_list[0]
dataframe_list = pd.DataFrame()
for file in files:
    if file.endswith('xlsx'):
        file_name = os.path.join(root_dir,file)
        df = pd.read_excel(file_name)
        dataframe_list = dataframe_list.append(df)
dataframe_list.drop(['创角账号数'],axis=1,inplace=True)
dataframe_list.columns = ['plan_name','channel_id','source_id','data_win','source_run_datetime','create_time','plan_id',
                         'game_id','platform','source_run_date_amount','create_role_num','create_role_cost','create_role_pay_num',
                          'create_role_pay_cost','create_role_pay_sum','create_role_roi','create_role_retain_1d']
dataframe_list.dropna(how='any',inplace=True)

In [4]:
dataframe_list['media_id'] = 10

In [6]:
# 入参
# "columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform",
#      "source_run_date_amount","create_role_num","create_role_cost","create_role_pay_num","create_role_pay_cost",
#             "create_role_pay_sum","create_role_roi","create_role_retain_1d"]

In [5]:
data = dataframe_list[["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform",
     "source_run_date_amount","create_role_num","create_role_cost","create_role_pay_num","create_role_pay_cost",
            "create_role_pay_sum","create_role_roi","create_role_retain_1d"]]

In [6]:
data.head()

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
0,304184,0,2020-05-01,2020-04-30,1000840,10,29.31,0,9.770000,0,0.00,0,0.000000,0.6667,0.00000,12
1,304185,0,2020-05-01,2020-04-30,1000840,10,496.71,0,41.392500,0,0.00,0,0.000000,0.0000,0.00000,0
2,304186,0,2020-05-01,2020-04-30,1000840,10,14863.08,0,170.840000,1,14863.08,12,0.000807,0.1149,0.01149,954
3,291448,0,2020-05-01,2020-04-09,1000840,10,1644.47,0,13.369675,0,0.00,0,0.000000,0.0976,0.00000,198
4,296967,0,2020-05-01,2020-04-20,1000840,10,1886.79,0,48.379231,0,0.00,0,0.000000,0.1795,0.00000,12


In [7]:
data.shape

(466200, 16)

In [8]:
data.values

array([[304184, 0, '2020-05-01', ..., 0.6667, 0.0, 12],
       [304185, 0, '2020-05-01', ..., 0.0, 0.0, 0],
       [304186, 0, '2020-05-01', ..., 0.1149, 0.01149, 954],
       ...,
       [337887, 7, '2020-07-31', ..., 0.0, 0.0, 0],
       [337888, 7, '2020-07-31', ..., 0.0, 0.0, 0],
       [337889, 7, '2020-07-31', ..., 0.0, 0.0, 0]], dtype=object)

In [65]:
aa = pd.DataFrame({'A':[1,2,3],'B':[4,5,6]})

In [141]:
aa = data.sample(100000)

In [103]:
aa['create_time'] = aa['create_time'].astype(str)

In [84]:
aa = aa.values.tolist()

In [86]:
file=open('data.txt','w') 
file.write(str(aa)); 
file.close() 

In [9]:
jsondata = {"columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"],
           "data":aa.values.tolist()}

NameError: name 'aa' is not defined

In [107]:
type(jsondata)

dict

In [110]:
jsondata

{'columns': ['channel_id',
  'source_id',
  'data_win',
  'source_run_datetime',
  'create_time',
  'media_id',
  'game_id',
  'platform',
  'source_run_date_amount',
  'create_role_num',
  'create_role_cost',
  'create_role_pay_num',
  'create_role_pay_cost',
  'create_role_pay_sum',
  'create_role_roi',
  'create_role_retain_1d'],
 'data': array([[7162.0, 261134.0, 7.0, ..., 0.0, 0.0, 0.0],
        [6867.0, 294573.0, 5.0, ..., 0.0, 0.0, 0.0],
        [7162.0, 217274.0, 6.0, ..., 0.0, 0.0, 0.0],
        ...,
        [21088.0, 333893.0, 5.0, ..., 0.0, 0.0, 0.0],
        [20754.0, 292204.0, 2.0, ..., 0.0, 0.0, 0.0],
        [20143.0, 314932.0, 2.0, ..., 0.0, 0.0, 0.0]], dtype=object)}

In [117]:
# j = json.dumps(jsondata)

In [116]:
# import json

# with open("tmp.txt", "w") as fp:
#     fp.write(json.dumps(jsondata))

In [112]:
columns_1 = source_predict(jsondata=jsondata)['columns']
data_1 = source_predict(jsondata=jsondata)['list']

In [45]:
result = pd.DataFrame(data=data_1,columns=columns_1)

In [47]:
result.shape

(30194, 17)

In [48]:
result['source_run_date_amount'].min()

500.15

In [49]:
groups = result.groupby(['channel_id','source_id','source_run_datetime'])
groups.ngroups

30194

In [50]:
result['label'].value_counts()

1    27087
0     3107
Name: label, dtype: int64

In [51]:
result[result['label']==0]['data_win'].value_counts()

0.0    1848
3.0     708
7.0     373
2.0     178
Name: data_win, dtype: int64

In [53]:
result[result['label']==0].head()

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,label
7,20954.0,336540.0,0.0,2020-07-30,2020-07-28,10,1000993.0,2.0,22838.39,648.0,35.353545,5.0,4567.678000,158.0,0.006918,0.0867,0
12,20755.0,336867.0,0.0,2020-07-30,2020-07-29,10,1000993.0,2.0,1286.17,46.0,27.960217,1.0,1286.170000,12.0,0.009330,0.0435,0
18,20755.0,307729.0,0.0,2020-05-09,2020-05-09,10,1000993.0,2.0,4528.30,137.0,33.053285,1.0,4528.300000,30.0,0.006625,0.0657,0
22,20954.0,336540.0,0.0,2020-07-29,2020-07-28,10,1000993.0,2.0,42549.78,982.0,43.418143,11.0,3868.161818,700.0,0.016451,0.0918,0
32,20954.0,307422.0,0.0,2020-05-09,2020-05-09,10,1000993.0,2.0,3695.65,98.0,37.710714,1.0,3695.650000,60.0,0.016235,0.0510,0


In [56]:
result.to_csv('./result.csv')

In [90]:
aa.head()

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
15390,20545.0,301828.0,5.0,2020-07-21,2020-04-28,10,1000840.0,1.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0000
18046,20755.0,243404.0,1.0,2020-05-11,2019-12-07,10,1000954.0,2.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0000
41770,20953.0,319112.0,4.0,2020-06-17,2020-06-13,10,1000840.0,1.0,558.25,7.0,79.75,0.0,0.0,0.0,0.0,0.1429
14267,20754.0,332926.0,6.0,2020-07-28,2020-07-19,10,1000840.0,1.0,0.19,1.0,0.19,0.0,0.0,0.0,0.0,0.0000
3088,20754.0,222773.0,2.0,2020-06-06,2019-10-22,10,1000960.0,1.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0000


In [262]:
bb = pd.DataFrame(data=[[1,1,3,"2020-03-30","2020-03-30",10,80024,1,1000,5,100,2,250,20,0.25,0.2],
      [1,1,7,"2020-03-30","2020-03-30",10,80023,2,2000,10,0,4,500,40,0.5,0.2]],columns = ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"])

In [263]:
bb

,channel_id,source_id,data_win,source_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
0,1,1,3,2020-03-30,2020-03-30,10,80024,1,1000,5,100,2,250,20,0.25,0.2
1,1,1,7,2020-03-30,2020-03-30,10,80023,2,2000,10,0,4,500,40,0.50,0.2


In [264]:
jsondata = {"columns": ["channel_id","source_id","data_win","source_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d"],
           "data":bb.values}

In [265]:
columns_1 = source_predict(jsondata)['columns']
data_1 = source_predict(jsondata)['list']

In [266]:
data_1

[[1,
  1,
  3,
  '2020-03-30',
  '2020-03-30',
  10,
  80024,
  1,
  1000,
  5,
  100,
  2,
  250,
  20,
  0.25,
  0.2,
  1]]

In [54]:
state_1_data = pd.DataFrame(data=[[1,1,"末日","2020-03-30","2020-03-30",10,80024,1,400,5,90,1,3500,32,0.02,0.2],
      [1,1,"末日_2","2020-03-30","2020-03-30",10,80023,2,2000,10,300,0,0,0,0,0]],columns = ["channel_id","source_id","plan_name","model_run_datetime","create_time","media_id","game_id","platform","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_pay_rate"])

In [55]:
state_1_data

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_pay_rate
0,1,1,末日,2020-03-30,2020-03-30,10,80024,1,400,5,90,1,3500,32,0.02,0.2
1,1,1,末日_2,2020-03-30,2020-03-30,10,80023,2,2000,10,300,0,0,0,0.00,0.0


In [56]:
jsondata = {"columns": state_1_data.columns,
           "data":state_1_data.values}

In [57]:
columns_1 = source_predict_state_2(jsondata)['columns']
data_1 = source_predict_state_2(jsondata)['list']

In [58]:
df1 = pd.DataFrame(data_1,columns=columns_1)

In [59]:
df1

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_pay_rate,label
0,1,1,末日_2,2020-03-30,2020-03-30,10,80023,2,2000,10,300,0,0,0,0.00,0.0,1
1,1,1,末日,2020-03-30,2020-03-30,10,80024,1,400,5,90,1,3500,32,0.02,0.2,3


In [10]:
def source_predict_state_1(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","plan_name", "model_run_datetime","create_time","media_id",
    "game_id","platform","source_run_date_amount","create_role_num","create_role_cost","create_role_pay_num",
    "create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_pay_rate"]}
    :return:
    '''
    data = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])
    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]    # 删除消耗为0的数据
    # 去重，处理某天没有消耗的情况
    data.drop_duplicates(subset=['channel_id','source_id','source_run_date_amount','create_role_num'],keep='first'
                               ,inplace=True)
    # 对消耗300元以下的不作判断，对数据按平台、win进行分类处理
#   data_ignore = data[data['source_run_date_amount'] < 300]
    data_not_ignore = data[data['source_run_date_amount'] >= 300]
    data_win_0 = data_not_ignore[data_not_ignore['platform'] == 1]
    data_win_0_ios = data_not_ignore[data_not_ignore['platform'] == 2]    
    
    result_df = pd.DataFrame()
    temp_df = pd.DataFrame()
    # win=0预判
    if data_win_0.shape[0]!=0:
        data_win_0['label'] = data_win_0.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_pay_sum==0)&
            (x.create_role_cost>=80) else(2 if (x.create_role_pay_sum==0)&(x.create_role_cost<80) else 
            (2 if (x.create_role_pay_sum!=0)&(x.create_role_pay_cost>=7000) else (2 if (x.create_role_pay_sum!=0)&
            (x.create_role_pay_cost<7000)&(x.create_role_cost>=140) else 0)))),axis=1)
        result_df =  result_df.append(data_win_0[(data_win_0['label'] == 1)| (data_win_0['label'] == 2)])
        temp_df = temp_df.append(data_win_0[data_win_0['label'] == 0])
    
    if data_win_0_ios.shape[0]!=0:
        data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if x.create_role_num == 0 else (1 if (x.create_role_pay_sum==0)&
            (x.create_role_cost>=200) else(2 if (x.create_role_pay_sum==0)&(x.create_role_cost<200) else 
            (2 if (x.create_role_pay_sum!=0)&(x.create_role_pay_cost>=12000) else (2 if (x.create_role_pay_sum!=0)&
            (x.create_role_pay_cost<12000)&(x.create_role_cost>=300) else 0)))),axis=1)
        result_df =  result_df.append(data_win_0_ios[(data_win_0_ios['label'] == 1)| (data_win_0_ios['label'] == 2)])
        temp_df = temp_df.append(data_win_0_ios[data_win_0_ios['label'] == 0])
     
    # 模型预测
    if temp_df.shape[0] != 0:
        temp_df['label'] = gbdt_win0.predict(temp_df[['source_run_date_amount',
           'create_role_num', 'create_role_cost', 'create_role_pay_num',
           'create_role_pay_cost', 'create_role_pay_sum', 'create_role_roi', 'create_role_pay_rate']])
        temp_df['label'] = temp_df['label'].apply(lambda x:3 if x==1 else 4)
    
    source_predict = pd.concat([result_df,temp_df],axis=0)

    source_predict['label'] = source_predict['label'].astype(int)

    return {"columns": source_predict.columns.values.tolist(), "list": source_predict.values.tolist()}

In [80]:
state_2_data = pd.DataFrame(data=[[1,1,"末日","2020-03-30","2020-03-30",10,80024,1,3,350,5,90,1,3500,32,0.1,0.02,0.2],
                                  [1,3,"末日","2020-03-30","2020-03-30",10,80024,1,2,200,5,90,1,3500,32,0.1,0.02,0.2],
      [1,2,"末日_2","2020-03-30","2020-03-30",10,80023,2,1,2000,10,300,0,0,0,0,0.02,0]],columns = ["channel_id","source_id","plan_name","model_run_datetime","create_time",
    "media_id","game_id","platform","data_win","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi",
    "create_role_retain_1d","create_role_pay_rate"])

In [81]:
state_2_data

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,data_win,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,create_role_pay_rate
0,1,1,末日,2020-03-30,2020-03-30,10,80024,1,3,350,5,90,1,3500,32,0.1,0.02,0.2
1,1,3,末日,2020-03-30,2020-03-30,10,80024,1,2,200,5,90,1,3500,32,0.1,0.02,0.2
2,1,2,末日_2,2020-03-30,2020-03-30,10,80023,2,1,2000,10,300,0,0,0,0.0,0.02,0.0


In [82]:
jsondata = {"columns": state_2_data.columns,
           "data":state_2_data.values}

In [83]:
columns_2 = source_predict_state_2(jsondata)['columns']
data_2 = source_predict_state_2(jsondata)['list']
df2 = pd.DataFrame(data_2,columns=columns_2)

In [87]:
df2.head()

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,data_win,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,create_role_pay_rate,label
0,1,1,末日,2020-03-30,2020-03-30,10,80024,1,3,350,5,90,1,3500,32,0.1,0.02,0.2,0
1,1,2,末日_2,2020-03-30,2020-03-30,10,80023,2,1,2000,10,300,0,0,0,0.0,0.02,0.0,1
2,1,3,末日,2020-03-30,2020-03-30,10,80024,1,-1,200,5,90,1,3500,32,0.1,0.02,0.2,1


In [1]:
def source_predict_state_2(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","plan_name","model_run_datetime","create_time",
    "media_id","game_id","platform","data_win","source_run_date_amount","create_role_num",
    "create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi",
    "create_role_retain_1d","create_role_pay_rate","create_role_pay_num_cum","learning_type","learning_time_dt",
    "learning_time_hr","deep_bid_type"]}
    :return:
    '''
    data = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])

    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]  # 删除消耗为0的数据
    # 去重，处理某天没有消耗的情况
    data.sort_values(by='data_win', inplace=True)
    data.drop_duplicates(subset=['channel_id', 'source_id', 'source_run_date_amount', 'create_role_num'], keep='first'
                         , inplace=True)
    # 对消耗300元以下的不作判断，对数据按平台、win进行分类处理
    data_not_ignore = data[(data['source_run_date_amount'] >= 300) | (data['data_win'] == 1)]
    data_ignore = data.drop(data_not_ignore.index)

    data_win_1 = data_not_ignore[(data_not_ignore['data_win'] == 1) & (data_not_ignore['platform'] == 1)]
    data_win_2 = data_not_ignore[(data_not_ignore['data_win'] == 2) & (data_not_ignore['platform'] == 1)]
    data_win_3 = data_not_ignore[(data_not_ignore['data_win'] == 3) & (data_not_ignore['platform'] == 1)]

    data_win_1_ios = data_not_ignore[(data_not_ignore['data_win'] == 1) & (data_not_ignore['platform'] == 2)]
    data_win_2_ios = data_not_ignore[(data_not_ignore['data_win'] == 2) & (data_not_ignore['platform'] == 2)]
    data_win_3_ios = data_not_ignore[(data_not_ignore['data_win'] == 3) & (data_not_ignore['platform'] == 2)]

    # win=1预判
    result_win1 = pd.DataFrame()
    temp_win1 = pd.DataFrame()

    # data_win_1中再分deep_bid_type
    data_win_1_pre_action = data_win_1[data_win_1['deep_bid_type'] == 'BID_PER_ACTION']
    data_win_1 = data_win_1[data_win_1['deep_bid_type'] != 'BID_PER_ACTION']

    if data_win_1_pre_action.shape[0] != 0:
        data_win_1_pre_action['label'] = data_win_1_pre_action.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.02 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.022) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.03) & (x.source_run_date_amount >= 500) else 1 if
                   (x.create_role_pay_cost > 8000) | (x.create_role_cost > 220) else 2))), axis=1)

        result_win1 = result_win1.append(
            data_win_1_pre_action[(data_win_1_pre_action['label'] == 1) | (data_win_1_pre_action['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1_pre_action[data_win_1_pre_action['label'] == 2])

    if data_win_1.shape[0] != 0:
        data_win_1['label'] = data_win_1.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.02 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.022) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.03) & (x.source_run_date_amount >= 500) else 1 if
                   (x.create_role_pay_cost > 8000) | (x.create_role_cost > 100) else 2))), axis=1)

        result_win1 = result_win1.append(data_win_1[(data_win_1['label'] == 1) | (data_win_1['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1[data_win_1['label'] == 2])

    data_win_1_ios_pre_action = data_win_1_ios[data_win_1_ios['deep_bid_type'] == 'BID_PER_ACTION']
    data_win_1_ios = data_win_1_ios[data_win_1_ios['deep_bid_type'] != 'BID_PER_ACTION']

    if data_win_1_ios_pre_action.shape[0] != 0:
        data_win_1_ios_pre_action['label'] = data_win_1_ios_pre_action.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.018 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.02) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.03) & (x.source_run_date_amount >= 500) else 1 if
                   (x.create_role_pay_cost > 12000) | (x.create_role_cost > 300) else 2))), axis=1)

        result_win1 = result_win1.append(
            data_win_1_ios_pre_action[(data_win_1_ios_pre_action['label'] == 1) | (data_win_1_ios_pre_action['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1_ios_pre_action[data_win_1_ios_pre_action['label'] == 2])

    if data_win_1_ios.shape[0] != 0:
        data_win_1_ios['label'] = data_win_1_ios.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.018 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.02) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.03) & (x.source_run_date_amount) >= 500 else 1 if
                   (x.create_role_pay_cost > 12000) | (x.create_role_cost > 200) else 2))), axis=1)

        result_win1 = result_win1.append(
            data_win_1_ios[(data_win_1_ios['label'] == 1) | (data_win_1_ios['label'] == 0)])
        temp_win1 = temp_win1.append(data_win_1_ios[data_win_1_ios['label'] == 2])

    # win=1 模型预测
    if temp_win1.shape[0] != 0:
        temp_win1['label'] = gbdt_b_win1.predict(temp_win1[['create_role_cost',
                                                            'create_role_pay_cost', 'create_role_roi',
                                                            'create_role_pay_rate']])
    result_win1_data = pd.concat([result_win1, temp_win1], axis=0)

    # win=2预判
    result_win2 = pd.DataFrame()
    temp_win2 = pd.DataFrame()
    if data_win_2.shape[0] != 0:
        data_win_2['label'] = data_win_2.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
         x.create_role_roi <= 0.05 else (0 if (x.create_role_pay_num > 1) & (
            x.create_role_roi >= 0.06) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.07) & (x.source_run_date_amount >= 500) else 1 if
                  (x.create_role_pay_cost > 7000) | (x.create_role_cost > 150) else 2))), axis=1)

        result_win2 = result_win2.append(data_win_2[(data_win_2['label'] == 1) | (data_win_2['label'] == 0)])
        temp_win2 = temp_win2.append(data_win_2[data_win_2['label'] == 2])

    if data_win_2_ios.shape[0] != 0:
        data_win_2_ios['label'] = data_win_2_ios.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
           x.create_role_roi <= 0.045 else (0 if (x.create_role_pay_num > 1) & (
             x.create_role_roi >= 0.05) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.065) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 10000) | (x.create_role_cost > 250) else 2))), axis=1)

        result_win2 = result_win2.append(data_win_2_ios[(data_win_2_ios['label'] == 1) | (data_win_2_ios['label'] == 0)])
        temp_win2 = temp_win2.append(data_win_2_ios[data_win_2_ios['label'] == 2])

    # win=2 模型预测
    if temp_win2.shape[0] != 0:
        temp_win2['label'] = gbdt_b_win2.predict(temp_win2[['create_role_cost',
                                                            'create_role_pay_cost', 'create_role_roi',
                                                            'create_role_pay_rate', 'create_role_retain_1d']])
    result_win2_data = pd.concat([result_win2, temp_win2], axis=0)

    # win=3预判
    result_win3 = pd.DataFrame()
    temp_win3 = pd.DataFrame()
    if data_win_3.shape[0] != 0:
        data_win_3['label'] = data_win_3.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
           x.create_role_roi <= 0.07 else (
            0 if (x.create_role_pay_num > 1) & (
                    x.create_role_roi >= 0.09) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.11) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 6000) | (x.create_role_cost > 120) else 2))), axis=1)

        result_win3 = result_win3.append(data_win_3[(data_win_3['label'] == 1) | (data_win_3['label'] == 0)])
        temp_win3 = temp_win3.append(data_win_3[data_win_3['label'] == 2])

    if data_win_3_ios.shape[0] != 0:
        data_win_3_ios['label'] = data_win_3_ios.apply(lambda x: 1 if x.create_role_num == 0 else (1 if
           x.create_role_roi <= 0.06 else (0 if (x.create_role_pay_num > 1) & (
              x.create_role_roi >= 0.07) & (x.source_run_date_amount >= 500) else (
                0 if (x.create_role_roi >= 0.08) & (x.source_run_date_amount >= 500) else 1 if
                (x.create_role_pay_cost > 9000) | (x.create_role_cost > 200) else 2))), axis=1)

        result_win3 = result_win3.append(
            data_win_3_ios[(data_win_3_ios['label'] == 1) | (data_win_3_ios['label'] == 0)])
        temp_win3 = temp_win3.append(data_win_3_ios[data_win_3_ios['label'] == 2])

    # win=3 模型预测
    if temp_win3.shape[0] != 0:
        temp_win3['label'] = gbdt_b_win3.predict(temp_win3[['create_role_cost',
                                                            'create_role_pay_cost', 'create_role_roi',
                                                            'create_role_pay_rate', 'create_role_retain_1d']])
    result_win3_data = pd.concat([result_win3, temp_win3], axis=0)

    result = pd.concat([result_win1_data, result_win2_data, result_win3_data], axis=0)

    # 结果输出
    if result.shape[0] != 0:
        result_1_2_3 = result[result['data_win'] != 7]
        # win1,2,3按优先级3,2,1进行去重
        if result_1_2_3.shape[0] != 0:
            result_1_2_3.sort_values('data_win', ascending=False, inplace=True)
            result_1_2_3.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first', inplace=True)

        # # 再与win=7进行去重，win=7优先级小
        # result_all = pd.concat([result_1_2_3, result[result['data_win'] == 7]], axis=0)
        #
        # result_all.sort_values('data_win', inplace=True)
        source_predict = result_1_2_3.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')

    else:
        source_predict = pd.DataFrame(
            columns=["channel_id", "source_id", "plan_name", "model_run_datetime", "create_time",
                     "media_id", "game_id", "platform", "data_win", "source_run_date_amount", "create_role_num",
                     "create_role_cost", "create_role_pay_num", "create_role_pay_cost", "create_role_pay_sum",
                     "create_role_roi",
                     "create_role_retain_1d", "create_role_pay_rate", "create_role_pay_num_cum", "learning_type",
                     "learning_time_dt",
                     "learning_time_hr", "deep_bid_type", "label"])

    if data_ignore.shape[0] != 0:
        data_ignore['label'] = 1
        data_ignore.sort_values('data_win', ascending=False, inplace=True)
        data_ignore = data_ignore.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')
        data_ignore['data_win'] = -1

    source_predict = pd.concat([source_predict, data_ignore], axis=0)
    source_predict.sort_values('data_win', ascending=False, inplace=True)
    source_predict = source_predict.drop_duplicates(['channel_id', 'source_id', 'model_run_datetime'], keep='first')

    source_predict['label'] = source_predict['label'].astype(int)
    print(source_predict.columns.values.tolist())
    return {"columns": source_predict.columns.values.tolist(), "list": source_predict.values.tolist()}

In [ ]:
{"action":"source_predict_state_2","columns":["channel_id","source_id","plan_name","model_run_datetime","create_time","media_id","game_id","platform","data_win","source_run_date_amount","create_role_num","create_role_cost","create_role_pay_num","create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_retain_1d","create_role_pay_rate","create_role_pay_num_cum","learning_type","learning_time_dt","learning_time_hr","deep_bid_type"],"data":[[6867,529646,"0314-540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,3204.11,59,54.3069492,1,3204.110000,98.00,0.0305857,0.0000,0.01695,1,0,null,null,"BID_PER_ACTION"],[6867,529658,"0314-540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,12701.94,202,62.8808911,3,4233.980000,358.00,0.0281847,0.0000,0.01485,4,0,null,null,"BID_PER_ACTION"],[6867,529919,"0315-204-穿山甲-每次付费-幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,925.64,28,33.0585714,1,925.640000,140.00,0.1512467,0.0000,0.03571,1,0,null,null,"BID_PER_ACTION"],[20451,530717,"306-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,4328.00,139,31.1366906,1,4328.000000,30.00,0.0069316,0.0000,0.00719,1,0,null,null,"ROI_COEFFICIENT"],[20540,527691,"RICH-财神到我家_幸存者-每次付费_排3000_73011_0312_95325530","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2647.57,32,82.7365625,1,2647.570000,942.00,0.3557980,0.0000,0.03125,4,2,null,null,"BID_PER_ACTION"],[20540,530798,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2285315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,198.48,10,19.8480000,1,198.480000,12.00,0.0604595,0.0000,0.10000,1,0,null,null,"BID_PER_ACTION"],[20545,530715,"0315-幸存者393z每次付费排除3k_2792_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,3617.26,61,59.2993443,1,3617.260000,30.00,0.0082936,0.0000,0.01639,1,0,null,null,"BID_PER_ACTION"],[20637,526845,"263-幸存者-MCFC0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,2412.24,49,49.2293878,1,2412.240000,30.00,0.0124366,0.0000,0.02041,8,2,null,null,"BID_PER_ACTION"],[20637,526943,"05-幸存者-MCFC0311_7_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,2785.52,33,84.4096970,1,2785.520000,30.00,0.0107700,0.0000,0.03030,10,2,null,null,"BID_PER_ACTION"],[20754,523295,"zhan-每次付费--3.9_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,0,462.36,5,92.4720000,1,462.360000,30.00,0.0648845,0.0000,0.20000,2,2,null,null,"BID_PER_ACTION"],[20754,529159,"yo-roi系数-周六-3.13_44","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,36.06,14,2.5757143,1,36.060000,1040.00,28.8408209,0.0000,0.07143,6,2,null,null,"ROI_COEFFICIENT"],[21208,531039,"bdna-非海马云-幸存者挑战-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2803.74,66,42.4809091,1,2803.740000,12.00,0.0042800,0.0000,0.01515,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531040,"bdna-非海马云-幸存者挑战-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2664.20,7,380.6000000,1,2664.200000,12.00,0.0045042,0.0000,0.14286,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531042,"bdna-非海马云-幸存者挑战-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2393.44,25,95.7376000,1,2393.440000,74.00,0.0309178,0.0000,0.04000,1,0,null,null,"DEEP_BID_DEFAULT"],[21212,529546,"i-每次付费-幸存者挑战周日-03.14_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,3113.07,42,74.1207143,1,3113.070000,68.00,0.0218434,0.0000,0.02381,2,0,null,null,"BID_PER_ACTION"],[21245,529408,"每次-周末冲刺-0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,4637.03,35,132.4865714,1,4637.030000,274.00,0.0590895,0.0000,0.02857,3,0,null,null,"BID_PER_ACTION"],[20541,506968,"RICH-财神到 -研究-通投_横SSR2138_0226_16","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,0,4672.90,52,89.8634615,1,4672.900000,192.00,0.0410880,0.0000,0.01923,3,2,null,null,"DEEP_BID_DEFAULT"],[20541,507537,"RICH-财神到-研究所_IOS通投_横SSR2363_0226_3","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,0,3802.28,48,79.2141667,1,3802.280000,42.00,0.0110460,0.0000,0.02083,16,2,null,null,"DEEP_BID_DEFAULT"],[20541,528019,"RICH-研究所-财神到我家_通投_横SSR2710_0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,0,3738.32,32,116.8225000,2,1869.160000,54.00,0.0144450,0.0000,0.06250,2,2,null,null,"DEEP_BID_DEFAULT"],[20541,531196,"RICH-研究所-财神到我家-通投_混SSR2796_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,3208.41,17,188.7300000,2,1604.205000,110.00,0.0342849,0.0000,0.11765,3,0,null,null,"DEEP_BID_DEFAULT"],[20546,531301,"0315wtd末日研究所_2138_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,4501.64,47,95.7795745,1,4501.640000,60.00,0.0133285,0.0000,0.02128,1,0,null,null,"DEEP_BID_DEFAULT"],[20545,531701,"0315-508幸存危机-每次付费_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,280.87,6,46.8116667,1,280.870000,1274.00,4.5359063,0.0000,0.16667,1,0,null,null,"BID_PER_ACTION"],[21208,529912,"481IC-幸存危机-2756爆量爆R-3.5_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,478.87,5,95.7740000,1,478.870000,72.00,0.1503540,0.0000,0.20000,1,0,null,null,"BID_PER_ACTION"],[21208,529915,"481IC-幸存危机-2756爆量爆R-3.5_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,1227.01,14,87.6435714,1,1227.010000,384.00,0.3129559,0.0000,0.07143,1,0,null,null,"BID_PER_ACTION"],[20540,452869,"HSX-幸存者-自动ROI-通投-头抖_混SSR1909_0128_20","2021-03-16 00:00:00","2021-01-28 00:00:00",10,1001379,1,0,28.21,5,5.6420000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20540,489063,"RICH-幸存者_春节-沉浸付费_安卓通投_69686_0215_07317475","2021-03-16 00:00:00","2021-02-15 00:00:00",10,1001379,1,0,3301.89,92,35.8901087,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,513880,"RICH-研究所-招R体-通投_横SSR2248_0303_12","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,0,3802.28,37,102.7643243,0,0.000000,0.00,0E-7,0.0000,0.00000,4,2,null,null,"DEEP_BID_DEFAULT"],[20541,513881,"RICH-研究所-招R体-通投_横SSR2185_0303_1","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,0,3.89,0,3.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20754,520091,"yo-roi系数roi-想个好名字-2.6_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,0,142.09,1,142.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_COEFFICIENT"],[21212,520588,"r-自动ROI-幸存者充值是大哥大-03.08_6","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,0,17.36,0,17.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,520837,"by每次付费-xcztz-0308_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,0,2500.95,55,45.4718182,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21245,520854,"dm每次付费-xcztz-0308_15","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,0,476.90,1,476.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,522092,"r-自动ROI-幸存者yyds-03.09_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,0,9.99,4,2.4975000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_PACING"],[20545,522465,"0309-508幸存危机-每次付费_2271_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,0,9.83,0,9.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20541,523054,"RICH-研究所-财神到我家-通投_横SSR2138_0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,0,2953.49,19,155.4468421,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[20754,523280,"zhan-每次付费--3.9_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,0,3247.66,44,73.8104545,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,524145,"293bd-末日海马云测试_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,0,2568.77,23,111.6856522,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524146,"293bd-末日海马云测试_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,0,2590.05,0,2590.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524147,"293bd-末日海马云测试_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,0,2696.61,1,2696.6100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524148,"293bd-末日海马云测试_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,0,2467.83,0,2467.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524149,"293bd-末日海马云测试_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,0,2546.98,0,2546.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20754,524825,"hix-每次付费--3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,0,394.83,1,394.8300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524832,"jap-每次付费--3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,0,484.50,1,484.5000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525005,"DM每次付费-xcztz-拉环卡车-0310-_23","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,0,2775.02,80,34.6877500,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21245,525019,"DM每次付费-xcztz-拉环卡车-0310-_18","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,0,113.81,0,113.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,525044,"ka-roi系数-爆好-3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,0,12.03,2,6.0150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21212,525586,"k-每次付费-幸存危机加量-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,0,675.12,1,675.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20540,526528,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2363_0311_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,1131.93,5,226.3860000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20647,526683,"cxq-tt548N1计划-0311_12742","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,0,2116.54,19,111.3968421,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[20647,526684,"cxq-tt894N1计划-0311_22558","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,0,2590.79,1,2590.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526688,"cxq-tt894N1计划-0311_6小2723","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,0,1355.85,2,677.9250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,526703,"N1穿山甲-iOS-499-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,0,3802.28,4,950.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"DEEP_BID_DEFAULT"],[6867,526752,"0311-tve-每次付费-幸_20","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,301.71,19,15.8794737,0,0.000000,0.00,0E-7,0.0000,0.00000,4,2,null,null,"BID_PER_ACTION"],[6867,526753,"0311-tve-每次付费-幸_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,180.02,1,180.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526755,"0311-tve-每次付费-幸_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526754,"0311-tve-每次付费-幸_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,9.21,0,9.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526757,"0311-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,95.63,1,95.6300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526756,"0311-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,75.36,2,37.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526758,"0311-tve-每次付费-幸_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,32.13,1,32.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526759,"0311-tve-每次付费-幸_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,5.21,0,5.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526760,"0311-tve-每次付费-幸_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,9.05,0,9.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526761,"0311-tve-每次付费-幸_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,6.55,0,6.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526762,"0311-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,0.42,0,0.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526764,"0311-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,0.75,0,0.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526765,"0311-tve-每次付费-幸_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,0.64,0,0.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526766,"0311-tve-每次付费-幸_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,15.93,0,15.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526767,"0311-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,8.93,0,8.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526770,"0311-tve-每次付费-幸_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,303.56,10,30.3560000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20545,526883,"0311幸存者393z每次付费3k_2735_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,702.90,4,175.7250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526884,"0311幸存者393z每次付费3k_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,550.66,4,137.6650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526888,"0311全民534自动roi排除3k_2754_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,1.78,0,1.7800000,0,0.000000,0.00,0E-7,0.0000,0,2,2,null,null,"ROI_PACING"],[20546,526922,"0311wtd末日研究所_2754_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,0,706.13,0,706.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526923,"0311wtd末日研究所_2760_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,0,730.79,0,730.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526925,"0311wtd末日研究所_2736_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,0,633.75,1,633.7500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20754,527176,"hix-每次付费--3.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,0,353.28,3,117.7600000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21245,527266,"每次-2762-冲冲冲_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2824.93,27,104.6270370,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,527267,"每次-2762-冲冲冲_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2496.23,20,124.8115000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21208,527304,"295-blcb-每次付费爆R爆量_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,172.87,2,86.4350000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20451,527365,"SR-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.07,8,0.0087500,0,0.000000,0.00,0E-7,0.0000,0.00000,5,2,null,null,"BID_PER_ACTION"],[20451,527367,"SR-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.24,2,0.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,6,2,null,null,"BID_PER_ACTION"],[6867,527470,"0312540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.24,0,0.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527471,"0312540-每次付费-幸_15","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,8.47,0,8.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527472,"0312540-每次付费-幸_24","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,68.85,1,68.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527473,"0312540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,28.43,1,28.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[6867,527477,"0312540-每次付费-幸_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.53,0,0.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527478,"0312540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,39.56,1,39.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527479,"0312540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,38.54,0,38.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527481,"0312540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,22.73,0,22.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527484,"0312540-每次付费-幸_11","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,1.12,0,1.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527485,"0312540-每次付费-幸_12","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,9.03,0,9.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527487,"0312540-每次付费-幸_14","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,27.20,1,27.2000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527489,"0312540-每次付费-幸_17","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,39.73,0,39.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527495,"0312540-每次付费-幸_23","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,12.91,0,12.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527496,"0312540-每次付费-幸_25","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,28.21,0,28.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527497,"0312540-每次付费-幸_26","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,13.52,2,6.7600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527498,"0312540-每次付费-幸_27","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,188.03,0,188.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20541,528016,"RICH-研究所-财神到我家_通投_横SSR2285_0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,0,102.60,0,102.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,528066,"7tt_幸_自动ROI__0312_88256792","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.11,0,0.1100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20754,528136,"hix-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,30.28,0,30.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528157,"tw-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,439.79,12,36.6491667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528353,"ka-roi系数-周五-3.12_4_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,1098.52,29,37.8800000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_COEFFICIENT"],[20754,528356,"ka-roi系数-周五-3.12_4_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,54.78,4,13.6950000,0,0.000000,0.00,0E-7,0.0000,0.00000,6,2,null,null,"ROI_COEFFICIENT"],[21212,528558,"g-自动ROI-幸存者挑战-03.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,898.27,61,14.7257377,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_PACING"],[21212,528562,"g-自动ROI-幸存者挑战-03.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,1339.51,28,47.8396429,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_PACING"],[20637,528659,"990-幸存者-MCFC-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.06,0,0.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528774,"7tt_穿穿_每次付费__0312_02259785","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,1.47,0,1.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528783,"7tt_穿穿_每次付费__0312_022597911","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,364.67,0,364.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528784,"7tt_穿穿_每次付费__0312_02259772","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,17.41,0,17.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528778,"7tt_穿穿_每次付费__0312_02259786","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,3349.33,3,1116.4433333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528780,"7tt_穿穿_每次付费__0312_02259760","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2786.85,0,2786.8500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528785,"7tt_穿穿_每次付费__0312_02259798","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,30.27,1,30.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528777,"7tt_穿穿_每次付费__0312_02259799","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,17.13,0,17.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528786,"7tt_穿穿_每次付费__0312_022598012","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2988.09,0,2988.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528788,"7tt_穿穿_每次付费__0312_022598014","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2154.33,0,2154.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528789,"7tt_穿穿_每次付费__0312_022598015","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,1.23,0,1.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528791,"7tt_穿穿_每次付费__0312_022598017","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2350.84,26,90.4169231,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"BID_PER_ACTION"],[6770,528792,"7tt_穿穿_每次付费__0312_022598118","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,1.80,0,1.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528794,"7tt_穿穿_每次付费__0312_022598120","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528795,"7tt_穿穿_每次付费__0312_022598121","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2859.75,1,2859.7500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528797,"7tt_穿穿_每次付费__0312_022598223","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2980.88,1,2980.8800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528798,"7tt_穿穿_每次付费__0312_022598224","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.82,0,0.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528800,"7tt_穿穿_每次付费__0312_022598226","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2931.48,0,2931.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528801,"7tt_穿穿_每次付费__0312_022598227","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.70,1,0.7000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528803,"7tt_穿穿_每次付费__0312_022598329","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,48.28,0,48.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528859,"7tt_498_穿哥_每次付费_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,896.63,0,896.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528862,"7tt_498_穿哥_每次付费_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,2266.62,12,188.8850000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[6770,528863,"7tt_498_穿哥_每次付费_3.12_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,0.46,0,0.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528963,"263-幸存者-MCFC0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,0,480.67,3,160.2233333,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20540,529036,"RICH-财神到我家_幸存者-每次付费_排3000_70019_0313_08286060","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,771.28,7,110.1828571,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20540,529038,"RICH-财神到我家_幸存者-每次付费_排3000_77023_0313_08286082","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,790.50,8,98.8125000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[21212,529098,"r3-每次付费-幸存者挑战周六-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,802.53,2,401.2650000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[21212,529099,"r3-每次付费-幸存者挑战周六-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,1862.64,15,124.1760000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20754,529161,"yo-roi系数-周六-3.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,1390.67,33,42.1415152,0,0.000000,0.00,0E-7,0.0000,0.00000,7,2,null,null,"ROI_COEFFICIENT"],[20754,529154,"ka-自动roi-周六-3.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,6.59,0,6.5900000,0,0.000000,0.00,0E-7,0.0000,0,2,2,null,null,"ROI_PACING"],[20754,529158,"ka-自动roi-周六-3.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,5.87,1,5.8700000,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"ROI_PACING"],[6770,529170,"7tt_n_幸存危机_每次付费__0313_84190331","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,0,13.96,0,13.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529171,"7tt_穿穿_每次付费__0313_84191313","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,167.29,0,167.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529169,"7tt_穿穿_每次付费__0313_84191302","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,1.87,0,1.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529178,"7tt_n_幸存危机_每次付费__0313_84190348","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,0,16.14,0,16.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529186,"7tt_穿穿_每次付费__0313_84191290","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,3611.66,14,257.9757143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529187,"7tt_穿穿_每次付费__0313_84191301","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,2412.17,0,2412.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529189,"7tt_穿穿_每次付费__0313_84191315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,7.59,0,7.5900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529190,"7tt_穿穿_每次付费__0313_84191316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,6.98,1,6.9800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529191,"7tt_穿穿_每次付费__0313_84191317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,3303.24,1,3303.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529192,"7tt_穿穿_每次付费__0313_84191318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,1.12,0,1.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529193,"7tt_穿穿_每次付费__0313_84191329","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,2177.30,1,2177.3000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529194,"7tt_穿穿_每次付费__0313_841913210","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,0.63,0,0.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529196,"7tt_穿穿_每次付费__0313_841913212","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,27.53,0,27.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529197,"7tt_穿穿_每次付费__0313_841913213","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,3759.67,0,3759.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529198,"7tt_穿穿_每次付费__0313_841913315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,2474.09,0,2474.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529199,"7tt_穿穿_每次付费__0313_841913214","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,20.56,0,20.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529200,"7tt_穿穿_每次付费__0313_841913316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,2.38,0,2.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529201,"7tt_穿穿_每次付费__0313_841913317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,1727.84,0,1727.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529202,"7tt_穿穿_每次付费__0313_841913318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,2.08,0,2.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529203,"7tt_穿穿_每次付费__0313_841913319","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,1.32,0,1.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,529211,"RICH-财神到我家-幸存者-每次付费_通投-混SSR2762-313_8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,3898.75,19,205.1973684,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"BID_PER_ACTION"],[20540,529220,"RICH-末日-穿山甲-付费_通投_混SSR2746_0313_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,0,0.03,2,0.0150000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[20540,529216,"RICH-末日-穿山甲-付费_春节DMP_混SSR2746_0313_9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,0,552.04,1,552.0400000,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"DEEP_BID_DEFAULT"],[20646,529223,"cxq-tt338幸存者-每次付费-穿山甲0313_27622000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,3146.17,84,37.4544048,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"BID_PER_ACTION"],[20646,529228,"cxq-tt552幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,9.23,1,9.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529226,"cxq-tt552幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,3.00,0,3.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529227,"cxq-tt338幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,0.27,0,0.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529224,"cxq-tt338幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,1.36,0,1.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529234,"cxq-tt552幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,13.25,3,4.4166667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529232,"cxq-tt552幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,0.05,0,0.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529231,"cxq-tt338幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529233,"cxq-tt338幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,4.66,0,4.6600000,0,0.000000,0.00,0E-7,0.0000,0,1,2,null,null,"BID_PER_ACTION"],[20646,529259,"cxq-tt982幸存者-每次付费-0313_包1SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,0,1401.87,22,63.7213636,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21246,529419,"穿-n1-周末付费冲-0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,0,1070.64,1,1070.6400000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"DEEP_BID_DEFAULT"],[6770,529462,"7tt_穿穿_每次付费__0314_72230811","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,19.33,0,19.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529461,"7tt_穿穿_每次付费__0314_72230800","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,53.08,0,53.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529464,"7tt_穿穿_每次付费__0314_72230827","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,3023.31,1,3023.3100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529466,"7tt_穿穿_每次付费__0314_72230829","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,2522.51,2,1261.2550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529463,"7tt_穿穿_每次付费__0314_72230825","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,18.53,1,18.5300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529465,"7tt_穿穿_每次付费__0314_72230826","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,12.77,0,12.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529468,"7tt_穿穿_每次付费__0314_72230813","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,1244.45,1,1244.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529469,"7tt_穿穿_每次付费__0314_72230812","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,37.74,2,18.8700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529471,"7tt_穿穿_每次付费__0314_72230828","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,9.74,0,9.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529467,"7tt_穿穿_每次付费__0314_72230824","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,7.93,0,7.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529470,"7tt_穿穿_每次付费__0314_722308311","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,1047.23,0,1047.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529544,"i-每次付费-幸存者挑战周日-03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,103.40,1,103.4000000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[21212,529545,"r3-每次付费-幸存者挑战周日-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,971.04,26,37.3476923,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"BID_PER_ACTION"],[21212,529541,"r3-每次付费-幸存者挑战周日-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,1414.38,13,108.7984615,0,0.000000,0.00,0E-7,0.0000,0.00000,3,0,null,null,"BID_PER_ACTION"],[21212,529540,"r3-每次付费-幸存者挑战周日-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,2727.21,27,101.0077778,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[21212,529551,"r-自动ROI-幸存者挑战带穿山甲-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,45.20,2,22.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"ROI_PACING"],[21212,529557,"r-每次付费-幸存者挑战-穿山甲专场-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,792.68,3,264.2266667,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[20754,529598,"zhan-每次付费--3.13_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529600,"zhan-每次付费--3.13_17","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,303.83,1,303.8300000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"BID_PER_ACTION"],[6867,529648,"0314-540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,330.11,2,165.0550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529647,"0314-540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,2879.32,58,49.6434483,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529650,"0314-540-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,653.64,4,163.4100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529652,"0314-540-每次付费-幸_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,3064.78,136,22.5351471,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"BID_PER_ACTION"],[6867,529654,"0314-540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,367.75,0,367.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529656,"0314-540-每次付费-幸_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,2936.27,37,79.3586486,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529655,"0314-540-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,3037.92,76,39.9726316,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529660,"0314-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,183.66,10,18.3660000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529668,"0314-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,161.11,2,80.5550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529667,"0314-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,6.46,0,6.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529673,"0314-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,41.68,0,41.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529666,"0314-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,0,50.93,0,50.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529719,"470-tv-每次付费拉环爆R爆量_3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,5.55,0,5.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529715,"470-tv-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,72.73,4,18.1825000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529716,"470-tv-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2.99,4,0.7475000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529718,"470-tv-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,260.60,7,37.2285714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529720,"470-tv-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,55.72,4,13.9300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529722,"470-tv-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,24.61,0,24.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529723,"470-tv-每次付费拉环爆R爆量_3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,38.50,0,38.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529724,"470-tv-每次付费拉环爆R爆量_3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,22.76,0,22.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529747,"295-blcb-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,15.97,0,15.9700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529749,"295-blcb-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,77.92,4,19.4800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529754,"295-blcb-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,66.67,1,66.6700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529755,"295-blcb-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,97.30,4,24.3250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529752,"295-blcb-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,5.55,1,5.5500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529851,"263-幸存者-MCFC0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2851.71,63,45.2652381,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[20637,529865,"263-幸存者-MCFC0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,969.89,15,64.6593333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529910,"0315-204-穿山甲-每次付费-幸_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,15.19,0,15.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529909,"0315-204-穿山甲-每次付费-幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,7.65,0,7.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529914,"0315-204-穿山甲-每次付费-幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,75.10,4,18.7750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529916,"481IC-幸存危机-2756爆量爆R-3.5_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,2803.74,34,82.4629412,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529917,"0315-204-穿山甲-每次付费-幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,50.88,5,10.1760000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529920,"0315-204-穿山甲-每次付费-幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,483.17,19,25.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529911,"481IC-幸存危机-2756爆量爆R-3.5_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,1743.56,17,102.5623529,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[6867,529921,"0315-204-穿山甲-每次付费-幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,248.71,16,15.5443750,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529922,"0315-204-穿山甲-每次付费-幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,9.63,0,9.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529923,"0315-204-穿山甲-每次付费-幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,3.84,0,3.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529924,"0315-204-穿山甲-每次付费-幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,56.00,1,56.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529925,"0315-204-穿山甲-每次付费-幸_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,40.05,0,40.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529926,"0315-204-穿山甲-每次付费-幸_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,10.65,0,10.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529927,"0315-204-穿山甲-每次付费-幸_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,4.54,0,4.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529974,"990-幸存者-MCFC-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,129.02,5,25.8040000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[20637,529975,"990-幸存者-MCFC-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.01,0,0.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529977,"990-幸存者-MCFC-0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,43.72,0,43.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529981,"990-幸存者-MCFC-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529980,"990-幸存者-MCFC-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.01,0,0.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529976,"990-幸存者-MCFC-0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,148.86,4,37.2150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529983,"990-幸存者-MCFC-0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,35.52,3,11.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529982,"990-幸存者-MCFC-0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,32.26,1,32.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530102,"每0315-tt-xcztz-wyt-rk_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,18.10,0,18.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530101,"每0315-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,15.23,0,15.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530104,"每0315-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,3.78,0,3.7800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530109,"每0315-tt-xcztz-wyt-rk_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,5.76,0,5.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530100,"每0315-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.08,0,0.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530111,"每0315-tt-xcztz-wyt-rk_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,7.22,0,7.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530112,"每0315-tt-xcztz-wyt-rk_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,13.48,0,13.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530113,"每0315-tt-xcztz-wyt-rk_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,49.71,0,49.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530114,"每0315-tt-xcztz-wyt-rk_21","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,100.41,0,100.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530115,"每0315-tt-xcztz-wyt-rk_22","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1.05,0,1.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530117,"每0315-tt-xcztz-wyt-rk_24","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,83.54,0,83.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530122,"每0315-tt-xcztz-wyt-rk_31","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.38,0,0.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530125,"每0315-tt-xcztz-wyt-rk_34","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,3.23,0,3.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530126,"每0315-tt-xcztz-wyt-rk_37","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.56,0,0.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530129,"每0315-tt-xcztz-wyt-rk_40","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2.62,0,2.6200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530130,"每0315-tt-xcztz-wyt-rk_41","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1.22,0,1.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530131,"每0315-tt-xcztz-wyt-rk_42","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.05,0,0.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530132,"每0315-tt-xcztz-wyt-rk_45","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.09,0,0.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530133,"每0315-tt-xcztz-wyt-rk_46","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,84.01,2,42.0050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530134,"每0315-tt-xcztz-wyt-rk_47","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1.03,0,1.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530152,"每0315-tt-xcztz-wyt-rk_68","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1475.73,7,210.8185714,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"BID_PER_ACTION"],[21212,530168,"r3-每次付费-幸存者挑战优选位-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.08,0,0.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,530175,"i-每次付费-幸存者挑战只限男优先位-03.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,4.20,1,4.2000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530174,"r3-每次付费-幸存者挑战优选位-03.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,43.95,0,43.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,530177,"i-每次付费-幸存者挑战只限男优先位-03.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,80.03,1,80.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530178,"i-每次付费-幸存者挑战只限男优先位-03.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2794.70,32,87.3343750,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[21212,530210,"i-每次付费-幸存者挑战只限男优先位-03.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,29.36,0,29.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,530212,"r3-每次付费-幸存者挑战优选位-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,41.69,1,41.6900000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[21212,530388,"ROI系数-幸存者挑战-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,23.46,1,23.4600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530514,"0315幸存者393z每次付费3k_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1838.51,0,1838.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530516,"0315幸存者393z每次付费3k_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1802.40,4,450.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530515,"0315幸存者393z每次付费3k_2731_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2191.47,1,2191.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530517,"0315幸存者393z每次付费3k_2746_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1247.99,11,113.4536364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530518,"0315幸存者393z每次付费3k_2735_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1163.25,0,1163.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530526,"0315oyo393-roi系数-排除3k_2138_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,792.29,7,113.1842857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530531,"0315oyo393-roi系数-排除3k_2731_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1140.58,2,570.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530529,"0315oyo393-roi系数-排除3k_2522_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,773.13,4,193.2825000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530530,"0315oyo393-roi系数-排除3k_2762_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1161.37,9,129.0411111,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20635,530537,"每0315-tt-xcztz-wyt-yan_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,50.72,1,50.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530540,"每0315-tt-xcztz-wyt-yan_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,96.80,2,48.4000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530541,"每0315-tt-xcztz-wyt-yan_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.08,0,0.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530542,"每0315-tt-xcztz-wyt-yan_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,41.53,0,41.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530543,"每0315-tt-xcztz-wyt-yan_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530544,"每0315-tt-xcztz-wyt-yan_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,109.36,4,27.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530545,"每0315-tt-xcztz-wyt-yan_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,261.54,10,26.1540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,530670,"cxq-tt982幸存者-ROI系数0315_2762","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,578.90,16,36.1812500,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"ROI_COEFFICIENT"],[20646,530671,"cxq-tt747幸存者-ROI系数0315_2738","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,181.30,8,22.6625000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530672,"cxq-tt549幸存者-ROI系数0315_2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,458.54,21,21.8352381,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530674,"cxq-tt982幸存者-ROI系数0315_2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530676,"cxq-tt530幸存者-ROI系数0315_录2793","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,251.01,0,251.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530680,"cxq-tt549幸存者-ROI系数0315_2781","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,437.21,4,109.3025000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530693,"0315-508幸存危机-每次付费竖版沉浸_2762_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,109.13,0,109.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530692,"0315-508幸存危机-每次付费竖版沉浸_2332_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,11.97,0,11.9700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530698,"0315-508幸存危机-每次付费竖版沉浸_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,564.89,5,112.9780000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530696,"0315-508幸存危机-每次付费竖版沉浸_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,30.51,0,30.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530718,"0315-幸存者393z每次付费排除3k_2784_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,183.79,1,183.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530713,"0315-幸存者393z每次付费排除3k_2783_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1057.20,3,352.4000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530716,"0315-幸存者393z每次付费排除3k_2786_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,41.64,0,41.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530779,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2332_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,407.07,15,27.1380000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530781,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2731_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,9.67,0,9.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530780,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,23.56,0,23.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530782,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2751_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.56,0,0.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530783,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2794_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,103.55,0,103.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530784,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2271_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,303.16,14,21.6542857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530786,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2484_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,69.24,9,7.6933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530785,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,653.25,30,21.7750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530787,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,135.51,0,135.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530788,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1.39,0,1.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530802,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2789315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,15.21,0,15.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530803,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2761315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,6.26,0,6.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530800,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2746315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,21.03,2,10.5150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,530806,"0315FP-ROI系数-MG幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2525.07,3,841.6900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530805,"0315FP-ROI系数-MG幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1138.81,7,162.6871429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530807,"0315FP-ROI系数-MG幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2220.54,4,555.1350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530810,"0315FP-ROI系数-MG幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,997.06,1,997.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530809,"0315FP-ROI系数-MG幸_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1191.77,0,1191.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[6867,530808,"0315FP-ROI系数-MG幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2848.37,2,1424.1850000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"ROI_COEFFICIENT"],[6867,530844,"0315FP-ROI系数-MG幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,635.76,1,635.7600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530843,"0315FP-ROI系数-MG幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1331.58,5,266.3160000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530846,"0315FP-ROI系数-MG幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1592.50,5,318.5000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530845,"0315FP-ROI系数-MG幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1146.38,1,1146.3800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530906,"jap-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,590.79,4,147.6975000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[20754,530912,"jap-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.06,0,0.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530909,"jap-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1356.96,15,90.4640000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530919,"jap-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.09,0,0.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530921,"zhan-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,625.36,6,104.2266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530922,"jap-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.13,0,0.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530924,"tw-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,403.51,2,201.7550000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"BID_PER_ACTION"],[20754,530927,"zhan-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,251.53,4,62.8825000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530928,"zhan-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,930.83,7,132.9757143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530958,"yo-roi系数-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,107.97,0,107.9700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20754,530961,"ka-roi系数-周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,238.08,7,34.0114286,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"ROI_COEFFICIENT"],[20754,530965,"yo-roi系数-周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,143.29,5,28.6580000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530967,"yo-roi系数-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,62.54,4,15.6350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530969,"yo-roi系数-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,94.51,2,47.2550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530968,"ka-roi系数-周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,4.49,5,0.8980000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"ROI_COEFFICIENT"],[20635,530999,"每0315-tt-xcztz-wyt-zhong_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.49,0,0.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530998,"每0315-tt-xcztz-wyt-zhong_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,735.36,6,122.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531000,"每0315-tt-xcztz-wyt-zhong_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531001,"每0315-tt-xcztz-wyt-zhong_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,17.86,0,17.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531002,"每0315-tt-xcztz-wyt-zhong_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1.17,1,1.1700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531006,"每0315-tt-xcztz-wyt-zhong_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,7.88,0,7.8800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531004,"每0315-tt-xcztz-wyt-zhong_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,388.83,11,35.3481818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531003,"每0315-tt-xcztz-wyt-zhong_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,5.56,0,5.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531008,"每0315-tt-xcztz-wyt-zhong_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,15.43,0,15.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531011,"每0315-tt-xcztz-wyt-zhong_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.56,0,0.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21213,531013,"n96-ios-黎明血战-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,0.03,1,0.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531014,"n96-ios-黎明血战-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,0.23,2,0.1150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531016,"n96-ios-黎明血战-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,5.76,2,2.8800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531038,"bdna-非海马云-幸存者挑战-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2701.47,91,29.6864835,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531041,"bdna-非海马云-幸存者挑战-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2345.34,50,46.9068000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531187,"au-黎明血战--周一-3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,254.78,3,84.9266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531189,"th-黎明血战--周一-3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,5.73,0,5.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531190,"au-黎明血战--周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,0.13,0,0.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531194,"th-黎明血战--周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,1461.44,20,73.0720000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531193,"roi系数-0315-BY-幸存_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,830.10,65,12.7707692,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"ROI_COEFFICIENT"],[21245,531188,"roi系数-0315-BY-幸存_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,38.55,4,9.6375000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20541,531195,"RICH-研究所-财神到我家-通投_混SSR2791_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,1893.71,2,946.8550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531197,"RICH-研究所-财神到我家-通投_混SSR2797_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,20.07,0,20.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531198,"RICH-研究所-财神到我家-通投_混SSR2792_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,2174.01,12,181.1675000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"DEEP_BID_DEFAULT"],[20541,531199,"RICH-研究所-财神到我家-通投_混SSR2749_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,1090.63,1,1090.6300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531200,"RICH-研究所-财神到我家-通投_混SSR2789_0315_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,783.68,0,783.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531201,"RICH-研究所-财神到我家-通投_混SSR2783_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,1410.81,1,1410.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531208,"RICH-研究所-财神到我家-通投_创意组8_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,1540.55,1,1540.5500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531211,"th-黎明血战--周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,2.30,0,2.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531207,"roi系数-0315-BY-幸存_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,124.69,2,62.3450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531209,"roi系数-0315-BY-幸存_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,51.38,2,25.6900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20755,531215,"th-黎明血战--周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,12.84,1,12.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531214,"roi系数-0315-BY-幸存_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,143.87,1,143.8700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531213,"roi系数-0315-BY-幸存_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,25.85,0,25.8500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[21245,531206,"roi系数-0315-BY-幸存_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,274.43,0,274.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20755,531227,"th-黎明血战--周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,1.21,1,1.2100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531229,"au-黎明血战--周一-3.15_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,7.37,0,7.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531233,"au-黎明血战--周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,0.05,0,0.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531234,"roi系数-0315-BY-幸存_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,112.14,0,112.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20755,531252,"au-黎明血战--周一-3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,6.51,0,6.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531257,"th-黎明血战--周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,5.46,0,5.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531259,"au-黎明血战--周一-3.15_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,0,11.96,0,11.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20545,531288,"0315幸存者393z每次付费排除3K_2796_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,3668.47,65,56.4380000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"BID_PER_ACTION"],[20545,531287,"0315幸存者393z每次付费排除3K_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2803.74,14,200.2671429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531293,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1076.84,9,119.6488889,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531294,"cxq-tt338幸存者-穿山甲每次付费-0315_30002797","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,76.39,0,76.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531298,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.40,0,0.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531297,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2731","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,118.73,0,118.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531296,"cxq-tt338幸存者-穿山甲每次付费-0315_30002798","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,52.44,1,52.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531295,"cxq-tt338幸存者-穿山甲每次付费-0315_30002796","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,878.24,6,146.3733333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20546,531300,"0315wtd末日研究所_2332_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,1805.84,17,106.2258824,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531303,"0315wtd末日研究所_2730_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,830.36,1,830.3600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531304,"0315wtd末日研究所_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,1831.95,3,610.6500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531306,"263-幸存者-ROI-0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,140.89,7,20.1271429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531314,"263-幸存者-ROI-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,67.80,5,13.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531308,"263-幸存者-ROI-0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2851.71,118,24.1670339,0,0.000000,0.00,0E-7,0.0000,0.00000,4,0,null,null,"ROI_COEFFICIENT"],[20637,531312,"263-幸存者-ROI-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,572.98,6,95.4966667,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"ROI_COEFFICIENT"],[20637,531307,"263-幸存者-ROI-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,58.47,2,29.2350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531315,"263-幸存者-ROI-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,42.53,1,42.5300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20546,531576,"0315ph648末日研究所_2800_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,164.73,1,164.7300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531577,"0315ph648末日研究所_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,2401.78,26,92.3761538,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"DEEP_BID_DEFAULT"],[20546,531578,"0315ph648末日研究所_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,3714.21,52,71.4271154,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531579,"0315ph648末日研究所_2783_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,69.78,0,69.7800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531580,"0315ph648末日研究所_2519_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,0,13.32,0,13.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531586,"263-幸存者-MCFC0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2829.28,20,141.4640000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[20545,531702,"0315-508幸存危机-每次付费_2053_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,1834.54,11,166.7763636,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[20545,531706,"0315-508幸存危机-每次付费_2800_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,0,475.74,1,475.7400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,531772,"r-自动ROI-幸存者挑战晚上七点-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2374.93,12,197.9108333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531773,"r-自动ROI-幸存者挑战晚上七点-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,1117.60,3,372.5333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531774,"r-自动ROI-幸存者挑战晚上七点-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,0.11,0,0.1100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21212,531775,"r-自动ROI-幸存者挑战晚上七点-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,0,2.58,1,2.5800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[6770,525572,"7tt_529_幸存者挑战_自动ROI_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,3043.46,105,28.9853333,2,1521.730000,146.00,0.0479717,0.0000,0.01905,4,2,null,null,"ROI_PACING"],[6770,528791,"7tt_穿穿_每次付费__0312_022598017","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,4130.36,89,46.4085393,3,1376.786667,236.00,0.0571379,0.0000,0.03371,3,2,null,null,"BID_PER_ACTION"],[6770,528862,"7tt_498_穿哥_每次付费_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,4980.10,121,41.1578512,2,2490.050000,322.00,0.0646573,0.0000,0.01653,2,2,null,null,"BID_PER_ACTION"],[6867,529646,"0314-540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,4310.70,79,54.5658228,1,4310.700000,98.00,0.0227341,0.0000,0.01266,1,0,null,null,"BID_PER_ACTION"],[6867,529652,"0314-540-每次付费-幸_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,12389.74,515,24.0577476,2,6194.870000,1322.00,0.1067012,0.0000,0.00388,2,0,null,null,"BID_PER_ACTION"],[6867,529655,"0314-540-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,5653.59,139,40.6733094,1,5653.590000,110.00,0.0194567,0.0000,0.00719,0,0,null,null,"BID_PER_ACTION"],[6867,529656,"0314-540-每次付费-幸_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,4422.75,52,85.0528846,1,4422.750000,12.00,0.0027132,0.0000,0.01923,0,0,null,null,"BID_PER_ACTION"],[6867,529658,"0314-540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,18874.63,268,70.4277239,4,4718.657500,418.00,0.0221461,0.0000,0.01493,4,0,null,null,"BID_PER_ACTION"],[6867,529664,"0314-tve-每次付费-幸_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,2803.64,43,65.2009302,1,2803.640000,30.00,0.0107004,0.0000,0.02326,0,0,null,null,"BID_PER_ACTION"],[6867,529672,"0314-tve-每次付费-幸_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,2803.74,48,58.4112500,1,2803.740000,12.00,0.0042800,0.0000,0.02083,0,0,null,null,"BID_PER_ACTION"],[6867,529919,"0315-204-穿山甲-每次付费-幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2010.23,62,32.4230645,2,1005.115000,518.00,0.2576820,0.0000,0.03226,1,0,null,null,"BID_PER_ACTION"],[6867,529921,"0315-204-穿山甲-每次付费-幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,439.52,21,20.9295238,1,439.520000,12.00,0.0273025,0.0000,0.04762,0,0,null,null,"BID_PER_ACTION"],[6867,530808,"0315FP-ROI系数-MG幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3766.77,14,269.0550000,1,3766.770000,2624.00,0.6966181,0.0000,0.07143,1,0,null,null,"ROI_COEFFICIENT"],[20451,499074,"306-幸存者-ROI02.21_5","2021-03-16 00:00:00","2021-02-21 00:00:00",10,1001379,1,1,3360.27,157,21.4029936,1,3360.270000,12.00,0.0035711,0.0000,0.00637,1,2,null,null,"ROI_PACING"],[20451,527365,"SR-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,6485.42,101,64.2120792,1,6485.420000,42.00,0.0064761,0.0000,0.00990,5,2,null,null,"BID_PER_ACTION"],[20451,527367,"SR-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,1199.19,21,57.1042857,1,1199.190000,208.00,0.1734504,0.0000,0.04762,6,2,null,null,"BID_PER_ACTION"],[20451,527374,"PB-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,857.04,14,61.2171429,1,857.040000,340.00,0.3967143,0.0000,0.07143,1,2,null,null,"BID_PER_ACTION"],[20451,527376,"PB-财神到-每次03.12-9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,3997.40,95,42.0778947,1,3997.400000,42.00,0.0105068,0.0000,0.01053,6,2,null,null,"BID_PER_ACTION"],[20451,528913,"FJ-财神到-每次03.13-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,10100.00,152,66.4473684,4,2525.000000,162.00,0.0160396,0.0000,0.02632,9,2,null,null,"BID_PER_ACTION"],[20451,528914,"FJ-财神到-每次03.13-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,6701.87,111,60.3772072,2,3350.935000,140.00,0.0208897,0.0000,0.01802,3,2,null,null,"BID_PER_ACTION"],[20451,528918,"FJ-财神到-每次03.13-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,6701.87,86,77.9287209,3,2233.956667,120.00,0.0179055,0.0000,0.03488,4,2,null,null,"BID_PER_ACTION"],[20451,530717,"306-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,7688.27,237,32.4399578,1,7688.270000,30.00,0.0039020,0.0000,0.00422,1,0,null,null,"ROI_COEFFICIENT"],[20451,530719,"303-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3356.80,89,37.7168539,1,3356.800000,256.00,0.0762631,0.0000,0.01124,1,0,null,null,"ROI_COEFFICIENT"],[20451,530936,"FJ-幸存者-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3302.07,47,70.2568085,1,3302.070000,42.00,0.0127193,0.0000,0.02128,0,0,null,null,"BID_PER_ACTION"],[20540,527691,"RICH-财神到我家_幸存者-每次付费_排3000_73011_0312_95325530","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,6843.88,63,108.6330159,3,2281.293333,2200.00,0.3214551,0.0000,0.04762,4,2,null,null,"BID_PER_ACTION"],[20540,529211,"RICH-财神到我家-幸存者-每次付费_通投-混SSR2762-313_8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,15672.03,154,101.7664286,2,7836.015000,2750.00,0.1754718,0.0000,0.01299,3,2,null,null,"BID_PER_ACTION"],[20540,530798,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2285315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2534.93,64,39.6082813,1,2534.930000,12.00,0.0047339,0.0000,0.01563,1,0,null,null,"BID_PER_ACTION"],[20545,526888,"0311全民534自动roi排除3k_2754_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,12.96,3,4.3200000,1,12.960000,12.00,0.9259259,0.0000,0.33333,2,2,null,null,"ROI_PACING"],[20545,530526,"0315oyo393-roi系数-排除3k_2138_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1656.50,19,87.1842105,1,1656.500000,12.00,0.0072442,0.0000,0.05263,0,0,null,null,"ROI_COEFFICIENT"],[20545,530714,"0315-幸存者393z每次付费排除3k_2789_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,124.21,1,124.2100000,1,124.210000,298.00,2.3991627,0.0000,1.00000,1,0,null,null,"BID_PER_ACTION"],[20545,530715,"0315-幸存者393z每次付费排除3k_2792_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,4240.95,69,61.4630435,1,4240.950000,30.00,0.0070739,0.0000,0.01449,1,0,null,null,"BID_PER_ACTION"],[20545,531288,"0315幸存者393z每次付费排除3K_2796_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,10905.36,259,42.1056371,3,3635.120000,3214.00,0.2947175,0.0000,0.01158,2,0,null,null,"BID_PER_ACTION"],[20635,530152,"每0315-tt-xcztz-wyt-rk_68","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,8937.37,78,114.5816667,2,4468.685000,250.00,0.0279724,0.0000,0.02564,2,0,null,null,"BID_PER_ACTION"],[20637,526845,"263-幸存者-MCFC0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,4231.43,110,38.4675455,3,1410.476667,54.00,0.0127616,0.0000,0.02727,8,2,null,null,"BID_PER_ACTION"],[20637,526943,"05-幸存者-MCFC0311_7_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,5589.26,72,77.6286111,1,5589.260000,30.00,0.0053674,0.0000,0.01389,10,2,null,null,"BID_PER_ACTION"],[20637,529563,"05-幸存者-MCFC0314_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,289.23,7,41.3185714,2,144.615000,298.00,1.0303219,0.0000,0.28571,9,0,null,null,"BID_PER_ACTION"],[20637,529851,"263-幸存者-MCFC0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,7889.73,135,58.4424444,2,3944.865000,168.00,0.0212935,0.0000,0.01481,1,0,null,null,"BID_PER_ACTION"],[20637,529866,"263-幸存者-MCFC0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2851.71,50,57.0342000,1,2851.710000,128.00,0.0448853,0.0000,0.02000,0,0,null,null,"BID_PER_ACTION"],[20637,529974,"990-幸存者-MCFC-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1068.89,18,59.3827778,1,1068.890000,128.00,0.1197504,0.0000,0.05556,1,0,null,null,"BID_PER_ACTION"],[20637,531204,"05-幸存者-MCFC0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,167.78,3,55.9266667,1,167.780000,392.00,2.3363929,0.0000,0.33333,1,0,null,null,"BID_PER_ACTION"],[20637,531251,"05-幸存者-MCFC0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1168.82,16,73.0512500,1,1168.820000,226.00,0.1933574,0.0000,0.06250,1,0,null,null,"BID_PER_ACTION"],[20637,531308,"263-幸存者-ROI-0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,16963.69,458,37.0386245,4,4240.922500,602.00,0.0354876,0.0000,0.00873,4,0,null,null,"ROI_COEFFICIENT"],[20637,531312,"263-幸存者-ROI-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1496.10,13,115.0846154,1,1496.100000,226.00,0.1510594,0.0000,0.07692,1,0,null,null,"ROI_COEFFICIENT"],[20637,531586,"263-幸存者-MCFC0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,7135.46,49,145.6216327,1,7135.460000,42.00,0.0058861,0.0000,0.02041,1,0,null,null,"BID_PER_ACTION"],[20646,529234,"cxq-tt552幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,1359.62,50,27.1924000,1,1359.620000,12.00,0.0088260,0.0000,0.02000,0,2,null,null,"BID_PER_ACTION"],[20646,530670,"cxq-tt982幸存者-ROI系数0315_2762","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1648.92,48,34.3525000,1,1648.920000,188.00,0.1140140,0.0000,0.02083,1,0,null,null,"ROI_COEFFICIENT"],[20754,520091,"yo-roi系数roi-想个好名字-2.6_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,1,1055.01,14,75.3578571,1,1055.010000,30.00,0.0284357,0.0000,0.07143,2,2,null,null,"ROI_COEFFICIENT"],[20754,523295,"zhan-每次付费--3.9_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,1,789.63,8,98.7037500,2,394.815000,958.00,1.2132264,0.0000,0.25000,2,2,null,null,"BID_PER_ACTION"],[20754,528148,"zhan-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,280.52,8,35.0650000,1,280.520000,158.00,0.5632397,0.0000,0.12500,1,2,null,null,"BID_PER_ACTION"],[20754,528356,"ka-roi系数-周五-3.12_4_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,672.57,26,25.8680769,1,672.570000,3438.00,5.1117356,0.0000,0.03846,6,2,null,null,"ROI_COEFFICIENT"],[20754,529159,"yo-roi系数-周六-3.13_44","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,6448.24,178,36.2260674,3,2149.413333,1148.00,0.1780331,0.0000,0.01685,6,2,null,null,"ROI_COEFFICIENT"],[20754,529161,"yo-roi系数-周六-3.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,1390.67,33,42.1415152,1,1390.670000,30.00,0.0215723,0.0000,0.03030,7,2,null,null,"ROI_COEFFICIENT"],[20754,530906,"jap-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1142.78,6,190.4633333,1,1142.780000,200.00,0.1750118,0.0000,0.16667,1,0,null,null,"BID_PER_ACTION"],[20754,530924,"tw-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1087.12,17,63.9482353,2,543.560000,3096.00,2.8478917,0.0000,0.11765,2,0,null,null,"BID_PER_ACTION"],[20754,530959,"ka-roi系数-周一-3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,4853.92,117,41.4864957,4,1213.480000,182.00,0.0374955,0.0000,0.03419,1,0,null,null,"ROI_COEFFICIENT"],[20754,530961,"ka-roi系数-周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,5097.48,91,56.0162637,2,2548.740000,652.00,0.1279063,0.0000,0.02198,1,0,null,null,"ROI_COEFFICIENT"],[20754,530962,"ka-roi系数-周一-3.15_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2803.74,72,38.9408333,1,2803.740000,12.00,0.0042800,0.0000,0.01389,0,0,null,null,"ROI_COEFFICIENT"],[20754,530966,"yo-roi系数-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2659.43,26,102.2857692,1,2659.430000,42.00,0.0157929,0.0000,0.03846,1,0,null,null,"ROI_COEFFICIENT"],[20754,530968,"ka-roi系数-周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2115.48,31,68.2412903,2,1057.740000,238.00,0.1125040,0.0000,0.06452,2,0,null,null,"ROI_COEFFICIENT"],[21208,527304,"295-blcb-每次付费爆R爆量_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,1189.66,24,49.5691667,1,1189.660000,214.00,0.1798833,0.0000,0.04167,2,2,null,null,"BID_PER_ACTION"],[21208,531039,"bdna-非海马云-幸存者挑战-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2803.74,66,42.4809091,1,2803.740000,12.00,0.0042800,0.0000,0.01515,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531040,"bdna-非海马云-幸存者挑战-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2664.20,7,380.6000000,1,2664.200000,12.00,0.0045042,0.0000,0.14286,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531042,"bdna-非海马云-幸存者挑战-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2393.44,25,95.7376000,1,2393.440000,74.00,0.0309178,0.0000,0.04000,1,0,null,null,"DEEP_BID_DEFAULT"],[21212,529098,"r3-每次付费-幸存者挑战周六-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,802.53,2,401.2650000,1,802.530000,298.00,0.3713257,0.0000,0.50000,2,2,null,null,"BID_PER_ACTION"],[21212,529099,"r3-每次付费-幸存者挑战周六-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,3373.55,26,129.7519231,1,3373.550000,42.00,0.0124498,0.0000,0.03846,2,2,null,null,"BID_PER_ACTION"],[21212,529541,"r3-每次付费-幸存者挑战周日-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,2900.57,26,111.5603846,2,1450.285000,692.00,0.2385738,0.0000,0.07692,3,0,null,null,"BID_PER_ACTION"],[21212,529545,"r3-每次付费-幸存者挑战周日-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3006.99,66,45.5604545,1,3006.990000,324.00,0.1077489,0.0000,0.01515,2,0,null,null,"BID_PER_ACTION"],[21212,529546,"i-每次付费-幸存者挑战周日-03.14_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3113.07,42,74.1207143,1,3113.070000,68.00,0.0218434,0.0000,0.02381,2,0,null,null,"BID_PER_ACTION"],[21212,530169,"r3-每次付费-幸存者挑战优选位-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2708.30,13,208.3307692,1,2708.300000,12.00,0.0044308,0.0000,0.07692,0,0,null,null,"BID_PER_ACTION"],[21212,530178,"i-每次付费-幸存者挑战只限男优先位-03.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,6533.02,84,77.7740476,1,6533.020000,678.00,0.1037805,0.0000,0.01190,1,0,null,null,"BID_PER_ACTION"],[21212,530212,"r3-每次付费-幸存者挑战优选位-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2035.04,22,92.5018182,1,2035.040000,30.00,0.0147417,0.0000,0.04545,1,0,null,null,"BID_PER_ACTION"],[21245,525005,"DM每次付费-xcztz-拉环卡车-0310-_23","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,5578.76,141,39.5656738,1,5578.760000,208.00,0.0372843,0.0000,0.00709,1,2,null,null,"BID_PER_ACTION"],[21245,527266,"每次-2762-冲冲冲_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,5628.67,49,114.8708163,1,5628.670000,60.00,0.0106597,0.0000,0.02041,0,2,null,null,"BID_PER_ACTION"],[21245,529408,"每次-周末冲刺-0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,7440.77,58,128.2891379,2,3720.385000,474.00,0.0637031,0.0000,0.03448,3,0,null,null,"BID_PER_ACTION"],[21245,531193,"roi系数-0315-BY-幸存_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2820.12,162,17.4081481,1,2820.120000,110.00,0.0390054,0.0000,0.00617,1,0,null,null,"ROI_COEFFICIENT"],[21245,531228,"roi系数-0315-BY-幸存_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2948.60,41,71.9170732,1,2948.600000,30.00,0.0101743,0.0000,0.02439,0,0,null,null,"ROI_COEFFICIENT"],[20541,506968,"RICH-财神到 -研究-通投_横SSR2138_0226_16","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,1,8411.22,103,81.6623301,1,8411.220000,192.00,0.0228267,0.0000,0.00971,3,2,null,null,"DEEP_BID_DEFAULT"],[20541,507537,"RICH-财神到-研究所_IOS通投_横SSR2363_0226_3","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,1,13307.98,191,69.6752880,3,4435.993333,458.00,0.0344154,0.0000,0.01571,16,2,null,null,"DEEP_BID_DEFAULT"],[20541,528019,"RICH-研究所-财神到我家_通投_横SSR2710_0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,1,4519.48,37,122.1481081,2,2259.740000,54.00,0.0119483,0.0000,0.05405,2,2,null,null,"DEEP_BID_DEFAULT"],[20541,531196,"RICH-研究所-财神到我家-通投_混SSR2796_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,3588.63,19,188.8752632,3,1196.210000,122.00,0.0339963,0.0000,0.15789,3,0,null,null,"DEEP_BID_DEFAULT"],[20541,531198,"RICH-研究所-财神到我家-通投_混SSR2792_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,2750.84,18,152.8244444,1,2750.840000,98.00,0.0356255,0.0000,0.05556,1,0,null,null,"DEEP_BID_DEFAULT"],[20546,531301,"0315wtd末日研究所_2138_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,7969.71,71,112.2494366,1,7969.710000,60.00,0.0075285,0.0000,0.01408,1,0,null,null,"DEEP_BID_DEFAULT"],[20546,531577,"0315ph648末日研究所_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,3836.34,34,112.8335294,1,3836.340000,886.00,0.2309493,0.0000,0.02941,1,0,null,null,"DEEP_BID_DEFAULT"],[20647,526683,"cxq-tt548N1计划-0311_12742","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,1,7819.62,60,130.3270000,2,3909.810000,128.00,0.0163691,0.0000,0.03333,1,2,null,null,"DEEP_BID_DEFAULT"],[20647,526684,"cxq-tt894N1计划-0311_22558","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,1,3416.07,30,113.8690000,1,3416.070000,12.00,0.0035128,0.0000,0.03333,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,526703,"N1穿山甲-iOS-499-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,1,4359.13,13,335.3176923,2,2179.565000,352.00,0.0807501,0.0000,0.15385,3,2,null,null,"DEEP_BID_DEFAULT"],[21246,529419,"穿-n1-周末付费冲-0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,1,2605.54,12,217.1283333,1,2605.540000,60.00,0.0230279,0.0000,0.08333,1,0,null,null,"DEEP_BID_DEFAULT"],[20451,530040,"CJ-幸存危机-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,2256.19,17,132.7170588,1,2256.190000,30.00,0.0132968,0.0000,0.05882,1,0,null,null,"BID_PER_ACTION"],[20545,531701,"0315-508幸存危机-每次付费_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,2764.52,47,58.8195745,2,1382.260000,1354.00,0.4897776,0.0000,0.04255,1,0,null,null,"BID_PER_ACTION"],[20545,531702,"0315-508幸存危机-每次付费_2053_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,4513.16,74,60.9886486,1,4513.160000,30.00,0.0066472,0.0000,0.01351,1,0,null,null,"BID_PER_ACTION"],[20545,531704,"0315-508幸存危机-每次付费_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,2778.84,28,99.2442857,1,2778.840000,12.00,0.0043183,0.0000,0.03571,0,0,null,null,"BID_PER_ACTION"],[21208,527873,"481IC-幸存危机-648大集合-3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,173.44,4,43.3600000,1,173.440000,48.00,0.2767528,0.0000,0.25000,2,2,null,null,"BID_PER_ACTION"],[21208,529911,"481IC-幸存危机-2756爆量爆R-3.5_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,4546.33,62,73.3279032,1,4546.330000,12.00,0.0026395,0.0000,0.01613,1,0,null,null,"BID_PER_ACTION"],[21208,529912,"481IC-幸存危机-2756爆量爆R-3.5_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,1254.35,10,125.4350000,1,1254.350000,72.00,0.0574002,0.0000,0.10000,1,0,null,null,"BID_PER_ACTION"],[21208,529915,"481IC-幸存危机-2756爆量爆R-3.5_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,2643.53,27,97.9085185,1,2643.530000,384.00,0.1452603,0.0000,0.03704,1,0,null,null,"BID_PER_ACTION"],[20540,452869,"HSX-幸存者-自动ROI-通投-头抖_混SSR1909_0128_20","2021-03-16 00:00:00","2021-01-28 00:00:00",10,1001379,1,1,1018.77,55,18.5230909,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20540,489063,"RICH-幸存者_春节-沉浸付费_安卓通投_69686_0215_07317475","2021-03-16 00:00:00","2021-02-15 00:00:00",10,1001379,1,1,6493.52,139,46.7159712,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,507233,"RICH-财神保佑-幸存者-每次付费_春节游戏DMP_竖SSR2285_0226_5","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001379,1,1,2381.33,76,31.3332895,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20541,513880,"RICH-研究所-招R体-通投_横SSR2248_0303_12","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,1,8549.94,112,76.3387500,0,0.000000,0.00,0E-7,0.0000,0.00000,4,2,null,null,"DEEP_BID_DEFAULT"],[20541,513881,"RICH-研究所-招R体-通投_横SSR2185_0303_1","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,1,143.34,1,143.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,513883,"RICH-研究所-招R体-通投_横SSR2666_0303_10","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,1,107.73,0,107.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,517614,"RICH-幸存者-我也爆R-每次付费_安卓游戏_竖SSR2402_0305_3","2021-03-16 00:00:00","2021-03-05 00:00:00",10,1001379,1,1,0.77,0,0.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,520097,"yo-roi系数roi-想个好名字-2.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,1,106.86,5,21.3720000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,520100,"ka-roi系数-想个好名字-2.6_11","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,1,91.62,5,18.3240000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,520177,"ka-自动roi-周天-2.7_6","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,1,166.26,1,166.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,520180,"yo-自动roi-周天-2.7_3","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,1,9.68,0,9.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[21212,520588,"r-自动ROI-幸存者充值是大哥大-03.08_6","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,1,56.98,3,18.9933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20541,520818,"RICH-研究所-财神到我家-通投_横SSR2724_0308_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,1,79.38,1,79.3800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,520837,"by每次付费-xcztz-0308_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,1,2500.95,55,45.4718182,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21245,520854,"dm每次付费-xcztz-0308_15","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,1,476.90,1,476.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,521273,"g-自动ROI-幸存者加批计划跑晚上-03.08_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,1,0.07,19,0.0036842,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,521878,"自动ROI-JK-排3K包-拉环-_0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,1,146.79,20,7.3395000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20637,521886,"05-幸存者-MCFC0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,1,407.12,4,101.7800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,521889,"05-幸存者-MCFC0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,1,53.31,0,53.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,522092,"r-自动ROI-幸存者yyds-03.09_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,1,74.03,20,3.7015000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_PACING"],[6770,522169,"7tt_n_每次付费_幸存危机_0309_9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,1,108.23,2,54.1150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522175,"7tt_n_每次付费_幸存危机_0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,1,90.10,7,12.8714286,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20545,522465,"0309-508幸存危机-每次付费_2271_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,1,61.69,3,20.5633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20541,523054,"RICH-研究所-财神到我家-通投_横SSR2138_0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,1,4168.63,32,130.2696875,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[20541,523053,"RICH-研究所-财神到我家-通投_横SSR2363_0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,1,1107.54,23,48.1539130,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"DEEP_BID_DEFAULT"],[20541,523056,"RICH-研究所-财神到我家-通投_横SSR2536_0309_19","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,1,10.26,5,2.0520000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[20541,523061,"RICH-研究所-财神到我家-通投_横SSR1909_0309_18","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,1,22.35,2,11.1750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20754,523280,"zhan-每次付费--3.9_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,1,3247.66,44,73.8104545,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,524145,"293bd-末日海马云测试_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,1,2568.77,23,111.6856522,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524146,"293bd-末日海马云测试_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,1,2590.05,0,2590.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524147,"293bd-末日海马云测试_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,1,2696.61,1,2696.6100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524148,"293bd-末日海马云测试_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,1,2467.83,0,2467.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524149,"293bd-末日海马云测试_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,1,2546.98,0,2546.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21212,524367,"k-每次付费-危机窄定向-03.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,1,854.03,3,284.6766667,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21245,524567,"自动ROI-排3K包-0310_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,56.13,6,9.3550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524575,"自动ROI-排3K包-0310_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,72.64,4,18.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524581,"自动ROI-排3K包-0310_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,2.17,1,2.1700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524582,"自动ROI-排3K包-0310_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,8.20,0,8.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20754,524825,"hix-每次付费--3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,705.68,1,705.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524832,"jap-每次付费--3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,919.60,1,919.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525019,"DM每次付费-xcztz-拉环卡车-0310-_18","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,113.81,0,113.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,525044,"ka-roi系数-爆好-3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,96.57,8,12.0712500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[6770,525577,"7tt_498_幸存者挑战_自动ROI_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,1,3045.17,121,25.1666942,0,0.000000,0.00,0E-7,0.0000,0.00000,5,2,null,null,"ROI_PACING"],[21212,525586,"k-每次付费-幸存危机加量-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,1,1394.68,3,464.8933333,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20541,526248,"RICH-研究所-财神到我家_通投-新用户-_SSR2248_0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,1,26.05,1,26.0500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526251,"RICH-研究所-财神到我家_通投-包过滤-_SSR2248_0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,1,184.77,3,61.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,526325,"7tt_n_幸存危机_每次付费__0311_52245342","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,1,37.14,0,37.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,526408,"263-幸存者-ROI-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,1536.53,29,52.9837931,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"ROI_COEFFICIENT"],[21212,526440,"r3-每次付费-幸存者挑战-03.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,2771.21,64,43.3001563,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526444,"r3-每次付费-幸存者挑战-03.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,2763.86,30,92.1286667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526453,"r3-每次付费-幸存者挑战-03.11_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,2773.18,22,126.0536364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526529,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2271_0311_25","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,10.81,1,10.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526528,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2363_0311_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,2229.43,14,159.2450000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20540,526532,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2484_0311_27","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,5.56,0,5.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526533,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2484_0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526530,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2723_0311_26","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,48.52,0,48.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526535,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2735_0311_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,8.15,0,8.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20647,526685,"cxq-tt894N1计划-0311_4小2753","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,1,402.06,3,134.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526687,"cxq-tt894N1计划-0311_3小2755","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,1,427.40,3,142.4666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526688,"cxq-tt894N1计划-0311_6小2723","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,1,1963.94,5,392.7880000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526694,"cxq-tt548N1计划-0311__52750","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,1,238.74,2,119.3700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526696,"cxq-tt894N1计划-0311_12742","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,1,20.62,1,20.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6867,526752,"0311-tve-每次付费-幸_20","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,2278.62,121,18.8315702,0,0.000000,0.00,0E-7,0.0000,0.00000,4,2,null,null,"BID_PER_ACTION"],[6867,526753,"0311-tve-每次付费-幸_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,594.28,11,54.0254545,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526755,"0311-tve-每次付费-幸_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,0.31,0,0.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526754,"0311-tve-每次付费-幸_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,14.40,0,14.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526757,"0311-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,198.24,5,39.6480000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526756,"0311-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,307.79,16,19.2368750,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526758,"0311-tve-每次付费-幸_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,127.33,5,25.4660000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526759,"0311-tve-每次付费-幸_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,11.48,1,11.4800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526760,"0311-tve-每次付费-幸_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,14.69,0,14.6900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526761,"0311-tve-每次付费-幸_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,10.89,0,10.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526762,"0311-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,0.68,0,0.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526764,"0311-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,1.39,0,1.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526765,"0311-tve-每次付费-幸_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,1.14,0,1.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526766,"0311-tve-每次付费-幸_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,35.59,0,35.5900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526767,"0311-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,26.32,0,26.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526770,"0311-tve-每次付费-幸_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,576.69,20,28.8345000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20540,526860,"RICH-财神到我家-幸存者-自动roi-_通投_横SSR2736_0311_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526861,"RICH-财神到我家-幸存者-自动roi-_春节DMP_横SSR1909_0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526862,"RICH-财神到我家-幸存者-自动roi-_春节DMP_横SSR2736_0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20451,526868,"306-幸存者-ROI系03.11-2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,1604.79,27,59.4366667,0,0.000000,0.00,0E-7,0.0000,0.00000,10,2,null,null,"ROI_COEFFICIENT"],[20545,526883,"0311幸存者393z每次付费3k_2735_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,725.72,5,145.1440000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526884,"0311幸存者393z每次付费3k_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,558.27,5,111.6540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20546,526922,"0311wtd末日研究所_2754_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,1,996.19,8,124.5237500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526923,"0311wtd末日研究所_2760_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,1,1073.00,2,536.5000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526925,"0311wtd末日研究所_2736_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,1,1290.20,7,184.3142857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20754,527176,"hix-每次付费--3.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,446.19,8,55.7737500,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20754,527183,"zhan-每次付费--3.11_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,1,404.13,7,57.7328571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,527267,"每次-2762-冲冲冲_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,5299.97,60,88.3328333,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21208,527307,"295-blcb-每次付费爆R爆量_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,68.40,0,68.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527311,"295-blcb-每次付费爆R爆量_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,32.58,0,32.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527310,"295-blcb-每次付费爆R爆量_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,73.36,1,73.3600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527314,"295-blcb-每次付费爆R爆量_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,27.43,1,27.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527315,"295-blcb-每次付费爆R爆量_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,9.42,1,9.4200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527342,"470-tv-每次付费爆R爆量_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.01,0,0.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527344,"470-tv-每次付费爆R爆量_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,9.96,0,9.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527345,"470-tv-每次付费爆R爆量_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,3.93,0,3.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527346,"470-tv-每次付费爆R爆量_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,200.59,1,200.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527343,"470-tv-每次付费爆R爆量_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.21,0,0.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527347,"470-tv-每次付费爆R爆量_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2.83,0,2.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,527371,"PB-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,3289.64,26,126.5246154,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527470,"0312540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.57,0,0.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527471,"0312540-每次付费-幸_15","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,15.59,0,15.5900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527472,"0312540-每次付费-幸_24","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,113.46,1,113.4600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527473,"0312540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,53.51,4,13.3775000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[6867,527477,"0312540-每次付费-幸_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,4.94,0,4.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527478,"0312540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,70.00,1,70.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527479,"0312540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,54.57,1,54.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527481,"0312540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,61.64,1,61.6400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527484,"0312540-每次付费-幸_11","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,4.45,0,4.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527485,"0312540-每次付费-幸_12","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,14.02,0,14.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527487,"0312540-每次付费-幸_14","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,90.11,2,45.0550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527489,"0312540-每次付费-幸_17","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,67.08,0,67.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527495,"0312540-每次付费-幸_23","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,26.97,0,26.9700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527496,"0312540-每次付费-幸_25","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,81.05,1,81.0500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527497,"0312540-每次付费-幸_26","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,28.01,2,14.0050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527498,"0312540-每次付费-幸_27","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,331.64,0,331.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527517,"每0312-tt-xcztz-wyt-rk_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,9.45,0,9.4500000,0,0.000000,0.00,0E-7,0.0000,0,1,0,null,null,"BID_PER_ACTION"],[20646,527536,"cxq-tt338幸存者-每次付费穿山甲0312_排30002248","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,150.89,2,75.4450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527539,"cxq-tt552幸存者-每次付费穿山甲0312_排30002762小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,6.53,10,0.6530000,0,0.000000,0.00,0E-7,0.0000,0.00000,5,2,null,null,"BID_PER_ACTION"],[20646,527540,"cxq-tt338幸存者-每次付费穿山甲0312_排负2762小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,115.40,1,115.4000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527543,"cxq-tt338幸存者-每次付费穿山甲0312_排负2751小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.95,1,0.9500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,527690,"RICH-财神到我家_幸存者-每次付费_排3000_71833_0312_95325541","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,69.66,1,69.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,527695,"RICH-财神到我家_幸存者-每次付费_排3000_78397_0312_95325553","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,8.72,0,8.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527697,"RICH-财神到我家_幸存者-每次付费_排3000_78431_0312_95325566","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2.03,0,2.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527794,"RICH-财神到我家_幸存者-每次付费_排3000_75017_0312_95325552","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,5.94,0,5.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,527861,"0312oyo393自动roi莱卡_2742_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,89.39,5,17.8780000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_PACING"],[20540,527863,"RICH-财神到我家_幸存者-自动ROI_排3000_67560_0312_01169274","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,11.47,0,11.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527866,"RICH-财神到我家_幸存者-自动ROI_排3000_77952_0312_01169251","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527868,"RICH-财神到我家_幸存者-自动ROI_排3000_73141_0312_01169273","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.01,0,0.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21208,527869,"481IC-幸存危机-648大集合-3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,22.55,0,22.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527872,"481IC-幸存危机-648大集合-3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,9.69,0,9.6900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527870,"481IC-幸存危机-648大集合-3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,71.73,0,71.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,527895,"0312幸存者393z每次付费3k_2735_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,5.64,0,5.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,527896,"0312幸存者393z每次付费3k_2731_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,57.81,0,57.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,527897,"0312幸存者393z每次付费3k_2746_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2.20,0,2.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527965,"7tt_n_幸存危机_每次付费__0312_16295360","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,15.05,0,15.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527967,"7tt_n_幸存危机_每次付费__0312_16295373","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,18.07,1,18.0700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527969,"7tt_n_幸存危机_每次付费__0312_16295385","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,5.35,0,5.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527971,"7tt_n_幸存危机_每次付费__0312_16295387","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,6.74,0,6.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527972,"7tt_n_幸存危机_每次付费__0312_16295388","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,2.58,0,2.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527973,"7tt_n_幸存危机_每次付费__0312_16295389","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,145.79,1,145.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527974,"7tt_n_幸存危机_每次付费__0312_162953910","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,37.93,1,37.9300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527975,"7tt_n_幸存危机_每次付费__0312_162953911","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,18.28,0,18.2800000,0,0.000000,0.00,0E-7,0.0000,0,1,2,null,null,"BID_PER_ACTION"],[6770,527978,"7tt_n_幸存危机_每次付费__0312_162953914","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,10.09,0,10.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,528001,"RICH-财神到我家_幸存者-每次付费_排3000_69832_0312_22237942","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,42.82,1,42.8200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20541,528016,"RICH-研究所-财神到我家_通投_横SSR2285_0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,1,1046.00,10,104.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528017,"RICH-研究所-财神到我家_通投_横SSR2495_0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,1,44.73,3,14.9100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528020,"RICH-研究所-财神到我家_通投_横SSR2138_0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,1,74.87,0,74.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528021,"RICH-研究所-财神到我家_通投_横SSR1909_0312_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,1,14.35,3,4.7833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528025,"RICH-研究所-财神到我家_通投_横SSR2138_0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,1,2531.95,11,230.1772727,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[6770,528062,"7tt_幸_自动ROI__0312_88256793","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,1.57,0,1.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528066,"7tt_幸_自动ROI__0312_88256792","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,94.65,2,47.3250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528071,"7tt_幸_自动ROI__0312_88256829","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2.57,0,2.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528068,"7tt_幸_自动ROI__0312_882568211","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,11.23,0,11.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528074,"7tt_幸_自动ROI__0312_882568314","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.55,1,0.5500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[6770,528075,"7tt_幸_自动ROI__0312_882568415","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.36,0,0.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528077,"7tt_幸_自动ROI__0312_882568417","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,21.74,0,21.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528079,"7tt_幸_自动ROI__0312_882568520","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,39.95,0,39.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528085,"7tt_幸_自动ROI__0312_882568926","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.04,0,0.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528086,"7tt_幸_自动ROI__0312_882569027","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528088,"7tt_幸_自动ROI__0312_882569129","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,9.75,0,9.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[20545,528127,"0312-507幸存危机每次付费_2332_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,32.25,1,32.2500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528136,"hix-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,47.22,1,47.2200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528157,"tw-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,439.79,12,36.6491667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528353,"ka-roi系数-周五-3.12_4_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,1098.52,29,37.8800000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_COEFFICIENT"],[20545,528547,"0312-508幸存危机-每次付费竖版沉浸_2332_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,1.71,0,1.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,528550,"0312-508幸存危机-每次付费竖版沉浸_2777_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,1,0.49,0,0.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,528558,"g-自动ROI-幸存者挑战-03.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,898.27,61,14.7257377,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_PACING"],[21212,528562,"g-自动ROI-幸存者挑战-03.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,1339.51,28,47.8396429,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_PACING"],[20637,528659,"990-幸存者-MCFC-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,12.14,0,12.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528657,"990-幸存者-MCFC-0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,1.14,0,1.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528655,"990-幸存者-MCFC-0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528656,"990-幸存者-MCFC-0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.41,0,0.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528658,"990-幸存者-MCFC-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.01,0,0.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528774,"7tt_穿穿_每次付费__0312_02259785","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,7.27,0,7.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528783,"7tt_穿穿_每次付费__0312_022597911","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,387.51,1,387.5100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528784,"7tt_穿穿_每次付费__0312_02259772","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,20.28,0,20.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528778,"7tt_穿穿_每次付费__0312_02259786","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,3583.93,8,447.9912500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528780,"7tt_穿穿_每次付费__0312_02259760","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2850.13,3,950.0433333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528785,"7tt_穿穿_每次付费__0312_02259798","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,50.83,2,25.4150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528777,"7tt_穿穿_每次付费__0312_02259799","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,52.87,1,52.8700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528786,"7tt_穿穿_每次付费__0312_022598012","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,3039.50,3,1013.1666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528788,"7tt_穿穿_每次付费__0312_022598014","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2315.06,3,771.6866667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528789,"7tt_穿穿_每次付费__0312_022598015","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2.42,0,2.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528792,"7tt_穿穿_每次付费__0312_022598118","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,6.43,0,6.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528794,"7tt_穿穿_每次付费__0312_022598120","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,0.29,0,0.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528795,"7tt_穿穿_每次付费__0312_022598121","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2892.82,2,1446.4100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528797,"7tt_穿穿_每次付费__0312_022598223","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,3023.20,3,1007.7333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528798,"7tt_穿穿_每次付费__0312_022598224","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2.08,0,2.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528800,"7tt_穿穿_每次付费__0312_022598226","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,2994.65,2,1497.3250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528801,"7tt_穿穿_每次付费__0312_022598227","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,1.97,1,1.9700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528803,"7tt_穿穿_每次付费__0312_022598329","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,61.72,0,61.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20143,528804,"0312XM-MGTTN1计划_3_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,5.65,0,5.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528805,"0312XM-MGTTN1计划_3_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,42.37,0,42.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528806,"0312XM-MGTTN1计划_3_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,90.71,1,90.7100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528807,"0312XM-MGTTN1计划_3_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,87.86,2,43.9300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528809,"0312XM-MGTTN1计划_3_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,20.31,0,20.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528808,"0312XM-MGTTN1计划_3_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,13.44,0,13.4400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528810,"0312XM-MGTTN1计划_3_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,20.36,0,20.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528811,"0312XM-MGTTN1计划_3_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,12.65,1,12.6500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528812,"0312XM-MGTTN1计划_3_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,0.93,6,0.1550000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[21246,528854,"付费-穿山甲-n1-499-0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,60.38,0,60.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528856,"付费-穿山甲-n1-499-0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,0.01,0,0.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528855,"付费-穿山甲-n1-499-0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,2.79,0,2.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528858,"付费-穿山甲-n1-499-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,0.25,0,0.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528857,"付费-穿山甲-n1-499-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,1,171.00,3,57.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,528859,"7tt_498_穿哥_每次付费_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,1342.52,21,63.9295238,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528860,"7tt_498_穿哥_每次付费_3.12_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,5.75,0,5.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528861,"7tt_498_穿哥_每次付费_3.12_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,6.60,1,6.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528864,"7tt_498_穿哥_每次付费_3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,145.17,3,48.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528865,"7tt_498_穿哥_每次付费_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,4.50,1,4.5000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528866,"7tt_498_穿哥_每次付费_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,22.92,0,22.9200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528867,"7tt_498_穿哥_每次付费_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,7.71,0,7.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528868,"7tt_498_穿哥_每次付费_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,99.27,2,49.6350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528863,"7tt_498_穿哥_每次付费_3.12_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,95.22,5,19.0440000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528963,"263-幸存者-MCFC0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,1,1168.56,9,129.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20540,529036,"RICH-财神到我家_幸存者-每次付费_排3000_70019_0313_08286060","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,1211.56,13,93.1969231,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20540,529037,"RICH-财神到我家_幸存者-每次付费_排3000_73012_0313_08286071","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,0.21,0,0.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,529038,"RICH-财神到我家_幸存者-每次付费_排3000_77023_0313_08286082","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,1656.74,15,110.4493333,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20754,529155,"ka-自动roi-周六-3.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,0.09,2,0.0450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20754,529154,"ka-自动roi-周六-3.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,6.59,0,6.5900000,0,0.000000,0.00,0E-7,0.0000,0,2,2,null,null,"ROI_PACING"],[20754,529158,"ka-自动roi-周六-3.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,49.61,5,9.9220000,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"ROI_PACING"],[6770,529170,"7tt_n_幸存危机_每次付费__0313_84190331","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,1,18.69,0,18.6900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529171,"7tt_穿穿_每次付费__0313_84191313","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,256.16,2,128.0800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529169,"7tt_穿穿_每次付费__0313_84191302","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,4.69,0,4.6900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529178,"7tt_n_幸存危机_每次付费__0313_84190348","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,1,20.64,0,20.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529186,"7tt_穿穿_每次付费__0313_84191290","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,3611.68,14,257.9771429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529187,"7tt_穿穿_每次付费__0313_84191301","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,2420.63,0,2420.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529189,"7tt_穿穿_每次付费__0313_84191315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,9.50,0,9.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529190,"7tt_穿穿_每次付费__0313_84191316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,11.08,2,5.5400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529191,"7tt_穿穿_每次付费__0313_84191317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,3318.27,1,3318.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529192,"7tt_穿穿_每次付费__0313_84191318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,3.22,0,3.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529193,"7tt_穿穿_每次付费__0313_84191329","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,2179.64,1,2179.6400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529194,"7tt_穿穿_每次付费__0313_841913210","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,0.99,0,0.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529196,"7tt_穿穿_每次付费__0313_841913212","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,38.55,1,38.5500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529197,"7tt_穿穿_每次付费__0313_841913213","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,3770.92,0,3770.9200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529198,"7tt_穿穿_每次付费__0313_841913315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,2482.83,2,1241.4150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529199,"7tt_穿穿_每次付费__0313_841913214","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,22.40,1,22.4000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529200,"7tt_穿穿_每次付费__0313_841913316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,4.47,0,4.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529201,"7tt_穿穿_每次付费__0313_841913317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,1729.34,0,1729.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529202,"7tt_穿穿_每次付费__0313_841913318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,5.49,0,5.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529203,"7tt_穿穿_每次付费__0313_841913319","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,1.54,0,1.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,529214,"RICH-末日-穿山甲-付费_通投_混SSR2778_0313_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,1,77.75,3,25.9166667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529215,"RICH-末日-穿山甲-付费_春节DMP_混SSR2762_0313_7","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,1,3.11,6,0.5183333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529221,"RICH-末日-穿山甲-付费_春节DMP_混SSR2778_0313_8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,1,82.35,4,20.5875000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529220,"RICH-末日-穿山甲-付费_通投_混SSR2746_0313_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,1,3126.11,73,42.8234247,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[20540,529217,"RICH-末日-穿山甲-付费_通投_混SSR2762_0313_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,1,356.62,19,18.7694737,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529218,"RICH-末日-穿山甲-付费_游戏_混SSR2778_0313_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,1,0.97,0,0.9700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529216,"RICH-末日-穿山甲-付费_春节DMP_混SSR2746_0313_9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,1,1880.29,45,41.7842222,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"DEEP_BID_DEFAULT"],[20540,529222,"RICH-末日-穿山甲-付费_游戏_混SSR2746_0313_6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,1,6.72,0,6.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529219,"RICH-末日-穿山甲-付费_游戏_混SSR2762_0313_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,1,0.49,0,0.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,529229,"cxq-tt552幸存者-每次付费-穿山甲0313_27462000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,1.32,1,1.3200000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20646,529223,"cxq-tt338幸存者-每次付费-穿山甲0313_27622000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,5196.19,115,45.1842609,0,0.000000,0.00,0E-7,0.0000,0.00000,3,2,null,null,"BID_PER_ACTION"],[20646,529228,"cxq-tt552幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,478.63,19,25.1910526,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529226,"cxq-tt552幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,202.19,7,28.8842857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529225,"cxq-tt338幸存者-每次付费-穿山甲0313_27462000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,5.72,9,0.6355556,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20646,529227,"cxq-tt338幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,14.92,0,14.9200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529224,"cxq-tt338幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,35.76,0,35.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529232,"cxq-tt552幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,0.46,0,0.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529231,"cxq-tt338幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,2.01,0,2.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529233,"cxq-tt338幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,838.36,20,41.9180000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20646,529259,"cxq-tt982幸存者-每次付费-0313_包1SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,1,1401.87,22,63.7213636,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20451,529366,"VR-危机-每次03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,1,0.15,0,0.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,529369,"VR-危机-每次03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,1,0.02,2,0.0100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529374,"JK-自动ROI排3K周日发力-0313_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,238.50,21,11.3571429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21245,529375,"JK-自动ROI排3K周日发力-0313_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,151.19,15,10.0793333,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"ROI_PACING"],[21245,529376,"JK-自动ROI排3K周日发力-0313_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,653.07,21,31.0985714,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"DEEP_BID_DEFAULT"],[21245,529377,"JK-自动ROI排3K周日发力-0313_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,9.93,0,9.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21245,529378,"JK-自动ROI排3K周日发力-0313_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,13.66,1,13.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21246,529417,"穿-n1-周末付费冲-0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,1,9.77,0,9.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529418,"穿-n1-周末付费冲-0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,1,3.35,0,3.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529420,"穿-n1-周末付费冲-0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,1,5.60,0,5.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529421,"穿-n1-周末付费冲-0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,1,16.87,0,16.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529422,"穿-n1-周末付费冲-0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,1,34.18,0,34.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[6770,529462,"7tt_穿穿_每次付费__0314_72230811","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,23.38,0,23.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529461,"7tt_穿穿_每次付费__0314_72230800","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,56.09,0,56.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529464,"7tt_穿穿_每次付费__0314_72230827","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3307.61,10,330.7610000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529466,"7tt_穿穿_每次付费__0314_72230829","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,2533.43,3,844.4766667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529463,"7tt_穿穿_每次付费__0314_72230825","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,27.21,1,27.2100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529465,"7tt_穿穿_每次付费__0314_72230826","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,29.43,0,29.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529468,"7tt_穿穿_每次付费__0314_72230813","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,1269.24,2,634.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529469,"7tt_穿穿_每次付费__0314_72230812","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,63.57,2,31.7850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529471,"7tt_穿穿_每次付费__0314_72230828","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,11.05,0,11.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529467,"7tt_穿穿_每次付费__0314_72230824","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,18.35,0,18.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529470,"7tt_穿穿_每次付费__0314_722308311","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,1048.45,0,1048.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529474,"每0314-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,0.39,0,0.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529482,"每0314-tt-xcztz-wyt-rk新_24","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,11.48,0,11.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529475,"每0314-tt-xcztz-wyt-rk新_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3.44,0,3.4400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529483,"每0314-tt-xcztz-wyt-rk新_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,20.18,0,20.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529495,"每0314-tt-xcztz-wyt-rk新_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,20.47,0,20.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529496,"每0314-tt-xcztz-wyt-rk新_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,15.06,0,15.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529502,"每0314-tt-xcztz-wyt-rk新_23","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,0.09,0,0.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529504,"每0314-tt-xcztz-wyt-rk新_27","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3.16,0,3.1600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529544,"i-每次付费-幸存者挑战周日-03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,338.92,5,67.7840000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[21212,529540,"r3-每次付费-幸存者挑战周日-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,2727.21,27,101.0077778,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[21212,529551,"r-自动ROI-幸存者挑战带穿山甲-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,45.20,2,22.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"ROI_PACING"],[21212,529550,"g-自动ROI-幸存者挑战带穿山甲-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,5.31,18,0.2950000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"ROI_PACING"],[21212,529555,"g-自动ROI-幸存者挑战带穿山甲-03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,0.05,1,0.0500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,529556,"r-每次付费-幸存者挑战-穿山甲专场-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,0.36,0,0.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529558,"r-每次付费-幸存者挑战-穿山甲专场-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,0.66,0,0.6600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529559,"r-每次付费-幸存者挑战-穿山甲专场-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,1.57,0,1.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529560,"r-每次付费-幸存者挑战-穿山甲专场-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529557,"r-每次付费-幸存者挑战-穿山甲专场-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,792.68,3,264.2266667,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[20637,529561,"05-幸存者-MCFC0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,2803.74,37,75.7767568,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"BID_PER_ACTION"],[20637,529568,"05-幸存者-MCFC0314_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,2803.74,24,116.8225000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[20637,529569,"05-幸存者-MCFC0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,267.70,7,38.2428571,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[20637,529571,"05-幸存者-MCFC0314_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,2803.74,28,100.1335714,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[21213,529573,"n96-iOS-黎明血战周日-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,1,2376.43,75,31.6857333,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"DEEP_BID_DEFAULT"],[21213,529574,"n96-iOS-黎明血战周日-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,1,9.03,2,4.5150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,529575,"n96-iOS-黎明血战周日-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,1,3.90,0,3.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20754,529587,"hix-每次付费--3.13_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,55.88,1,55.8800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529591,"jap-每次付费--3.13_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,54.14,1,54.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529593,"tw-每次付费--3.13_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,0.02,2,0.0100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529598,"zhan-每次付费--3.13_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,9.93,0,9.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529600,"zhan-每次付费--3.13_17","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,663.94,7,94.8485714,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"BID_PER_ACTION"],[6867,529645,"0314-540-每次付费-幸_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3042.01,142,21.4226056,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529648,"0314-540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,601.80,2,300.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529643,"0314-540-每次付费-幸_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3374.57,60,56.2428333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529644,"0314-540-每次付费-幸_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3434.77,33,104.0839394,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529647,"0314-540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,4294.94,81,53.0239506,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529650,"0314-540-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,1410.25,12,117.5208333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529649,"0314-540-每次付费-幸_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3594.66,158,22.7510127,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529651,"0314-540-每次付费-幸_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,2926.56,53,55.2181132,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529653,"0314-540-每次付费-幸_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3451.77,59,58.5045763,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529654,"0314-540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,945.63,0,945.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529657,"0314-540-每次付费-幸_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,641.13,2,320.5650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529659,"0314-tve-每次付费-幸_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,0.11,5,0.0220000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,0,null,null,"BID_PER_ACTION"],[6867,529660,"0314-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,976.15,32,30.5046875,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529668,"0314-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,915.98,12,76.3316667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529667,"0314-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,8.30,0,8.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529669,"0314-tve-每次付费-幸_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,3274.98,38,86.1836842,0,0.000000,0.00,0E-7,0.0000,0.00000,1,0,null,null,"BID_PER_ACTION"],[6867,529673,"0314-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,181.54,1,181.5400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529666,"0314-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,1,95.81,2,47.9050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529719,"470-tv-每次付费拉环爆R爆量_3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,13.85,1,13.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529715,"470-tv-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,345.64,14,24.6885714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529716,"470-tv-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,851.44,16,53.2150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529718,"470-tv-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,688.95,25,27.5580000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529720,"470-tv-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,118.75,5,23.7500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529722,"470-tv-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,100.57,3,33.5233333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529721,"470-tv-每次付费拉环爆R爆量_3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2803.74,31,90.4432258,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529723,"470-tv-每次付费拉环爆R爆量_3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,137.01,5,27.4020000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529724,"470-tv-每次付费拉环爆R爆量_3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,38.90,0,38.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529725,"470-tv-每次付费拉环爆R爆量_3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,589.15,0,589.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529747,"295-blcb-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,114.76,3,38.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529749,"295-blcb-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,176.98,8,22.1225000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529748,"295-blcb-每次付费拉环爆R爆量_3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2803.74,18,155.7633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529754,"295-blcb-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,133.70,2,66.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529753,"295-blcb-每次付费拉环爆R爆量_3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1149.38,4,287.3450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529755,"295-blcb-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,213.23,8,26.6537500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529752,"295-blcb-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,8.18,1,8.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529847,"263-幸存者-MCFC0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2619.13,33,79.3675758,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529845,"263-幸存者-MCFC0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,422.61,8,52.8262500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529863,"263-幸存者-MCFC0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,171.92,1,171.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529864,"263-幸存者-MCFC0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,415.87,0,415.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529865,"263-幸存者-MCFC0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1586.14,27,58.7459259,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529910,"0315-204-穿山甲-每次付费-幸_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,17.04,0,17.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529909,"0315-204-穿山甲-每次付费-幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,63.17,0,63.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529918,"481IC-幸存危机-2756爆量爆R-3.5_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,287.28,0,287.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529914,"0315-204-穿山甲-每次付费-幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,95.97,4,23.9925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529913,"0315-204-穿山甲-每次付费-幸_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2991.81,59,50.7086441,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529916,"481IC-幸存危机-2756爆量爆R-3.5_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,5607.48,81,69.2281481,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529917,"0315-204-穿山甲-每次付费-幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,89.67,7,12.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529920,"0315-204-穿山甲-每次付费-幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,806.49,32,25.2028125,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529922,"0315-204-穿山甲-每次付费-幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,14.31,0,14.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529923,"0315-204-穿山甲-每次付费-幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,8.55,0,8.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529924,"0315-204-穿山甲-每次付费-幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,193.22,4,48.3050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529925,"0315-204-穿山甲-每次付费-幸_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,42.37,0,42.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529926,"0315-204-穿山甲-每次付费-幸_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,24.58,0,24.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529927,"0315-204-穿山甲-每次付费-幸_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,5.17,0,5.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529975,"990-幸存者-MCFC-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,9.48,0,9.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529977,"990-幸存者-MCFC-0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,288.92,5,57.7840000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529978,"990-幸存者-MCFC-0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.29,0,1.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529981,"990-幸存者-MCFC-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,5.06,0,5.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529979,"990-幸存者-MCFC-0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.67,0,0.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529980,"990-幸存者-MCFC-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3.95,0,3.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529976,"990-幸存者-MCFC-0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,792.00,15,52.8000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529983,"990-幸存者-MCFC-0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,616.76,26,23.7215385,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529982,"990-幸存者-MCFC-0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,546.02,6,91.0033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530038,"CJ-幸存危机-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,28.17,0,28.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530041,"CJ-幸存危机-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,3303.74,60,55.0623333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530039,"CJ-幸存危机-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,614.50,11,55.8636364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530050,"CJ-幸存危机-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,16.87,0,16.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530102,"每0315-tt-xcztz-wyt-rk_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,18.10,0,18.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530101,"每0315-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,15.23,0,15.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530104,"每0315-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3.78,0,3.7800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530109,"每0315-tt-xcztz-wyt-rk_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,5.76,0,5.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530100,"每0315-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.08,0,0.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530111,"每0315-tt-xcztz-wyt-rk_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,7.22,0,7.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530112,"每0315-tt-xcztz-wyt-rk_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,13.48,0,13.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530113,"每0315-tt-xcztz-wyt-rk_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,49.71,0,49.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530114,"每0315-tt-xcztz-wyt-rk_21","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,100.41,0,100.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530115,"每0315-tt-xcztz-wyt-rk_22","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.05,0,1.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530117,"每0315-tt-xcztz-wyt-rk_24","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,83.54,0,83.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530122,"每0315-tt-xcztz-wyt-rk_31","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,6.31,0,6.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530125,"每0315-tt-xcztz-wyt-rk_34","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,54.63,0,54.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530126,"每0315-tt-xcztz-wyt-rk_37","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,8.21,0,8.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530127,"每0315-tt-xcztz-wyt-rk_38","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.24,0,0.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530128,"每0315-tt-xcztz-wyt-rk_39","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.36,0,1.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530129,"每0315-tt-xcztz-wyt-rk_40","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,11.37,0,11.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530130,"每0315-tt-xcztz-wyt-rk_41","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,92.33,0,92.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530131,"每0315-tt-xcztz-wyt-rk_42","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.49,0,1.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530132,"每0315-tt-xcztz-wyt-rk_45","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2.00,0,2.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530133,"每0315-tt-xcztz-wyt-rk_46","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,469.30,4,117.3250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530134,"每0315-tt-xcztz-wyt-rk_47","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,13.72,0,13.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530135,"每0315-tt-xcztz-wyt-rk_48","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,302.18,1,302.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530139,"每0315-tt-xcztz-wyt-rk_52","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.23,0,0.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530140,"每0315-tt-xcztz-wyt-rk_53","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.49,0,0.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530141,"每0315-tt-xcztz-wyt-rk_54","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.41,0,0.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530145,"每0315-tt-xcztz-wyt-rk_58","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,21.41,0,21.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530146,"每0315-tt-xcztz-wyt-rk_59","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530147,"每0315-tt-xcztz-wyt-rk_60","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530148,"每0315-tt-xcztz-wyt-rk_62","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.22,0,0.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530149,"每0315-tt-xcztz-wyt-rk_61","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3.46,0,3.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530150,"每0315-tt-xcztz-wyt-rk_64","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,9.99,0,9.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530151,"每0315-tt-xcztz-wyt-rk_63","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,29.08,0,29.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530155,"每0315-tt-xcztz-wyt-rk_70","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,7.02,0,7.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530154,"每0315-tt-xcztz-wyt-rk_65","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,408.44,1,408.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530153,"每0315-tt-xcztz-wyt-rk_67","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,951.02,11,86.4563636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530157,"每0315-tt-xcztz-wyt-rk_66","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,265.58,3,88.5266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530156,"每0315-tt-xcztz-wyt-rk_69","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,8.07,0,8.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530158,"每0315-tt-xcztz-wyt-rk_71","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530159,"每0315-tt-xcztz-wyt-rk_72","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,275.30,2,137.6500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530168,"r3-每次付费-幸存者挑战优选位-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2336.53,65,35.9466154,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530173,"r3-每次付费-幸存者挑战优选位-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1658.99,14,118.4992857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530175,"i-每次付费-幸存者挑战只限男优先位-03.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,783.36,20,39.1680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530174,"r3-每次付费-幸存者挑战优选位-03.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1913.11,24,79.7129167,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530176,"i-每次付费-幸存者挑战只限男优先位-03.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1174.93,9,130.5477778,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530167,"r3-每次付费-幸存者挑战优选位-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,485.07,3,161.6900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530177,"i-每次付费-幸存者挑战只限男优先位-03.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,309.76,3,103.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530210,"i-每次付费-幸存者挑战只限男优先位-03.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,430.16,1,430.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530211,"i-每次付费-幸存者挑战只限男优先位-03.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,577.04,4,144.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530385,"ROI系数-幸存者挑战-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3107.77,112,27.7479464,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530386,"ROI系数-幸存者挑战-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3070.08,51,60.1976471,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530387,"ROI系数-幸存者挑战-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3015.56,96,31.4120833,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530388,"ROI系数-幸存者挑战-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1111.52,34,32.6917647,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530389,"ROI系数-幸存者挑战-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2983.30,81,36.8308642,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530514,"0315幸存者393z每次付费3k_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1844.31,1,1844.3100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530516,"0315幸存者393z每次付费3k_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1863.95,5,372.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530515,"0315幸存者393z每次付费3k_2731_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2252.57,2,1126.2850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530517,"0315幸存者393z每次付费3k_2746_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3237.84,38,85.2063158,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530518,"0315幸存者393z每次付费3k_2735_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1199.35,0,1199.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530524,"0315oyo393-roi系数-排除3k_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2885.28,52,55.4861538,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530531,"0315oyo393-roi系数-排除3k_2731_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1394.48,4,348.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530529,"0315oyo393-roi系数-排除3k_2522_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1046.06,12,87.1716667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530530,"0315oyo393-roi系数-排除3k_2762_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1331.56,12,110.9633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20635,530537,"每0315-tt-xcztz-wyt-yan_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,173.89,4,43.4725000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530540,"每0315-tt-xcztz-wyt-yan_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,314.88,5,62.9760000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530541,"每0315-tt-xcztz-wyt-yan_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,294.41,3,98.1366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530542,"每0315-tt-xcztz-wyt-yan_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,95.29,1,95.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530543,"每0315-tt-xcztz-wyt-yan_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,302.67,3,100.8900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530544,"每0315-tt-xcztz-wyt-yan_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,496.49,10,49.6490000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530545,"每0315-tt-xcztz-wyt-yan_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2486.77,40,62.1692500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530546,"每0315-tt-xcztz-wyt-yan_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.28,0,1.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530547,"每0315-tt-xcztz-wyt-yan_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.94,0,0.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530548,"每0315-tt-xcztz-wyt-yan_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,4.45,0,4.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530549,"每0315-tt-xcztz-wyt-yan_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.09,0,0.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530550,"每0315-tt-xcztz-wyt-yan_19","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2.71,0,2.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530551,"每0315-tt-xcztz-wyt-yan_20","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2.11,1,2.1100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530552,"每0315-tt-xcztz-wyt-yan_21","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,4.98,0,4.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,530671,"cxq-tt747幸存者-ROI系数0315_2738","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1968.98,62,31.7577419,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530672,"cxq-tt549幸存者-ROI系数0315_2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2855.54,85,33.5945882,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530673,"cxq-tt747幸存者-ROI系数0315_2332","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,739.36,4,184.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530674,"cxq-tt982幸存者-ROI系数0315_2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,150.86,2,75.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530675,"cxq-tt530幸存者-ROI系数0315_录2789","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,314.35,0,314.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530676,"cxq-tt530幸存者-ROI系数0315_录2793","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,261.52,0,261.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530678,"cxq-tt530幸存者-ROI系数0315_录2764","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,260.82,0,260.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530677,"cxq-tt549幸存者-ROI系数0315_2731","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,361.83,5,72.3660000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530679,"cxq-tt982幸存者-ROI系数0315_2746","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,337.08,4,84.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530680,"cxq-tt549幸存者-ROI系数0315_2781","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,437.21,4,109.3025000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530693,"0315-508幸存危机-每次付费竖版沉浸_2762_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,135.38,1,135.3800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530692,"0315-508幸存危机-每次付费竖版沉浸_2332_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,23.62,0,23.6200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530698,"0315-508幸存危机-每次付费竖版沉浸_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,681.82,7,97.4028571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530696,"0315-508幸存危机-每次付费竖版沉浸_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,161.82,1,161.8200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530721,"303-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,318.02,1,318.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530722,"306-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,61.94,3,20.6466667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530718,"0315-幸存者393z每次付费排除3k_2784_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,315.11,4,78.7775000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530713,"0315-幸存者393z每次付费排除3k_2783_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1101.04,4,275.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530720,"306-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,559.71,7,79.9585714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530716,"0315-幸存者393z每次付费排除3k_2786_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,49.52,0,49.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530724,"301-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,342.01,3,114.0033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530725,"301-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,316.00,3,105.3333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530723,"303-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1383.69,36,38.4358333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530726,"301-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,6.18,0,6.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530728,"306-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,95.06,1,95.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530729,"306-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,13.65,0,13.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530731,"306-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,8.09,0,8.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530732,"303-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2.37,0,2.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530733,"306-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,27.41,0,27.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530734,"303-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,303.88,3,101.2933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530736,"306-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2153.20,55,39.1490909,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530735,"303-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.21,0,1.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530730,"306-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,5.67,0,5.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530737,"306-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,367.57,8,45.9462500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530738,"303-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.83,0,1.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530739,"303-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,29.18,2,14.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530740,"303-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.41,0,1.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530741,"301-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,44.92,2,22.4600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530742,"301-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,438.56,6,73.0933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530743,"301-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,722.79,20,36.1395000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530744,"301-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,304.04,1,304.0400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530745,"301-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,4.43,0,4.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530746,"301-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2.79,0,2.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530747,"301-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,79.66,1,79.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530759,"303-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.02,0,1.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530779,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2332_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,648.74,19,34.1442105,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530781,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2731_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,12.56,0,12.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530780,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,34.53,0,34.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530782,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2751_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2.03,0,2.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530783,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2794_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,119.91,0,119.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530784,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2271_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,361.69,15,24.1126667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530786,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2484_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,234.18,17,13.7752941,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530785,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1692.29,57,29.6892982,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530787,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,155.94,0,155.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530788,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,4.05,0,4.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530799,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2622315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,956.08,8,119.5100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20540,530802,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2789315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,99.25,0,99.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530803,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2761315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,75.60,0,75.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530800,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2746315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,697.88,6,116.3133333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,530806,"0315FP-ROI系数-MG幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3269.14,10,326.9140000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530805,"0315FP-ROI系数-MG幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1537.46,20,76.8730000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530807,"0315FP-ROI系数-MG幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2333.97,7,333.4242857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530810,"0315FP-ROI系数-MG幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1061.06,2,530.5300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530809,"0315FP-ROI系数-MG幸_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1255.06,1,1255.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530844,"0315FP-ROI系数-MG幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1171.66,4,292.9150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530843,"0315FP-ROI系数-MG幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1840.60,6,306.7666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530846,"0315FP-ROI系数-MG幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2177.36,20,108.8680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530845,"0315FP-ROI系数-MG幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1762.77,5,352.5540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530908,"tw-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,970.04,5,194.0080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530913,"tw-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,107.09,0,107.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530914,"hix-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1105.87,8,138.2337500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530915,"zhan-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,389.75,0,389.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530912,"jap-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,31.46,0,31.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530910,"hix-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,418.61,2,209.3050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530911,"hix-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,388.89,2,194.4450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530907,"tw-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2713.20,32,84.7875000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530916,"zhan-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,230.98,0,230.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530917,"zhan-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,518.88,4,129.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530909,"jap-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2588.93,30,86.2976667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530920,"jap-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.39,0,1.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530918,"hix-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,8.43,0,8.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530919,"jap-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,285.03,1,285.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530921,"zhan-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,701.53,6,116.9216667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530925,"hix-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,29.89,0,29.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530926,"hix-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,38.19,0,38.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530923,"tw-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,156.00,1,156.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530922,"jap-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,13.37,0,13.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530927,"zhan-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,285.16,4,71.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530928,"zhan-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,995.17,7,142.1671429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530929,"tw-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,54.24,1,54.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530931,"FJ-幸存者-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,753.31,7,107.6157143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530933,"FW-幸存者-每次03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,26.64,0,26.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530930,"FJ-幸存者-每次03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,258.76,3,86.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530932,"FW-幸存者-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,719.29,5,143.8580000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530934,"FW-幸存者-每次03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,528.73,6,88.1216667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530935,"FW-幸存者-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,956.90,8,119.6125000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530937,"FJ-幸存者-每次03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,42.93,0,42.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530940,"FJ-幸存者-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,35.09,0,35.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530939,"FJ-幸存者-每次03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,5.68,0,5.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530938,"FW-幸存者-每次03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,374.85,1,374.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530943,"FW-幸存者-每次03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1606.62,15,107.1080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530948,"FJ-幸存者-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,164.54,0,164.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530947,"FJ-幸存者-每次03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,602.25,5,120.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530946,"FW-幸存者-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,413.23,0,413.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530949,"FJ-幸存者-每次03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,647.84,4,161.9600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530944,"FJ-幸存者-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,400.58,0,400.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530945,"FW-幸存者-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1025.53,11,93.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530942,"FW-幸存者-每次03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3303.74,39,84.7112821,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530941,"FW-幸存者-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,509.93,5,101.9860000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530964,"yo-roi系数-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2803.74,76,36.8913158,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530963,"ka-roi系数-周一-3.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1648.62,24,68.6925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530958,"yo-roi系数-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,110.73,0,110.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20754,530960,"yo-roi系数-周一-3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2803.74,40,70.0935000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530965,"yo-roi系数-周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,736.83,17,43.3429412,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530967,"yo-roi系数-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,365.61,6,60.9350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530969,"yo-roi系数-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,834.29,14,59.5921429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530973,"ka-roi系数-周一-3.15_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,630.42,10,63.0420000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530970,"ka-roi系数-周一-3.15_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,791.80,17,46.5764706,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530972,"ka-roi系数-周一-3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,407.12,2,203.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530971,"yo-roi系数-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,440.39,1,440.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20635,530999,"每0315-tt-xcztz-wyt-zhong_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,4.12,0,4.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530998,"每0315-tt-xcztz-wyt-zhong_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1302.96,14,93.0685714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531000,"每0315-tt-xcztz-wyt-zhong_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531001,"每0315-tt-xcztz-wyt-zhong_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,620.65,8,77.5812500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531002,"每0315-tt-xcztz-wyt-zhong_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3.62,1,3.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531006,"每0315-tt-xcztz-wyt-zhong_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,71.09,1,71.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531004,"每0315-tt-xcztz-wyt-zhong_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,672.76,13,51.7507692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531003,"每0315-tt-xcztz-wyt-zhong_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,921.25,3,307.0833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531005,"每0315-tt-xcztz-wyt-zhong_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,0.19,0,0.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531007,"每0315-tt-xcztz-wyt-zhong_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,22.40,0,22.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531008,"每0315-tt-xcztz-wyt-zhong_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,37.46,0,37.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531009,"每0315-tt-xcztz-wyt-zhong_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,3.99,0,3.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531010,"每0315-tt-xcztz-wyt-zhong_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1.51,0,1.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531011,"每0315-tt-xcztz-wyt-zhong_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,11.20,0,11.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21213,531012,"n96-ios-黎明血战-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,39.96,1,39.9600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531013,"n96-ios-黎明血战-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,63.84,2,31.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531014,"n96-ios-黎明血战-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,1659.45,33,50.2863636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531015,"n96-ios-黎明血战-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,53.39,1,53.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531016,"n96-ios-黎明血战-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,1534.32,11,139.4836364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531038,"bdna-非海马云-幸存者挑战-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2701.47,91,29.6864835,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531041,"bdna-非海马云-幸存者挑战-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2345.34,50,46.9068000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531187,"au-黎明血战--周一-3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,679.51,11,61.7736364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531189,"th-黎明血战--周一-3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,179.24,0,179.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531190,"au-黎明血战--周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,75.27,0,75.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531194,"th-黎明血战--周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,2508.92,43,58.3469767,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531188,"roi系数-0315-BY-幸存_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,305.27,10,30.5270000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20541,531195,"RICH-研究所-财神到我家-通投_混SSR2791_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,2183.63,7,311.9471429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531197,"RICH-研究所-财神到我家-通投_混SSR2797_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,88.80,0,88.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531199,"RICH-研究所-财神到我家-通投_混SSR2749_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,1143.12,1,1143.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531200,"RICH-研究所-财神到我家-通投_混SSR2789_0315_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,813.08,0,813.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531201,"RICH-研究所-财神到我家-通投_混SSR2783_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,1568.03,2,784.0150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531203,"05-幸存者-MCFC0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,433.57,7,61.9385714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531202,"05-幸存者-MCFC0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2803.74,51,54.9752941,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531205,"05-幸存者-MCFC0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2803.74,39,71.8907692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20541,531208,"RICH-研究所-财神到我家-通投_创意组8_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,1653.34,1,1653.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531211,"th-黎明血战--周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,49.77,0,49.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531212,"th-黎明血战--周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,390.00,1,390.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531207,"roi系数-0315-BY-幸存_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,559.87,8,69.9837500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531209,"roi系数-0315-BY-幸存_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,267.01,2,133.5050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20755,531215,"th-黎明血战--周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,386.82,2,193.4100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531214,"roi系数-0315-BY-幸存_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,508.28,2,254.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531213,"roi系数-0315-BY-幸存_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,50.73,0,50.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[21245,531210,"roi系数-0315-BY-幸存_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,784.04,5,156.8080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531206,"roi系数-0315-BY-幸存_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,529.12,1,529.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531216,"05-幸存者-MCFC0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,8.14,0,8.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,531217,"05-幸存者-MCFC0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,36.43,0,36.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20755,531227,"th-黎明血战--周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,106.53,2,53.2650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531229,"au-黎明血战--周一-3.15_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,17.58,0,17.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531230,"th-黎明血战--周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,637.41,3,212.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531231,"au-黎明血战--周一-3.15_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,418.74,0,418.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531232,"au-黎明血战--周一-3.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,476.02,1,476.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531233,"au-黎明血战--周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,69.94,0,69.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531234,"roi系数-0315-BY-幸存_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,304.39,4,76.0975000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20755,531252,"au-黎明血战--周一-3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,48.58,0,48.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531257,"th-黎明血战--周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,415.40,2,207.7000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531259,"au-黎明血战--周一-3.15_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,1,159.31,0,159.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531256,"05-幸存者-MCFC0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1073.08,20,53.6540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531258,"05-幸存者-MCFC0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,926.95,18,51.4972222,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531254,"05-幸存者-MCFC0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2.74,1,2.7400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531290,"0315幸存者393z每次付费排除3K_2797_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1202.22,3,400.7400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531289,"0315幸存者393z每次付费排除3K_2742_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1781.97,7,254.5671429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531291,"0315幸存者393z每次付费排除3K_2791_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1612.04,15,107.4693333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531292,"0315幸存者393z每次付费排除3K_2798_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1320.65,14,94.3321429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531287,"0315幸存者393z每次付费排除3K_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,5530.46,74,74.7359459,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531293,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1155.31,10,115.5310000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531294,"cxq-tt338幸存者-穿山甲每次付费-0315_30002797","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,93.94,0,93.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531298,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,395.33,7,56.4757143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531297,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2731","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,180.30,0,180.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531296,"cxq-tt338幸存者-穿山甲每次付费-0315_30002798","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,98.57,1,98.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531295,"cxq-tt338幸存者-穿山甲每次付费-0315_30002796","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1798.86,18,99.9366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20546,531300,"0315wtd末日研究所_2332_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,3440.76,28,122.8842857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531303,"0315wtd末日研究所_2730_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,890.32,1,890.3200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531302,"0315wtd末日研究所_2762_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,1949.39,11,177.2172727,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531304,"0315wtd末日研究所_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,2917.04,14,208.3600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531306,"263-幸存者-ROI-0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,763.58,22,34.7081818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531314,"263-幸存者-ROI-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2728.54,80,34.1067500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531307,"263-幸存者-ROI-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2910.18,58,50.1755172,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531315,"263-幸存者-ROI-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,236.45,2,118.2250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20546,531576,"0315ph648末日研究所_2800_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,236.37,1,236.3700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531578,"0315ph648末日研究所_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,7286.80,81,89.9604938,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531579,"0315ph648末日研究所_2783_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,93.76,0,93.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531580,"0315ph648末日研究所_2519_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,1,29.25,0,29.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531583,"263-幸存者-MCFC0315_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,330.96,0,330.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,531582,"263-幸存者-MCFC0315_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,470.23,2,235.1150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531585,"263-幸存者-MCFC0315_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1102.03,5,220.4060000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531581,"263-幸存者-MCFC0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,68.49,1,68.4900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531584,"263-幸存者-MCFC0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,2672.80,33,80.9939394,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531706,"0315-508幸存危机-每次付费_2800_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,1,742.06,1,742.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,531771,"r-自动ROI-幸存者挑战晚上七点-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,16.65,0,16.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21212,531772,"r-自动ROI-幸存者挑战晚上七点-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,4590.59,90,51.0065556,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531773,"r-自动ROI-幸存者挑战晚上七点-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,1305.40,6,217.5666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531774,"r-自动ROI-幸存者挑战晚上七点-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,6.50,0,6.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21212,531775,"r-自动ROI-幸存者挑战晚上七点-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,1,644.52,3,214.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[20540,529214,"RICH-末日-穿山甲-付费_通投_混SSR2778_0313_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,2,337.75,9,37.5277778,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529215,"RICH-末日-穿山甲-付费_春节DMP_混SSR2762_0313_7","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,2,2845.07,60,47.4178333,0,0.000000,0.00,0E-7,0.0741,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529216,"RICH-末日-穿山甲-付费_春节DMP_混SSR2746_0313_9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,2,5207.29,115,45.2807826,2,2603.645000,238.00,0.0457052,0.0571,0.01739,3,2,null,null,"DEEP_BID_DEFAULT"],[20540,529217,"RICH-末日-穿山甲-付费_通投_混SSR2762_0313_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,2,2576.57,64,40.2589063,0,0.000000,0.00,0E-7,0.0889,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529220,"RICH-末日-穿山甲-付费_通投_混SSR2746_0313_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,2,6003.34,139,43.1894964,1,6003.340000,30.00,0.0049972,0.0606,0.00719,1,2,null,null,"DEEP_BID_DEFAULT"],[20540,529221,"RICH-末日-穿山甲-付费_春节DMP_混SSR2778_0313_8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,2,738.19,14,52.7278571,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529222,"RICH-末日-穿山甲-付费_游戏_混SSR2746_0313_6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,2,46.51,1,46.5100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,525572,"7tt_529_幸存者挑战_自动ROI_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,6108.12,241,25.3448963,3,2036.040000,212.00,0.0347079,0.0588,0.01245,4,2,null,null,"ROI_PACING"],[6770,525577,"7tt_498_幸存者挑战_自动ROI_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,8761.02,271,32.3284871,4,2190.255000,260.00,0.0296769,0.1000,0.01476,5,2,null,null,"ROI_PACING"],[6770,528066,"7tt_幸_自动ROI__0312_88256792","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,408.77,9,45.4188889,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528068,"7tt_幸_自动ROI__0312_882568211","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,40.03,1,40.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528077,"7tt_幸_自动ROI__0312_882568417","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,46.45,1,46.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528079,"7tt_幸_自动ROI__0312_882568520","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,94.74,2,47.3700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528774,"7tt_穿穿_每次付费__0312_02259785","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,34.46,3,11.4866667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528778,"7tt_穿穿_每次付费__0312_02259786","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3701.79,11,336.5263636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528780,"7tt_穿穿_每次付费__0312_02259760","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2969.62,6,494.9366667,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528783,"7tt_穿穿_每次付费__0312_022597911","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,545.10,4,136.2750000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528788,"7tt_穿穿_每次付费__0312_022598014","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2364.49,4,591.1225000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528791,"7tt_穿穿_每次付费__0312_022598017","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6221.77,203,30.6491133,3,2073.923333,236.00,0.0379313,0.0702,0.01478,3,2,null,null,"BID_PER_ACTION"],[6770,528797,"7tt_穿穿_每次付费__0312_022598223","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3170.72,6,528.4533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528800,"7tt_穿穿_每次付费__0312_022598226","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3010.01,3,1003.3366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528859,"7tt_498_穿哥_每次付费_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1621.08,33,49.1236364,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528860,"7tt_498_穿哥_每次付费_3.12_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,9.69,0,9.6900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528861,"7tt_498_穿哥_每次付费_3.12_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,26.95,3,8.9833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528862,"7tt_498_穿哥_每次付费_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6099.43,163,37.4198160,2,3049.715000,322.00,0.0527918,0.1667,0.01227,2,2,null,null,"BID_PER_ACTION"],[6770,528863,"7tt_498_穿哥_每次付费_3.12_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,165.27,8,20.6587500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528864,"7tt_498_穿哥_每次付费_3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,282.33,9,31.3700000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528865,"7tt_498_穿哥_每次付费_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,22.27,2,11.1350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528868,"7tt_498_穿哥_每次付费_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,198.38,3,66.1266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529187,"7tt_穿穿_每次付费__0313_84191301","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,2445.08,1,2445.0800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529192,"7tt_穿穿_每次付费__0313_84191318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3.94,1,3.9400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529198,"7tt_穿穿_每次付费__0313_841913315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,2572.60,5,514.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529467,"7tt_穿穿_每次付费__0314_72230824","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,30.03,1,30.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529470,"7tt_穿穿_每次付费__0314_722308311","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1064.14,1,1064.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,526752,"0311-tve-每次付费-幸_20","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,9025.58,272,33.1822794,4,2256.395000,280.00,0.0310229,0.0993,0.01471,4,2,null,null,"BID_PER_ACTION"],[6867,526754,"0311-tve-每次付费-幸_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,226.90,2,113.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526756,"0311-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2073.89,47,44.1253191,0,0.000000,0.00,0E-7,0.0968,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526757,"0311-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,786.78,12,65.5650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526758,"0311-tve-每次付费-幸_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,578.17,7,82.5957143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526763,"0311-tve-每次付费-幸_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,849.35,6,141.5583333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526769,"0311-tve-每次付费-幸_18","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2.57,4,0.6425000,0,0.000000,0.00,0E-7,0.2500,0.00000,1,2,null,null,"BID_PER_ACTION"],[6867,526770,"0311-tve-每次付费-幸_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2205.11,48,45.9397917,1,2205.110000,98.00,0.0444422,0.1786,0.02083,1,2,null,null,"BID_PER_ACTION"],[6867,527472,"0312540-每次付费-幸_24","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,981.51,17,57.7358824,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527473,"0312540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,670.06,18,37.2255556,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[6867,527478,"0312540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,579.49,8,72.4362500,0,0.000000,0.00,0E-7,0.2857,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527479,"0312540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1080.02,8,135.0025000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527481,"0312540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,387.31,3,129.1033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527483,"0312540-每次付费-幸_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2964.35,45,65.8744444,0,0.000000,0.00,0E-7,0.0444,0.00000,3,2,null,null,"BID_PER_ACTION"],[6867,527484,"0312540-每次付费-幸_11","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,82.25,2,41.1250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527485,"0312540-每次付费-幸_12","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,161.95,6,26.9916667,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527487,"0312540-每次付费-幸_14","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,385.25,3,128.4166667,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527489,"0312540-每次付费-幸_17","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,283.00,3,94.3333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527492,"0312540-每次付费-幸_20","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,732.43,5,146.4860000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[6867,527495,"0312540-每次付费-幸_23","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,365.82,2,182.9100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527496,"0312540-每次付费-幸_25","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,510.78,2,255.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527497,"0312540-每次付费-幸_26","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,162.54,5,32.5080000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527498,"0312540-每次付费-幸_27","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,581.13,1,581.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,529659,"0314-tve-每次付费-幸_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4522.57,47,96.2248936,2,2261.285000,42.00,0.0092868,0.1667,0.04255,2,0,null,null,"BID_PER_ACTION"],[6867,529660,"0314-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2054.95,43,47.7895349,0,0.000000,0.00,0E-7,0.0909,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529661,"0314-tve-每次付费-幸_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1641.12,17,96.5364706,0,0.000000,0.00,0E-7,0.0588,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529662,"0314-tve-每次付费-幸_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1691.61,17,99.5064706,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529663,"0314-tve-每次付费-幸_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1702.45,17,100.1441176,2,851.225000,212.00,0.1245264,0.3529,0.11765,1,0,null,null,"BID_PER_ACTION"],[6867,529664,"0314-tve-每次付费-幸_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,5517.03,70,78.8147143,1,5517.030000,30.00,0.0054377,0.1111,0.01429,0,0,null,null,"BID_PER_ACTION"],[6867,529665,"0314-tve-每次付费-幸_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,832.97,8,104.1212500,0,0.000000,0.00,0E-7,0.1250,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529666,"0314-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,187.94,4,46.9850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529668,"0314-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1035.91,13,79.6853846,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529669,"0314-tve-每次付费-幸_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,7948.23,95,83.6655789,1,7948.230000,102.00,0.0128330,0.0702,0.01053,1,0,null,null,"BID_PER_ACTION"],[6867,529670,"0314-tve-每次付费-幸_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2632.16,27,97.4874074,0,0.000000,0.00,0E-7,0.0370,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529671,"0314-tve-每次付费-幸_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2616.41,32,81.7628125,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529672,"0314-tve-每次付费-幸_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,5607.30,118,47.5194915,2,2803.650000,24.00,0.0042801,0.0429,0.01695,0,0,null,null,"BID_PER_ACTION"],[6867,529674,"0314-tve-每次付费-幸_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2762.90,24,115.1208333,1,2762.900000,60.00,0.0217163,0.0833,0.04167,1,0,null,null,"BID_PER_ACTION"],[20451,499074,"306-幸存者-ROI02.21_5","2021-03-16 00:00:00","2021-02-21 00:00:00",10,1001379,1,2,3360.27,157,21.4029936,1,3360.270000,12.00,0.0035711,0.0000,0.00637,1,2,null,null,"ROI_PACING"],[20451,526868,"306-幸存者-ROI系03.11-2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,8113.55,149,54.4533557,2,4056.775000,162.00,0.0199666,0.0574,0.01342,10,2,null,null,"ROI_COEFFICIENT"],[20451,526878,"301-幸存者-ROI系03.11-4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.01,67,0.0001493,1,0.010000,60.00,6000.0000000,0.1045,0.01493,10,2,null,null,"ROI_COEFFICIENT"],[20451,527365,"SR-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,9789.16,161,60.8022360,1,9789.160000,42.00,0.0042905,0.0667,0.00621,5,2,null,null,"BID_PER_ACTION"],[20451,527366,"SR-财神到-每次03.12-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6701.87,120,55.8489167,2,3350.935000,736.00,0.1098201,0.0583,0.01667,6,2,null,null,"BID_PER_ACTION"],[20451,527367,"SR-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,11565.91,109,106.1092661,3,3855.303333,684.00,0.0591393,0.1250,0.02752,6,2,null,null,"BID_PER_ACTION"],[20451,527371,"PB-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3289.64,26,126.5246154,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527374,"PB-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3643.83,48,75.9131250,1,3643.830000,340.00,0.0933084,0.0882,0.02083,1,2,null,null,"BID_PER_ACTION"],[20451,527376,"PB-财神到-每次03.12-9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,7301.14,160,45.6321250,1,7301.140000,42.00,0.0057525,0.1231,0.00625,6,2,null,null,"BID_PER_ACTION"],[20451,527377,"PB-财神到-每次03.12-0","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3303.74,103,32.0751456,0,0.000000,0.00,0E-7,0.0777,0.00000,8,2,null,null,"BID_PER_ACTION"],[20451,528505,"FJ-财神到-每次03.12-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6701.95,76,88.1835526,3,2233.983333,218.00,0.0325278,0.1053,0.03947,3,2,null,null,"BID_PER_ACTION"],[20451,528508,"FJ-财神到-每次03.12-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6701.87,80,83.7733750,1,6701.870000,98.00,0.0146228,0.0750,0.01250,1,2,null,null,"BID_PER_ACTION"],[20451,528913,"FJ-财神到-每次03.13-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,16801.87,269,62.4604833,6,2800.311667,644.00,0.0383291,0.0769,0.02230,9,2,null,null,"BID_PER_ACTION"],[20451,528914,"FJ-财神到-每次03.13-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6701.87,111,60.3772072,2,3350.935000,140.00,0.0208897,0.0000,0.01802,3,2,null,null,"BID_PER_ACTION"],[20451,528917,"FW-财神到-每次03.13-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3303.74,68,48.5844118,1,3303.740000,30.00,0.0090806,0.0882,0.01471,1,2,null,null,"BID_PER_ACTION"],[20451,528918,"FJ-财神到-每次03.13-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6701.87,86,77.9287209,3,2233.956667,120.00,0.0179055,0.0000,0.03488,4,2,null,null,"BID_PER_ACTION"],[20540,452869,"HSX-幸存者-自动ROI-通投-头抖_混SSR1909_0128_20","2021-03-16 00:00:00","2021-01-28 00:00:00",10,1001379,1,2,2398.01,108,22.2037963,0,0.000000,0.00,0E-7,0.0943,0.00000,0,2,null,null,"ROI_PACING"],[20540,489063,"RICH-幸存者_春节-沉浸付费_安卓通投_69686_0215_07317475","2021-03-16 00:00:00","2021-02-15 00:00:00",10,1001379,1,2,6493.52,139,46.7159712,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,521086,"RICH-财神到我家-幸存者-每次付费_竖SSR2728_XTSC1614654964-5_0308_15","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,2,2891.65,31,93.2790323,0,0.000000,0.00,0E-7,0.0323,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526529,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2271_0311_25","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,202.64,8,25.3300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526531,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2332_0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2776.37,42,66.1040476,0,0.000000,0.00,0E-7,0.0238,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526535,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2735_0311_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,173.35,1,173.3500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526536,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2622_0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,3232.53,51,63.3829412,0,0.000000,0.00,0E-7,0.0588,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,527690,"RICH-财神到我家_幸存者-每次付费_排3000_71833_0312_95325541","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1362.71,15,90.8473333,0,0.000000,0.00,0E-7,0.0714,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,527692,"RICH-财神到我家_幸存者-每次付费_排3000_70989_0312_95325565","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2741.20,52,52.7153846,0,0.000000,0.00,0E-7,0.0192,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,527698,"RICH-财神到我家_幸存者-每次付费_排3000_76461_0312_95325564","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1169.04,7,167.0057143,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529036,"RICH-财神到我家_幸存者-每次付费_排3000_70019_0313_08286060","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3147.79,34,92.5820588,1,3147.790000,718.00,0.2280965,0.2381,0.02941,1,2,null,null,"BID_PER_ACTION"],[20540,529038,"RICH-财神到我家_幸存者-每次付费_排3000_77023_0313_08286082","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,1656.74,15,110.4493333,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20540,529211,"RICH-财神到我家-幸存者-每次付费_通投-混SSR2762-313_8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,15672.03,154,101.7664286,2,7836.015000,2750.00,0.1754718,0.0000,0.01299,3,2,null,null,"BID_PER_ACTION"],[20545,525304,"0310全民534自动roi基定排除2k_2742_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,8.85,4,2.2125000,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"ROI_PACING"],[20545,526883,"0311幸存者393z每次付费3k_2735_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,796.67,9,88.5188889,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526884,"0311幸存者393z每次付费3k_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,566.72,5,113.3440000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526887,"0311全民534自动roi排除3k_2731_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,310.61,15,20.7073333,0,0.000000,0.00,0E-7,0.2000,0.00000,1,2,null,null,"ROI_PACING"],[20545,526888,"0311全民534自动roi排除3k_2754_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,1048.04,24,43.6683333,1,1048.040000,12.00,0.0114499,0.1905,0.04167,2,2,null,null,"ROI_PACING"],[20545,526889,"0311全民534自动roi排除3k_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,199.32,10,19.9320000,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"ROI_PACING"],[20545,526890,"0311全民534自动roi排除3k_2738_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,237.29,11,21.5718182,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"ROI_PACING"],[20545,527861,"0312oyo393自动roi莱卡_2742_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2731.52,54,50.5837037,1,2731.520000,30.00,0.0109829,0.1020,0.01852,2,2,null,null,"ROI_PACING"],[20545,527862,"0312oyo393自动roi莱卡_2735_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2.83,13,0.2176923,0,0.000000,0.00,0E-7,0.0769,0.00000,3,2,null,null,"ROI_PACING"],[20545,527895,"0312幸存者393z每次付费3k_2735_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,52.68,2,26.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,527896,"0312幸存者393z每次付费3k_2731_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,119.71,1,119.7100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,527897,"0312幸存者393z每次付费3k_2746_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,39.34,1,39.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528105,"0312全民534自动roi排除3k_2766_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1.17,7,0.1671429,0,0.000000,0.00,0E-7,0.1429,0.00000,1,2,null,null,"ROI_PACING"],[20545,528106,"0312全民534自动roi排除3k_2764_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,24.64,6,4.1066667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,528107,"0312全民534自动roi排除3k_2768_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3139.48,79,39.7402532,0,0.000000,0.00,0E-7,0.0506,0.00000,3,2,null,null,"ROI_PACING"],[20545,528839,"0312全民534自动roi穿山甲_2622_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,635.32,19,33.4378947,1,635.320000,12.00,0.0188881,0.0526,0.05263,2,2,null,null,"BID_PER_ACTION"],[20545,528841,"0312全民534自动roi穿山甲_2762_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2865.33,98,29.2380612,1,2865.330000,30.00,0.0104700,0.0408,0.01020,4,2,null,null,"BID_PER_ACTION"],[20545,528846,"0312全民534自动roi穿山甲_2778_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,363.64,5,72.7280000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527500,"每0312-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,311.35,2,155.6750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,527501,"每0312-tt-xcztz-wyt-rk_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,768.90,3,256.3000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,527517,"每0312-tt-xcztz-wyt-rk_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,83.48,1,83.4800000,1,83.480000,68.00,0.8145664,1.0000,1.00000,1,0,null,null,"BID_PER_ACTION"],[20635,527531,"每0312-tt-xcztz-wyt-rk_24","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,430.58,7,61.5114286,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[20635,527532,"每0312-tt-xcztz-wyt-rk_27","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,51.72,1,51.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527533,"每0312-tt-xcztz-wyt-rk_28","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,56.26,2,28.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,529473,"每0314-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,810.03,3,270.0100000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529480,"每0314-tt-xcztz-wyt-rk新_19","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,802.69,5,160.5380000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529494,"每0314-tt-xcztz-wyt-rk新_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,484.59,4,121.1475000,1,484.590000,266.00,0.5489176,0.2500,0.25000,1,0,null,null,"BID_PER_ACTION"],[20635,529495,"每0314-tt-xcztz-wyt-rk新_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,466.24,5,93.2480000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529499,"每0314-tt-xcztz-wyt-rk新_20","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,452.98,4,113.2450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529500,"每0314-tt-xcztz-wyt-rk新_21","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,25,112.1496000,1,2803.740000,30.00,0.0107000,0.1200,0.04000,0,0,null,null,"BID_PER_ACTION"],[20635,529503,"每0314-tt-xcztz-wyt-rk新_26","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,788.60,3,262.8666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529504,"每0314-tt-xcztz-wyt-rk新_27","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,253.04,5,50.6080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529508,"每0314-tt-xcztz-wyt-rk新_33","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,411.79,1,411.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,521886,"05-幸存者-MCFC0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,2,3210.86,34,94.4370588,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,521889,"05-幸存者-MCFC0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,2,315.44,2,157.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526408,"263-幸存者-ROI-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,5982.19,111,53.8936036,2,2991.095000,268.00,0.0447996,0.1463,0.01802,3,2,null,null,"ROI_COEFFICIENT"],[20637,526410,"263-幸存者-ROI-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,772.40,28,27.5857143,0,0.000000,0.00,0E-7,0.0714,0.00000,2,2,null,null,"DEEP_BID_DEFAULT"],[20637,526845,"263-幸存者-MCFC0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,4781.59,129,37.0665891,3,1593.863333,54.00,0.0112933,0.1053,0.02326,8,2,null,null,"BID_PER_ACTION"],[20637,526847,"263-幸存者-MCFC0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,9490.94,115,82.5299130,2,4745.470000,422.00,0.0444635,0.0783,0.01739,2,2,null,null,"BID_PER_ACTION"],[20637,526943,"05-幸存者-MCFC0311_7_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,8393.00,154,54.5000000,1,8393.000000,30.00,0.0035744,0.0854,0.00649,10,2,null,null,"BID_PER_ACTION"],[20637,528659,"990-幸存者-MCFC-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,28.95,1,28.9500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528894,"05-幸存者-MCFC0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,255.47,3,85.1566667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528896,"05-幸存者-MCFC0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,475.13,2,237.5650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528898,"05-幸存者-MCFC0312_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,208.38,1,208.3800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528900,"05-幸存者-MCFC0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,242.56,1,242.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528901,"05-幸存者-MCFC0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,98.76,2,49.3800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528902,"05-幸存者-MCFC0312_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,83.84,1,83.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528924,"263-幸存者-MCFC0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1826.56,8,228.3200000,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528925,"263-幸存者-MCFC0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,571.92,6,95.3200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528928,"263-幸存者-MCFC0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,647.48,6,107.9133333,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20637,528930,"263-幸存者-MCFC0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,399.48,1,399.4800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528963,"263-幸存者-MCFC0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3456.42,24,144.0175000,2,1728.210000,128.00,0.0370325,0.1333,0.08333,2,2,null,null,"BID_PER_ACTION"],[20637,529561,"05-幸存者-MCFC0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,7757.01,103,75.3107767,2,3878.505000,392.00,0.0505349,0.0606,0.01942,2,0,null,null,"BID_PER_ACTION"],[20637,529562,"05-幸存者-MCFC0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,26,107.8361538,0,0.000000,0.00,0E-7,0.0769,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529563,"05-幸存者-MCFC0314_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,17360.96,128,135.6325000,9,1928.995556,6320.00,0.3640352,0.0826,0.07031,9,0,null,null,"BID_PER_ACTION"],[20637,529564,"05-幸存者-MCFC0314_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1937.23,24,80.7179167,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529565,"05-幸存者-MCFC0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,334.05,2,167.0250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529566,"05-幸存者-MCFC0314_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,25,112.1496000,0,0.000000,0.00,0E-7,0.0400,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529567,"05-幸存者-MCFC0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.63,16,175.2268750,0,0.000000,0.00,0E-7,0.1250,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529568,"05-幸存者-MCFC0314_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,5514.36,42,131.2942857,1,5514.360000,98.00,0.0177718,0.1667,0.02381,1,0,null,null,"BID_PER_ACTION"],[20637,529569,"05-幸存者-MCFC0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3729.36,50,74.5872000,2,1864.680000,392.00,0.1051119,0.0698,0.04000,1,0,null,null,"BID_PER_ACTION"],[20637,529570,"05-幸存者-MCFC0314_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2606.03,22,118.4559091,0,0.000000,0.00,0E-7,0.0909,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529571,"05-幸存者-MCFC0314_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,7757.01,91,85.2418681,1,7757.010000,2170.00,0.2797470,0.0159,0.01099,1,0,null,null,"BID_PER_ACTION"],[20637,529572,"05-幸存者-MCFC0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,34,82.4629412,0,0.000000,0.00,0E-7,0.0882,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,526460,"cxq-tt幸存者-每次付费-排除3000-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,494.05,5,98.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526479,"cxq-tt幸存者-每次付费-排除3000-0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2983.61,53,56.2945283,0,0.000000,0.00,0E-7,0.0377,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526546,"cxq-tt幸存者-每次付费-排除负向0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.05,6,0.0083333,1,0.050000,60.00,1200.0000000,0.1667,0.16667,1,2,null,null,"BID_PER_ACTION"],[20646,527535,"cxq-tt338幸存者-每次付费穿山甲0312_排30002736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,0.03,1,0.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527536,"cxq-tt338幸存者-每次付费穿山甲0312_排30002248","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,320.62,4,80.1550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527539,"cxq-tt552幸存者-每次付费穿山甲0312_排30002762小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2969.10,105,28.2771429,0,0.000000,0.00,0E-7,0.0316,0.00000,5,2,null,null,"BID_PER_ACTION"],[20646,527540,"cxq-tt338幸存者-每次付费穿山甲0312_排负2762小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,263.33,7,37.6185714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527541,"cxq-tt552幸存者-每次付费穿山甲0312_排30002746小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,416.54,18,23.1411111,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20646,527543,"cxq-tt338幸存者-每次付费穿山甲0312_排负2751小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,300.65,2,150.3250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527680,"cxq-tt549幸存者-每次付费-0312_排负+男优2729","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,290.54,2,145.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20646,527683,"cxq-tt747幸存者-每次付费-0312_排负+通全选小2762","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2721.60,43,63.2930233,2,1360.800000,54.00,0.0198413,0.0698,0.04651,4,2,null,null,"DEEP_BID_DEFAULT"],[20646,528876,"cxq-tt530幸存者-每次付费-优选-0312_小2746843-19","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,532.60,6,88.7666667,0,0.000000,0.00,0E-7,0.0000,0.00000,4,2,null,null,"DEEP_BID_DEFAULT"],[20646,529223,"cxq-tt338幸存者-每次付费-穿山甲0313_27622000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,8301.74,190,43.6933684,2,4150.870000,90.00,0.0108411,0.1067,0.01053,3,2,null,null,"BID_PER_ACTION"],[20646,529225,"cxq-tt338幸存者-每次付费-穿山甲0313_27462000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3260.12,107,30.4684112,1,3260.120000,30.00,0.0092021,0.1224,0.00935,2,2,null,null,"BID_PER_ACTION"],[20646,529226,"cxq-tt552幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,390.84,12,32.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529229,"cxq-tt552幸存者-每次付费-穿山甲0313_27462000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,2295.29,55,41.7325455,1,2295.290000,72.00,0.0313686,0.0926,0.01818,2,2,null,null,"BID_PER_ACTION"],[20646,529230,"cxq-tt552幸存者-每次付费-穿山甲0313_27622000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,1193.24,23,51.8800000,0,0.000000,0.00,0E-7,0.0435,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529233,"cxq-tt338幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,1757.74,50,35.1548000,1,1757.740000,12.00,0.0068269,0.2000,0.02000,1,2,null,null,"BID_PER_ACTION"],[20646,529234,"cxq-tt552幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,2721.14,78,34.8864103,1,2721.140000,12.00,0.0044099,0.0714,0.01282,0,2,null,null,"BID_PER_ACTION"],[20646,529252,"cxq-tt549幸存者-每次付费-0313_122-1SSR2285","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3170.88,219,14.4789041,1,3170.880000,212.00,0.0668584,0.0320,0.00457,1,2,null,null,"BID_PER_ACTION"],[20646,529253,"cxq-tt747幸存者-每次付费-0313_843-18SSR2762","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,1345.51,10,134.5510000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529254,"cxq-tt982幸存者-每次付费-0313_包1SSR2285","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3178.40,80,39.7300000,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529255,"cxq-tt747幸存者-每次付费-0313_843-18SSR2285","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3236.73,70,46.2390000,0,0.000000,0.00,0E-7,0.0571,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529257,"cxq-tt747幸存者-每次付费-0313_843-18SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,2215.01,11,201.3645455,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529259,"cxq-tt982幸存者-每次付费-0313_包1SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,4642.94,68,68.2785294,2,2321.470000,308.00,0.0663373,0.0652,0.02941,1,2,null,null,"BID_PER_ACTION"],[20646,529260,"cxq-tt747幸存者-每次付费-0313_843-18SSR1414","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,681.85,14,48.7035714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529261,"cxq-tt549幸存者-每次付费-0313_122-1SSR1414","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3262.92,97,33.6383505,2,1631.460000,24.00,0.0073554,0.0412,0.02062,2,2,null,null,"BID_PER_ACTION"],[20646,529262,"cxq-tt549幸存者-每次付费-0313_122-1SSR2762","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,1115.56,13,85.8123077,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529263,"cxq-tt549幸存者-每次付费-0313_122-1SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,5663.97,194,29.1957216,2,2831.985000,110.00,0.0194210,0.0670,0.01031,2,2,null,null,"BID_PER_ACTION"],[20754,519396,"hix-每次付费-ssr-3.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,3043.57,56,54.3494643,0,0.000000,0.00,0E-7,0.0357,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519397,"zhan-每次付费-ssr-3.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,3271.03,66,49.5610606,0,0.000000,0.00,0E-7,0.0606,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519401,"zhan-每次付费-ssr-3.6_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,8440.01,88,95.9092045,3,2813.336667,598.00,0.0708530,0.0909,0.03409,3,2,null,null,"BID_PER_ACTION"],[20754,519405,"hix-每次付费-ssr-3.6_8","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,498.37,1,498.3700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519409,"zhan-每次付费-ssr-3.6_6","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,662.59,1,662.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519411,"zhan-每次付费-ssr-3.6_8","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,3271.03,45,72.6895556,1,3271.030000,98.00,0.0299600,0.0222,0.02222,1,2,null,null,"BID_PER_ACTION"],[20754,520091,"yo-roi系数roi-想个好名字-2.6_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,1055.01,14,75.3578571,1,1055.010000,30.00,0.0284357,0.0000,0.07143,2,2,null,null,"ROI_COEFFICIENT"],[20754,520097,"yo-roi系数roi-想个好名字-2.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,106.86,5,21.3720000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,520100,"ka-roi系数-想个好名字-2.6_11","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,91.62,5,18.3240000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,523280,"zhan-每次付费--3.9_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,2,3247.66,44,73.8104545,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,523295,"zhan-每次付费--3.9_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,2,3676.13,31,118.5848387,2,1838.065000,958.00,0.2606001,0.0870,0.06452,2,2,null,null,"BID_PER_ACTION"],[20754,524817,"zhan-每次付费--3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,3175.76,48,66.1616667,0,0.000000,0.00,0E-7,0.0417,0.00000,3,2,null,null,"DEEP_BID_DEFAULT"],[20754,524825,"hix-每次付费--3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,1194.24,3,398.0800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524831,"jap-每次付费--3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,2988.42,29,103.0489655,1,2988.420000,60.00,0.0200775,0.0690,0.03448,1,2,null,null,"BID_PER_ACTION"],[20754,524832,"jap-每次付费--3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,1514.70,2,757.3500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524834,"hix-每次付费--3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,2066.19,22,93.9177273,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,525044,"ka-roi系数-爆好-3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,96.57,8,12.0712500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,527176,"hix-每次付费--3.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,5119.87,59,86.7774576,1,5119.870000,110.00,0.0214849,0.0980,0.01695,1,2,null,null,"BID_PER_ACTION"],[20754,527178,"tw-每次付费--3.11_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,559.21,14,39.9435714,0,0.000000,0.00,0E-7,0.1429,0.00000,2,2,null,null,"DEEP_BID_DEFAULT"],[20754,527183,"zhan-每次付费--3.11_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,840.73,10,84.0730000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528135,"hix-每次付费--3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,173.11,1,173.1100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528141,"jap-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,0.01,1,0.0100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528148,"zhan-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2311.04,39,59.2574359,1,2311.040000,158.00,0.0683675,0.0968,0.02564,1,2,null,null,"BID_PER_ACTION"],[20754,528153,"hix-每次付费--3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3786.04,48,78.8758333,2,1893.020000,104.00,0.0274693,0.0625,0.04167,2,2,null,null,"BID_PER_ACTION"],[20754,528157,"tw-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,787.70,16,49.2312500,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528349,"ka-roi系数-周五-3.12_4_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3271.03,51,64.1378431,0,0.000000,0.00,0E-7,0.0196,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,528353,"ka-roi系数-周五-3.12_4_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1098.52,29,37.8800000,0,0.000000,0.00,0E-7,0.1176,0.00000,2,2,null,null,"ROI_COEFFICIENT"],[20754,528354,"yo-roi系数-周五-3.12_4_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,77.20,17,4.5411765,0,0.000000,0.00,0E-7,0.0588,0.00000,3,2,null,null,"DEEP_BID_DEFAULT"],[20754,528356,"ka-roi系数-周五-3.12_4_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,5136.21,100,51.3621000,2,2568.105000,3516.00,0.6845514,0.0811,0.02000,6,2,null,null,"ROI_COEFFICIENT"],[20754,529154,"ka-自动roi-周六-3.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,2719.24,87,31.2556322,2,1359.620000,60.00,0.0220650,0.0690,0.02299,2,2,null,null,"ROI_PACING"],[20754,529155,"ka-自动roi-周六-3.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,1142.66,32,35.7081250,0,0.000000,0.00,0E-7,0.0333,0.00000,0,2,null,null,"ROI_PACING"],[20754,529158,"ka-自动roi-周六-3.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,2373.54,67,35.4259701,0,0.000000,0.00,0E-7,0.0645,0.00000,3,2,null,null,"ROI_PACING"],[20754,529159,"yo-roi系数-周六-3.13_44","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,6448.24,178,36.2260674,3,2149.413333,1148.00,0.1780331,0.0000,0.01685,6,2,null,null,"ROI_COEFFICIENT"],[20754,529161,"yo-roi系数-周六-3.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,4661.70,102,45.7029412,2,2330.850000,90.00,0.0193063,0.0290,0.01961,7,2,null,null,"ROI_COEFFICIENT"],[20754,529587,"hix-每次付费--3.13_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,504.15,9,56.0166667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529589,"jap-每次付费--3.13_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,581.66,4,145.4150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529590,"hix-每次付费--3.13_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,36,77.8816667,0,0.000000,0.00,0E-7,0.0556,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529591,"jap-每次付费--3.13_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1627.32,32,50.8537500,0,0.000000,0.00,0E-7,0.0645,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529592,"jap-每次付费--3.13_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,37,75.7767568,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529593,"tw-每次付费--3.13_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1682.15,41,41.0280488,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529594,"hix-每次付费--3.13_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,456.30,2,228.1500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529595,"hix-每次付费--3.13_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,44.73,2,22.3650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529596,"tw-每次付费--3.13_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4857.47,177,27.4433333,1,4857.470000,494.00,0.1016990,0.0395,0.00565,1,0,null,null,"BID_PER_ACTION"],[20754,529597,"tw-每次付费--3.13_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,42,66.7557143,0,0.000000,0.00,0E-7,0.0476,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529598,"zhan-每次付费--3.13_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,92.43,3,30.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529599,"tw-每次付费--3.13_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4859.81,74,65.6731081,1,4859.810000,102.00,0.0209885,0.0270,0.01351,1,0,null,null,"BID_PER_ACTION"],[20754,529600,"zhan-每次付费--3.13_17","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4492.21,77,58.3403896,3,1497.403333,304.00,0.0676727,0.0857,0.03896,2,0,null,null,"BID_PER_ACTION"],[20754,529601,"zhan-每次付费--3.13_18","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,463.50,2,231.7500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529602,"zhan-每次付费--3.13_19","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,32.47,1,32.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,527304,"295-blcb-每次付费爆R爆量_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1911.83,42,45.5197619,2,955.915000,244.00,0.1276264,0.0556,0.04762,2,2,null,null,"BID_PER_ACTION"],[21208,527307,"295-blcb-每次付费爆R爆量_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,261.47,6,43.5783333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527310,"295-blcb-每次付费爆R爆量_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1639.82,11,149.0745455,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527311,"295-blcb-每次付费爆R爆量_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,177.99,2,88.9950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527314,"295-blcb-每次付费爆R爆量_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,68.70,3,22.9000000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527346,"470-tv-每次付费爆R爆量_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1205.72,7,172.2457143,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,520588,"r-自动ROI-幸存者充值是大哥大-03.08_6","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,2,400.38,5,80.0760000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,521273,"g-自动ROI-幸存者加批计划跑晚上-03.08_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,2,2486.10,82,30.3182927,0,0.000000,0.00,0E-7,0.0952,0.00000,0,2,null,null,"ROI_PACING"],[21212,522092,"r-自动ROI-幸存者yyds-03.09_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,2,2907.25,130,22.3634615,0,0.000000,0.00,0E-7,0.0727,0.00000,2,2,null,null,"ROI_PACING"],[21212,525391,"i-每次付费-幸存者挑战-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,607.88,5,121.5760000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526440,"r3-每次付费-幸存者挑战-03.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,5520.72,84,65.7228571,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526444,"r3-每次付费-幸存者挑战-03.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2763.86,30,92.1286667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526453,"r3-每次付费-幸存者挑战-03.11_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2773.18,22,126.0536364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,528558,"g-自动ROI-幸存者挑战-03.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,898.27,61,14.7257377,1,898.270000,238.00,0.2649537,0.1429,0.01639,2,2,null,null,"ROI_PACING"],[21212,528562,"g-自动ROI-幸存者挑战-03.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1339.51,28,47.8396429,1,1339.510000,30.00,0.0223962,0.0714,0.03571,2,2,null,null,"ROI_PACING"],[21212,529098,"r3-每次付费-幸存者挑战周六-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3500.17,26,134.6219231,1,3500.170000,298.00,0.0851387,0.0833,0.03846,2,2,null,null,"BID_PER_ACTION"],[21212,529099,"r3-每次付费-幸存者挑战周六-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,4743.82,34,139.5241176,1,4743.820000,42.00,0.0088536,0.0000,0.02941,2,2,null,null,"BID_PER_ACTION"],[21212,529109,"r-自动ROI-幸存者挑战带穿山甲-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,8.48,6,1.4133333,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"ROI_PACING"],[21212,529110,"r-自动ROI-幸存者挑战带穿山甲-03.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,0.18,1,0.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,529111,"r-自动ROI-幸存者挑战带穿山甲-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,15.35,2,7.6750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,529113,"r-自动ROI-幸存者挑战带穿山甲-03.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,5.27,1,5.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,529538,"i-每次付费-幸存者挑战周日-03.14_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1325.42,10,132.5420000,0,0.000000,0.00,0E-7,0.3000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529539,"i-每次付费-幸存者挑战周日-03.14_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,42,66.7557143,0,0.000000,0.00,0E-7,0.0238,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529540,"r3-每次付费-幸存者挑战周日-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4871.34,46,105.8986957,0,0.000000,0.00,0E-7,0.0526,0.00000,1,0,null,null,"BID_PER_ACTION"],[21212,529541,"r3-每次付费-幸存者挑战周日-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3518.33,29,121.3217241,3,1172.776667,1008.00,0.2864996,0.3333,0.10345,3,0,null,null,"BID_PER_ACTION"],[21212,529542,"i-每次付费-幸存者挑战周日-03.14_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,544.15,5,108.8300000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529543,"r3-每次付费-幸存者挑战周日-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,52,53.9180769,0,0.000000,0.00,0E-7,0.0577,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529544,"i-每次付费-幸存者挑战周日-03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,984.42,11,89.4927273,1,984.420000,434.00,0.4408687,0.1667,0.09091,1,0,null,null,"BID_PER_ACTION"],[21212,529545,"r3-每次付费-幸存者挑战周日-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,6139.32,105,58.4697143,2,3069.660000,422.00,0.0687373,0.0769,0.01905,2,0,null,null,"BID_PER_ACTION"],[21212,529546,"i-每次付费-幸存者挑战周日-03.14_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,5878.93,111,52.9633333,2,2939.465000,550.00,0.0935544,0.0580,0.01802,2,0,null,null,"BID_PER_ACTION"],[21212,529547,"r3-每次付费-幸存者挑战周日-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,494.71,2,247.3550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529548,"g-自动ROI-幸存者挑战带穿山甲-03.14_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3226.82,209,15.4393301,0,0.000000,0.00,0E-7,0.0478,0.00000,0,0,null,null,"ROI_PACING"],[21212,529549,"g-自动ROI-幸存者挑战带穿山甲-03.14_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1128.39,55,20.5161818,0,0.000000,0.00,0E-7,0.0364,0.00000,0,0,null,null,"ROI_PACING"],[21212,529550,"g-自动ROI-幸存者挑战带穿山甲-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4918.00,228,21.5701754,1,4918.000000,188.00,0.0382269,0.1000,0.00439,1,0,null,null,"ROI_PACING"],[21212,529551,"r-自动ROI-幸存者挑战带穿山甲-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2723.69,180,15.1316111,1,2723.690000,250.00,0.0917872,0.0281,0.00556,1,0,null,null,"ROI_PACING"],[21212,529552,"r-自动ROI-幸存者挑战带穿山甲-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3128.51,123,25.4350407,1,3128.510000,30.00,0.0095892,0.0813,0.00813,1,0,null,null,"ROI_PACING"],[21212,529553,"r-自动ROI-幸存者挑战带穿山甲-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2893.99,204,14.1862255,1,2893.990000,12.00,0.0041465,0.0343,0.00490,1,0,null,null,"ROI_PACING"],[21212,529554,"r-自动ROI-幸存者挑战带穿山甲-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,291.06,10,29.1060000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,529555,"g-自动ROI-幸存者挑战带穿山甲-03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,940.88,32,29.4025000,0,0.000000,0.00,0E-7,0.0645,0.00000,0,0,null,null,"ROI_PACING"],[21212,529557,"r-每次付费-幸存者挑战-穿山甲专场-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1855.91,11,168.7190909,1,1855.910000,60.00,0.0323292,0.2500,0.09091,1,0,null,null,"BID_PER_ACTION"],[21245,519366,"BY每次付费-拉环小游戏-xcztz-周六-0306_4","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,157.82,5,31.5640000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520837,"by每次付费-xcztz-0308_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,2,2500.95,55,45.4718182,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21245,521878,"自动ROI-JK-排3K包-拉环-_0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,2,146.79,20,7.3395000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524567,"自动ROI-排3K包-0310_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,56.26,7,8.0371429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524575,"自动ROI-排3K包-0310_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,72.64,4,18.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524576,"自动ROI-排3K包-0310_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,2.94,5,0.5880000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524581,"自动ROI-排3K包-0310_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,2.17,1,2.1700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524999,"BY每次付费-xcztz-拉环卡车-0310-_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,59.90,1,59.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525000,"BY每次付费-xcztz-拉环卡车-0310-_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,51.04,2,25.5200000,0,0.000000,0.00,0E-7,0.5000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21245,525003,"BY每次付费-xcztz-拉环卡车-0310-_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,221.86,14,15.8471429,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21245,525009,"BY每次付费-xcztz-拉环卡车-0310-_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,31.88,5,6.3760000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21245,525014,"BY每次付费-xcztz-拉环卡车-0310-_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,127.87,5,25.5740000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,526396,"jk自动ROI-排3K-xcztz-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.82,1,0.8200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,526397,"jk自动ROI-排3K-xcztz-0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.93,1,0.9300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526576,"DM自动ROI-xcztz-排3K-0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,20.73,10,2.0730000,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526578,"DM自动ROI-xcztz-排3K-0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,7.32,4,1.8300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526580,"DM自动ROI-xcztz-排3K-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2.42,1,2.4200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,527266,"每次-2762-冲冲冲_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,7099.46,58,122.4044828,1,7099.460000,60.00,0.0084513,0.0000,0.01724,0,2,null,null,"BID_PER_ACTION"],[21245,527267,"每次-2762-冲冲冲_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,8103.71,85,95.3377647,1,8103.710000,12.00,0.0014808,0.0800,0.01176,1,2,null,null,"BID_PER_ACTION"],[21245,529374,"JK-自动ROI排3K周日发力-0313_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,524.14,48,10.9195833,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21245,529375,"JK-自动ROI排3K周日发力-0313_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,666.41,45,14.8091111,1,666.410000,60.00,0.0900347,0.0667,0.02222,1,0,null,null,"ROI_PACING"],[21245,529376,"JK-自动ROI排3K周日发力-0313_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3473.89,103,33.7270874,1,3473.890000,208.00,0.0598752,0.0854,0.00971,1,0,null,null,"DEEP_BID_DEFAULT"],[21245,529378,"JK-自动ROI排3K周日发力-0313_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,102.88,3,34.2933333,0,0.000000,0.00,0E-7,0.5000,0.00000,0,0,null,null,"ROI_PACING"],[21245,529399,"每次-周末冲刺-0314_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2802.70,31,90.4096774,1,2802.700000,12.00,0.0042816,0.1613,0.03226,1,0,null,null,"BID_PER_ACTION"],[21245,529400,"每次-周末冲刺-0314_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,26,107.8361538,0,0.000000,0.00,0E-7,0.1154,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529401,"每次-周末冲刺-0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,38,73.7826316,0,0.000000,0.00,0E-7,0.0526,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529402,"每次-周末冲刺-0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,62,45.2216129,0,0.000000,0.00,0E-7,0.0323,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529403,"每次-周末冲刺-0314_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1781.02,20,89.0510000,0,0.000000,0.00,0E-7,0.0500,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529404,"每次-周末冲刺-0314_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,173,16.2065896,2,1401.870000,422.00,0.1505132,0.0578,0.01156,2,0,null,null,"BID_PER_ACTION"],[21245,529405,"每次-周末冲刺-0314_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,25,112.1496000,0,0.000000,0.00,0E-7,0.0400,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529406,"每次-周末冲刺-0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,21,133.5114286,1,2803.740000,12.00,0.0042800,0.1429,0.04762,0,0,null,null,"BID_PER_ACTION"],[21245,529407,"每次-周末冲刺-0314_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,74,37.8883784,1,2803.740000,42.00,0.0149800,0.0405,0.01351,1,0,null,null,"BID_PER_ACTION"],[21245,529408,"每次-周末冲刺-0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,10140.91,84,120.7251190,3,3380.303333,3212.00,0.3167369,0.0769,0.03571,3,0,null,null,"BID_PER_ACTION"],[21245,529409,"每次-周末冲刺-0314_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1051.60,3,350.5333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529410,"每次-周末冲刺-0314_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,41,68.3839024,0,0.000000,0.00,0E-7,0.1220,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529411,"每次-周末冲刺-0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,457.52,1,457.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529413,"每次-周末冲刺-0314_17","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,472.52,1,472.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529415,"每次-周末冲刺-0314_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2803.74,18,155.7633333,2,1401.870000,200.00,0.0713333,0.0556,0.11111,2,0,null,null,"BID_PER_ACTION"],[6867,494510,"0219-自动ROI-MG540迷岛_31304077","2021-03-16 00:00:00","2021-02-19 00:00:00",10,1001400,1,2,2975.03,31,95.9687097,0,0.000000,0.00,0E-7,0.0645,0.00000,0,2,null,null,"ROI_PACING"],[20541,506968,"RICH-财神到 -研究-通投_横SSR2138_0226_16","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,2,12149.54,152,79.9311842,2,6074.770000,290.00,0.0238692,0.0408,0.01316,3,2,null,null,"DEEP_BID_DEFAULT"],[20541,507537,"RICH-财神到-研究所_IOS通投_横SSR2363_0226_3","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,2,40682.60,460,88.4404348,11,3698.418182,1932.00,0.0474896,0.0929,0.02391,16,2,null,null,"DEEP_BID_DEFAULT"],[20541,513880,"RICH-研究所-招R体-通投_横SSR2248_0303_12","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,2,14918.29,194,76.8984021,3,4972.763333,408.00,0.0273490,0.1829,0.01546,4,2,null,null,"DEEP_BID_DEFAULT"],[20541,513883,"RICH-研究所-招R体-通投_横SSR2666_0303_10","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,2,107.73,0,107.7300000,0,0.000000,0.00,0E-7,1.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,520818,"RICH-研究所-财神到我家-通投_横SSR2724_0308_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,2,760.91,3,253.6366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,523053,"RICH-研究所-财神到我家-通投_横SSR2363_0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,2,4845.86,82,59.0958537,2,2422.930000,60.00,0.0123817,0.1017,0.02439,2,2,null,null,"DEEP_BID_DEFAULT"],[20541,523054,"RICH-研究所-财神到我家-通投_横SSR2138_0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,2,7786.03,74,105.2166216,1,7786.030000,474.00,0.0608783,0.0952,0.01351,1,2,null,null,"DEEP_BID_DEFAULT"],[20541,523056,"RICH-研究所-财神到我家-通投_横SSR2536_0309_19","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,2,3717.29,46,80.8106522,2,1858.645000,54.00,0.0145267,0.1707,0.04348,1,2,null,null,"DEEP_BID_DEFAULT"],[20541,523061,"RICH-研究所-财神到我家-通投_横SSR1909_0309_18","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,2,2203.24,23,95.7930435,0,0.000000,0.00,0E-7,0.0476,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526248,"RICH-研究所-财神到我家_通投-新用户-_SSR2248_0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,2,1009.76,11,91.7963636,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526251,"RICH-研究所-财神到我家_通投-包过滤-_SSR2248_0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,2,1918.92,16,119.9325000,0,0.000000,0.00,0E-7,0.2308,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528016,"RICH-研究所-财神到我家_通投_横SSR2285_0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,2,1772.94,16,110.8087500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528017,"RICH-研究所-财神到我家_通投_横SSR2495_0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,2,1022.70,7,146.1000000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528019,"RICH-研究所-财神到我家_通投_横SSR2710_0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,2,5721.17,51,112.1798039,2,2860.585000,54.00,0.0094386,0.0714,0.03922,2,2,null,null,"DEEP_BID_DEFAULT"],[20541,528020,"RICH-研究所-财神到我家_通投_横SSR2138_0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,2,1160.76,9,128.9733333,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528021,"RICH-研究所-财神到我家_通投_横SSR1909_0312_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,2,417.57,17,24.5629412,1,417.570000,30.00,0.0718442,0.1429,0.05882,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528025,"RICH-研究所-财神到我家_通投_横SSR2138_0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,2,2592.85,11,235.7136364,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[20546,526922,"0311wtd末日研究所_2754_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,2,1913.12,13,147.1630769,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526923,"0311wtd末日研究所_2760_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,2,1616.34,6,269.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526925,"0311wtd末日研究所_2736_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,2,2553.07,11,232.0972727,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528806,"0312XM-MGTTN1计划_3_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,565.14,9,62.7933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528807,"0312XM-MGTTN1计划_3_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,217.34,3,72.4466667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528808,"0312XM-MGTTN1计划_3_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,109.09,1,109.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528811,"0312XM-MGTTN1计划_3_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,40.59,2,20.2950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528812,"0312XM-MGTTN1计划_3_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,1073.72,28,38.3471429,1,1073.720000,268.00,0.2495995,0.0455,0.03571,1,2,null,null,"DEEP_BID_DEFAULT"],[20647,525144,"cxq-tt548N1计划-0310_包12742","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,2,305.17,14,21.7978571,0,0.000000,0.00,0E-7,0.0714,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525151,"cxq-tt894N1计划-0310_春节包通投2741","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,2,787.83,7,112.5471429,1,787.830000,12.00,0.0152317,0.0000,0.14286,2,2,null,null,"DEEP_BID_DEFAULT"],[20647,526681,"cxq-tt548N1计划-0311__3小2755","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,2,1928.17,22,87.6440909,0,0.000000,0.00,0E-7,0.0455,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526683,"cxq-tt548N1计划-0311_12742","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,2,7986.37,62,128.8124194,2,3993.185000,128.00,0.0160273,0.0000,0.03226,1,2,null,null,"DEEP_BID_DEFAULT"],[20647,526684,"cxq-tt894N1计划-0311_22558","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,2,5592.78,58,96.4272414,1,5592.780000,12.00,0.0021456,0.1071,0.01724,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526685,"cxq-tt894N1计划-0311_4小2753","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,2,703.79,4,175.9475000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526688,"cxq-tt894N1计划-0311_6小2723","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,2,2275.99,6,379.3316667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526694,"cxq-tt548N1计划-0311__52750","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,2,359.19,3,119.7300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526696,"cxq-tt894N1计划-0311_12742","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,2,32.09,3,10.6966667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,526703,"N1穿山甲-iOS-499-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,2,5327.56,30,177.5853333,3,1775.853333,528.00,0.0991073,0.0588,0.10000,3,2,null,null,"DEEP_BID_DEFAULT"],[21246,528854,"付费-穿山甲-n1-499-0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,177.57,3,59.1900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528855,"付费-穿山甲-n1-499-0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,15.92,1,15.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528857,"付费-穿山甲-n1-499-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,536.22,6,89.3700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,529419,"穿-n1-周末付费冲-0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,2,3039.72,18,168.8733333,2,1519.860000,90.00,0.0296080,0.1667,0.11111,1,0,null,null,"DEEP_BID_DEFAULT"],[21246,529421,"穿-n1-周末付费冲-0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,2,129.76,1,129.7600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529422,"穿-n1-周末付费冲-0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,2,267.40,1,267.4000000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[6770,522169,"7tt_n_每次付费_幸存危机_0309_9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,2,792.32,18,44.0177778,0,0.000000,0.00,0E-7,0.3125,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522172,"7tt_n_每次付费_幸存危机_0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,2,219.89,2,109.9450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522175,"7tt_n_每次付费_幸存危机_0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,2,928.59,34,27.3114706,0,0.000000,0.00,0E-7,0.0370,0.00000,1,2,null,null,"BID_PER_ACTION"],[6770,526325,"7tt_n_幸存危机_每次付费__0311_52245342","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,586.33,7,83.7614286,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526326,"7tt_n_幸存危机_每次付费__0311_52245343","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,100.98,3,33.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526336,"7tt_n_幸存危机_每次付费__0311_522453713","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,35.14,1,35.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527935,"7tt_n_幸存危机_每次付费__0312_16295372","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,474.59,2,237.2950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527967,"7tt_n_幸存危机_每次付费__0312_16295373","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,166.34,3,55.4466667,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527969,"7tt_n_幸存危机_每次付费__0312_16295385","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,69.14,1,69.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527973,"7tt_n_幸存危机_每次付费__0312_16295389","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,1044.06,19,54.9505263,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527974,"7tt_n_幸存危机_每次付费__0312_162953910","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,239.43,2,119.7150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527975,"7tt_n_幸存危机_每次付费__0312_162953911","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,119.73,1,119.7300000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[6770,527976,"7tt_n_幸存危机_每次付费__0312_162953912","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,374.93,3,124.9766667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529179,"7tt_n_幸存危机_每次付费__0313_84190334","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,2,375.11,2,187.5550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529180,"7tt_n_幸存危机_每次付费__0313_84190335","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,2,2824.35,16,176.5218750,1,2824.350000,12.00,0.0042488,0.1250,0.06250,1,2,null,null,"BID_PER_ACTION"],[6770,529183,"7tt_n_幸存危机_每次付费__0313_841903412","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,2,379.43,3,126.4766667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529184,"7tt_n_幸存危机_每次付费__0313_841903513","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,2,410.56,4,102.6400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,529024,"CJ-财神到-ROI系03.13-9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,2,3.21,0,3.2100000,0,0.000000,0.00,0E-7,0.0000,0,1,2,null,null,"ROI_COEFFICIENT"],[20451,529364,"VR-危机-每次03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,547.21,2,273.6050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529365,"CJ-危机-每次03.14_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,726.81,2,363.4050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529366,"VR-危机-每次03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,156.87,1,156.8700000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529367,"CJ-危机-每次03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,1938.18,20,96.9090000,1,1938.180000,110.00,0.0567543,0.2000,0.05000,1,0,null,null,"BID_PER_ACTION"],[20451,529370,"CJ-危机-每次03.14_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,749.56,4,187.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529371,"CJ-危机-每次03.14_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,720.60,1,720.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529372,"VR-危机-每次03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,1100.06,7,157.1514286,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529373,"VR-危机-每次03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,679.15,3,226.3833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,522465,"0309-508幸存危机-每次付费_2271_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,2,170.04,5,34.0080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526911,"0311-508幸存危机-每次付费竖版沉浸_2332_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,15.81,1,15.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528127,"0312-507幸存危机每次付费_2332_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,1052.92,9,116.9911111,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528547,"0312-508幸存危机-每次付费竖版沉浸_2332_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,251.33,3,83.7766667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527870,"481IC-幸存危机-648大集合-3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,961.15,2,480.5750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527871,"481IC-幸存危机-648大集合-3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,3103.88,14,221.7057143,1,3103.880000,182.00,0.0586363,0.2143,0.07143,1,2,null,null,"BID_PER_ACTION"],[21208,527872,"481IC-幸存危机-648大集合-3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,250.09,2,125.0450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527873,"481IC-幸存危机-648大集合-3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,2444.19,10,244.4190000,1,2444.190000,48.00,0.0196384,0.1667,0.10000,2,2,null,null,"BID_PER_ACTION"],[21212,524367,"k-每次付费-危机窄定向-03.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,2,854.03,3,284.6766667,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[21212,525586,"k-每次付费-幸存危机加量-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,2,2006.11,7,286.5871429,0,0.000000,0.00,0E-7,0.0000,0.00000,2,2,null,null,"BID_PER_ACTION"],[21213,526689,"n-ios-xy-黎明血战-03.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,2,210.56,5,42.1120000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"DEEP_BID_DEFAULT"],[21213,526691,"n-ios-xy-黎明血战-03.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,2,2803.74,28,100.1335714,0,0.000000,0.00,0E-7,0.0714,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,529573,"n96-iOS-黎明血战周日-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,2,4752.88,123,38.6413008,1,4752.880000,80.00,0.0168319,0.0625,0.00813,1,0,null,null,"DEEP_BID_DEFAULT"],[21213,529574,"n96-iOS-黎明血战周日-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,2,1174.32,13,90.3323077,0,0.000000,0.00,0E-7,0.0909,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,529575,"n96-iOS-黎明血战周日-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,2,1038.24,22,47.1927273,0,0.000000,0.00,0E-7,0.1364,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[6867,529646,"0314-540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4310.70,79,54.5658228,1,4310.700000,98.00,0.0227341,0.0000,0.01266,1,0,null,null,"BID_PER_ACTION"],[6867,529652,"0314-540-每次付费-幸_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,12389.74,515,24.0577476,2,6194.870000,1322.00,0.1067012,0.0000,0.00388,2,0,null,null,"BID_PER_ACTION"],[6867,529655,"0314-540-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,5653.59,139,40.6733094,1,5653.590000,110.00,0.0194567,0.0000,0.00719,0,0,null,null,"BID_PER_ACTION"],[6867,529656,"0314-540-每次付费-幸_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4422.75,52,85.0528846,1,4422.750000,12.00,0.0027132,0.0000,0.01923,0,0,null,null,"BID_PER_ACTION"],[6867,529658,"0314-540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,18874.63,268,70.4277239,4,4718.657500,418.00,0.0221461,0.0000,0.01493,4,0,null,null,"BID_PER_ACTION"],[6867,529919,"0315-204-穿山甲-每次付费-幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2010.23,62,32.4230645,2,1005.115000,518.00,0.2576820,0.0000,0.03226,1,0,null,null,"BID_PER_ACTION"],[6867,529921,"0315-204-穿山甲-每次付费-幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,439.52,21,20.9295238,1,439.520000,12.00,0.0273025,0.0000,0.04762,0,0,null,null,"BID_PER_ACTION"],[6867,530808,"0315FP-ROI系数-MG幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3766.77,14,269.0550000,1,3766.770000,2624.00,0.6966181,0.0000,0.07143,1,0,null,null,"ROI_COEFFICIENT"],[20451,530717,"306-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,7688.27,237,32.4399578,1,7688.270000,30.00,0.0039020,0.0000,0.00422,1,0,null,null,"ROI_COEFFICIENT"],[20451,530719,"303-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3356.80,89,37.7168539,1,3356.800000,256.00,0.0762631,0.0000,0.01124,1,0,null,null,"ROI_COEFFICIENT"],[20451,530936,"FJ-幸存者-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3302.07,47,70.2568085,1,3302.070000,42.00,0.0127193,0.0000,0.02128,0,0,null,null,"BID_PER_ACTION"],[20540,527691,"RICH-财神到我家_幸存者-每次付费_排3000_73011_0312_95325530","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6843.88,63,108.6330159,3,2281.293333,2200.00,0.3214551,0.0000,0.04762,4,2,null,null,"BID_PER_ACTION"],[20540,530798,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2285315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2534.93,64,39.6082813,1,2534.930000,12.00,0.0047339,0.0000,0.01563,1,0,null,null,"BID_PER_ACTION"],[20545,530526,"0315oyo393-roi系数-排除3k_2138_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1656.50,19,87.1842105,1,1656.500000,12.00,0.0072442,0.0000,0.05263,0,0,null,null,"ROI_COEFFICIENT"],[20545,530714,"0315-幸存者393z每次付费排除3k_2789_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,124.21,1,124.2100000,1,124.210000,298.00,2.3991627,0.0000,1.00000,1,0,null,null,"BID_PER_ACTION"],[20545,530715,"0315-幸存者393z每次付费排除3k_2792_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,4240.95,69,61.4630435,1,4240.950000,30.00,0.0070739,0.0000,0.01449,1,0,null,null,"BID_PER_ACTION"],[20545,531288,"0315幸存者393z每次付费排除3K_2796_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,10905.36,259,42.1056371,3,3635.120000,3214.00,0.2947175,0.0000,0.01158,2,0,null,null,"BID_PER_ACTION"],[20635,530152,"每0315-tt-xcztz-wyt-rk_68","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,8937.37,78,114.5816667,2,4468.685000,250.00,0.0279724,0.0000,0.02564,2,0,null,null,"BID_PER_ACTION"],[20637,529851,"263-幸存者-MCFC0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,7889.73,135,58.4424444,2,3944.865000,168.00,0.0212935,0.0000,0.01481,1,0,null,null,"BID_PER_ACTION"],[20637,529866,"263-幸存者-MCFC0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2851.71,50,57.0342000,1,2851.710000,128.00,0.0448853,0.0000,0.02000,0,0,null,null,"BID_PER_ACTION"],[20637,529974,"990-幸存者-MCFC-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1068.89,18,59.3827778,1,1068.890000,128.00,0.1197504,0.0000,0.05556,1,0,null,null,"BID_PER_ACTION"],[20637,531204,"05-幸存者-MCFC0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,167.78,3,55.9266667,1,167.780000,392.00,2.3363929,0.0000,0.33333,1,0,null,null,"BID_PER_ACTION"],[20637,531251,"05-幸存者-MCFC0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1168.82,16,73.0512500,1,1168.820000,226.00,0.1933574,0.0000,0.06250,1,0,null,null,"BID_PER_ACTION"],[20637,531308,"263-幸存者-ROI-0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,16963.69,458,37.0386245,4,4240.922500,602.00,0.0354876,0.0000,0.00873,4,0,null,null,"ROI_COEFFICIENT"],[20637,531312,"263-幸存者-ROI-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1496.10,13,115.0846154,1,1496.100000,226.00,0.1510594,0.0000,0.07692,1,0,null,null,"ROI_COEFFICIENT"],[20637,531586,"263-幸存者-MCFC0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,7135.46,49,145.6216327,1,7135.460000,42.00,0.0058861,0.0000,0.02041,1,0,null,null,"BID_PER_ACTION"],[20646,530670,"cxq-tt982幸存者-ROI系数0315_2762","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1648.92,48,34.3525000,1,1648.920000,188.00,0.1140140,0.0000,0.02083,1,0,null,null,"ROI_COEFFICIENT"],[20754,530906,"jap-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1142.78,6,190.4633333,1,1142.780000,200.00,0.1750118,0.0000,0.16667,1,0,null,null,"BID_PER_ACTION"],[20754,530924,"tw-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1087.12,17,63.9482353,2,543.560000,3096.00,2.8478917,0.0000,0.11765,2,0,null,null,"BID_PER_ACTION"],[20754,530959,"ka-roi系数-周一-3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,4853.92,117,41.4864957,4,1213.480000,182.00,0.0374955,0.0000,0.03419,1,0,null,null,"ROI_COEFFICIENT"],[20754,530961,"ka-roi系数-周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,5097.48,91,56.0162637,2,2548.740000,652.00,0.1279063,0.0000,0.02198,1,0,null,null,"ROI_COEFFICIENT"],[20754,530962,"ka-roi系数-周一-3.15_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2803.74,72,38.9408333,1,2803.740000,12.00,0.0042800,0.0000,0.01389,0,0,null,null,"ROI_COEFFICIENT"],[20754,530966,"yo-roi系数-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2659.43,26,102.2857692,1,2659.430000,42.00,0.0157929,0.0000,0.03846,1,0,null,null,"ROI_COEFFICIENT"],[20754,530968,"ka-roi系数-周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2115.48,31,68.2412903,2,1057.740000,238.00,0.1125040,0.0000,0.06452,2,0,null,null,"ROI_COEFFICIENT"],[21208,531039,"bdna-非海马云-幸存者挑战-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2803.74,66,42.4809091,1,2803.740000,12.00,0.0042800,0.0000,0.01515,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531040,"bdna-非海马云-幸存者挑战-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2664.20,7,380.6000000,1,2664.200000,12.00,0.0045042,0.0000,0.14286,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531042,"bdna-非海马云-幸存者挑战-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2393.44,25,95.7376000,1,2393.440000,74.00,0.0309178,0.0000,0.04000,1,0,null,null,"DEEP_BID_DEFAULT"],[21212,530169,"r3-每次付费-幸存者挑战优选位-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2708.30,13,208.3307692,1,2708.300000,12.00,0.0044308,0.0000,0.07692,0,0,null,null,"BID_PER_ACTION"],[21212,530178,"i-每次付费-幸存者挑战只限男优先位-03.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,6533.02,84,77.7740476,1,6533.020000,678.00,0.1037805,0.0000,0.01190,1,0,null,null,"BID_PER_ACTION"],[21212,530212,"r3-每次付费-幸存者挑战优选位-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2035.04,22,92.5018182,1,2035.040000,30.00,0.0147417,0.0000,0.04545,1,0,null,null,"BID_PER_ACTION"],[21245,525005,"DM每次付费-xcztz-拉环卡车-0310-_23","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,5578.76,141,39.5656738,1,5578.760000,208.00,0.0372843,0.0000,0.00709,1,2,null,null,"BID_PER_ACTION"],[21245,531193,"roi系数-0315-BY-幸存_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2820.12,162,17.4081481,1,2820.120000,110.00,0.0390054,0.0000,0.00617,1,0,null,null,"ROI_COEFFICIENT"],[21245,531228,"roi系数-0315-BY-幸存_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2948.60,41,71.9170732,1,2948.600000,30.00,0.0101743,0.0000,0.02439,0,0,null,null,"ROI_COEFFICIENT"],[20541,531196,"RICH-研究所-财神到我家-通投_混SSR2796_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,3588.63,19,188.8752632,3,1196.210000,122.00,0.0339963,0.0000,0.15789,3,0,null,null,"DEEP_BID_DEFAULT"],[20541,531198,"RICH-研究所-财神到我家-通投_混SSR2792_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,2750.84,18,152.8244444,1,2750.840000,98.00,0.0356255,0.0000,0.05556,1,0,null,null,"DEEP_BID_DEFAULT"],[20546,531301,"0315wtd末日研究所_2138_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,7969.71,71,112.2494366,1,7969.710000,60.00,0.0075285,0.0000,0.01408,1,0,null,null,"DEEP_BID_DEFAULT"],[20546,531577,"0315ph648末日研究所_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,3836.34,34,112.8335294,1,3836.340000,886.00,0.2309493,0.0000,0.02941,1,0,null,null,"DEEP_BID_DEFAULT"],[20451,530040,"CJ-幸存危机-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,2256.19,17,132.7170588,1,2256.190000,30.00,0.0132968,0.0000,0.05882,1,0,null,null,"BID_PER_ACTION"],[20545,531701,"0315-508幸存危机-每次付费_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,2764.52,47,58.8195745,2,1382.260000,1354.00,0.4897776,0.0000,0.04255,1,0,null,null,"BID_PER_ACTION"],[20545,531702,"0315-508幸存危机-每次付费_2053_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,4513.16,74,60.9886486,1,4513.160000,30.00,0.0066472,0.0000,0.01351,1,0,null,null,"BID_PER_ACTION"],[20545,531704,"0315-508幸存危机-每次付费_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,2778.84,28,99.2442857,1,2778.840000,12.00,0.0043183,0.0000,0.03571,0,0,null,null,"BID_PER_ACTION"],[21208,529911,"481IC-幸存危机-2756爆量爆R-3.5_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,4546.33,62,73.3279032,1,4546.330000,12.00,0.0026395,0.0000,0.01613,1,0,null,null,"BID_PER_ACTION"],[21208,529912,"481IC-幸存危机-2756爆量爆R-3.5_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,1254.35,10,125.4350000,1,1254.350000,72.00,0.0574002,0.0000,0.10000,1,0,null,null,"BID_PER_ACTION"],[21208,529915,"481IC-幸存危机-2756爆量爆R-3.5_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,2643.53,27,97.9085185,1,2643.530000,384.00,0.1452603,0.0000,0.03704,1,0,null,null,"BID_PER_ACTION"],[20540,507233,"RICH-财神保佑-幸存者-每次付费_春节游戏DMP_竖SSR2285_0226_5","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001379,1,2,2381.33,76,31.3332895,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20541,513881,"RICH-研究所-招R体-通投_横SSR2185_0303_1","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,2,228.82,1,228.8200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,517614,"RICH-幸存者-我也爆R-每次付费_安卓游戏_竖SSR2402_0305_3","2021-03-16 00:00:00","2021-03-05 00:00:00",10,1001379,1,2,0.77,0,0.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,519399,"hix-每次付费-ssr-3.6_6","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,2,382.33,0,382.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,520177,"ka-自动roi-周天-2.7_6","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,2,166.26,1,166.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,520180,"yo-自动roi-周天-2.7_3","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,2,9.68,0,9.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[21245,520854,"dm每次付费-xcztz-0308_15","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,2,476.90,1,476.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,520869,"0308-508幸存危机-每次付费_2332_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,2,6.90,0,6.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,520870,"0308-508幸存危机-每次付费_2724_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,2,0.34,0,0.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,522170,"7tt_n_每次付费_幸存危机_0309_7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,2,53.18,0,53.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,524145,"293bd-末日海马云测试_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,2,2568.77,23,111.6856522,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524146,"293bd-末日海马云测试_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,2,2590.05,0,2590.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524147,"293bd-末日海马云测试_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,2,2696.61,1,2696.6100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524148,"293bd-末日海马云测试_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,2,2467.83,0,2467.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524149,"293bd-末日海马云测试_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,2,2546.98,0,2546.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,524582,"自动ROI-排3K包-0310_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,8.20,0,8.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,525012,"BY每次付费-xcztz-拉环卡车-0310-_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,3.98,0,3.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525011,"BY每次付费-xcztz-拉环卡车-0310-_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,20.68,0,20.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525019,"DM每次付费-xcztz-拉环卡车-0310-_18","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,113.81,0,113.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20647,525149,"cxq-tt894N1计划-0310_春节包通投2743","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,2,27.89,0,27.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,525568,"7tt_529_幸存者挑战_自动ROI_3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,0.20,0,0.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525570,"7tt_529_幸存者挑战_自动ROI_3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,0.20,0,0.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525569,"7tt_529_幸存者挑战_自动ROI_3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525576,"7tt_529_幸存者挑战_自动ROI_3.10_9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,2,0.19,0,0.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,526311,"7tt_n_幸存危机_每次付费__0311_52245341","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,4.77,0,4.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526327,"7tt_n_幸存危机_每次付费__0311_52245354","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,32.15,0,32.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526328,"7tt_n_幸存危机_每次付费__0311_52245355","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,138.22,0,138.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526329,"7tt_n_幸存危机_每次付费__0311_52245356","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,37.05,0,37.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526330,"7tt_n_幸存危机_每次付费__0311_52245357","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,54.31,0,54.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526331,"7tt_n_幸存危机_每次付费__0311_52245358","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,11.13,0,11.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526334,"7tt_n_幸存危机_每次付费__0311_522453611","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,48.01,0,48.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526333,"7tt_n_幸存危机_每次付费__0311_522453610","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,22.32,0,22.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526337,"7tt_n_幸存危机_每次付费__0311_522453714","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,187.19,0,187.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,526395,"jk自动ROI-排3K-xcztz-0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.68,0,0.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,526394,"jk自动ROI-排3K-xcztz-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.15,0,0.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,526400,"jk自动ROI-排3K-xcztz-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.11,0,0.1100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526528,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2363_0311_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2229.43,14,159.2450000,0,0.000000,0.00,0E-7,0.0000,0.00000,1,2,null,null,"BID_PER_ACTION"],[20540,526532,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2484_0311_27","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,19.61,0,19.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526533,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2484_0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2.75,0,2.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526530,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2723_0311_26","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,134.63,0,134.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,526577,"DM自动ROI-xcztz-排3K-0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.69,0,0.6900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,526579,"DM自动ROI-xcztz-排3K-0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,2.57,0,2.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,526581,"DM自动ROI-xcztz-排3K-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,4.93,0,4.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,526582,"DM自动ROI-xcztz-排3K-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,1.45,0,1.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20647,526687,"cxq-tt894N1计划-0311_3小2755","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,2,491.09,3,163.6966667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6867,526753,"0311-tve-每次付费-幸_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,727.91,11,66.1736364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526755,"0311-tve-每次付费-幸_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,15.95,0,15.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526759,"0311-tve-每次付费-幸_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,108.06,1,108.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526760,"0311-tve-每次付费-幸_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,41.52,0,41.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526761,"0311-tve-每次付费-幸_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,78.32,0,78.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526762,"0311-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,11.57,0,11.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526764,"0311-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,59.86,0,59.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526765,"0311-tve-每次付费-幸_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,10.53,0,10.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526766,"0311-tve-每次付费-幸_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,460.09,0,460.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526767,"0311-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,145.39,0,145.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526860,"RICH-财神到我家-幸存者-自动roi-_通投_横SSR2736_0311_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,1.35,0,1.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526861,"RICH-财神到我家-幸存者-自动roi-_春节DMP_横SSR1909_0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526864,"RICH-财神到我家-幸存者-自动roi-_游戏_横SSR2736_0311_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.60,0,0.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526862,"RICH-财神到我家-幸存者-自动roi-_春节DMP_横SSR2736_0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,4.78,0,4.7800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526865,"RICH-财神到我家-幸存者-自动roi-_游戏_横SSR1909_0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.44,0,0.4400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526863,"RICH-财神到我家-幸存者-自动roi-_通投_横SSR1909_0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.08,0,0.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,526885,"0311幸存者393z每次付费3k_2724_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,2,0.21,0,0.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,526912,"0311-508幸存危机-每次付费竖版沉浸_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,13.37,0,13.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,526915,"0311-508幸存危机-每次付费竖版沉浸_2697_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,0.37,0,0.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,526914,"0311-508幸存危机-每次付费竖版沉浸_2695_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,2,0.33,0,0.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527315,"295-blcb-每次付费爆R爆量_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,68.26,1,68.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527342,"470-tv-每次付费爆R爆量_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,5.70,0,5.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527344,"470-tv-每次付费爆R爆量_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,84.40,0,84.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527345,"470-tv-每次付费爆R爆量_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,40.99,0,40.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527343,"470-tv-每次付费爆R爆量_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,17.73,0,17.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527347,"470-tv-每次付费爆R爆量_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,80.90,0,80.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527470,"0312540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,13.82,0,13.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527471,"0312540-每次付费-幸_15","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,105.71,0,105.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527477,"0312540-每次付费-幸_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,45.42,0,45.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527513,"每0312-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,287.17,0,287.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,527525,"每0312-tt-xcztz-wyt-rk_15","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,4.91,0,4.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527527,"每0312-tt-xcztz-wyt-rk_19","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,17.27,0,17.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527528,"每0312-tt-xcztz-wyt-rk_20","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,1.62,0,1.6200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527529,"每0312-tt-xcztz-wyt-rk_21","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2.88,0,2.8800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527530,"每0312-tt-xcztz-wyt-rk_22","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,22.62,0,22.6200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,527685,"cxq-tt549幸存者-每次付费-0312_排负+男优2736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,294.63,0,294.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,527686,"cxq-tt549幸存者-每次付费-0312_排负+男优2765","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,0.96,0,0.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527695,"RICH-财神到我家_幸存者-每次付费_排3000_78397_0312_95325553","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,40.70,0,40.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527697,"RICH-财神到我家_幸存者-每次付费_排3000_78431_0312_95325566","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,9.18,0,9.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527794,"RICH-财神到我家_幸存者-每次付费_排3000_75017_0312_95325552","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,22.76,0,22.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527859,"RICH-财神到我家_幸存者-自动ROI_排3000_78449_0312_01169240","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,0.04,0,0.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527863,"RICH-财神到我家_幸存者-自动ROI_排3000_67560_0312_01169274","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,187.58,0,187.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527865,"RICH-财神到我家_幸存者-自动ROI_排3000_63730_0312_01169262","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,0.54,0,0.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527866,"RICH-财神到我家_幸存者-自动ROI_排3000_77952_0312_01169251","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,4.12,0,4.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527867,"RICH-财神到我家_幸存者-自动ROI_排3000_78397_0312_01169275","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,0.13,0,0.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527868,"RICH-财神到我家_幸存者-自动ROI_排3000_73141_0312_01169273","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,0.37,0,0.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21208,527869,"481IC-幸存危机-648大集合-3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,451.29,0,451.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527965,"7tt_n_幸存危机_每次付费__0312_16295360","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,66.50,0,66.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527971,"7tt_n_幸存危机_每次付费__0312_16295387","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,66.12,0,66.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527972,"7tt_n_幸存危机_每次付费__0312_16295388","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,87.49,0,87.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527978,"7tt_n_幸存危机_每次付费__0312_162953914","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,109.40,0,109.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,528001,"RICH-财神到我家_幸存者-每次付费_排3000_69832_0312_22237942","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,217.94,1,217.9400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528062,"7tt_幸_自动ROI__0312_88256793","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,10.21,0,10.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528071,"7tt_幸_自动ROI__0312_88256829","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6.14,0,6.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528074,"7tt_幸_自动ROI__0312_882568314","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3.43,1,3.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[6770,528075,"7tt_幸_自动ROI__0312_882568415","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2.70,0,2.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528085,"7tt_幸_自动ROI__0312_882568926","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528086,"7tt_幸_自动ROI__0312_882569027","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3.51,0,3.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528088,"7tt_幸_自动ROI__0312_882569129","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,31.56,0,31.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[20754,528136,"hix-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,47.22,1,47.2200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528550,"0312-508幸存危机-每次付费竖版沉浸_2777_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,2,30.41,0,30.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528657,"990-幸存者-MCFC-0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,11.29,0,11.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528655,"990-幸存者-MCFC-0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,11.87,0,11.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528656,"990-幸存者-MCFC-0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,39.58,0,39.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528658,"990-幸存者-MCFC-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2.81,0,2.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528784,"7tt_穿穿_每次付费__0312_02259772","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,25.32,0,25.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528785,"7tt_穿穿_每次付费__0312_02259798","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,123.28,2,61.6400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528777,"7tt_穿穿_每次付费__0312_02259799","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,79.27,1,79.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528786,"7tt_穿穿_每次付费__0312_022598012","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3050.35,3,1016.7833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528789,"7tt_穿穿_每次付费__0312_022598015","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,6.07,0,6.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528792,"7tt_穿穿_每次付费__0312_022598118","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,12.04,0,12.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528794,"7tt_穿穿_每次付费__0312_022598120","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,0.60,0,0.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528795,"7tt_穿穿_每次付费__0312_022598121","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2977.27,2,1488.6350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528798,"7tt_穿穿_每次付费__0312_022598224","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,2.95,0,2.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528801,"7tt_穿穿_每次付费__0312_022598227","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,3.44,1,3.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528803,"7tt_穿穿_每次付费__0312_022598329","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,79.76,0,79.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20143,528804,"0312XM-MGTTN1计划_3_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,18.18,0,18.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528805,"0312XM-MGTTN1计划_3_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,176.06,0,176.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528809,"0312XM-MGTTN1计划_3_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,54.96,0,54.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528810,"0312XM-MGTTN1计划_3_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,49.28,0,49.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20545,528840,"0312全民534自动roi穿山甲_2332_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,15.81,0,15.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21246,528852,"付费-穿山甲-n1-499-0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,0.42,0,0.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528856,"付费-穿山甲-n1-499-0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,0.19,0,0.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528858,"付费-穿山甲-n1-499-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,2,20.27,0,20.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,528866,"7tt_498_穿哥_每次付费_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,52.94,0,52.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528867,"7tt_498_穿哥_每次付费_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,19.61,0,19.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528895,"05-幸存者-MCFC0312_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,120.35,0,120.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528897,"05-幸存者-MCFC0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,58.07,0,58.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528899,"05-幸存者-MCFC0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,75.50,0,75.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528927,"263-幸存者-MCFC0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,132.51,0,132.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,528964,"263-幸存者-MCFC0312_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,2,49.35,0,49.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,529020,"CJ-财神到-ROI系03.13-0","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,2,0.05,0,0.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[20540,529037,"RICH-财神到我家_幸存者-每次付费_排3000_73012_0313_08286071","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3.92,0,3.9200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,529112,"r-自动ROI-幸存者挑战带穿山甲-03.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,0.28,0,0.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,529170,"7tt_n_幸存危机_每次付费__0313_84190331","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,2,53.26,0,53.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529171,"7tt_穿穿_每次付费__0313_84191313","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,275.00,2,137.5000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529169,"7tt_穿穿_每次付费__0313_84191302","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,5.99,0,5.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529178,"7tt_n_幸存危机_每次付费__0313_84190348","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,2,47.72,0,47.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529185,"7tt_n_幸存危机_每次付费__0313_841903514","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,2,188.12,0,188.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529186,"7tt_穿穿_每次付费__0313_84191290","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3611.70,14,257.9785714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529189,"7tt_穿穿_每次付费__0313_84191315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,13.26,0,13.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529190,"7tt_穿穿_每次付费__0313_84191316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,16.19,2,8.0950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529191,"7tt_穿穿_每次付费__0313_84191317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3329.56,1,3329.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529193,"7tt_穿穿_每次付费__0313_84191329","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,2187.08,1,2187.0800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529194,"7tt_穿穿_每次付费__0313_841913210","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,1.74,0,1.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529196,"7tt_穿穿_每次付费__0313_841913212","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,42.86,1,42.8600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529197,"7tt_穿穿_每次付费__0313_841913213","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,3778.58,0,3778.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529199,"7tt_穿穿_每次付费__0313_841913214","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,27.45,1,27.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529200,"7tt_穿穿_每次付费__0313_841913316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,10.26,0,10.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529201,"7tt_穿穿_每次付费__0313_841913317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,1734.89,0,1734.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529202,"7tt_穿穿_每次付费__0313_841913318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,18.40,0,18.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529203,"7tt_穿穿_每次付费__0313_841913319","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,2.33,0,2.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,529218,"RICH-末日-穿山甲-付费_游戏_混SSR2778_0313_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,2,7.99,0,7.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529219,"RICH-末日-穿山甲-付费_游戏_混SSR2762_0313_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,2,2.59,0,2.5900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,529228,"cxq-tt552幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,538.72,19,28.3536842,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529227,"cxq-tt338幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,22.62,0,22.6200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529224,"cxq-tt338幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,59.86,0,59.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529232,"cxq-tt552幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,0.83,0,0.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529231,"cxq-tt338幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,2,7.14,0,7.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,529368,"CJ-危机-每次03.14_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,7.65,0,7.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,529369,"VR-危机-每次03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,2,67.25,2,33.6250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529377,"JK-自动ROI排3K周日发力-0313_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,101.30,0,101.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21245,529412,"每次-周末冲刺-0314_18","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,33.57,0,33.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21245,529414,"每次-周末冲刺-0314_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,170.04,0,170.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21245,529416,"每次-周末冲刺-0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,205.53,0,205.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21246,529417,"穿-n1-周末付费冲-0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,2,62.60,0,62.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529418,"穿-n1-周末付费冲-0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,2,48.83,0,48.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529420,"穿-n1-周末付费冲-0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,2,49.93,0,49.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[6770,529462,"7tt_穿穿_每次付费__0314_72230811","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,27.15,0,27.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529461,"7tt_穿穿_每次付费__0314_72230800","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,60.08,0,60.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529464,"7tt_穿穿_每次付费__0314_72230827","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3382.48,10,338.2480000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529466,"7tt_穿穿_每次付费__0314_72230829","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2541.11,3,847.0366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529463,"7tt_穿穿_每次付费__0314_72230825","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,32.16,1,32.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529465,"7tt_穿穿_每次付费__0314_72230826","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,41.00,0,41.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529468,"7tt_穿穿_每次付费__0314_72230813","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1275.59,2,637.7950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529469,"7tt_穿穿_每次付费__0314_72230812","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,164.99,2,82.4950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529471,"7tt_穿穿_每次付费__0314_72230828","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,13.34,0,13.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529472,"7tt_穿穿_每次付费__0314_722308210","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529474,"每0314-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,33.83,0,33.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529482,"每0314-tt-xcztz-wyt-rk新_24","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,244.84,0,244.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529475,"每0314-tt-xcztz-wyt-rk新_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,41.80,0,41.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529483,"每0314-tt-xcztz-wyt-rk新_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,296.09,0,296.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529481,"每0314-tt-xcztz-wyt-rk新_25","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1.34,0,1.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529484,"每0314-tt-xcztz-wyt-rk新_30","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,12.79,0,12.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529485,"每0314-tt-xcztz-wyt-rk新_31","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,0.32,0,0.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529486,"每0314-tt-xcztz-wyt-rk新_36","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,321.38,0,321.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529496,"每0314-tt-xcztz-wyt-rk新_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,166.99,0,166.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529502,"每0314-tt-xcztz-wyt-rk新_23","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1.51,0,1.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529506,"每0314-tt-xcztz-wyt-rk新_29","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3.64,0,3.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529507,"每0314-tt-xcztz-wyt-rk新_32","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,280.60,0,280.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529510,"每0314-tt-xcztz-wyt-rk新_35","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3.50,0,3.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529556,"r-每次付费-幸存者挑战-穿山甲专场-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,7.95,0,7.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529558,"r-每次付费-幸存者挑战-穿山甲专场-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,9.04,0,9.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529559,"r-每次付费-幸存者挑战-穿山甲专场-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,15.14,0,15.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529560,"r-每次付费-幸存者挑战-穿山甲专场-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4.65,0,4.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529588,"jap-每次付费--3.13_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,84.21,0,84.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529606,"tw-每次付费--3.13_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,50.42,0,50.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529605,"hix-每次付费--3.13_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,5.80,0,5.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529608,"jap-每次付费--3.13_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4.31,0,4.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529609,"zhan-每次付费--3.13_20","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3.99,0,3.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529645,"0314-540-每次付费-幸_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3042.01,142,21.4226056,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529648,"0314-540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,601.80,2,300.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529643,"0314-540-每次付费-幸_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3374.57,60,56.2428333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529644,"0314-540-每次付费-幸_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3434.77,33,104.0839394,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529647,"0314-540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,4294.94,81,53.0239506,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529650,"0314-540-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,1410.25,12,117.5208333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529649,"0314-540-每次付费-幸_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3594.66,158,22.7510127,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529651,"0314-540-每次付费-幸_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,2926.56,53,55.2181132,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529653,"0314-540-每次付费-幸_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,3451.77,59,58.5045763,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529654,"0314-540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,945.63,0,945.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529657,"0314-540-每次付费-幸_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,641.13,2,320.5650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529667,"0314-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,25.86,0,25.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529673,"0314-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,2,247.68,1,247.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529719,"470-tv-每次付费拉环爆R爆量_3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,13.85,1,13.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529715,"470-tv-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,345.64,14,24.6885714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529716,"470-tv-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,851.44,16,53.2150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529718,"470-tv-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,688.95,25,27.5580000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529720,"470-tv-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,118.75,5,23.7500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529722,"470-tv-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,100.57,3,33.5233333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529721,"470-tv-每次付费拉环爆R爆量_3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2803.74,31,90.4432258,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529723,"470-tv-每次付费拉环爆R爆量_3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,137.01,5,27.4020000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529724,"470-tv-每次付费拉环爆R爆量_3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,38.90,0,38.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529725,"470-tv-每次付费拉环爆R爆量_3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,589.15,0,589.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529747,"295-blcb-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,114.76,3,38.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529749,"295-blcb-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,176.98,8,22.1225000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529748,"295-blcb-每次付费拉环爆R爆量_3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2803.74,18,155.7633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529754,"295-blcb-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,133.70,2,66.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529753,"295-blcb-每次付费拉环爆R爆量_3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1149.38,4,287.3450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529755,"295-blcb-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,213.23,8,26.6537500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529752,"295-blcb-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,8.18,1,8.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529847,"263-幸存者-MCFC0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2619.13,33,79.3675758,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529845,"263-幸存者-MCFC0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,422.61,8,52.8262500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529863,"263-幸存者-MCFC0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,171.92,1,171.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529864,"263-幸存者-MCFC0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,415.87,0,415.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529865,"263-幸存者-MCFC0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1586.14,27,58.7459259,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529910,"0315-204-穿山甲-每次付费-幸_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,17.04,0,17.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529909,"0315-204-穿山甲-每次付费-幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,63.17,0,63.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529918,"481IC-幸存危机-2756爆量爆R-3.5_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,287.28,0,287.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529914,"0315-204-穿山甲-每次付费-幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,95.97,4,23.9925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529913,"0315-204-穿山甲-每次付费-幸_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2991.81,59,50.7086441,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529916,"481IC-幸存危机-2756爆量爆R-3.5_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,5607.48,81,69.2281481,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529917,"0315-204-穿山甲-每次付费-幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,89.67,7,12.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529920,"0315-204-穿山甲-每次付费-幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,806.49,32,25.2028125,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529922,"0315-204-穿山甲-每次付费-幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,14.31,0,14.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529923,"0315-204-穿山甲-每次付费-幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,8.55,0,8.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529924,"0315-204-穿山甲-每次付费-幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,193.22,4,48.3050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529925,"0315-204-穿山甲-每次付费-幸_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,42.37,0,42.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529926,"0315-204-穿山甲-每次付费-幸_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,24.58,0,24.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529927,"0315-204-穿山甲-每次付费-幸_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,5.17,0,5.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529975,"990-幸存者-MCFC-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,9.48,0,9.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529977,"990-幸存者-MCFC-0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,288.92,5,57.7840000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529978,"990-幸存者-MCFC-0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.29,0,1.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529981,"990-幸存者-MCFC-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,5.06,0,5.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529979,"990-幸存者-MCFC-0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.67,0,0.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529980,"990-幸存者-MCFC-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3.95,0,3.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529976,"990-幸存者-MCFC-0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,792.00,15,52.8000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529983,"990-幸存者-MCFC-0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,616.76,26,23.7215385,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529982,"990-幸存者-MCFC-0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,546.02,6,91.0033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530038,"CJ-幸存危机-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,28.17,0,28.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530041,"CJ-幸存危机-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,3303.74,60,55.0623333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530039,"CJ-幸存危机-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,614.50,11,55.8636364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530050,"CJ-幸存危机-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,16.87,0,16.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530102,"每0315-tt-xcztz-wyt-rk_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,18.10,0,18.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530101,"每0315-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,15.23,0,15.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530104,"每0315-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3.78,0,3.7800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530109,"每0315-tt-xcztz-wyt-rk_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,5.76,0,5.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530100,"每0315-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.08,0,0.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530111,"每0315-tt-xcztz-wyt-rk_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,7.22,0,7.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530112,"每0315-tt-xcztz-wyt-rk_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,13.48,0,13.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530113,"每0315-tt-xcztz-wyt-rk_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,49.71,0,49.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530114,"每0315-tt-xcztz-wyt-rk_21","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,100.41,0,100.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530115,"每0315-tt-xcztz-wyt-rk_22","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.05,0,1.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530117,"每0315-tt-xcztz-wyt-rk_24","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,83.54,0,83.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530122,"每0315-tt-xcztz-wyt-rk_31","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,6.31,0,6.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530125,"每0315-tt-xcztz-wyt-rk_34","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,54.63,0,54.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530126,"每0315-tt-xcztz-wyt-rk_37","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,8.21,0,8.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530127,"每0315-tt-xcztz-wyt-rk_38","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.24,0,0.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530128,"每0315-tt-xcztz-wyt-rk_39","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.36,0,1.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530129,"每0315-tt-xcztz-wyt-rk_40","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,11.37,0,11.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530130,"每0315-tt-xcztz-wyt-rk_41","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,92.33,0,92.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530131,"每0315-tt-xcztz-wyt-rk_42","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.49,0,1.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530132,"每0315-tt-xcztz-wyt-rk_45","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2.00,0,2.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530133,"每0315-tt-xcztz-wyt-rk_46","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,469.30,4,117.3250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530134,"每0315-tt-xcztz-wyt-rk_47","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,13.72,0,13.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530135,"每0315-tt-xcztz-wyt-rk_48","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,302.18,1,302.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530139,"每0315-tt-xcztz-wyt-rk_52","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.23,0,0.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530140,"每0315-tt-xcztz-wyt-rk_53","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.49,0,0.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530141,"每0315-tt-xcztz-wyt-rk_54","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.41,0,0.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530145,"每0315-tt-xcztz-wyt-rk_58","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,21.41,0,21.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530146,"每0315-tt-xcztz-wyt-rk_59","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530147,"每0315-tt-xcztz-wyt-rk_60","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530148,"每0315-tt-xcztz-wyt-rk_62","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.22,0,0.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530149,"每0315-tt-xcztz-wyt-rk_61","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3.46,0,3.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530150,"每0315-tt-xcztz-wyt-rk_64","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,9.99,0,9.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530151,"每0315-tt-xcztz-wyt-rk_63","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,29.08,0,29.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530155,"每0315-tt-xcztz-wyt-rk_70","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,7.02,0,7.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530154,"每0315-tt-xcztz-wyt-rk_65","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,408.44,1,408.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530153,"每0315-tt-xcztz-wyt-rk_67","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,951.02,11,86.4563636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530157,"每0315-tt-xcztz-wyt-rk_66","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,265.58,3,88.5266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530156,"每0315-tt-xcztz-wyt-rk_69","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,8.07,0,8.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530158,"每0315-tt-xcztz-wyt-rk_71","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530159,"每0315-tt-xcztz-wyt-rk_72","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,275.30,2,137.6500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530168,"r3-每次付费-幸存者挑战优选位-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2336.53,65,35.9466154,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530173,"r3-每次付费-幸存者挑战优选位-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1658.99,14,118.4992857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530175,"i-每次付费-幸存者挑战只限男优先位-03.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,783.36,20,39.1680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530174,"r3-每次付费-幸存者挑战优选位-03.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1913.11,24,79.7129167,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530176,"i-每次付费-幸存者挑战只限男优先位-03.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1174.93,9,130.5477778,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530167,"r3-每次付费-幸存者挑战优选位-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,485.07,3,161.6900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530177,"i-每次付费-幸存者挑战只限男优先位-03.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,309.76,3,103.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530210,"i-每次付费-幸存者挑战只限男优先位-03.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,430.16,1,430.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530211,"i-每次付费-幸存者挑战只限男优先位-03.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,577.04,4,144.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530385,"ROI系数-幸存者挑战-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3107.77,112,27.7479464,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530386,"ROI系数-幸存者挑战-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3070.08,51,60.1976471,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530387,"ROI系数-幸存者挑战-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3015.56,96,31.4120833,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530388,"ROI系数-幸存者挑战-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1111.52,34,32.6917647,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530389,"ROI系数-幸存者挑战-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2983.30,81,36.8308642,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530514,"0315幸存者393z每次付费3k_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1844.31,1,1844.3100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530516,"0315幸存者393z每次付费3k_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1863.95,5,372.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530515,"0315幸存者393z每次付费3k_2731_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2252.57,2,1126.2850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530517,"0315幸存者393z每次付费3k_2746_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3237.84,38,85.2063158,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530518,"0315幸存者393z每次付费3k_2735_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1199.35,0,1199.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530524,"0315oyo393-roi系数-排除3k_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2885.28,52,55.4861538,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530531,"0315oyo393-roi系数-排除3k_2731_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1394.48,4,348.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530529,"0315oyo393-roi系数-排除3k_2522_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1046.06,12,87.1716667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530530,"0315oyo393-roi系数-排除3k_2762_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1331.56,12,110.9633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20635,530537,"每0315-tt-xcztz-wyt-yan_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,173.89,4,43.4725000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530540,"每0315-tt-xcztz-wyt-yan_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,314.88,5,62.9760000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530541,"每0315-tt-xcztz-wyt-yan_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,294.41,3,98.1366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530542,"每0315-tt-xcztz-wyt-yan_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,95.29,1,95.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530543,"每0315-tt-xcztz-wyt-yan_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,302.67,3,100.8900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530544,"每0315-tt-xcztz-wyt-yan_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,496.49,10,49.6490000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530545,"每0315-tt-xcztz-wyt-yan_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2486.77,40,62.1692500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530546,"每0315-tt-xcztz-wyt-yan_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.28,0,1.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530547,"每0315-tt-xcztz-wyt-yan_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.94,0,0.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530548,"每0315-tt-xcztz-wyt-yan_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,4.45,0,4.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530549,"每0315-tt-xcztz-wyt-yan_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.09,0,0.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530550,"每0315-tt-xcztz-wyt-yan_19","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2.71,0,2.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530551,"每0315-tt-xcztz-wyt-yan_20","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2.11,1,2.1100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530552,"每0315-tt-xcztz-wyt-yan_21","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,4.98,0,4.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,530671,"cxq-tt747幸存者-ROI系数0315_2738","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1968.98,62,31.7577419,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530672,"cxq-tt549幸存者-ROI系数0315_2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2855.54,85,33.5945882,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530673,"cxq-tt747幸存者-ROI系数0315_2332","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,739.36,4,184.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530674,"cxq-tt982幸存者-ROI系数0315_2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,150.86,2,75.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530675,"cxq-tt530幸存者-ROI系数0315_录2789","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,314.35,0,314.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530676,"cxq-tt530幸存者-ROI系数0315_录2793","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,261.52,0,261.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530678,"cxq-tt530幸存者-ROI系数0315_录2764","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,260.82,0,260.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530677,"cxq-tt549幸存者-ROI系数0315_2731","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,361.83,5,72.3660000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530679,"cxq-tt982幸存者-ROI系数0315_2746","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,337.08,4,84.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530680,"cxq-tt549幸存者-ROI系数0315_2781","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,437.21,4,109.3025000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530693,"0315-508幸存危机-每次付费竖版沉浸_2762_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,135.38,1,135.3800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530692,"0315-508幸存危机-每次付费竖版沉浸_2332_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,23.62,0,23.6200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530698,"0315-508幸存危机-每次付费竖版沉浸_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,681.82,7,97.4028571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530696,"0315-508幸存危机-每次付费竖版沉浸_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,161.82,1,161.8200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530721,"303-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,318.02,1,318.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530722,"306-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,61.94,3,20.6466667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530718,"0315-幸存者393z每次付费排除3k_2784_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,315.11,4,78.7775000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530713,"0315-幸存者393z每次付费排除3k_2783_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1101.04,4,275.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530720,"306-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,559.71,7,79.9585714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530716,"0315-幸存者393z每次付费排除3k_2786_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,49.52,0,49.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530724,"301-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,342.01,3,114.0033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530725,"301-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,316.00,3,105.3333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530723,"303-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1383.69,36,38.4358333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530726,"301-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,6.18,0,6.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530728,"306-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,95.06,1,95.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530729,"306-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,13.65,0,13.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530731,"306-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,8.09,0,8.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530732,"303-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2.37,0,2.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530733,"306-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,27.41,0,27.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530734,"303-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,303.88,3,101.2933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530736,"306-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2153.20,55,39.1490909,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530735,"303-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.21,0,1.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530730,"306-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,5.67,0,5.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530737,"306-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,367.57,8,45.9462500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530738,"303-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.83,0,1.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530739,"303-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,29.18,2,14.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530740,"303-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.41,0,1.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530741,"301-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,44.92,2,22.4600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530742,"301-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,438.56,6,73.0933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530743,"301-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,722.79,20,36.1395000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530744,"301-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,304.04,1,304.0400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530745,"301-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,4.43,0,4.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530746,"301-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2.79,0,2.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530747,"301-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,79.66,1,79.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530759,"303-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.02,0,1.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530779,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2332_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,648.74,19,34.1442105,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530781,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2731_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,12.56,0,12.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530780,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,34.53,0,34.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530782,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2751_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2.03,0,2.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530783,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2794_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,119.91,0,119.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530784,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2271_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,361.69,15,24.1126667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530786,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2484_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,234.18,17,13.7752941,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530785,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1692.29,57,29.6892982,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530787,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,155.94,0,155.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530788,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,4.05,0,4.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530799,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2622315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,956.08,8,119.5100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20540,530802,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2789315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,99.25,0,99.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530803,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2761315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,75.60,0,75.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530800,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2746315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,697.88,6,116.3133333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,530806,"0315FP-ROI系数-MG幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3269.14,10,326.9140000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530805,"0315FP-ROI系数-MG幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1537.46,20,76.8730000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530807,"0315FP-ROI系数-MG幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2333.97,7,333.4242857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530810,"0315FP-ROI系数-MG幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1061.06,2,530.5300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530809,"0315FP-ROI系数-MG幸_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1255.06,1,1255.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530844,"0315FP-ROI系数-MG幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1171.66,4,292.9150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530843,"0315FP-ROI系数-MG幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1840.60,6,306.7666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530846,"0315FP-ROI系数-MG幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2177.36,20,108.8680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530845,"0315FP-ROI系数-MG幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1762.77,5,352.5540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530908,"tw-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,970.04,5,194.0080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530913,"tw-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,107.09,0,107.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530914,"hix-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1105.87,8,138.2337500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530915,"zhan-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,389.75,0,389.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530912,"jap-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,31.46,0,31.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530910,"hix-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,418.61,2,209.3050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530911,"hix-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,388.89,2,194.4450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530907,"tw-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2713.20,32,84.7875000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530916,"zhan-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,230.98,0,230.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530917,"zhan-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,518.88,4,129.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530909,"jap-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2588.93,30,86.2976667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530920,"jap-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.39,0,1.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530918,"hix-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,8.43,0,8.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530919,"jap-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,285.03,1,285.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530921,"zhan-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,701.53,6,116.9216667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530925,"hix-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,29.89,0,29.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530926,"hix-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,38.19,0,38.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530923,"tw-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,156.00,1,156.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530922,"jap-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,13.37,0,13.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530927,"zhan-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,285.16,4,71.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530928,"zhan-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,995.17,7,142.1671429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530929,"tw-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,54.24,1,54.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530931,"FJ-幸存者-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,753.31,7,107.6157143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530933,"FW-幸存者-每次03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,26.64,0,26.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530930,"FJ-幸存者-每次03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,258.76,3,86.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530932,"FW-幸存者-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,719.29,5,143.8580000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530934,"FW-幸存者-每次03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,528.73,6,88.1216667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530935,"FW-幸存者-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,956.90,8,119.6125000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530937,"FJ-幸存者-每次03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,42.93,0,42.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530940,"FJ-幸存者-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,35.09,0,35.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530939,"FJ-幸存者-每次03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,5.68,0,5.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530938,"FW-幸存者-每次03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,374.85,1,374.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530943,"FW-幸存者-每次03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1606.62,15,107.1080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530948,"FJ-幸存者-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,164.54,0,164.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530947,"FJ-幸存者-每次03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,602.25,5,120.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530946,"FW-幸存者-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,413.23,0,413.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530949,"FJ-幸存者-每次03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,647.84,4,161.9600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530944,"FJ-幸存者-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,400.58,0,400.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530945,"FW-幸存者-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1025.53,11,93.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530942,"FW-幸存者-每次03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3303.74,39,84.7112821,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530941,"FW-幸存者-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,509.93,5,101.9860000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530964,"yo-roi系数-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2803.74,76,36.8913158,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530963,"ka-roi系数-周一-3.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1648.62,24,68.6925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530958,"yo-roi系数-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,110.73,0,110.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20754,530960,"yo-roi系数-周一-3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2803.74,40,70.0935000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530965,"yo-roi系数-周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,736.83,17,43.3429412,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530967,"yo-roi系数-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,365.61,6,60.9350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530969,"yo-roi系数-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,834.29,14,59.5921429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530973,"ka-roi系数-周一-3.15_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,630.42,10,63.0420000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530970,"ka-roi系数-周一-3.15_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,791.80,17,46.5764706,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530972,"ka-roi系数-周一-3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,407.12,2,203.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530971,"yo-roi系数-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,440.39,1,440.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20635,530999,"每0315-tt-xcztz-wyt-zhong_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,4.12,0,4.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530998,"每0315-tt-xcztz-wyt-zhong_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1302.96,14,93.0685714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531000,"每0315-tt-xcztz-wyt-zhong_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531001,"每0315-tt-xcztz-wyt-zhong_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,620.65,8,77.5812500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531002,"每0315-tt-xcztz-wyt-zhong_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3.62,1,3.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531006,"每0315-tt-xcztz-wyt-zhong_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,71.09,1,71.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531004,"每0315-tt-xcztz-wyt-zhong_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,672.76,13,51.7507692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531003,"每0315-tt-xcztz-wyt-zhong_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,921.25,3,307.0833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531005,"每0315-tt-xcztz-wyt-zhong_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,0.19,0,0.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531007,"每0315-tt-xcztz-wyt-zhong_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,22.40,0,22.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531008,"每0315-tt-xcztz-wyt-zhong_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,37.46,0,37.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531009,"每0315-tt-xcztz-wyt-zhong_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,3.99,0,3.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531010,"每0315-tt-xcztz-wyt-zhong_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1.51,0,1.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531011,"每0315-tt-xcztz-wyt-zhong_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,11.20,0,11.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21213,531012,"n96-ios-黎明血战-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,39.96,1,39.9600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531013,"n96-ios-黎明血战-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,63.84,2,31.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531014,"n96-ios-黎明血战-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,1659.45,33,50.2863636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531015,"n96-ios-黎明血战-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,53.39,1,53.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531016,"n96-ios-黎明血战-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,1534.32,11,139.4836364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531038,"bdna-非海马云-幸存者挑战-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2701.47,91,29.6864835,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531041,"bdna-非海马云-幸存者挑战-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2345.34,50,46.9068000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531187,"au-黎明血战--周一-3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,679.51,11,61.7736364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531189,"th-黎明血战--周一-3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,179.24,0,179.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531190,"au-黎明血战--周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,75.27,0,75.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531194,"th-黎明血战--周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,2508.92,43,58.3469767,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531188,"roi系数-0315-BY-幸存_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,305.27,10,30.5270000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20541,531195,"RICH-研究所-财神到我家-通投_混SSR2791_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,2183.63,7,311.9471429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531197,"RICH-研究所-财神到我家-通投_混SSR2797_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,88.80,0,88.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531199,"RICH-研究所-财神到我家-通投_混SSR2749_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,1143.12,1,1143.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531200,"RICH-研究所-财神到我家-通投_混SSR2789_0315_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,813.08,0,813.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531201,"RICH-研究所-财神到我家-通投_混SSR2783_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,1568.03,2,784.0150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531203,"05-幸存者-MCFC0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,433.57,7,61.9385714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531202,"05-幸存者-MCFC0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2803.74,51,54.9752941,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531205,"05-幸存者-MCFC0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2803.74,39,71.8907692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20541,531208,"RICH-研究所-财神到我家-通投_创意组8_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,1653.34,1,1653.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531211,"th-黎明血战--周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,49.77,0,49.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531212,"th-黎明血战--周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,390.00,1,390.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531207,"roi系数-0315-BY-幸存_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,559.87,8,69.9837500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531209,"roi系数-0315-BY-幸存_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,267.01,2,133.5050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20755,531215,"th-黎明血战--周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,386.82,2,193.4100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531214,"roi系数-0315-BY-幸存_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,508.28,2,254.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531213,"roi系数-0315-BY-幸存_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,50.73,0,50.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[21245,531210,"roi系数-0315-BY-幸存_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,784.04,5,156.8080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531206,"roi系数-0315-BY-幸存_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,529.12,1,529.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531216,"05-幸存者-MCFC0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,8.14,0,8.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,531217,"05-幸存者-MCFC0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,36.43,0,36.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20755,531227,"th-黎明血战--周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,106.53,2,53.2650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531229,"au-黎明血战--周一-3.15_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,17.58,0,17.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531230,"th-黎明血战--周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,637.41,3,212.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531231,"au-黎明血战--周一-3.15_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,418.74,0,418.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531232,"au-黎明血战--周一-3.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,476.02,1,476.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531233,"au-黎明血战--周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,69.94,0,69.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531234,"roi系数-0315-BY-幸存_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,304.39,4,76.0975000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20755,531252,"au-黎明血战--周一-3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,48.58,0,48.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531257,"th-黎明血战--周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,415.40,2,207.7000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531259,"au-黎明血战--周一-3.15_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,2,159.31,0,159.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531256,"05-幸存者-MCFC0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1073.08,20,53.6540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531258,"05-幸存者-MCFC0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,926.95,18,51.4972222,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531254,"05-幸存者-MCFC0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2.74,1,2.7400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531290,"0315幸存者393z每次付费排除3K_2797_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1202.22,3,400.7400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531289,"0315幸存者393z每次付费排除3K_2742_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1781.97,7,254.5671429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531291,"0315幸存者393z每次付费排除3K_2791_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1612.04,15,107.4693333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531292,"0315幸存者393z每次付费排除3K_2798_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1320.65,14,94.3321429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531287,"0315幸存者393z每次付费排除3K_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,5530.46,74,74.7359459,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531293,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1155.31,10,115.5310000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531294,"cxq-tt338幸存者-穿山甲每次付费-0315_30002797","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,93.94,0,93.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531298,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,395.33,7,56.4757143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531297,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2731","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,180.30,0,180.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531296,"cxq-tt338幸存者-穿山甲每次付费-0315_30002798","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,98.57,1,98.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531295,"cxq-tt338幸存者-穿山甲每次付费-0315_30002796","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1798.86,18,99.9366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20546,531300,"0315wtd末日研究所_2332_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,3440.76,28,122.8842857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531303,"0315wtd末日研究所_2730_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,890.32,1,890.3200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531302,"0315wtd末日研究所_2762_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,1949.39,11,177.2172727,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531304,"0315wtd末日研究所_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,2917.04,14,208.3600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531306,"263-幸存者-ROI-0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,763.58,22,34.7081818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531314,"263-幸存者-ROI-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2728.54,80,34.1067500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531307,"263-幸存者-ROI-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2910.18,58,50.1755172,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531315,"263-幸存者-ROI-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,236.45,2,118.2250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20546,531576,"0315ph648末日研究所_2800_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,236.37,1,236.3700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531578,"0315ph648末日研究所_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,7286.80,81,89.9604938,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531579,"0315ph648末日研究所_2783_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,93.76,0,93.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531580,"0315ph648末日研究所_2519_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,2,29.25,0,29.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531583,"263-幸存者-MCFC0315_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,330.96,0,330.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,531582,"263-幸存者-MCFC0315_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,470.23,2,235.1150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531585,"263-幸存者-MCFC0315_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1102.03,5,220.4060000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531581,"263-幸存者-MCFC0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,68.49,1,68.4900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531584,"263-幸存者-MCFC0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,2672.80,33,80.9939394,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531706,"0315-508幸存危机-每次付费_2800_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,2,742.06,1,742.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,531771,"r-自动ROI-幸存者挑战晚上七点-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,16.65,0,16.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21212,531772,"r-自动ROI-幸存者挑战晚上七点-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,4590.59,90,51.0065556,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531773,"r-自动ROI-幸存者挑战晚上七点-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,1305.40,6,217.5666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531774,"r-自动ROI-幸存者挑战晚上七点-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,6.50,0,6.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21212,531775,"r-自动ROI-幸存者挑战晚上七点-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,2,644.52,3,214.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[20540,529214,"RICH-末日-穿山甲-付费_通投_混SSR2778_0313_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,3,386.41,9,42.9344444,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529215,"RICH-末日-穿山甲-付费_春节DMP_混SSR2762_0313_7","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,3,3431.44,80,42.8930000,0,0.000000,0.00,0E-7,0.0946,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529216,"RICH-末日-穿山甲-付费_春节DMP_混SSR2746_0313_9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,3,6253.95,138,45.3184783,3,2084.650000,280.00,0.0447717,0.0860,0.02174,3,2,null,null,"DEEP_BID_DEFAULT"],[20540,529217,"RICH-末日-穿山甲-付费_通投_混SSR2762_0313_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,3,2767.96,68,40.7052941,0,0.000000,0.00,0E-7,0.1020,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529220,"RICH-末日-穿山甲-付费_通投_混SSR2746_0313_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,3,6139.13,139,44.1664029,1,6139.130000,30.00,0.0048867,0.0606,0.00719,1,2,null,null,"DEEP_BID_DEFAULT"],[20540,529221,"RICH-末日-穿山甲-付费_春节DMP_混SSR2778_0313_8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,3,767.90,15,51.1933333,0,0.000000,0.00,0E-7,0.1818,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529222,"RICH-末日-穿山甲-付费_游戏_混SSR2746_0313_6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,3,51.60,1,51.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,525568,"7tt_529_幸存者挑战_自动ROI_3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,148.76,6,24.7933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525569,"7tt_529_幸存者挑战_自动ROI_3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,89.14,2,44.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525570,"7tt_529_幸存者挑战_自动ROI_3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,98.19,3,32.7300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525571,"7tt_529_幸存者挑战_自动ROI_3.10_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,27.72,1,27.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525572,"7tt_529_幸存者挑战_自动ROI_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,9007.00,353,25.5155807,5,1801.400000,400.00,0.0444099,0.0685,0.01416,4,2,null,null,"ROI_PACING"],[6770,525576,"7tt_529_幸存者挑战_自动ROI_3.10_9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,86.87,1,86.8700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525577,"7tt_498_幸存者挑战_自动ROI_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,10715.92,343,31.2417493,6,1785.986667,632.00,0.0589777,0.0901,0.01749,5,2,null,null,"ROI_PACING"],[6770,527399,"7tt_529_幸存者挑战_每次付费3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,417.34,5,83.4680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528066,"7tt_幸_自动ROI__0312_88256792","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,601.46,11,54.6781818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528068,"7tt_幸_自动ROI__0312_882568211","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,51.96,2,25.9800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528077,"7tt_幸_自动ROI__0312_882568417","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,50.26,1,50.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528079,"7tt_幸_自动ROI__0312_882568520","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,118.99,3,39.6633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528774,"7tt_穿穿_每次付费__0312_02259785","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,47.75,3,15.9166667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528778,"7tt_穿穿_每次付费__0312_02259786","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3781.08,15,252.0720000,0,0.000000,0.00,0E-7,0.2857,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528780,"7tt_穿穿_每次付费__0312_02259760","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,2972.61,6,495.4350000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528783,"7tt_穿穿_每次付费__0312_022597911","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,552.58,4,138.1450000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528785,"7tt_穿穿_每次付费__0312_02259798","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,152.63,3,50.8766667,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528788,"7tt_穿穿_每次付费__0312_022598014","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,2393.39,6,398.8983333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528791,"7tt_穿穿_每次付费__0312_022598017","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,6802.92,244,27.8808197,6,1133.820000,370.00,0.0543884,0.0839,0.02459,3,2,null,null,"BID_PER_ACTION"],[6770,528795,"7tt_穿穿_每次付费__0312_022598121","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3101.13,5,620.2260000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528797,"7tt_穿穿_每次付费__0312_022598223","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3192.50,6,532.0833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528800,"7tt_穿穿_每次付费__0312_022598226","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3013.47,3,1004.4900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528859,"7tt_498_穿哥_每次付费_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1811.93,46,39.3897826,0,0.000000,0.00,0E-7,0.1600,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528860,"7tt_498_穿哥_每次付费_3.12_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,12.13,0,12.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528861,"7tt_498_穿哥_每次付费_3.12_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,383.43,13,29.4946154,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528862,"7tt_498_穿哥_每次付费_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,6385.11,168,38.0066071,2,3192.555000,322.00,0.0504298,0.1489,0.01190,2,2,null,null,"BID_PER_ACTION"],[6770,528863,"7tt_498_穿哥_每次付费_3.12_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,204.06,11,18.5509091,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528864,"7tt_498_穿哥_每次付费_3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,301.58,9,33.5088889,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528865,"7tt_498_穿哥_每次付费_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,25.32,2,12.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528868,"7tt_498_穿哥_每次付费_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,314.99,12,26.2491667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529171,"7tt_穿穿_每次付费__0313_84191313","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,277.75,3,92.5833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529187,"7tt_穿穿_每次付费__0313_84191301","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2452.32,1,2452.3200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529192,"7tt_穿穿_每次付费__0313_84191318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,4.85,1,4.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529198,"7tt_穿穿_每次付费__0313_841913315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2718.67,8,339.8337500,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529201,"7tt_穿穿_每次付费__0313_841913317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1739.88,1,1739.8800000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529202,"7tt_穿穿_每次付费__0313_841913318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,23.21,1,23.2100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529467,"7tt_穿穿_每次付费__0314_72230824","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,30.03,1,30.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529470,"7tt_穿穿_每次付费__0314_722308311","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1064.14,1,1064.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,526752,"0311-tve-每次付费-幸_20","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,10288.33,304,33.8431908,4,2572.082500,280.00,0.0272153,0.0984,0.01316,4,2,null,null,"BID_PER_ACTION"],[6867,526754,"0311-tve-每次付费-幸_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,250.60,3,83.5333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526756,"0311-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,3413.94,65,52.5221538,0,0.000000,0.00,0E-7,0.0612,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526757,"0311-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,1092.57,18,60.6983333,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526758,"0311-tve-每次付费-幸_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,765.49,11,69.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526760,"0311-tve-每次付费-幸_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,49.49,1,49.4900000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526763,"0311-tve-每次付费-幸_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,1097.62,6,182.9366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526766,"0311-tve-每次付费-幸_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,645.62,1,645.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526767,"0311-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,246.11,1,246.1100000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526769,"0311-tve-每次付费-幸_18","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,2983.64,64,46.6193750,0,0.000000,0.00,0E-7,0.1094,0.00000,1,2,null,null,"BID_PER_ACTION"],[6867,526770,"0311-tve-每次付费-幸_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,2328.57,52,44.7801923,1,2328.570000,98.00,0.0420859,0.1563,0.01923,1,2,null,null,"BID_PER_ACTION"],[6867,527472,"0312540-每次付费-幸_24","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1369.91,21,65.2338095,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527473,"0312540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,933.39,25,37.3356000,1,933.390000,188.00,0.2014163,0.0476,0.04000,1,2,null,null,"BID_PER_ACTION"],[6867,527475,"0312540-每次付费-幸_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3267.13,30,108.9043333,0,0.000000,0.00,0E-7,0.0645,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6867,527478,"0312540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1217.07,9,135.2300000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527479,"0312540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1213.18,10,121.3180000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527481,"0312540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,708.51,4,177.1275000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527483,"0312540-每次付费-幸_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,11329.37,153,74.0481699,3,3776.456667,4810.00,0.4245602,0.0458,0.01961,3,2,null,null,"BID_PER_ACTION"],[6867,527484,"0312540-每次付费-幸_11","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,159.55,2,79.7750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527485,"0312540-每次付费-幸_12","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,309.29,10,30.9290000,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527487,"0312540-每次付费-幸_14","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,524.29,3,174.7633333,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527488,"0312540-每次付费-幸_16","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,646.57,2,323.2850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527489,"0312540-每次付费-幸_17","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,662.43,4,165.6075000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527492,"0312540-每次付费-幸_20","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1121.26,13,86.2507692,1,1121.260000,30.00,0.0267556,0.0769,0.07692,1,2,null,null,"BID_PER_ACTION"],[6867,527495,"0312540-每次付费-幸_23","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,376.75,2,188.3750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527496,"0312540-每次付费-幸_25","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1051.41,4,262.8525000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527497,"0312540-每次付费-幸_26","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,270.53,8,33.8162500,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527498,"0312540-每次付费-幸_27","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,687.76,1,687.7600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,529659,"0314-tve-每次付费-幸_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4522.57,47,96.2248936,2,2261.285000,42.00,0.0092868,0.1667,0.04255,2,0,null,null,"BID_PER_ACTION"],[6867,529660,"0314-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2054.95,43,47.7895349,0,0.000000,0.00,0E-7,0.0909,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529661,"0314-tve-每次付费-幸_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1641.12,17,96.5364706,0,0.000000,0.00,0E-7,0.0588,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529662,"0314-tve-每次付费-幸_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1691.61,17,99.5064706,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529663,"0314-tve-每次付费-幸_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1702.45,17,100.1441176,2,851.225000,212.00,0.1245264,0.3529,0.11765,1,0,null,null,"BID_PER_ACTION"],[6867,529664,"0314-tve-每次付费-幸_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,5517.03,70,78.8147143,1,5517.030000,30.00,0.0054377,0.1111,0.01429,0,0,null,null,"BID_PER_ACTION"],[6867,529665,"0314-tve-每次付费-幸_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,832.97,8,104.1212500,0,0.000000,0.00,0E-7,0.1250,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529666,"0314-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,187.94,4,46.9850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529668,"0314-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1035.91,13,79.6853846,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529669,"0314-tve-每次付费-幸_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,7948.23,95,83.6655789,1,7948.230000,102.00,0.0128330,0.0702,0.01053,1,0,null,null,"BID_PER_ACTION"],[6867,529670,"0314-tve-每次付费-幸_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2632.16,27,97.4874074,0,0.000000,0.00,0E-7,0.0370,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529671,"0314-tve-每次付费-幸_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2616.41,32,81.7628125,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529672,"0314-tve-每次付费-幸_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,5607.30,118,47.5194915,2,2803.650000,24.00,0.0042801,0.0429,0.01695,0,0,null,null,"BID_PER_ACTION"],[6867,529674,"0314-tve-每次付费-幸_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2762.90,24,115.1208333,1,2762.900000,60.00,0.0217163,0.0833,0.04167,1,0,null,null,"BID_PER_ACTION"],[20451,499074,"306-幸存者-ROI02.21_5","2021-03-16 00:00:00","2021-02-21 00:00:00",10,1001379,1,3,3360.27,157,21.4029936,1,3360.270000,12.00,0.0035711,0.0000,0.00637,1,2,null,null,"ROI_PACING"],[20451,522259,"303搜-幸存者-ROI系03.09-0","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,3,6816.54,86,79.2620930,3,2272.180000,122.00,0.0178976,0.1957,0.03488,3,2,null,null,"ROI_COEFFICIENT"],[20451,526868,"306-幸存者-ROI系03.11-2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,10398.55,360,28.8848611,10,1039.855000,6318.00,0.6075847,0.0661,0.02778,10,2,null,null,"ROI_COEFFICIENT"],[20451,526878,"301-幸存者-ROI系03.11-4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,18926.19,1010,18.7388020,10,1892.619000,980.00,0.0517801,0.0693,0.00990,10,2,null,null,"ROI_COEFFICIENT"],[20451,527348,"EZ-财神到-每次03.12-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,914.28,8,114.2850000,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527356,"EZ-财神到-每次03.12-9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,809.51,30,26.9836667,0,0.000000,0.00,0E-7,0.0303,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527358,"EZ-财神到-每次03.12-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1391.16,11,126.4690909,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527359,"EZ-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3295.25,50,65.9050000,1,3295.250000,72.00,0.0218496,0.0192,0.02000,1,2,null,null,"BID_PER_ACTION"],[20451,527360,"EZ-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,102.31,2,51.1550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527362,"EZ-财神到-每次03.12-8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,656.11,11,59.6463636,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527365,"SR-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,16491.03,272,60.6287868,5,3298.206000,586.00,0.0355345,0.1053,0.01838,5,2,null,null,"BID_PER_ACTION"],[20451,527366,"SR-财神到-每次03.12-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,13384.31,198,67.5975253,6,2230.718333,1328.00,0.0992207,0.0606,0.03030,6,2,null,null,"BID_PER_ACTION"],[20451,527367,"SR-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,14923.17,157,95.0520382,6,2487.195000,12076.00,0.8092114,0.1618,0.03822,6,2,null,null,"BID_PER_ACTION"],[20451,527371,"PB-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,6591.80,58,113.6517241,0,0.000000,0.00,0E-7,0.0294,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527374,"PB-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3659.19,48,76.2331250,1,3659.190000,340.00,0.0929167,0.0882,0.02083,1,2,null,null,"BID_PER_ACTION"],[20451,527376,"PB-财神到-每次03.12-9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,23580.82,464,50.8207328,6,3930.136667,9782.00,0.4148287,0.0732,0.01293,6,2,null,null,"BID_PER_ACTION"],[20451,527377,"PB-财神到-每次03.12-0","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,24818.79,666,37.2654505,8,3102.348750,750.00,0.0302190,0.0661,0.01201,8,2,null,null,"BID_PER_ACTION"],[20451,528505,"FJ-财神到-每次03.12-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,6745.07,79,85.3806329,3,2248.356667,218.00,0.0323199,0.1013,0.03797,3,2,null,null,"BID_PER_ACTION"],[20451,528506,"FJ-财神到-每次03.12-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,0.01,7,0.0014286,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528507,"FW-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,450.46,2,225.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528508,"FJ-财神到-每次03.12-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,6759.90,81,83.4555556,1,6759.900000,98.00,0.0144973,0.0741,0.01235,1,2,null,null,"BID_PER_ACTION"],[20451,528509,"FW-财神到-每次03.12-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3294.95,54,61.0175926,1,3294.950000,60.00,0.0182097,0.0351,0.01852,1,2,null,null,"BID_PER_ACTION"],[20451,528510,"FW-财神到-每次03.12-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,466.29,3,155.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528511,"FW-财神到-每次03.12-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3303.74,55,60.0680000,0,0.000000,0.00,0E-7,0.0545,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528912,"FW-财神到-每次03.13-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,623.15,6,103.8583333,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528913,"FJ-财神到-每次03.13-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,26540.03,398,66.6834925,10,2654.003000,1828.00,0.0688771,0.1057,0.02513,9,2,null,null,"BID_PER_ACTION"],[20451,528914,"FJ-财神到-每次03.13-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,10005.61,161,62.1466460,4,2501.402500,182.00,0.0181898,0.0962,0.02484,3,2,null,null,"BID_PER_ACTION"],[20451,528915,"FW-财神到-每次03.13-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3303.74,57,57.9603509,0,0.000000,0.00,0E-7,0.0333,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528917,"FW-财神到-每次03.13-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,4622.29,77,60.0297403,2,2311.145000,3838.00,0.8303244,0.0909,0.02597,1,2,null,null,"BID_PER_ACTION"],[20451,528918,"FJ-财神到-每次03.13-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,7231.92,92,78.6078261,3,2410.640000,120.00,0.0165931,0.1667,0.03261,4,2,null,null,"BID_PER_ACTION"],[20451,528920,"FJ-财神到-每次03.13-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,473.87,1,473.8700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528921,"FW-财神到-每次03.13-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,727.99,6,121.3316667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528923,"FW-财神到-每次03.13-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,794.58,8,99.3225000,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,452869,"HSX-幸存者-自动ROI-通投-头抖_混SSR1909_0128_20","2021-03-16 00:00:00","2021-01-28 00:00:00",10,1001379,1,3,2601.22,119,21.8589916,0,0.000000,0.00,0E-7,0.0781,0.00000,0,2,null,null,"ROI_PACING"],[20540,489063,"RICH-幸存者_春节-沉浸付费_安卓通投_69686_0215_07317475","2021-03-16 00:00:00","2021-02-15 00:00:00",10,1001379,1,3,6493.52,139,46.7159712,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,521086,"RICH-财神到我家-幸存者-每次付费_竖SSR2728_XTSC1614654964-5_0308_15","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,3,3552.63,45,78.9473333,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526528,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2363_0311_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,3941.07,28,140.7525000,1,3941.070000,42.00,0.0106570,0.0714,0.03571,1,2,null,null,"BID_PER_ACTION"],[20540,526529,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2271_0311_25","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,242.68,9,26.9644444,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526531,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2332_0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,4758.85,81,58.7512346,0,0.000000,0.00,0E-7,0.0864,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526533,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2484_0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,18.39,1,18.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526535,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2735_0311_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,218.78,1,218.7800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526536,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2622_0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,3729.78,58,64.3065517,0,0.000000,0.00,0E-7,0.0517,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526865,"RICH-财神到我家-幸存者-自动roi-_游戏_横SSR1909_0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,0.48,1,0.4800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20540,527690,"RICH-财神到我家_幸存者-每次付费_排3000_71833_0312_95325541","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,2217.35,28,79.1910714,0,0.000000,0.00,0E-7,0.0370,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,527691,"RICH-财神到我家_幸存者-每次付费_排3000_73011_0312_95325530","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,7634.56,69,110.6457971,4,1908.640000,2464.00,0.3227429,0.1667,0.05797,4,2,null,null,"BID_PER_ACTION"],[20540,527692,"RICH-财神到我家_幸存者-每次付费_排3000_70989_0312_95325565","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3917.57,77,50.8775325,1,3917.570000,42.00,0.0107209,0.0390,0.01299,0,2,null,null,"BID_PER_ACTION"],[20540,527698,"RICH-财神到我家_幸存者-每次付费_排3000_76461_0312_95325564","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1210.11,7,172.8728571,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529036,"RICH-财神到我家_幸存者-每次付费_排3000_70019_0313_08286060","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3838.44,45,85.2986667,2,1919.220000,766.00,0.1995602,0.2188,0.04444,1,2,null,null,"BID_PER_ACTION"],[20540,529038,"RICH-财神到我家_幸存者-每次付费_排3000_77023_0313_08286082","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3535.06,33,107.1230303,2,1767.530000,2502.00,0.7077673,0.1053,0.06061,2,2,null,null,"BID_PER_ACTION"],[20540,529204,"RICH-财神到我家-幸存者-每次付费_春节DMP-混SSR2762-313_6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1072.18,9,119.1311111,0,0.000000,0.00,0E-7,0.2222,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529205,"RICH-财神到我家-幸存者-每次付费_游戏-混SSR2762-313_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2650.25,42,63.1011905,1,2650.250000,170.00,0.0641449,0.2000,0.02381,1,2,null,null,"BID_PER_ACTION"],[20540,529206,"RICH-财神到我家-幸存者-每次付费_游戏-混SSR2762-313_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1164.24,8,145.5300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529207,"RICH-财神到我家-幸存者-每次付费_通投-混SSR2762-313_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1345.35,10,134.5350000,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529208,"RICH-财神到我家-幸存者-每次付费_游戏-混SSR2762-313_7","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1220.10,11,110.9181818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529209,"RICH-财神到我家-幸存者-每次付费_通投-混SSR2762-313_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,754.22,4,188.5550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529210,"RICH-财神到我家-幸存者-每次付费_春节DMP-混SSR2762-313_9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2721.20,39,69.7743590,0,0.000000,0.00,0E-7,0.0476,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529211,"RICH-财神到我家-幸存者-每次付费_通投-混SSR2762-313_8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,18349.82,201,91.2926368,4,4587.455000,2774.00,0.1511731,0.0408,0.01990,3,2,null,null,"BID_PER_ACTION"],[20540,529212,"RICH-财神到我家-幸存者-每次付费_春节DMP-混SSR2762-313_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2706.90,36,75.1916667,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,523130,"0309oyo393自动roi莱卡_2738_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,3,15.33,5,3.0660000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,525269,"0310oyo393自动roi莱卡_2332_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,21.32,0,21.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,525274,"0310oyo393自动roi莱卡_2746_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,9.95,1,9.9500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,525303,"0310全民534自动roi基定排除2k_2745_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,90.10,5,18.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,525304,"0310全民534自动roi基定排除2k_2742_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,3113.23,57,54.6180702,2,1556.615000,238.00,0.0764479,0.1404,0.03509,2,2,null,null,"ROI_PACING"],[20545,525306,"0310全民534自动roi基定排除2k_2733_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,8.90,1,8.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,526883,"0311幸存者393z每次付费3k_2735_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,800.26,11,72.7509091,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526884,"0311幸存者393z每次付费3k_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,569.89,7,81.4128571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526887,"0311全民534自动roi排除3k_2731_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,1747.83,48,36.4131250,1,1747.830000,30.00,0.0171641,0.1458,0.02083,1,2,null,null,"ROI_PACING"],[20545,526888,"0311全民534自动roi排除3k_2754_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,2528.88,54,46.8311111,2,1264.440000,78.00,0.0308437,0.1569,0.03704,2,2,null,null,"ROI_PACING"],[20545,526889,"0311全民534自动roi排除3k_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,708.07,24,29.5029167,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"ROI_PACING"],[20545,526890,"0311全民534自动roi排除3k_2738_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,980.42,39,25.1389744,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"ROI_PACING"],[20545,527858,"0312oyo393自动roi莱卡_2332_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,4.20,3,1.4000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,527861,"0312oyo393自动roi莱卡_2742_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,5075.53,88,57.6764773,3,1691.843333,140.00,0.0275833,0.0964,0.03409,2,2,null,null,"ROI_PACING"],[20545,527862,"0312oyo393自动roi莱卡_2735_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3056.47,137,22.3100000,1,3056.470000,42.00,0.0137413,0.0657,0.00730,3,2,null,null,"ROI_PACING"],[20545,527895,"0312幸存者393z每次付费3k_2735_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,54.91,4,13.7275000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,527896,"0312幸存者393z每次付费3k_2731_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,124.39,3,41.4633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,527897,"0312幸存者393z每次付费3k_2746_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,44.32,2,22.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528104,"0312全民534自动roi排除3k_2762_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,53.27,16,3.3293750,0,0.000000,0.00,0E-7,0.0588,0.00000,0,2,null,null,"ROI_PACING"],[20545,528105,"0312全民534自动roi排除3k_2766_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3136.59,98,32.0060204,1,3136.590000,316.00,0.1007464,0.0714,0.01020,1,2,null,null,"ROI_PACING"],[20545,528106,"0312全民534自动roi排除3k_2764_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1377.72,19,72.5115789,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,528107,"0312全民534自动roi排除3k_2768_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,6939.89,201,34.5268159,3,2313.296667,54.00,0.0077811,0.0647,0.01493,3,2,null,null,"ROI_PACING"],[20545,528839,"0312全民534自动roi穿山甲_2622_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,2427.19,78,31.1178205,2,1213.595000,220.00,0.0906398,0.0897,0.02564,2,2,null,null,"BID_PER_ACTION"],[20545,528840,"0312全民534自动roi穿山甲_2332_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,56.43,2,28.2150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528841,"0312全民534自动roi穿山甲_2762_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,9392.70,308,30.4957792,5,1878.540000,668.00,0.0711191,0.0747,0.01623,4,2,null,null,"BID_PER_ACTION"],[20545,528844,"0312全民534自动roi穿山甲_2779_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3142.52,122,25.7583607,1,3142.520000,98.00,0.0311852,0.0400,0.00820,1,2,null,null,"BID_PER_ACTION"],[20545,528846,"0312全民534自动roi穿山甲_2778_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,763.01,15,50.8673333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,526435,"每0311-tt-xcztz-wyt-rk_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,300.75,2,150.3750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527500,"每0312-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,311.35,2,155.6750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,527501,"每0312-tt-xcztz-wyt-rk_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,768.90,3,256.3000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,527503,"每0312-tt-xcztz-wyt-rk_16","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,10.40,1,10.4000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527515,"每0312-tt-xcztz-wyt-rk_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,893.63,4,223.4075000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527517,"每0312-tt-xcztz-wyt-rk_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,83.48,1,83.4800000,1,83.480000,68.00,0.8145664,1.0000,1.00000,1,0,null,null,"BID_PER_ACTION"],[20635,527525,"每0312-tt-xcztz-wyt-rk_15","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,48.30,1,48.3000000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527526,"每0312-tt-xcztz-wyt-rk_17","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,368.62,5,73.7240000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527529,"每0312-tt-xcztz-wyt-rk_21","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,63.23,1,63.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527531,"每0312-tt-xcztz-wyt-rk_24","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,4852.29,45,107.8286667,2,2426.145000,2714.00,0.5593235,0.0667,0.04444,2,2,null,null,"BID_PER_ACTION"],[20635,527532,"每0312-tt-xcztz-wyt-rk_27","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,179.93,4,44.9825000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527533,"每0312-tt-xcztz-wyt-rk_28","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,144.97,3,48.3233333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,529473,"每0314-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,810.03,3,270.0100000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529480,"每0314-tt-xcztz-wyt-rk新_19","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,802.69,5,160.5380000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529494,"每0314-tt-xcztz-wyt-rk新_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,484.59,4,121.1475000,1,484.590000,266.00,0.5489176,0.2500,0.25000,1,0,null,null,"BID_PER_ACTION"],[20635,529495,"每0314-tt-xcztz-wyt-rk新_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,466.24,5,93.2480000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529499,"每0314-tt-xcztz-wyt-rk新_20","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,452.98,4,113.2450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529500,"每0314-tt-xcztz-wyt-rk新_21","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,25,112.1496000,1,2803.740000,30.00,0.0107000,0.1200,0.04000,0,0,null,null,"BID_PER_ACTION"],[20635,529503,"每0314-tt-xcztz-wyt-rk新_26","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,788.60,3,262.8666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529504,"每0314-tt-xcztz-wyt-rk新_27","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,253.04,5,50.6080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529508,"每0314-tt-xcztz-wyt-rk新_33","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,411.79,1,411.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,521886,"05-幸存者-MCFC0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,3,3923.36,40,98.0840000,0,0.000000,0.00,0E-7,0.0556,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,521889,"05-幸存者-MCFC0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,3,584.97,8,73.1212500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526408,"263-幸存者-ROI-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,8032.50,170,47.2500000,3,2677.500000,370.00,0.0460629,0.1277,0.01765,3,2,null,null,"ROI_COEFFICIENT"],[20637,526410,"263-幸存者-ROI-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,3120.78,85,36.7150588,2,1560.390000,336.00,0.1076654,0.0824,0.02353,2,2,null,null,"DEEP_BID_DEFAULT"],[20637,526845,"263-幸存者-MCFC0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,9399.49,234,40.1687607,8,1174.936250,10364.00,1.1026130,0.1129,0.03419,8,2,null,null,"BID_PER_ACTION"],[20637,526847,"263-幸存者-MCFC0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,12342.65,155,79.6300000,2,6171.325000,422.00,0.0341904,0.0645,0.01290,2,2,null,null,"BID_PER_ACTION"],[20637,526943,"05-幸存者-MCFC0311_7_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,36178.72,624,57.9787179,11,3288.974545,11940.00,0.3300283,0.0725,0.01763,10,2,null,null,"BID_PER_ACTION"],[20637,528659,"990-幸存者-MCFC-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,32.13,1,32.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528893,"05-幸存者-MCFC0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1253.07,11,113.9154545,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528894,"05-幸存者-MCFC0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,275.32,4,68.8300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528896,"05-幸存者-MCFC0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,841.65,3,280.5500000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528898,"05-幸存者-MCFC0312_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,333.49,5,66.6980000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528900,"05-幸存者-MCFC0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,255.57,1,255.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528901,"05-幸存者-MCFC0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,134.88,2,67.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528902,"05-幸存者-MCFC0312_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,87.80,1,87.8000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528924,"263-幸存者-MCFC0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,4678.27,49,95.4748980,0,0.000000,0.00,0E-7,0.0612,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528925,"263-幸存者-MCFC0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,794.33,7,113.4757143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528927,"263-幸存者-MCFC0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,182.01,2,91.0050000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,528928,"263-幸存者-MCFC0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,832.62,7,118.9457143,1,832.620000,60.00,0.0720617,0.1429,0.14286,1,2,null,null,"BID_PER_ACTION"],[20637,528929,"263-幸存者-MCFC0312_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1453.67,9,161.5188889,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528930,"263-幸存者-MCFC0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,2091.59,16,130.7243750,0,0.000000,0.00,0E-7,0.0625,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528963,"263-幸存者-MCFC0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3502.08,24,145.9200000,2,1751.040000,128.00,0.0365497,0.1333,0.08333,2,2,null,null,"BID_PER_ACTION"],[20637,529561,"05-幸存者-MCFC0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,7757.01,103,75.3107767,2,3878.505000,392.00,0.0505349,0.0606,0.01942,2,0,null,null,"BID_PER_ACTION"],[20637,529562,"05-幸存者-MCFC0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,26,107.8361538,0,0.000000,0.00,0E-7,0.0769,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529563,"05-幸存者-MCFC0314_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,17360.96,128,135.6325000,9,1928.995556,6320.00,0.3640352,0.0826,0.07031,9,0,null,null,"BID_PER_ACTION"],[20637,529564,"05-幸存者-MCFC0314_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1937.23,24,80.7179167,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529565,"05-幸存者-MCFC0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,334.05,2,167.0250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529566,"05-幸存者-MCFC0314_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,25,112.1496000,0,0.000000,0.00,0E-7,0.0400,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529567,"05-幸存者-MCFC0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.63,16,175.2268750,0,0.000000,0.00,0E-7,0.1250,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529568,"05-幸存者-MCFC0314_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,5514.36,42,131.2942857,1,5514.360000,98.00,0.0177718,0.1667,0.02381,1,0,null,null,"BID_PER_ACTION"],[20637,529569,"05-幸存者-MCFC0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3729.36,50,74.5872000,2,1864.680000,392.00,0.1051119,0.0698,0.04000,1,0,null,null,"BID_PER_ACTION"],[20637,529570,"05-幸存者-MCFC0314_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2606.03,22,118.4559091,0,0.000000,0.00,0E-7,0.0909,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529571,"05-幸存者-MCFC0314_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,7757.01,91,85.2418681,1,7757.010000,2170.00,0.2797470,0.0159,0.01099,1,0,null,null,"BID_PER_ACTION"],[20637,529572,"05-幸存者-MCFC0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,34,82.4629412,0,0.000000,0.00,0E-7,0.0882,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,526460,"cxq-tt幸存者-每次付费-排除3000-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,769.14,9,85.4600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526479,"cxq-tt幸存者-每次付费-排除3000-0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,4262.87,83,51.3598795,0,0.000000,0.00,0E-7,0.0602,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526484,"cxq-tt幸存者-每次付费-排除3000-0311_26","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,548.98,3,182.9933333,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526546,"cxq-tt幸存者-每次付费-排除负向0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,2960.21,61,48.5280328,1,2960.210000,60.00,0.0202688,0.0820,0.01639,1,2,null,null,"BID_PER_ACTION"],[20646,526551,"cxq-tt幸存者-每次付费-排除负向0311_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,332.40,4,83.1000000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526572,"cxq-tt幸存者-每次付费-排除负向0311_20","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,626.64,6,104.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527535,"cxq-tt338幸存者-每次付费穿山甲0312_排30002736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,344.93,5,68.9860000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527536,"cxq-tt338幸存者-每次付费穿山甲0312_排30002248","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,354.35,4,88.5875000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527537,"cxq-tt338幸存者-每次付费穿山甲0312_排负2746小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,314.78,9,34.9755556,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,527538,"cxq-tt552幸存者-每次付费穿山甲0312_排30002751小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3151.47,83,37.9695181,2,1575.735000,24.00,0.0076155,0.0941,0.02410,2,2,null,null,"BID_PER_ACTION"],[20646,527539,"cxq-tt552幸存者-每次付费穿山甲0312_排30002762小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,13952.56,349,39.9786819,5,2790.512000,834.00,0.0597740,0.0649,0.01433,5,2,null,null,"BID_PER_ACTION"],[20646,527540,"cxq-tt338幸存者-每次付费穿山甲0312_排负2762小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1063.57,25,42.5428000,0,0.000000,0.00,0E-7,0.0417,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527541,"cxq-tt552幸存者-每次付费穿山甲0312_排30002746小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3743.54,128,29.2464063,1,3743.540000,12.00,0.0032055,0.0469,0.00781,1,2,null,null,"BID_PER_ACTION"],[20646,527542,"cxq-tt552幸存者-每次付费穿山甲0312_排负2248","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,431.99,4,107.9975000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527543,"cxq-tt338幸存者-每次付费穿山甲0312_排负2751小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,758.61,11,68.9645455,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527546,"cxq-tt552幸存者-每次付费穿山甲0312_排负2765","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,130.08,4,32.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527679,"cxq-tt747幸存者-每次付费-0312_排负+通全选小2735","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,316.14,2,158.0700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527680,"cxq-tt549幸存者-每次付费-0312_排负+男优2729","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1016.92,11,92.4472727,1,1016.920000,98.00,0.0963694,0.0909,0.09091,1,2,null,null,"BID_PER_ACTION"],[20646,527683,"cxq-tt747幸存者-每次付费-0312_排负+通全选小2762","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,8219.11,125,65.7528800,4,2054.777500,138.00,0.0167901,0.1040,0.03200,4,2,null,null,"DEEP_BID_DEFAULT"],[20646,527685,"cxq-tt549幸存者-每次付费-0312_排负+男优2736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,432.20,1,432.2000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527686,"cxq-tt549幸存者-每次付费-0312_排负+男优2765","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,250.88,1,250.8800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,528876,"cxq-tt530幸存者-每次付费-优选-0312_小2746843-19","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,6913.68,112,61.7292857,4,1728.420000,8692.00,1.2572176,0.0536,0.03571,4,2,null,null,"DEEP_BID_DEFAULT"],[20646,529223,"cxq-tt338幸存者-每次付费-穿山甲0313_27622000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,9633.67,228,42.2529386,4,2408.417500,230.00,0.0238746,0.1327,0.01754,3,2,null,null,"BID_PER_ACTION"],[20646,529225,"cxq-tt338幸存者-每次付费-穿山甲0313_27462000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3813.67,113,33.7492920,2,1906.835000,132.00,0.0346123,0.1250,0.01770,2,2,null,null,"BID_PER_ACTION"],[20646,529226,"cxq-tt552幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,508.14,14,36.2957143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529229,"cxq-tt552幸存者-每次付费-穿山甲0313_27462000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,5148.18,107,48.1138318,2,2574.090000,132.00,0.0256401,0.0943,0.01869,2,2,null,null,"BID_PER_ACTION"],[20646,529230,"cxq-tt552幸存者-每次付费-穿山甲0313_27622000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3769.27,69,54.6271014,0,0.000000,0.00,0E-7,0.1014,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529233,"cxq-tt338幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1776.97,50,35.5394000,1,1776.970000,12.00,0.0067531,0.2000,0.02000,1,2,null,null,"BID_PER_ACTION"],[20646,529234,"cxq-tt552幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3609.21,95,37.9916842,1,3609.210000,12.00,0.0033248,0.0667,0.01053,0,2,null,null,"BID_PER_ACTION"],[20646,529252,"cxq-tt549幸存者-每次付费-0313_122-1SSR2285","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3197.10,219,14.5986301,1,3197.100000,212.00,0.0663101,0.0320,0.00457,1,2,null,null,"BID_PER_ACTION"],[20646,529253,"cxq-tt747幸存者-每次付费-0313_843-18SSR2762","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1398.83,11,127.1663636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529254,"cxq-tt982幸存者-每次付费-0313_包1SSR2285","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3330.96,80,41.6370000,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529255,"cxq-tt747幸存者-每次付费-0313_843-18SSR2285","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3275.23,71,46.1300000,0,0.000000,0.00,0E-7,0.0563,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529256,"cxq-tt982幸存者-每次付费-0313_包1SSR2762","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,521.93,1,521.9300000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529257,"cxq-tt747幸存者-每次付费-0313_843-18SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2225.61,11,202.3281818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529258,"cxq-tt982幸存者-每次付费-0313_包1SSR1414","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,831.45,4,207.8625000,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529259,"cxq-tt982幸存者-每次付费-0313_包1SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,4673.02,68,68.7208824,2,2336.510000,308.00,0.0659103,0.0652,0.02941,1,2,null,null,"BID_PER_ACTION"],[20646,529260,"cxq-tt747幸存者-每次付费-0313_843-18SSR1414","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,842.08,14,60.1485714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529261,"cxq-tt549幸存者-每次付费-0313_122-1SSR1414","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3539.65,100,35.3965000,2,1769.825000,24.00,0.0067803,0.0400,0.02000,2,2,null,null,"BID_PER_ACTION"],[20646,529262,"cxq-tt549幸存者-每次付费-0313_122-1SSR2762","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1150.25,13,88.4807692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529263,"cxq-tt549幸存者-每次付费-0313_122-1SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,5698.00,195,29.2205128,2,2849.000000,110.00,0.0193050,0.0667,0.01026,2,2,null,null,"BID_PER_ACTION"],[20754,519396,"hix-每次付费-ssr-3.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,5600.17,103,54.3705825,0,0.000000,0.00,0E-7,0.0485,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519397,"zhan-每次付费-ssr-3.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,6542.06,157,41.6691720,0,0.000000,0.00,0E-7,0.0701,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519401,"zhan-每次付费-ssr-3.6_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,11495.69,121,95.0057025,4,2873.922500,664.00,0.0577608,0.0992,0.03306,3,2,null,null,"BID_PER_ACTION"],[20754,519405,"hix-每次付费-ssr-3.6_8","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,1025.76,2,512.8800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519406,"hix-每次付费-ssr-3.6_2","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,803.51,4,200.8775000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519409,"zhan-每次付费-ssr-3.6_6","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,662.59,1,662.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519411,"zhan-每次付费-ssr-3.6_8","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,6542.06,76,86.0797368,1,6542.060000,98.00,0.0149800,0.0658,0.01316,1,2,null,null,"BID_PER_ACTION"],[20754,520091,"yo-roi系数roi-想个好名字-2.6_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,3500.18,44,79.5495455,2,1750.090000,72.00,0.0205704,0.1471,0.04545,2,2,null,null,"ROI_COEFFICIENT"],[20754,520097,"yo-roi系数roi-想个好名字-2.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,1456.11,32,45.5034375,0,0.000000,0.00,0E-7,0.1379,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,520100,"ka-roi系数-想个好名字-2.6_11","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,91.62,5,18.3240000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,520178,"ka-自动roi-周天-2.7_5","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,3,3159.76,150,21.0650667,1,3159.760000,12.00,0.0037978,0.0621,0.00667,1,2,null,null,"DEEP_BID_DEFAULT"],[20754,523280,"zhan-每次付费--3.9_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,3,6518.69,79,82.5150633,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,523295,"zhan-每次付费--3.9_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,3,4064.82,32,127.0256250,2,2032.410000,958.00,0.2356808,0.0833,0.06250,2,2,null,null,"BID_PER_ACTION"],[20754,524817,"zhan-每次付费--3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,8480.69,145,58.4875172,3,2826.896667,1560.00,0.1839473,0.1034,0.02069,3,2,null,null,"DEEP_BID_DEFAULT"],[20754,524825,"hix-每次付费--3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,1598.17,5,319.6340000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524831,"jap-每次付费--3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,3289.51,35,93.9860000,1,3289.510000,60.00,0.0182398,0.0571,0.02857,1,2,null,null,"BID_PER_ACTION"],[20754,524832,"jap-每次付费--3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,3006.38,13,231.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524834,"hix-每次付费--3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,2452.52,24,102.1883333,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524842,"jap-每次付费--3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,403.20,1,403.2000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,525044,"ka-roi系数-爆好-3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,2889.23,87,33.2095402,0,0.000000,0.00,0E-7,0.1047,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,527176,"hix-每次付费--3.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,5545.75,60,92.4291667,1,5545.750000,110.00,0.0198350,0.0962,0.01667,1,2,null,null,"BID_PER_ACTION"],[20754,527178,"tw-每次付费--3.11_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,4211.27,90,46.7918889,2,2105.635000,8688.00,2.0630356,0.0889,0.02222,2,2,null,null,"DEEP_BID_DEFAULT"],[20754,527182,"tw-每次付费--3.11_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,545.27,4,136.3175000,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527183,"zhan-每次付费--3.11_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,1399.77,27,51.8433333,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528134,"jap-每次付费--3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,139.54,1,139.5400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528135,"hix-每次付费--3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,419.14,5,83.8280000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528136,"hix-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,218.04,2,109.0200000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528138,"hix-每次付费--3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,237.09,2,118.5450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528141,"jap-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,359.26,4,89.8150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528145,"zhan-每次付费--3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,199.18,1,199.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528146,"zhan-每次付费--3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,93.90,2,46.9500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528147,"zhan-每次付费--3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,457.65,3,152.5500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528148,"zhan-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,4109.17,56,73.3780357,2,2054.585000,358.00,0.0871222,0.1042,0.03571,1,2,null,null,"BID_PER_ACTION"],[20754,528149,"zhan-每次付费--3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,40.54,1,40.5400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528153,"hix-每次付费--3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,6930.97,82,84.5240244,2,3465.485000,104.00,0.0150051,0.0610,0.02439,2,2,null,null,"BID_PER_ACTION"],[20754,528154,"hix-每次付费--3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,341.44,2,170.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528157,"tw-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,787.70,16,49.2312500,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528345,"yo-roi系数-周五-3.12_4_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3226.11,82,39.3428049,0,0.000000,0.00,0E-7,0.0682,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,528349,"ka-roi系数-周五-3.12_4_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3271.03,51,64.1378431,0,0.000000,0.00,0E-7,0.0182,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,528351,"yo-roi系数-周五-3.12_4_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3271.03,128,25.5549219,3,1090.343333,192.00,0.0586971,0.1111,0.02344,4,2,null,null,"DEEP_BID_DEFAULT"],[20754,528353,"ka-roi系数-周五-3.12_4_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,5032.30,127,39.6244094,2,2516.150000,54.00,0.0107307,0.0957,0.01575,2,2,null,null,"ROI_COEFFICIENT"],[20754,528354,"yo-roi系数-周五-3.12_4_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,6894.38,177,38.9512994,3,2298.126667,168.00,0.0243677,0.0452,0.01695,3,2,null,null,"DEEP_BID_DEFAULT"],[20754,528356,"ka-roi系数-周五-3.12_4_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,10737.37,168,63.9129167,6,1789.561667,3794.00,0.3533454,0.0915,0.03571,6,2,null,null,"ROI_COEFFICIENT"],[20754,529154,"ka-自动roi-周六-3.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,4298.62,131,32.8138931,2,2149.310000,60.00,0.0139580,0.0687,0.01527,2,2,null,null,"ROI_PACING"],[20754,529155,"ka-自动roi-周六-3.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2264.44,64,35.3818750,0,0.000000,0.00,0E-7,0.0323,0.00000,0,2,null,null,"ROI_PACING"],[20754,529156,"yo-roi系数-周六-3.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2803.74,54,51.9211111,1,2803.740000,12.00,0.0042800,0.0714,0.01852,0,2,null,null,"ROI_COEFFICIENT"],[20754,529157,"ka-自动roi-周六-3.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2043.72,53,38.5607547,0,0.000000,0.00,0E-7,0.0476,0.00000,0,2,null,null,"ROI_PACING"],[20754,529158,"ka-自动roi-周六-3.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,5631.22,167,33.7198802,3,1877.073333,236.00,0.0419092,0.0802,0.01796,3,2,null,null,"ROI_PACING"],[20754,529159,"yo-roi系数-周六-3.13_44","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,12865.06,349,36.8626361,6,2144.176667,1378.00,0.1071118,0.0575,0.01719,6,2,null,null,"ROI_COEFFICIENT"],[20754,529160,"yo-roi系数-周六-3.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2803.74,68,41.2314706,0,0.000000,0.00,0E-7,0.0137,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,529161,"yo-roi系数-周六-3.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,14223.76,289,49.2171626,7,2031.965714,504.00,0.0354337,0.0664,0.02422,7,2,null,null,"ROI_COEFFICIENT"],[20754,529162,"yo-roi系数-周六-3.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2803.74,47,59.6540426,0,0.000000,0.00,0E-7,0.0392,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,529587,"hix-每次付费--3.13_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,504.15,9,56.0166667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529589,"jap-每次付费--3.13_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,581.66,4,145.4150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529590,"hix-每次付费--3.13_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,36,77.8816667,0,0.000000,0.00,0E-7,0.0556,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529591,"jap-每次付费--3.13_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1627.32,32,50.8537500,0,0.000000,0.00,0E-7,0.0645,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529592,"jap-每次付费--3.13_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,37,75.7767568,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529593,"tw-每次付费--3.13_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1682.15,41,41.0280488,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529594,"hix-每次付费--3.13_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,456.30,2,228.1500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529595,"hix-每次付费--3.13_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,44.73,2,22.3650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529596,"tw-每次付费--3.13_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4857.47,177,27.4433333,1,4857.470000,494.00,0.1016990,0.0395,0.00565,1,0,null,null,"BID_PER_ACTION"],[20754,529597,"tw-每次付费--3.13_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,42,66.7557143,0,0.000000,0.00,0E-7,0.0476,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529598,"zhan-每次付费--3.13_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,92.43,3,30.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529599,"tw-每次付费--3.13_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4859.81,74,65.6731081,1,4859.810000,102.00,0.0209885,0.0270,0.01351,1,0,null,null,"BID_PER_ACTION"],[20754,529600,"zhan-每次付费--3.13_17","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4492.21,77,58.3403896,3,1497.403333,304.00,0.0676727,0.0857,0.03896,2,0,null,null,"BID_PER_ACTION"],[20754,529601,"zhan-每次付费--3.13_18","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,463.50,2,231.7500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529602,"zhan-每次付费--3.13_19","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,32.47,1,32.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,527304,"295-blcb-每次付费爆R爆量_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,2310.93,53,43.6024528,2,1155.465000,244.00,0.1055852,0.0345,0.03774,2,2,null,null,"BID_PER_ACTION"],[21208,527307,"295-blcb-每次付费爆R爆量_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,409.43,8,51.1787500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527310,"295-blcb-每次付费爆R爆量_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1750.59,14,125.0421429,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527311,"295-blcb-每次付费爆R爆量_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,698.77,11,63.5245455,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527314,"295-blcb-每次付费爆R爆量_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,78.03,3,26.0100000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527315,"295-blcb-每次付费爆R爆量_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,89.58,2,44.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527345,"470-tv-每次付费爆R爆量_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,133.56,1,133.5600000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527346,"470-tv-每次付费爆R爆量_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1483.03,8,185.3787500,0,0.000000,0.00,0E-7,0.2857,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,520588,"r-自动ROI-幸存者充值是大哥大-03.08_6","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,3,430.90,6,71.8166667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,520601,"r-自动ROI-幸存者充值是大哥大-03.08_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,3,309.12,24,12.8800000,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"ROI_PACING"],[21212,521273,"g-自动ROI-幸存者加批计划跑晚上-03.08_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,3,2486.10,82,30.3182927,0,0.000000,0.00,0E-7,0.1077,0.00000,0,2,null,null,"ROI_PACING"],[21212,522092,"r-自动ROI-幸存者yyds-03.09_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,3,3250.64,158,20.5736709,2,1625.320000,170.00,0.0522974,0.0870,0.01266,2,2,null,null,"ROI_PACING"],[21212,525391,"i-每次付费-幸存者挑战-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,607.88,5,121.5760000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526440,"r3-每次付费-幸存者挑战-03.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,5520.72,84,65.7228571,0,0.000000,0.00,0E-7,0.0952,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526444,"r3-每次付费-幸存者挑战-03.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,2763.86,30,92.1286667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526453,"r3-每次付费-幸存者挑战-03.11_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,5538.88,36,153.8577778,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,528035,"r-自动ROI-幸存者挑战-03.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,2.68,1,2.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,528056,"r-自动ROI-幸存者挑战-03.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,0.10,2,0.0500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,528058,"r-自动ROI-幸存者挑战-03.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,4.90,0,4.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21212,528558,"g-自动ROI-幸存者挑战-03.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3838.48,379,10.1279156,2,1919.240000,250.00,0.0651299,0.0431,0.00528,2,2,null,null,"ROI_PACING"],[21212,528559,"g-自动ROI-幸存者挑战-03.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,77.23,1,77.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,528560,"g-自动ROI-幸存者挑战-03.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,338.60,6,56.4333333,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"ROI_PACING"],[21212,528561,"g-自动ROI-幸存者挑战-03.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,2276.48,116,19.6248276,0,0.000000,0.00,0E-7,0.0583,0.00000,0,2,null,null,"ROI_PACING"],[21212,528562,"g-自动ROI-幸存者挑战-03.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,5018.34,129,38.9018605,2,2509.170000,806.00,0.1606109,0.0609,0.01550,2,2,null,null,"ROI_PACING"],[21212,529098,"r3-每次付费-幸存者挑战周六-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,4906.48,40,122.6620000,2,2453.240000,340.00,0.0692961,0.1579,0.05000,2,2,null,null,"BID_PER_ACTION"],[21212,529099,"r3-每次付费-幸存者挑战周六-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,5316.36,39,136.3169231,3,1772.120000,2726.00,0.5127568,0.1538,0.07692,2,2,null,null,"BID_PER_ACTION"],[21212,529100,"r3-每次付费-幸存者挑战周六-03.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2336.45,48,48.6760417,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529102,"r3-每次付费-幸存者挑战周六-03.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,274.95,1,274.9500000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529103,"r3-每次付费-幸存者挑战周六-03.13_6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2336.45,30,77.8816667,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529109,"r-自动ROI-幸存者挑战带穿山甲-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1713.43,81,21.1534568,1,1713.430000,12.00,0.0070035,0.0370,0.01235,1,2,null,null,"ROI_PACING"],[21212,529110,"r-自动ROI-幸存者挑战带穿山甲-03.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,50.34,1,50.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,529111,"r-自动ROI-幸存者挑战带穿山甲-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,768.57,35,21.9591429,0,0.000000,0.00,0E-7,0.1143,0.00000,0,2,null,null,"ROI_PACING"],[21212,529112,"r-自动ROI-幸存者挑战带穿山甲-03.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,172.43,4,43.1075000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,529113,"r-自动ROI-幸存者挑战带穿山甲-03.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,753.28,38,19.8231579,0,0.000000,0.00,0E-7,0.0526,0.00000,0,2,null,null,"ROI_PACING"],[21212,529538,"i-每次付费-幸存者挑战周日-03.14_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1325.42,10,132.5420000,0,0.000000,0.00,0E-7,0.3000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529539,"i-每次付费-幸存者挑战周日-03.14_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,42,66.7557143,0,0.000000,0.00,0E-7,0.0238,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529540,"r3-每次付费-幸存者挑战周日-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4871.34,46,105.8986957,0,0.000000,0.00,0E-7,0.0526,0.00000,1,0,null,null,"BID_PER_ACTION"],[21212,529541,"r3-每次付费-幸存者挑战周日-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3518.33,29,121.3217241,3,1172.776667,1008.00,0.2864996,0.3333,0.10345,3,0,null,null,"BID_PER_ACTION"],[21212,529542,"i-每次付费-幸存者挑战周日-03.14_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,544.15,5,108.8300000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529543,"r3-每次付费-幸存者挑战周日-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,52,53.9180769,0,0.000000,0.00,0E-7,0.0577,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529544,"i-每次付费-幸存者挑战周日-03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,984.42,11,89.4927273,1,984.420000,434.00,0.4408687,0.1667,0.09091,1,0,null,null,"BID_PER_ACTION"],[21212,529545,"r3-每次付费-幸存者挑战周日-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,6139.32,105,58.4697143,2,3069.660000,422.00,0.0687373,0.0769,0.01905,2,0,null,null,"BID_PER_ACTION"],[21212,529546,"i-每次付费-幸存者挑战周日-03.14_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,5878.93,111,52.9633333,2,2939.465000,550.00,0.0935544,0.0580,0.01802,2,0,null,null,"BID_PER_ACTION"],[21212,529547,"r3-每次付费-幸存者挑战周日-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,494.71,2,247.3550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529548,"g-自动ROI-幸存者挑战带穿山甲-03.14_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3226.82,209,15.4393301,0,0.000000,0.00,0E-7,0.0478,0.00000,0,0,null,null,"ROI_PACING"],[21212,529549,"g-自动ROI-幸存者挑战带穿山甲-03.14_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1128.39,55,20.5161818,0,0.000000,0.00,0E-7,0.0364,0.00000,0,0,null,null,"ROI_PACING"],[21212,529550,"g-自动ROI-幸存者挑战带穿山甲-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4918.00,228,21.5701754,1,4918.000000,188.00,0.0382269,0.1000,0.00439,1,0,null,null,"ROI_PACING"],[21212,529551,"r-自动ROI-幸存者挑战带穿山甲-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2723.69,180,15.1316111,1,2723.690000,250.00,0.0917872,0.0281,0.00556,1,0,null,null,"ROI_PACING"],[21212,529552,"r-自动ROI-幸存者挑战带穿山甲-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3128.51,123,25.4350407,1,3128.510000,30.00,0.0095892,0.0813,0.00813,1,0,null,null,"ROI_PACING"],[21212,529553,"r-自动ROI-幸存者挑战带穿山甲-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2893.99,204,14.1862255,1,2893.990000,12.00,0.0041465,0.0343,0.00490,1,0,null,null,"ROI_PACING"],[21212,529554,"r-自动ROI-幸存者挑战带穿山甲-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,291.06,10,29.1060000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,529555,"g-自动ROI-幸存者挑战带穿山甲-03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,940.88,32,29.4025000,0,0.000000,0.00,0E-7,0.0645,0.00000,0,0,null,null,"ROI_PACING"],[21212,529557,"r-每次付费-幸存者挑战-穿山甲专场-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1855.91,11,168.7190909,1,1855.910000,60.00,0.0323292,0.2500,0.09091,1,0,null,null,"BID_PER_ACTION"],[21245,519366,"BY每次付费-拉环小游戏-xcztz-周六-0306_4","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,1564.61,43,36.3862791,0,0.000000,0.00,0E-7,0.0698,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,519381,"BY每次付费-拉环小游戏-xcztz-周六-0306_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,2803.74,97,28.9045361,0,0.000000,0.00,0E-7,0.0505,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520837,"by每次付费-xcztz-0308_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,3,5575.12,159,35.0636478,2,2787.560000,822.00,0.1474408,0.0660,0.01258,1,2,null,null,"BID_PER_ACTION"],[21245,520854,"dm每次付费-xcztz-0308_15","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,3,476.90,1,476.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,521878,"自动ROI-JK-排3K包-拉环-_0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,3,146.79,20,7.3395000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524567,"自动ROI-排3K包-0310_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,163.87,25,6.5548000,0,0.000000,0.00,0E-7,0.1579,0.00000,0,2,null,null,"ROI_PACING"],[21245,524575,"自动ROI-排3K包-0310_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,72.64,4,18.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524576,"自动ROI-排3K包-0310_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,313.03,19,16.4752632,0,0.000000,0.00,0E-7,0.0526,0.00000,0,2,null,null,"ROI_PACING"],[21245,524581,"自动ROI-排3K包-0310_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,5.59,5,1.1180000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524582,"自动ROI-排3K包-0310_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,8.20,0,8.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,524999,"BY每次付费-xcztz-拉环卡车-0310-_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,886.34,19,46.6494737,0,0.000000,0.00,0E-7,0.1053,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525000,"BY每次付费-xcztz-拉环卡车-0310-_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,728.08,25,29.1232000,1,728.080000,128.00,0.1758049,0.2000,0.04000,1,2,null,null,"BID_PER_ACTION"],[21245,525003,"BY每次付费-xcztz-拉环卡车-0310-_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,2319.04,72,32.2088889,1,2319.040000,30.00,0.0129364,0.0833,0.01389,1,2,null,null,"BID_PER_ACTION"],[21245,525009,"BY每次付费-xcztz-拉环卡车-0310-_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,991.76,25,39.6704000,1,991.760000,30.00,0.0302493,0.1600,0.04000,1,2,null,null,"BID_PER_ACTION"],[21245,525014,"BY每次付费-xcztz-拉环卡车-0310-_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,612.95,12,51.0791667,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525019,"DM每次付费-xcztz-拉环卡车-0310-_18","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,113.81,0,113.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,526394,"jk自动ROI-排3K-xcztz-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,64.35,1,64.3500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526395,"jk自动ROI-排3K-xcztz-0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,16.18,3,5.3933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526396,"jk自动ROI-排3K-xcztz-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,176.33,27,6.5307407,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,526397,"jk自动ROI-排3K-xcztz-0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,119.46,19,6.2873684,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526398,"jk自动ROI-排3K-xcztz-0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,6.21,2,3.1050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526576,"DM自动ROI-xcztz-排3K-0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,714.60,108,6.6166667,0,0.000000,0.00,0E-7,0.0556,0.00000,0,2,null,null,"ROI_PACING"],[21245,526578,"DM自动ROI-xcztz-排3K-0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,134.69,7,19.2414286,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"ROI_PACING"],[21245,526579,"DM自动ROI-xcztz-排3K-0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,44.38,4,11.0950000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"ROI_PACING"],[21245,526580,"DM自动ROI-xcztz-排3K-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,78.78,1,78.7800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526582,"DM自动ROI-xcztz-排3K-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,15.81,5,3.1620000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,527266,"每次-2762-冲冲冲_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,7099.46,58,122.4044828,1,7099.460000,60.00,0.0084513,0.0769,0.01724,0,2,null,null,"BID_PER_ACTION"],[21245,527267,"每次-2762-冲冲冲_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,8103.71,85,95.3377647,1,8103.710000,12.00,0.0014808,0.0714,0.01176,1,2,null,null,"BID_PER_ACTION"],[21245,529374,"JK-自动ROI排3K周日发力-0313_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,524.14,48,10.9195833,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21245,529375,"JK-自动ROI排3K周日发力-0313_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,666.41,45,14.8091111,1,666.410000,60.00,0.0900347,0.0667,0.02222,1,0,null,null,"ROI_PACING"],[21245,529376,"JK-自动ROI排3K周日发力-0313_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3473.89,103,33.7270874,1,3473.890000,208.00,0.0598752,0.0854,0.00971,1,0,null,null,"DEEP_BID_DEFAULT"],[21245,529378,"JK-自动ROI排3K周日发力-0313_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,102.88,3,34.2933333,0,0.000000,0.00,0E-7,0.5000,0.00000,0,0,null,null,"ROI_PACING"],[21245,529399,"每次-周末冲刺-0314_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2802.70,31,90.4096774,1,2802.700000,12.00,0.0042816,0.1613,0.03226,1,0,null,null,"BID_PER_ACTION"],[21245,529400,"每次-周末冲刺-0314_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,26,107.8361538,0,0.000000,0.00,0E-7,0.1154,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529401,"每次-周末冲刺-0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,38,73.7826316,0,0.000000,0.00,0E-7,0.0526,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529402,"每次-周末冲刺-0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,62,45.2216129,0,0.000000,0.00,0E-7,0.0323,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529403,"每次-周末冲刺-0314_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1781.02,20,89.0510000,0,0.000000,0.00,0E-7,0.0500,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529404,"每次-周末冲刺-0314_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,173,16.2065896,2,1401.870000,422.00,0.1505132,0.0578,0.01156,2,0,null,null,"BID_PER_ACTION"],[21245,529405,"每次-周末冲刺-0314_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,25,112.1496000,0,0.000000,0.00,0E-7,0.0400,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529406,"每次-周末冲刺-0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,21,133.5114286,1,2803.740000,12.00,0.0042800,0.1429,0.04762,0,0,null,null,"BID_PER_ACTION"],[21245,529407,"每次-周末冲刺-0314_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,74,37.8883784,1,2803.740000,42.00,0.0149800,0.0405,0.01351,1,0,null,null,"BID_PER_ACTION"],[21245,529408,"每次-周末冲刺-0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,10140.91,84,120.7251190,3,3380.303333,3212.00,0.3167369,0.0769,0.03571,3,0,null,null,"BID_PER_ACTION"],[21245,529409,"每次-周末冲刺-0314_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1051.60,3,350.5333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529410,"每次-周末冲刺-0314_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,41,68.3839024,0,0.000000,0.00,0E-7,0.1220,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529411,"每次-周末冲刺-0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,457.52,1,457.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529413,"每次-周末冲刺-0314_17","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,472.52,1,472.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529415,"每次-周末冲刺-0314_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2803.74,18,155.7633333,2,1401.870000,200.00,0.0713333,0.0556,0.11111,2,0,null,null,"BID_PER_ACTION"],[6867,494510,"0219-自动ROI-MG540迷岛_31304077","2021-03-16 00:00:00","2021-02-19 00:00:00",10,1001400,1,3,5024.33,54,93.0431481,0,0.000000,0.00,0E-7,0.0741,0.00000,0,2,null,null,"ROI_PACING"],[20541,506968,"RICH-财神到 -研究-通投_横SSR2138_0226_16","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,3,15887.86,192,82.7492708,3,5295.953333,418.00,0.0263094,0.0674,0.01563,3,2,null,null,"DEEP_BID_DEFAULT"],[20541,507537,"RICH-财神到-研究所_IOS通投_横SSR2363_0226_3","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,3,47336.59,535,88.4796075,16,2958.536875,2404.00,0.0507852,0.1017,0.02991,16,2,null,null,"DEEP_BID_DEFAULT"],[20541,513880,"RICH-研究所-招R体-通投_横SSR2248_0303_12","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,3,18248.96,234,77.9870085,4,4562.240000,684.00,0.0374816,0.1639,0.01709,4,2,null,null,"DEEP_BID_DEFAULT"],[20541,513881,"RICH-研究所-招R体-通投_横SSR2185_0303_1","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,3,279.21,2,139.6050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,513883,"RICH-研究所-招R体-通投_横SSR2666_0303_10","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,3,107.73,0,107.7300000,0,0.000000,0.00,0E-7,0.5000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,520818,"RICH-研究所-财神到我家-通投_横SSR2724_0308_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,3,1272.36,4,318.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,523053,"RICH-研究所-财神到我家-通投_横SSR2363_0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,3,8584.18,132,65.0316667,2,4292.090000,60.00,0.0069896,0.0826,0.01515,2,2,null,null,"DEEP_BID_DEFAULT"],[20541,523054,"RICH-研究所-财神到我家-通投_横SSR2138_0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,3,9784.14,94,104.0865957,1,9784.140000,474.00,0.0484457,0.1290,0.01064,1,2,null,null,"DEEP_BID_DEFAULT"],[20541,523056,"RICH-研究所-财神到我家-通投_横SSR2536_0309_19","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,3,4853.76,61,79.5698361,2,2426.880000,54.00,0.0111254,0.1786,0.03279,1,2,null,null,"DEEP_BID_DEFAULT"],[20541,523061,"RICH-研究所-财神到我家-通投_横SSR1909_0309_18","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,3,2688.02,30,89.6006667,0,0.000000,0.00,0E-7,0.0357,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526238,"RICH-研究所-财神到我家_通投-网络-_SSR2248_0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,3,1689.48,11,153.5890909,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526248,"RICH-研究所-财神到我家_通投-新用户-_SSR2248_0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,3,2042.04,17,120.1200000,0,0.000000,0.00,0E-7,0.0625,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526251,"RICH-研究所-财神到我家_通投-包过滤-_SSR2248_0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,3,2671.02,23,116.1313043,0,0.000000,0.00,0E-7,0.1500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528016,"RICH-研究所-财神到我家_通投_横SSR2285_0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,3,2352.71,27,87.1374074,0,0.000000,0.00,0E-7,0.0588,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528017,"RICH-研究所-财神到我家_通投_横SSR2495_0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,3,1133.14,7,161.8771429,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528019,"RICH-研究所-财神到我家_通投_横SSR2710_0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,3,5885.79,53,111.0526415,2,2942.895000,54.00,0.0091746,0.1250,0.03774,2,2,null,null,"DEEP_BID_DEFAULT"],[20541,528020,"RICH-研究所-财神到我家_通投_横SSR2138_0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,3,1783.52,17,104.9129412,0,0.000000,0.00,0E-7,0.0588,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528021,"RICH-研究所-财神到我家_通投_横SSR1909_0312_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,3,598.24,18,33.2355556,1,598.240000,30.00,0.0501471,0.1333,0.05556,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528025,"RICH-研究所-财神到我家_通投_横SSR2138_0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,3,2817.81,15,187.8540000,1,2817.810000,98.00,0.0347788,0.7500,0.06667,1,2,null,null,"DEEP_BID_DEFAULT"],[20546,526922,"0311wtd末日研究所_2754_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,3,2293.59,14,163.8278571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526923,"0311wtd末日研究所_2760_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,3,1685.15,7,240.7357143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526925,"0311wtd末日研究所_2736_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,3,2705.34,12,225.4450000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528805,"0312XM-MGTTN1计划_3_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,261.56,1,261.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528806,"0312XM-MGTTN1计划_3_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,835.18,9,92.7977778,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528807,"0312XM-MGTTN1计划_3_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,299.99,4,74.9975000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528808,"0312XM-MGTTN1计划_3_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,158.74,4,39.6850000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528811,"0312XM-MGTTN1计划_3_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,66.07,2,33.0350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528812,"0312XM-MGTTN1计划_3_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,1505.22,38,39.6110526,1,1505.220000,268.00,0.1780471,0.0313,0.02632,1,2,null,null,"DEEP_BID_DEFAULT"],[20647,525144,"cxq-tt548N1计划-0310_包12742","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,3,2527.92,44,57.4527273,0,0.000000,0.00,0E-7,0.0682,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525151,"cxq-tt894N1计划-0310_春节包通投2741","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,3,841.69,9,93.5211111,2,420.845000,24.00,0.0285141,0.1111,0.22222,2,2,null,null,"DEEP_BID_DEFAULT"],[20647,526681,"cxq-tt548N1计划-0311__3小2755","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,3,2017.50,22,91.7045455,0,0.000000,0.00,0E-7,0.0455,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526683,"cxq-tt548N1计划-0311_12742","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,3,7997.18,62,128.9867742,2,3998.590000,128.00,0.0160056,0.0000,0.03226,1,2,null,null,"DEEP_BID_DEFAULT"],[20647,526684,"cxq-tt894N1计划-0311_22558","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,3,5847.29,59,99.1066102,1,5847.290000,12.00,0.0020522,0.1034,0.01695,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526685,"cxq-tt894N1计划-0311_4小2753","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,3,731.10,4,182.7750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526688,"cxq-tt894N1计划-0311_6小2723","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,3,2350.36,9,261.1511111,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526694,"cxq-tt548N1计划-0311__52750","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,3,370.47,4,92.6175000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526696,"cxq-tt894N1计划-0311_12742","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,3,35.37,3,11.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,526703,"N1穿山甲-iOS-499-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,3,5601.24,39,143.6215385,3,1867.080000,528.00,0.0942648,0.0385,0.07692,3,2,null,null,"DEEP_BID_DEFAULT"],[21246,528854,"付费-穿山甲-n1-499-0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,251.75,4,62.9375000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528855,"付费-穿山甲-n1-499-0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,36.64,1,36.6400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528857,"付费-穿山甲-n1-499-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,716.26,8,89.5325000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,529419,"穿-n1-周末付费冲-0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,3,3039.72,18,168.8733333,2,1519.860000,90.00,0.0296080,0.1667,0.11111,1,0,null,null,"DEEP_BID_DEFAULT"],[21246,529421,"穿-n1-周末付费冲-0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,3,129.76,1,129.7600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529422,"穿-n1-周末付费冲-0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,3,267.40,1,267.4000000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[6770,522169,"7tt_n_每次付费_幸存危机_0309_9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,3,1720.49,30,57.3496667,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522170,"7tt_n_每次付费_幸存危机_0309_7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,3,245.12,4,61.2800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522172,"7tt_n_每次付费_幸存危机_0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,3,637.99,9,70.8877778,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522175,"7tt_n_每次付费_幸存危机_0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,3,4204.13,128,32.8447656,1,4204.130000,3870.00,0.9205234,0.1074,0.00781,1,2,null,null,"BID_PER_ACTION"],[6770,524789,"7tt_n_幸存危机_每次付费__0310_72275170","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,64.29,2,32.1450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524805,"7tt_n_幸存危机_每次付费__0310_722752412","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,136.17,2,68.0850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526325,"7tt_n_幸存危机_每次付费__0311_52245342","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,867.16,8,108.3950000,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526326,"7tt_n_幸存危机_每次付费__0311_52245343","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,116.35,4,29.0875000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526329,"7tt_n_幸存危机_每次付费__0311_52245356","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,61.44,1,61.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526331,"7tt_n_幸存危机_每次付费__0311_52245358","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,20.51,1,20.5100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526336,"7tt_n_幸存危机_每次付费__0311_522453713","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,74.62,1,74.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527935,"7tt_n_幸存危机_每次付费__0312_16295372","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,792.54,5,158.5080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527967,"7tt_n_幸存危机_每次付费__0312_16295373","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,527.31,13,40.5623077,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527969,"7tt_n_幸存危机_每次付费__0312_16295385","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,106.68,1,106.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527970,"7tt_n_幸存危机_每次付费__0312_16295386","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,378.48,3,126.1600000,1,378.480000,286.00,0.7556542,0.6667,0.33333,1,2,null,null,"BID_PER_ACTION"],[6770,527973,"7tt_n_幸存危机_每次付费__0312_16295389","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,1306.28,21,62.2038095,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527974,"7tt_n_幸存危机_每次付费__0312_162953910","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,349.65,2,174.8250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527975,"7tt_n_幸存危机_每次付费__0312_162953911","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,145.98,2,72.9900000,0,0.000000,0.00,0E-7,0.5000,0.00000,1,2,null,null,"BID_PER_ACTION"],[6770,527976,"7tt_n_幸存危机_每次付费__0312_162953912","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,498.28,5,99.6560000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527977,"7tt_n_幸存危机_每次付费__0312_162953913","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,380.73,0,380.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529172,"7tt_n_幸存危机_每次付费__0313_84190332","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,228.24,1,228.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529173,"7tt_n_幸存危机_每次付费__0313_84190333","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,726.15,4,181.5375000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529175,"7tt_n_幸存危机_每次付费__0313_84190349","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,1663.14,17,97.8317647,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529176,"7tt_n_幸存危机_每次付费__0313_84190336","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,1766.09,21,84.0995238,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529177,"7tt_n_幸存危机_每次付费__0313_84190347","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,286.56,2,143.2800000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529179,"7tt_n_幸存危机_每次付费__0313_84190334","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,425.15,2,212.5750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529180,"7tt_n_幸存危机_每次付费__0313_84190335","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,4136.65,26,159.1019231,2,2068.325000,24.00,0.0058018,0.1538,0.07692,1,2,null,null,"BID_PER_ACTION"],[6770,529182,"7tt_n_幸存危机_每次付费__0313_841903411","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,776.36,4,194.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529183,"7tt_n_幸存危机_每次付费__0313_841903412","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,861.68,9,95.7422222,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529184,"7tt_n_幸存危机_每次付费__0313_841903513","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,595.81,5,119.1620000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,529020,"CJ-财神到-ROI系03.13-0","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,1330.09,33,40.3057576,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529021,"VR-财神到-ROI系03.13-1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,1076.89,10,107.6890000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529022,"VR-财神到-ROI系03.13-3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,3303.74,73,45.2567123,2,1651.870000,80.00,0.0242150,0.1096,0.02740,2,2,null,null,"ROI_COEFFICIENT"],[20451,529023,"CJ-财神到-ROI系03.13-8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,3300.27,28,117.8667857,0,0.000000,0.00,0E-7,0.0690,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529024,"CJ-财神到-ROI系03.13-9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,1612.81,31,52.0261290,1,1612.810000,184.00,0.1140866,0.1613,0.03226,1,2,null,null,"ROI_COEFFICIENT"],[20451,529025,"CJ-财神到-ROI系03.13-7","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,2549.25,51,49.9852941,0,0.000000,0.00,0E-7,0.2807,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529027,"VR-财神到-ROI系03.13-2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,424.82,4,106.2050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529028,"VR-财神到-ROI系03.13-5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,415.98,4,103.9950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529029,"CJ-财神到-ROI系03.13-6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,2161.62,29,74.5386207,0,0.000000,0.00,0E-7,0.0690,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529364,"VR-危机-每次03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,547.21,2,273.6050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529365,"CJ-危机-每次03.14_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,726.81,2,363.4050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529366,"VR-危机-每次03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,156.87,1,156.8700000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529367,"CJ-危机-每次03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,1938.18,20,96.9090000,1,1938.180000,110.00,0.0567543,0.2000,0.05000,1,0,null,null,"BID_PER_ACTION"],[20451,529370,"CJ-危机-每次03.14_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,749.56,4,187.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529371,"CJ-危机-每次03.14_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,720.60,1,720.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529372,"VR-危机-每次03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,1100.06,7,157.1514286,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529373,"VR-危机-每次03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,679.15,3,226.3833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,520869,"0308-508幸存危机-每次付费_2332_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,3,91.38,0,91.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,522465,"0309-508幸存危机-每次付费_2271_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,3,384.17,10,38.4170000,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,524344,"0310-508幸存危机-每次付费竖版沉浸_2332_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,6.43,0,6.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,524351,"0310-508幸存危机-每次付费竖版沉浸_2738_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,81.13,4,20.2825000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526911,"0311-508幸存危机-每次付费竖版沉浸_2332_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,134.77,2,67.3850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526912,"0311-508幸存危机-每次付费竖版沉浸_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,617.00,4,154.2500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528126,"0312-507幸存危机每次付费_2138_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,1520.26,7,217.1800000,1,1520.260000,452.00,0.2973176,0.1429,0.14286,1,2,null,null,"BID_PER_ACTION"],[20545,528127,"0312-507幸存危机每次付费_2332_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,1732.83,14,123.7735714,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528129,"0312-507幸存危机每次付费_2766_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,708.85,4,177.2125000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528133,"0312-507幸存危机每次付费_2768_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,5646.72,53,106.5418868,1,5646.720000,30.00,0.0053128,0.0370,0.01887,2,2,null,null,"BID_PER_ACTION"],[20545,528547,"0312-508幸存危机-每次付费竖版沉浸_2332_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,426.89,9,47.4322222,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528550,"0312-508幸存危机-每次付费竖版沉浸_2777_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,168.72,1,168.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528551,"0312-508幸存危机-每次付费竖版沉浸_2762_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,563.76,2,281.8800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528553,"0312-508幸存危机-每次付费竖版沉浸_2737_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,690.61,0,690.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527869,"481IC-幸存危机-648大集合-3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,516.94,1,516.9400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527870,"481IC-幸存危机-648大集合-3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,1271.51,2,635.7550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527871,"481IC-幸存危机-648大集合-3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,4648.81,18,258.2672222,1,4648.810000,182.00,0.0391498,0.1667,0.05556,1,2,null,null,"BID_PER_ACTION"],[21208,527872,"481IC-幸存危机-648大集合-3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,413.02,3,137.6733333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527873,"481IC-幸存危机-648大集合-3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,3610.72,15,240.7146667,1,3610.720000,48.00,0.0132937,0.3636,0.06667,2,2,null,null,"BID_PER_ACTION"],[21212,524367,"k-每次付费-危机窄定向-03.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,3324.78,21,158.3228571,1,3324.780000,128.00,0.0384988,0.0476,0.04762,1,2,null,null,"BID_PER_ACTION"],[21212,525586,"k-每次付费-幸存危机加量-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,2934.38,16,183.3987500,2,1467.190000,256.00,0.0872416,0.1538,0.12500,2,2,null,null,"BID_PER_ACTION"],[21212,529114,"k-每次付费-幸存危机-03.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,510.17,2,255.0850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529115,"k-每次付费-幸存危机-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,483.99,1,483.9900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529122,"k-每次付费-幸存危机-03.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,669.59,5,133.9180000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21213,526689,"n-ios-xy-黎明血战-03.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,3,1185.45,14,84.6750000,1,1185.450000,176.00,0.1484668,0.0714,0.07143,1,2,null,null,"DEEP_BID_DEFAULT"],[21213,526691,"n-ios-xy-黎明血战-03.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,3,3059.00,32,95.5937500,0,0.000000,0.00,0E-7,0.0938,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526706,"n96-ios-xy-黎明血战-03.11_18","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,3,4.15,1,4.1500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526709,"n96-ios-xy-黎明血战-03.11_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,3,0.81,0,0.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528877,"n96-ios-黎明血战-03.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,3,0.88,2,0.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528878,"n96-ios-黎明血战-03.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,3,0.07,2,0.0350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528879,"n96-ios-黎明血战-03.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,3,0.17,5,0.0340000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528883,"n96-ios-黎明血战-03.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,3,0.46,1,0.4600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,529573,"n96-iOS-黎明血战周日-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,3,4752.88,123,38.6413008,1,4752.880000,80.00,0.0168319,0.0625,0.00813,1,0,null,null,"DEEP_BID_DEFAULT"],[21213,529574,"n96-iOS-黎明血战周日-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,3,1174.32,13,90.3323077,0,0.000000,0.00,0E-7,0.0909,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,529575,"n96-iOS-黎明血战周日-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,3,1038.24,22,47.1927273,0,0.000000,0.00,0E-7,0.1364,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[6867,529646,"0314-540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4310.70,79,54.5658228,1,4310.700000,98.00,0.0227341,0.0000,0.01266,1,0,null,null,"BID_PER_ACTION"],[6867,529652,"0314-540-每次付费-幸_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,12389.74,515,24.0577476,2,6194.870000,1322.00,0.1067012,0.0000,0.00388,2,0,null,null,"BID_PER_ACTION"],[6867,529655,"0314-540-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,5653.59,139,40.6733094,1,5653.590000,110.00,0.0194567,0.0000,0.00719,0,0,null,null,"BID_PER_ACTION"],[6867,529656,"0314-540-每次付费-幸_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4422.75,52,85.0528846,1,4422.750000,12.00,0.0027132,0.0000,0.01923,0,0,null,null,"BID_PER_ACTION"],[6867,529658,"0314-540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,18874.63,268,70.4277239,4,4718.657500,418.00,0.0221461,0.0000,0.01493,4,0,null,null,"BID_PER_ACTION"],[6867,529919,"0315-204-穿山甲-每次付费-幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2010.23,62,32.4230645,2,1005.115000,518.00,0.2576820,0.0000,0.03226,1,0,null,null,"BID_PER_ACTION"],[6867,529921,"0315-204-穿山甲-每次付费-幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,439.52,21,20.9295238,1,439.520000,12.00,0.0273025,0.0000,0.04762,0,0,null,null,"BID_PER_ACTION"],[6867,530808,"0315FP-ROI系数-MG幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3766.77,14,269.0550000,1,3766.770000,2624.00,0.6966181,0.0000,0.07143,1,0,null,null,"ROI_COEFFICIENT"],[20451,530717,"306-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,7688.27,237,32.4399578,1,7688.270000,30.00,0.0039020,0.0000,0.00422,1,0,null,null,"ROI_COEFFICIENT"],[20451,530719,"303-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3356.80,89,37.7168539,1,3356.800000,256.00,0.0762631,0.0000,0.01124,1,0,null,null,"ROI_COEFFICIENT"],[20451,530936,"FJ-幸存者-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3302.07,47,70.2568085,1,3302.070000,42.00,0.0127193,0.0000,0.02128,0,0,null,null,"BID_PER_ACTION"],[20540,530798,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2285315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2534.93,64,39.6082813,1,2534.930000,12.00,0.0047339,0.0000,0.01563,1,0,null,null,"BID_PER_ACTION"],[20545,530526,"0315oyo393-roi系数-排除3k_2138_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1656.50,19,87.1842105,1,1656.500000,12.00,0.0072442,0.0000,0.05263,0,0,null,null,"ROI_COEFFICIENT"],[20545,530714,"0315-幸存者393z每次付费排除3k_2789_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,124.21,1,124.2100000,1,124.210000,298.00,2.3991627,0.0000,1.00000,1,0,null,null,"BID_PER_ACTION"],[20545,530715,"0315-幸存者393z每次付费排除3k_2792_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,4240.95,69,61.4630435,1,4240.950000,30.00,0.0070739,0.0000,0.01449,1,0,null,null,"BID_PER_ACTION"],[20545,531288,"0315幸存者393z每次付费排除3K_2796_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,10905.36,259,42.1056371,3,3635.120000,3214.00,0.2947175,0.0000,0.01158,2,0,null,null,"BID_PER_ACTION"],[20635,530152,"每0315-tt-xcztz-wyt-rk_68","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,8937.37,78,114.5816667,2,4468.685000,250.00,0.0279724,0.0000,0.02564,2,0,null,null,"BID_PER_ACTION"],[20637,529851,"263-幸存者-MCFC0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,7889.73,135,58.4424444,2,3944.865000,168.00,0.0212935,0.0000,0.01481,1,0,null,null,"BID_PER_ACTION"],[20637,529866,"263-幸存者-MCFC0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2851.71,50,57.0342000,1,2851.710000,128.00,0.0448853,0.0000,0.02000,0,0,null,null,"BID_PER_ACTION"],[20637,529974,"990-幸存者-MCFC-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1068.89,18,59.3827778,1,1068.890000,128.00,0.1197504,0.0000,0.05556,1,0,null,null,"BID_PER_ACTION"],[20637,531204,"05-幸存者-MCFC0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,167.78,3,55.9266667,1,167.780000,392.00,2.3363929,0.0000,0.33333,1,0,null,null,"BID_PER_ACTION"],[20637,531251,"05-幸存者-MCFC0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1168.82,16,73.0512500,1,1168.820000,226.00,0.1933574,0.0000,0.06250,1,0,null,null,"BID_PER_ACTION"],[20637,531308,"263-幸存者-ROI-0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,16963.69,458,37.0386245,4,4240.922500,602.00,0.0354876,0.0000,0.00873,4,0,null,null,"ROI_COEFFICIENT"],[20637,531312,"263-幸存者-ROI-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1496.10,14,106.8642857,1,1496.100000,226.00,0.1510594,0.0000,0.07143,1,0,null,null,"ROI_COEFFICIENT"],[20637,531586,"263-幸存者-MCFC0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,7135.46,49,145.6216327,1,7135.460000,42.00,0.0058861,0.0000,0.02041,1,0,null,null,"BID_PER_ACTION"],[20646,530670,"cxq-tt982幸存者-ROI系数0315_2762","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1648.92,48,34.3525000,1,1648.920000,188.00,0.1140140,0.0000,0.02083,1,0,null,null,"ROI_COEFFICIENT"],[20754,530906,"jap-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1142.78,6,190.4633333,1,1142.780000,200.00,0.1750118,0.0000,0.16667,1,0,null,null,"BID_PER_ACTION"],[20754,530924,"tw-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1087.12,17,63.9482353,2,543.560000,3096.00,2.8478917,0.0000,0.11765,2,0,null,null,"BID_PER_ACTION"],[20754,530959,"ka-roi系数-周一-3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,4853.92,117,41.4864957,4,1213.480000,182.00,0.0374955,0.0000,0.03419,1,0,null,null,"ROI_COEFFICIENT"],[20754,530961,"ka-roi系数-周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,5097.48,91,56.0162637,2,2548.740000,652.00,0.1279063,0.0000,0.02198,1,0,null,null,"ROI_COEFFICIENT"],[20754,530962,"ka-roi系数-周一-3.15_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2803.74,72,38.9408333,1,2803.740000,12.00,0.0042800,0.0000,0.01389,0,0,null,null,"ROI_COEFFICIENT"],[20754,530966,"yo-roi系数-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2659.43,26,102.2857692,1,2659.430000,42.00,0.0157929,0.0000,0.03846,1,0,null,null,"ROI_COEFFICIENT"],[20754,530968,"ka-roi系数-周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2115.48,31,68.2412903,2,1057.740000,238.00,0.1125040,0.0000,0.06452,2,0,null,null,"ROI_COEFFICIENT"],[21208,531039,"bdna-非海马云-幸存者挑战-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2803.74,66,42.4809091,1,2803.740000,12.00,0.0042800,0.0000,0.01515,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531040,"bdna-非海马云-幸存者挑战-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2664.20,7,380.6000000,1,2664.200000,12.00,0.0045042,0.0000,0.14286,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531042,"bdna-非海马云-幸存者挑战-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2393.44,25,95.7376000,1,2393.440000,74.00,0.0309178,0.0000,0.04000,1,0,null,null,"DEEP_BID_DEFAULT"],[21212,530169,"r3-每次付费-幸存者挑战优选位-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2708.30,13,208.3307692,1,2708.300000,12.00,0.0044308,0.0000,0.07692,0,0,null,null,"BID_PER_ACTION"],[21212,530178,"i-每次付费-幸存者挑战只限男优先位-03.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,6533.02,84,77.7740476,1,6533.020000,678.00,0.1037805,0.0000,0.01190,1,0,null,null,"BID_PER_ACTION"],[21212,530212,"r3-每次付费-幸存者挑战优选位-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2035.04,22,92.5018182,1,2035.040000,30.00,0.0147417,0.0000,0.04545,1,0,null,null,"BID_PER_ACTION"],[21245,525005,"DM每次付费-xcztz-拉环卡车-0310-_23","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,5578.76,141,39.5656738,1,5578.760000,208.00,0.0372843,0.0000,0.00709,1,2,null,null,"BID_PER_ACTION"],[21245,531193,"roi系数-0315-BY-幸存_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2820.12,162,17.4081481,1,2820.120000,110.00,0.0390054,0.0000,0.00617,1,0,null,null,"ROI_COEFFICIENT"],[21245,531228,"roi系数-0315-BY-幸存_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2948.60,41,71.9170732,1,2948.600000,30.00,0.0101743,0.0000,0.02439,0,0,null,null,"ROI_COEFFICIENT"],[20541,531196,"RICH-研究所-财神到我家-通投_混SSR2796_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,3588.63,19,188.8752632,3,1196.210000,122.00,0.0339963,0.0000,0.15789,3,0,null,null,"DEEP_BID_DEFAULT"],[20541,531198,"RICH-研究所-财神到我家-通投_混SSR2792_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,2750.84,18,152.8244444,1,2750.840000,98.00,0.0356255,0.0000,0.05556,1,0,null,null,"DEEP_BID_DEFAULT"],[20546,531301,"0315wtd末日研究所_2138_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,7969.71,71,112.2494366,1,7969.710000,60.00,0.0075285,0.0000,0.01408,1,0,null,null,"DEEP_BID_DEFAULT"],[20546,531577,"0315ph648末日研究所_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,3836.34,34,112.8335294,1,3836.340000,886.00,0.2309493,0.0000,0.02941,1,0,null,null,"DEEP_BID_DEFAULT"],[20451,530040,"CJ-幸存危机-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,2256.19,17,132.7170588,1,2256.190000,30.00,0.0132968,0.0000,0.05882,1,0,null,null,"BID_PER_ACTION"],[20545,531701,"0315-508幸存危机-每次付费_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,2764.52,47,58.8195745,2,1382.260000,1354.00,0.4897776,0.0000,0.04255,1,0,null,null,"BID_PER_ACTION"],[20545,531702,"0315-508幸存危机-每次付费_2053_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,4513.16,74,60.9886486,1,4513.160000,30.00,0.0066472,0.0000,0.01351,1,0,null,null,"BID_PER_ACTION"],[20545,531704,"0315-508幸存危机-每次付费_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,2778.84,28,99.2442857,1,2778.840000,12.00,0.0043183,0.0000,0.03571,0,0,null,null,"BID_PER_ACTION"],[21208,529911,"481IC-幸存危机-2756爆量爆R-3.5_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,4546.33,62,73.3279032,1,4546.330000,12.00,0.0026395,0.0000,0.01613,1,0,null,null,"BID_PER_ACTION"],[21208,529912,"481IC-幸存危机-2756爆量爆R-3.5_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,1254.35,10,125.4350000,1,1254.350000,72.00,0.0574002,0.0000,0.10000,1,0,null,null,"BID_PER_ACTION"],[21208,529915,"481IC-幸存危机-2756爆量爆R-3.5_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,2643.53,27,97.9085185,1,2643.530000,384.00,0.1452603,0.0000,0.03704,1,0,null,null,"BID_PER_ACTION"],[20540,507233,"RICH-财神保佑-幸存者-每次付费_春节游戏DMP_竖SSR2285_0226_5","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001379,1,3,2381.33,76,31.3332895,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,517614,"RICH-幸存者-我也爆R-每次付费_安卓游戏_竖SSR2402_0305_3","2021-03-16 00:00:00","2021-03-05 00:00:00",10,1001379,1,3,0.77,0,0.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,519399,"hix-每次付费-ssr-3.6_6","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,3,554.58,0,554.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,520177,"ka-自动roi-周天-2.7_6","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,3,166.26,1,166.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,520180,"yo-自动roi-周天-2.7_3","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,3,9.68,0,9.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[20545,520870,"0308-508幸存危机-每次付费_2724_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,3,17.57,0,17.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,524145,"293bd-末日海马云测试_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,3,2568.77,23,111.6856522,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524146,"293bd-末日海马云测试_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,3,2590.05,0,2590.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524147,"293bd-末日海马云测试_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,3,2696.61,1,2696.6100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524148,"293bd-末日海马云测试_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,3,2467.83,0,2467.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524149,"293bd-末日海马云测试_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,3,2546.98,0,2546.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[6770,524791,"7tt_n_幸存危机_每次付费__0310_72275202","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,1.20,0,1.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,524797,"7tt_n_幸存危机_每次付费__0310_72275215","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,5.12,0,5.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,524795,"7tt_n_幸存危机_每次付费__0310_72275227","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,131.93,0,131.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,524803,"7tt_n_幸存危机_每次付费__0310_722752310","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,0.16,0,0.1600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,524807,"7tt_n_幸存危机_每次付费__0310_722752514","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,2.06,0,2.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,524964,"0310-507幸存危机每次付费_2743_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,3,0.16,0,0.1600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525012,"BY每次付费-xcztz-拉环卡车-0310-_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,59.97,0,59.9700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525011,"BY每次付费-xcztz-拉环卡车-0310-_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,310.76,0,310.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20647,525149,"cxq-tt894N1计划-0310_春节包通投2743","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,3,74.79,0,74.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20545,525270,"0310oyo393自动roi莱卡_2363_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,3.06,0,3.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,525273,"0310oyo393自动roi莱卡_2743_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,3.15,0,3.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,525302,"0310全民534自动roi基定排除2k_2724_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,37.73,0,37.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525582,"7tt_529_幸存者挑战_自动ROI_3.10_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,205.79,0,205.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525585,"7tt_498_幸存者挑战_自动ROI_3.10_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,40.63,0,40.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525584,"7tt_498_幸存者挑战_自动ROI_3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,3,10.33,0,10.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,526311,"7tt_n_幸存危机_每次付费__0311_52245341","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,12.43,0,12.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526327,"7tt_n_幸存危机_每次付费__0311_52245354","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,59.33,0,59.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526328,"7tt_n_幸存危机_每次付费__0311_52245355","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,139.50,0,139.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526330,"7tt_n_幸存危机_每次付费__0311_52245357","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,69.68,0,69.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526334,"7tt_n_幸存危机_每次付费__0311_522453611","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,68.95,0,68.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526333,"7tt_n_幸存危机_每次付费__0311_522453610","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,23.01,0,23.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526337,"7tt_n_幸存危机_每次付费__0311_522453714","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,285.53,0,285.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,526400,"jk自动ROI-排3K-xcztz-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,48.25,0,48.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20635,526433,"每0311-tt-xcztz-wyt-rk_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,7.77,0,7.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,526434,"每0311-tt-xcztz-wyt-rk_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,27.77,0,27.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526480,"cxq-tt幸存者-每次付费-排除3000-0311_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,285.89,0,285.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526532,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2484_0311_27","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,24.13,0,24.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526530,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2723_0311_26","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,146.35,0,146.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,526577,"DM自动ROI-xcztz-排3K-0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,13.83,0,13.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,526581,"DM自动ROI-xcztz-排3K-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,49.39,0,49.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20647,526687,"cxq-tt894N1计划-0311_3小2755","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,3,527.51,3,175.8366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6867,526753,"0311-tve-每次付费-幸_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,743.10,11,67.5545455,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526755,"0311-tve-每次付费-幸_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,16.67,0,16.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526759,"0311-tve-每次付费-幸_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,115.78,1,115.7800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526761,"0311-tve-每次付费-幸_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,151.01,0,151.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526762,"0311-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,17.05,0,17.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526764,"0311-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,99.73,0,99.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526765,"0311-tve-每次付费-幸_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,11.56,0,11.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526860,"RICH-财神到我家-幸存者-自动roi-_通投_横SSR2736_0311_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,1.39,0,1.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526861,"RICH-财神到我家-幸存者-自动roi-_春节DMP_横SSR1909_0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526864,"RICH-财神到我家-幸存者-自动roi-_游戏_横SSR2736_0311_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,0.63,0,0.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526862,"RICH-财神到我家-幸存者-自动roi-_春节DMP_横SSR2736_0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,4.96,0,4.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526863,"RICH-财神到我家-幸存者-自动roi-_通投_横SSR1909_0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,0.12,0,0.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,526885,"0311幸存者393z每次付费3k_2724_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,3,1.97,0,1.9700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,526913,"0311-508幸存危机-每次付费竖版沉浸_2736_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,526915,"0311-508幸存危机-每次付费竖版沉浸_2697_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,7.73,0,7.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,526914,"0311-508幸存危机-每次付费竖版沉浸_2695_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,3,1.47,0,1.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20755,526949,"au-黎明血战-3.11_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,3,0.49,0,0.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526950,"th-黎明血战-3.11_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,3,0.11,0,0.1100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526954,"au-黎明血战-3.11_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,3,2.91,0,2.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,527342,"470-tv-每次付费爆R爆量_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,20.85,0,20.8500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527344,"470-tv-每次付费爆R爆量_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,97.09,0,97.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527343,"470-tv-每次付费爆R爆量_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,20.10,0,20.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527347,"470-tv-每次付费爆R爆量_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,99.40,0,99.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,527350,"SR-财神到-每次03.12-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,18.03,0,18.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,527368,"SR-财神到-每次03.12-8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,40.56,0,40.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527403,"7tt_498_幸存者挑战_每次付费3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,380.65,0,380.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527470,"0312540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,20.95,0,20.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527471,"0312540-每次付费-幸_15","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,204.65,0,204.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527477,"0312540-每次付费-幸_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,73.30,0,73.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527513,"每0312-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,287.17,0,287.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,527527,"每0312-tt-xcztz-wyt-rk_19","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,120.47,0,120.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527528,"每0312-tt-xcztz-wyt-rk_20","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,46.12,0,46.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527530,"每0312-tt-xcztz-wyt-rk_22","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,92.32,0,92.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,527534,"cxq-tt552幸存者-每次付费穿山甲0312_排负2736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,309.77,0,309.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527695,"RICH-财神到我家_幸存者-每次付费_排3000_78397_0312_95325553","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,51.56,0,51.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527697,"RICH-财神到我家_幸存者-每次付费_排3000_78431_0312_95325566","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,11.83,0,11.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527794,"RICH-财神到我家_幸存者-每次付费_排3000_75017_0312_95325552","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,26.73,0,26.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527859,"RICH-财神到我家_幸存者-自动ROI_排3000_78449_0312_01169240","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,0.09,0,0.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527863,"RICH-财神到我家_幸存者-自动ROI_排3000_67560_0312_01169274","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,192.94,0,192.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527865,"RICH-财神到我家_幸存者-自动ROI_排3000_63730_0312_01169262","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,0.61,0,0.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527866,"RICH-财神到我家_幸存者-自动ROI_排3000_77952_0312_01169251","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,4.13,0,4.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527867,"RICH-财神到我家_幸存者-自动ROI_排3000_78397_0312_01169275","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,0.21,0,0.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527868,"RICH-财神到我家_幸存者-自动ROI_排3000_73141_0312_01169273","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,0.37,0,0.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,527965,"7tt_n_幸存危机_每次付费__0312_16295360","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,77.75,0,77.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527971,"7tt_n_幸存危机_每次付费__0312_16295387","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,75.36,0,75.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527972,"7tt_n_幸存危机_每次付费__0312_16295388","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,94.55,0,94.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527978,"7tt_n_幸存危机_每次付费__0312_162953914","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,3,117.82,0,117.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,528001,"RICH-财神到我家_幸存者-每次付费_排3000_69832_0312_22237942","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,241.93,1,241.9300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20541,528018,"RICH-研究所-财神到我家_通投_横SSR2285_0312_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,3,9.13,0,9.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528024,"RICH-研究所-财神到我家_通投_横SSR2710_0312_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,3,0.06,0,0.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21212,528059,"r-自动ROI-幸存者挑战-03.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528062,"7tt_幸_自动ROI__0312_88256793","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,11.03,0,11.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528071,"7tt_幸_自动ROI__0312_88256829","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,9.36,0,9.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528074,"7tt_幸_自动ROI__0312_882568314","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3.43,1,3.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[6770,528075,"7tt_幸_自动ROI__0312_882568415","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,5.17,0,5.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528085,"7tt_幸_自动ROI__0312_882568926","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1.51,0,1.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528086,"7tt_幸_自动ROI__0312_882569027","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,4.26,0,4.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528088,"7tt_幸_自动ROI__0312_882569129","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,31.56,0,31.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[20754,528137,"jap-每次付费--3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,21.34,0,21.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528142,"jap-每次付费--3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,12.07,0,12.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528139,"hix-每次付费--3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,25.65,0,25.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528152,"jap-每次付费--3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,9.25,0,9.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528156,"jap-每次付费--3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,8.84,0,8.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528155,"hix-每次付费--3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,57.66,0,57.6600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528160,"zhan-每次付费--3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,39.93,0,39.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528161,"zhan-每次付费--3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,34.85,0,34.8500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,528512,"FJ-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528657,"990-幸存者-MCFC-0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,11.70,0,11.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528655,"990-幸存者-MCFC-0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,13.80,0,13.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528656,"990-幸存者-MCFC-0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,40.08,0,40.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528658,"990-幸存者-MCFC-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3.87,0,3.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528784,"7tt_穿穿_每次付费__0312_02259772","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,53.00,0,53.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528777,"7tt_穿穿_每次付费__0312_02259799","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,85.24,1,85.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528786,"7tt_穿穿_每次付费__0312_022598012","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3056.50,3,1018.8333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528789,"7tt_穿穿_每次付费__0312_022598015","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,7.23,0,7.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528792,"7tt_穿穿_每次付费__0312_022598118","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,14.01,0,14.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528794,"7tt_穿穿_每次付费__0312_022598120","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,1.17,0,1.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528798,"7tt_穿穿_每次付费__0312_022598224","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,3.68,0,3.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528801,"7tt_穿穿_每次付费__0312_022598227","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,5.59,1,5.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528803,"7tt_穿穿_每次付费__0312_022598329","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,81.61,0,81.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20143,528804,"0312XM-MGTTN1计划_3_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,18.78,0,18.7800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528809,"0312XM-MGTTN1计划_3_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,101.03,0,101.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528810,"0312XM-MGTTN1计划_3_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,52.18,0,52.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528852,"付费-穿山甲-n1-499-0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,3.28,0,3.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528856,"付费-穿山甲-n1-499-0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,0.88,0,0.8800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528858,"付费-穿山甲-n1-499-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,3,33.64,0,33.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,528866,"7tt_498_穿哥_每次付费_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,98.72,0,98.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528867,"7tt_498_穿哥_每次付费_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,24.14,0,24.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21213,528882,"n96-ios-黎明血战-03.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,3,0.44,0,0.4400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,528895,"05-幸存者-MCFC0312_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,127.81,0,127.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528897,"05-幸存者-MCFC0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,75.12,0,75.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528899,"05-幸存者-MCFC0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,88.04,0,88.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,528919,"FJ-财神到-每次03.13-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,11.88,0,11.8800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528964,"263-幸存者-MCFC0312_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,3,99.65,0,99.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,529026,"VR-财神到-ROI系03.13-4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,91.94,0,91.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[20540,529037,"RICH-财神到我家_幸存者-每次付费_排3000_73012_0313_08286071","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,5.40,0,5.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,529101,"r3-每次付费-幸存者挑战周六-03.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,257.39,0,257.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,529121,"k-每次付费-幸存危机-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,406.65,0,406.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,529123,"k-每次付费-幸存危机-03.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,58.37,0,58.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,529163,"ka-自动roi-周六-3.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,25.70,0,25.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,529170,"7tt_n_幸存危机_每次付费__0313_84190331","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,66.15,0,66.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529169,"7tt_穿穿_每次付费__0313_84191302","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,6.60,0,6.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529178,"7tt_n_幸存危机_每次付费__0313_84190348","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,61.70,0,61.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529181,"7tt_n_幸存危机_每次付费__0313_841903410","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,231.74,0,231.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529174,"7tt_n_幸存危机_每次付费__0313_84190320","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,198.09,0,198.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529185,"7tt_n_幸存危机_每次付费__0313_841903514","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,3,228.91,0,228.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529186,"7tt_穿穿_每次付费__0313_84191290","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3614.51,14,258.1792857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529189,"7tt_穿穿_每次付费__0313_84191315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,14.42,0,14.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529190,"7tt_穿穿_每次付费__0313_84191316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,17.70,2,8.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529191,"7tt_穿穿_每次付费__0313_84191317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3331.83,1,3331.8300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529193,"7tt_穿穿_每次付费__0313_84191329","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2188.97,1,2188.9700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529194,"7tt_穿穿_每次付费__0313_841913210","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2.47,0,2.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529196,"7tt_穿穿_每次付费__0313_841913212","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,44.24,1,44.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529197,"7tt_穿穿_每次付费__0313_841913213","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,3780.34,0,3780.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529199,"7tt_穿穿_每次付费__0313_841913214","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,30.13,1,30.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529200,"7tt_穿穿_每次付费__0313_841913316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,13.36,0,13.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529203,"7tt_穿穿_每次付费__0313_841913319","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,2.83,0,2.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,529218,"RICH-末日-穿山甲-付费_游戏_混SSR2778_0313_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,3,9.84,0,9.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529219,"RICH-末日-穿山甲-付费_游戏_混SSR2762_0313_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,3,3.93,0,3.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,529228,"cxq-tt552幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,616.77,19,32.4615789,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529227,"cxq-tt338幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,24.76,0,24.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529224,"cxq-tt338幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,61.07,0,61.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529232,"cxq-tt552幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,1.55,0,1.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529231,"cxq-tt338幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,3,7.40,0,7.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,529368,"CJ-危机-每次03.14_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,7.65,0,7.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,529369,"VR-危机-每次03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,3,67.25,2,33.6250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529377,"JK-自动ROI排3K周日发力-0313_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,101.30,0,101.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21245,529412,"每次-周末冲刺-0314_18","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,33.57,0,33.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21245,529414,"每次-周末冲刺-0314_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,170.04,0,170.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21245,529416,"每次-周末冲刺-0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,205.53,0,205.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21246,529417,"穿-n1-周末付费冲-0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,3,62.60,0,62.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529418,"穿-n1-周末付费冲-0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,3,48.83,0,48.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529420,"穿-n1-周末付费冲-0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,3,49.93,0,49.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[6770,529462,"7tt_穿穿_每次付费__0314_72230811","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,27.15,0,27.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529461,"7tt_穿穿_每次付费__0314_72230800","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,60.08,0,60.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529464,"7tt_穿穿_每次付费__0314_72230827","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3382.48,10,338.2480000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529466,"7tt_穿穿_每次付费__0314_72230829","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2541.11,3,847.0366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529463,"7tt_穿穿_每次付费__0314_72230825","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,32.16,1,32.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529465,"7tt_穿穿_每次付费__0314_72230826","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,41.00,0,41.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529468,"7tt_穿穿_每次付费__0314_72230813","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1275.59,2,637.7950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529469,"7tt_穿穿_每次付费__0314_72230812","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,164.99,2,82.4950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529471,"7tt_穿穿_每次付费__0314_72230828","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,13.34,0,13.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529472,"7tt_穿穿_每次付费__0314_722308210","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529474,"每0314-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,33.83,0,33.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529482,"每0314-tt-xcztz-wyt-rk新_24","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,244.84,0,244.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529475,"每0314-tt-xcztz-wyt-rk新_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,41.80,0,41.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529483,"每0314-tt-xcztz-wyt-rk新_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,296.09,0,296.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529481,"每0314-tt-xcztz-wyt-rk新_25","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1.34,0,1.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529484,"每0314-tt-xcztz-wyt-rk新_30","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,12.79,0,12.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529485,"每0314-tt-xcztz-wyt-rk新_31","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,0.32,0,0.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529486,"每0314-tt-xcztz-wyt-rk新_36","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,321.38,0,321.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529496,"每0314-tt-xcztz-wyt-rk新_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,166.99,0,166.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529502,"每0314-tt-xcztz-wyt-rk新_23","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1.51,0,1.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529506,"每0314-tt-xcztz-wyt-rk新_29","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3.64,0,3.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529507,"每0314-tt-xcztz-wyt-rk新_32","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,280.60,0,280.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529510,"每0314-tt-xcztz-wyt-rk新_35","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3.50,0,3.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529556,"r-每次付费-幸存者挑战-穿山甲专场-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,7.95,0,7.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529558,"r-每次付费-幸存者挑战-穿山甲专场-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,9.04,0,9.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529559,"r-每次付费-幸存者挑战-穿山甲专场-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,15.14,0,15.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529560,"r-每次付费-幸存者挑战-穿山甲专场-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4.65,0,4.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529588,"jap-每次付费--3.13_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,84.21,0,84.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529606,"tw-每次付费--3.13_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,50.42,0,50.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529605,"hix-每次付费--3.13_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,5.80,0,5.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529608,"jap-每次付费--3.13_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4.31,0,4.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529609,"zhan-每次付费--3.13_20","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3.99,0,3.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529645,"0314-540-每次付费-幸_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3042.01,142,21.4226056,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529648,"0314-540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,601.80,2,300.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529643,"0314-540-每次付费-幸_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3374.57,60,56.2428333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529644,"0314-540-每次付费-幸_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3434.77,33,104.0839394,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529647,"0314-540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,4294.94,81,53.0239506,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529650,"0314-540-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,1410.25,12,117.5208333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529649,"0314-540-每次付费-幸_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3594.66,158,22.7510127,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529651,"0314-540-每次付费-幸_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,2926.56,53,55.2181132,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529653,"0314-540-每次付费-幸_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,3451.77,59,58.5045763,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529654,"0314-540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,945.63,0,945.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529657,"0314-540-每次付费-幸_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,641.13,2,320.5650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529667,"0314-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,25.86,0,25.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529673,"0314-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,3,247.68,1,247.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529719,"470-tv-每次付费拉环爆R爆量_3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,13.85,1,13.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529715,"470-tv-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,345.64,14,24.6885714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529716,"470-tv-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,851.44,16,53.2150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529718,"470-tv-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,688.95,25,27.5580000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529720,"470-tv-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,118.75,5,23.7500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529722,"470-tv-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,100.57,3,33.5233333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529721,"470-tv-每次付费拉环爆R爆量_3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2803.74,31,90.4432258,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529723,"470-tv-每次付费拉环爆R爆量_3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,137.01,5,27.4020000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529724,"470-tv-每次付费拉环爆R爆量_3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,38.90,0,38.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529725,"470-tv-每次付费拉环爆R爆量_3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,589.15,0,589.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529747,"295-blcb-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,114.76,3,38.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529749,"295-blcb-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,176.98,8,22.1225000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529748,"295-blcb-每次付费拉环爆R爆量_3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2803.74,18,155.7633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529754,"295-blcb-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,133.70,2,66.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529753,"295-blcb-每次付费拉环爆R爆量_3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1149.38,4,287.3450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529755,"295-blcb-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,213.23,8,26.6537500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529752,"295-blcb-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,8.18,1,8.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529847,"263-幸存者-MCFC0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2619.13,33,79.3675758,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529845,"263-幸存者-MCFC0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,422.61,8,52.8262500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529863,"263-幸存者-MCFC0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,171.92,1,171.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529864,"263-幸存者-MCFC0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,415.87,0,415.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529865,"263-幸存者-MCFC0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1586.14,27,58.7459259,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529910,"0315-204-穿山甲-每次付费-幸_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,17.04,0,17.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529909,"0315-204-穿山甲-每次付费-幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,63.17,0,63.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529918,"481IC-幸存危机-2756爆量爆R-3.5_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,287.28,0,287.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529914,"0315-204-穿山甲-每次付费-幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,95.97,4,23.9925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529913,"0315-204-穿山甲-每次付费-幸_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2991.81,59,50.7086441,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529916,"481IC-幸存危机-2756爆量爆R-3.5_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,5607.48,81,69.2281481,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529917,"0315-204-穿山甲-每次付费-幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,89.67,7,12.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529920,"0315-204-穿山甲-每次付费-幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,806.49,32,25.2028125,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529922,"0315-204-穿山甲-每次付费-幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,14.31,0,14.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529923,"0315-204-穿山甲-每次付费-幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,8.55,0,8.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529924,"0315-204-穿山甲-每次付费-幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,193.22,4,48.3050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529925,"0315-204-穿山甲-每次付费-幸_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,42.37,0,42.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529926,"0315-204-穿山甲-每次付费-幸_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,24.58,0,24.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529927,"0315-204-穿山甲-每次付费-幸_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,5.17,0,5.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529975,"990-幸存者-MCFC-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,9.48,0,9.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529977,"990-幸存者-MCFC-0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,288.92,5,57.7840000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529978,"990-幸存者-MCFC-0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.29,0,1.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529981,"990-幸存者-MCFC-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,5.06,0,5.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529979,"990-幸存者-MCFC-0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.67,0,0.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529980,"990-幸存者-MCFC-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3.95,0,3.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529976,"990-幸存者-MCFC-0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,792.00,15,52.8000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529983,"990-幸存者-MCFC-0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,616.76,26,23.7215385,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529982,"990-幸存者-MCFC-0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,546.02,6,91.0033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530038,"CJ-幸存危机-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,28.17,0,28.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530041,"CJ-幸存危机-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,3303.74,60,55.0623333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530039,"CJ-幸存危机-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,614.50,11,55.8636364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530050,"CJ-幸存危机-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,16.87,0,16.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530102,"每0315-tt-xcztz-wyt-rk_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,18.10,0,18.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530101,"每0315-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,15.23,0,15.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530104,"每0315-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3.78,0,3.7800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530109,"每0315-tt-xcztz-wyt-rk_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,5.76,0,5.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530100,"每0315-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.08,0,0.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530111,"每0315-tt-xcztz-wyt-rk_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,7.22,0,7.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530112,"每0315-tt-xcztz-wyt-rk_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,13.48,0,13.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530113,"每0315-tt-xcztz-wyt-rk_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,49.71,0,49.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530114,"每0315-tt-xcztz-wyt-rk_21","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,100.41,0,100.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530115,"每0315-tt-xcztz-wyt-rk_22","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.05,0,1.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530117,"每0315-tt-xcztz-wyt-rk_24","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,83.54,0,83.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530122,"每0315-tt-xcztz-wyt-rk_31","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,6.31,0,6.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530125,"每0315-tt-xcztz-wyt-rk_34","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,54.63,0,54.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530126,"每0315-tt-xcztz-wyt-rk_37","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,8.21,0,8.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530127,"每0315-tt-xcztz-wyt-rk_38","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.24,0,0.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530128,"每0315-tt-xcztz-wyt-rk_39","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.36,0,1.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530129,"每0315-tt-xcztz-wyt-rk_40","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,11.37,0,11.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530130,"每0315-tt-xcztz-wyt-rk_41","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,92.33,0,92.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530131,"每0315-tt-xcztz-wyt-rk_42","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.49,0,1.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530132,"每0315-tt-xcztz-wyt-rk_45","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2.00,0,2.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530133,"每0315-tt-xcztz-wyt-rk_46","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,469.30,4,117.3250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530134,"每0315-tt-xcztz-wyt-rk_47","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,13.72,0,13.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530135,"每0315-tt-xcztz-wyt-rk_48","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,302.18,1,302.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530139,"每0315-tt-xcztz-wyt-rk_52","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.23,0,0.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530140,"每0315-tt-xcztz-wyt-rk_53","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.49,0,0.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530141,"每0315-tt-xcztz-wyt-rk_54","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.41,0,0.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530145,"每0315-tt-xcztz-wyt-rk_58","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,21.41,0,21.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530146,"每0315-tt-xcztz-wyt-rk_59","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530147,"每0315-tt-xcztz-wyt-rk_60","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530148,"每0315-tt-xcztz-wyt-rk_62","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.22,0,0.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530149,"每0315-tt-xcztz-wyt-rk_61","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3.46,0,3.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530150,"每0315-tt-xcztz-wyt-rk_64","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,9.99,0,9.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530151,"每0315-tt-xcztz-wyt-rk_63","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,29.08,0,29.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530155,"每0315-tt-xcztz-wyt-rk_70","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,7.02,0,7.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530154,"每0315-tt-xcztz-wyt-rk_65","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,408.44,1,408.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530153,"每0315-tt-xcztz-wyt-rk_67","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,951.02,11,86.4563636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530157,"每0315-tt-xcztz-wyt-rk_66","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,265.58,3,88.5266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530156,"每0315-tt-xcztz-wyt-rk_69","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,8.07,0,8.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530158,"每0315-tt-xcztz-wyt-rk_71","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530159,"每0315-tt-xcztz-wyt-rk_72","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,275.30,2,137.6500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530168,"r3-每次付费-幸存者挑战优选位-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2336.53,65,35.9466154,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530173,"r3-每次付费-幸存者挑战优选位-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1658.99,14,118.4992857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530175,"i-每次付费-幸存者挑战只限男优先位-03.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,783.36,20,39.1680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530174,"r3-每次付费-幸存者挑战优选位-03.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1913.11,24,79.7129167,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530176,"i-每次付费-幸存者挑战只限男优先位-03.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1174.93,9,130.5477778,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530167,"r3-每次付费-幸存者挑战优选位-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,485.07,3,161.6900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530177,"i-每次付费-幸存者挑战只限男优先位-03.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,309.76,3,103.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530210,"i-每次付费-幸存者挑战只限男优先位-03.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,430.16,1,430.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530211,"i-每次付费-幸存者挑战只限男优先位-03.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,577.04,4,144.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530385,"ROI系数-幸存者挑战-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3107.77,112,27.7479464,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530386,"ROI系数-幸存者挑战-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3070.08,51,60.1976471,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530387,"ROI系数-幸存者挑战-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3015.56,96,31.4120833,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530388,"ROI系数-幸存者挑战-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1111.52,34,32.6917647,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530389,"ROI系数-幸存者挑战-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2983.30,81,36.8308642,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530514,"0315幸存者393z每次付费3k_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1844.31,1,1844.3100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530516,"0315幸存者393z每次付费3k_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1863.95,5,372.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530515,"0315幸存者393z每次付费3k_2731_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2252.57,2,1126.2850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530517,"0315幸存者393z每次付费3k_2746_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3237.84,38,85.2063158,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530518,"0315幸存者393z每次付费3k_2735_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1199.35,0,1199.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530524,"0315oyo393-roi系数-排除3k_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2885.28,52,55.4861538,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530531,"0315oyo393-roi系数-排除3k_2731_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1394.48,4,348.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530529,"0315oyo393-roi系数-排除3k_2522_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1046.06,12,87.1716667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530530,"0315oyo393-roi系数-排除3k_2762_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1331.56,12,110.9633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20635,530537,"每0315-tt-xcztz-wyt-yan_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,173.89,4,43.4725000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530540,"每0315-tt-xcztz-wyt-yan_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,314.88,5,62.9760000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530541,"每0315-tt-xcztz-wyt-yan_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,294.41,3,98.1366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530542,"每0315-tt-xcztz-wyt-yan_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,95.29,1,95.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530543,"每0315-tt-xcztz-wyt-yan_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,302.67,3,100.8900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530544,"每0315-tt-xcztz-wyt-yan_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,496.49,10,49.6490000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530545,"每0315-tt-xcztz-wyt-yan_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2486.77,40,62.1692500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530546,"每0315-tt-xcztz-wyt-yan_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.28,0,1.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530547,"每0315-tt-xcztz-wyt-yan_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.94,0,0.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530548,"每0315-tt-xcztz-wyt-yan_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,4.45,0,4.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530549,"每0315-tt-xcztz-wyt-yan_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.09,0,0.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530550,"每0315-tt-xcztz-wyt-yan_19","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2.71,0,2.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530551,"每0315-tt-xcztz-wyt-yan_20","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2.11,1,2.1100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530552,"每0315-tt-xcztz-wyt-yan_21","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,4.98,0,4.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,530671,"cxq-tt747幸存者-ROI系数0315_2738","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1968.98,62,31.7577419,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530672,"cxq-tt549幸存者-ROI系数0315_2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2855.54,85,33.5945882,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530673,"cxq-tt747幸存者-ROI系数0315_2332","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,739.36,4,184.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530674,"cxq-tt982幸存者-ROI系数0315_2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,150.86,2,75.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530675,"cxq-tt530幸存者-ROI系数0315_录2789","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,314.35,0,314.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530676,"cxq-tt530幸存者-ROI系数0315_录2793","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,261.52,0,261.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530678,"cxq-tt530幸存者-ROI系数0315_录2764","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,260.82,0,260.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530677,"cxq-tt549幸存者-ROI系数0315_2731","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,361.83,5,72.3660000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530679,"cxq-tt982幸存者-ROI系数0315_2746","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,337.08,4,84.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530680,"cxq-tt549幸存者-ROI系数0315_2781","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,437.21,4,109.3025000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530693,"0315-508幸存危机-每次付费竖版沉浸_2762_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,135.38,1,135.3800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530692,"0315-508幸存危机-每次付费竖版沉浸_2332_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,23.62,0,23.6200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530698,"0315-508幸存危机-每次付费竖版沉浸_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,681.82,7,97.4028571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530696,"0315-508幸存危机-每次付费竖版沉浸_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,161.82,1,161.8200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530721,"303-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,318.02,1,318.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530722,"306-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,61.94,3,20.6466667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530718,"0315-幸存者393z每次付费排除3k_2784_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,315.11,4,78.7775000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530713,"0315-幸存者393z每次付费排除3k_2783_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1101.04,4,275.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530720,"306-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,559.71,7,79.9585714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530716,"0315-幸存者393z每次付费排除3k_2786_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,49.52,0,49.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530724,"301-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,342.01,3,114.0033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530725,"301-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,316.00,3,105.3333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530723,"303-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1383.69,36,38.4358333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530726,"301-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,6.18,0,6.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530728,"306-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,95.06,1,95.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530729,"306-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,13.65,0,13.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530731,"306-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,8.09,0,8.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530732,"303-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2.37,0,2.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530733,"306-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,27.41,0,27.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530734,"303-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,303.88,3,101.2933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530736,"306-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2153.20,55,39.1490909,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530735,"303-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.21,0,1.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530730,"306-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,5.67,0,5.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530737,"306-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,367.57,8,45.9462500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530738,"303-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.83,0,1.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530739,"303-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,29.18,2,14.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530740,"303-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.41,0,1.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530741,"301-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,44.92,2,22.4600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530742,"301-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,438.56,6,73.0933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530743,"301-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,722.79,20,36.1395000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530744,"301-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,304.04,1,304.0400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530745,"301-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,4.43,0,4.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530746,"301-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2.79,0,2.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530747,"301-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,79.66,1,79.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530759,"303-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.02,0,1.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530779,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2332_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,648.74,19,34.1442105,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530781,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2731_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,12.56,0,12.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530780,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,34.53,0,34.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530782,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2751_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2.03,0,2.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530783,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2794_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,119.91,0,119.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530784,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2271_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,361.69,15,24.1126667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530786,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2484_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,234.18,17,13.7752941,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530785,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1692.29,57,29.6892982,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530787,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,155.94,0,155.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530788,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,4.05,0,4.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530799,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2622315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,956.08,8,119.5100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20540,530802,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2789315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,99.25,0,99.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530803,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2761315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,75.60,0,75.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530800,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2746315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,697.88,6,116.3133333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,530806,"0315FP-ROI系数-MG幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3269.14,10,326.9140000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530805,"0315FP-ROI系数-MG幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1537.46,20,76.8730000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530807,"0315FP-ROI系数-MG幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2333.97,7,333.4242857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530810,"0315FP-ROI系数-MG幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1061.06,2,530.5300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530809,"0315FP-ROI系数-MG幸_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1255.06,1,1255.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530844,"0315FP-ROI系数-MG幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1171.66,4,292.9150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530843,"0315FP-ROI系数-MG幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1840.60,6,306.7666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530846,"0315FP-ROI系数-MG幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2177.36,20,108.8680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530845,"0315FP-ROI系数-MG幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1762.77,5,352.5540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530908,"tw-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,970.04,5,194.0080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530913,"tw-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,107.09,0,107.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530914,"hix-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1105.87,8,138.2337500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530915,"zhan-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,389.75,0,389.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530912,"jap-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,31.46,0,31.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530910,"hix-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,418.61,2,209.3050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530911,"hix-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,388.89,2,194.4450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530907,"tw-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2713.20,32,84.7875000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530916,"zhan-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,230.98,0,230.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530917,"zhan-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,518.88,4,129.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530909,"jap-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2588.93,30,86.2976667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530920,"jap-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.39,0,1.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530918,"hix-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,8.43,0,8.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530919,"jap-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,285.03,1,285.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530921,"zhan-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,701.53,6,116.9216667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530925,"hix-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,29.89,0,29.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530926,"hix-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,38.19,0,38.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530923,"tw-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,156.00,1,156.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530922,"jap-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,13.37,0,13.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530927,"zhan-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,285.16,4,71.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530928,"zhan-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,995.17,7,142.1671429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530929,"tw-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,54.24,1,54.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530931,"FJ-幸存者-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,753.31,7,107.6157143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530933,"FW-幸存者-每次03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,26.64,0,26.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530930,"FJ-幸存者-每次03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,258.76,3,86.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530932,"FW-幸存者-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,719.29,5,143.8580000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530934,"FW-幸存者-每次03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,528.73,6,88.1216667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530935,"FW-幸存者-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,956.90,8,119.6125000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530937,"FJ-幸存者-每次03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,42.93,0,42.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530940,"FJ-幸存者-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,35.09,0,35.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530939,"FJ-幸存者-每次03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,5.68,0,5.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530938,"FW-幸存者-每次03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,374.85,1,374.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530943,"FW-幸存者-每次03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1606.62,15,107.1080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530948,"FJ-幸存者-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,164.54,0,164.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530947,"FJ-幸存者-每次03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,602.25,5,120.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530946,"FW-幸存者-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,413.23,0,413.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530949,"FJ-幸存者-每次03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,647.84,4,161.9600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530944,"FJ-幸存者-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,400.58,0,400.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530945,"FW-幸存者-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1025.53,11,93.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530942,"FW-幸存者-每次03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3303.74,39,84.7112821,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530941,"FW-幸存者-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,509.93,5,101.9860000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530964,"yo-roi系数-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2803.74,76,36.8913158,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530963,"ka-roi系数-周一-3.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1648.62,24,68.6925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530958,"yo-roi系数-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,110.73,0,110.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20754,530960,"yo-roi系数-周一-3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2803.74,40,70.0935000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530965,"yo-roi系数-周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,736.83,17,43.3429412,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530967,"yo-roi系数-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,365.61,6,60.9350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530969,"yo-roi系数-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,834.29,14,59.5921429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530973,"ka-roi系数-周一-3.15_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,630.42,10,63.0420000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530970,"ka-roi系数-周一-3.15_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,791.80,17,46.5764706,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530972,"ka-roi系数-周一-3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,407.12,2,203.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530971,"yo-roi系数-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,440.39,1,440.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20635,530999,"每0315-tt-xcztz-wyt-zhong_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,4.12,0,4.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530998,"每0315-tt-xcztz-wyt-zhong_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1302.96,14,93.0685714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531000,"每0315-tt-xcztz-wyt-zhong_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531001,"每0315-tt-xcztz-wyt-zhong_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,620.65,8,77.5812500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531002,"每0315-tt-xcztz-wyt-zhong_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3.62,1,3.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531006,"每0315-tt-xcztz-wyt-zhong_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,71.09,1,71.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531004,"每0315-tt-xcztz-wyt-zhong_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,672.76,13,51.7507692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531003,"每0315-tt-xcztz-wyt-zhong_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,921.25,3,307.0833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531005,"每0315-tt-xcztz-wyt-zhong_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,0.19,0,0.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531007,"每0315-tt-xcztz-wyt-zhong_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,22.40,0,22.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531008,"每0315-tt-xcztz-wyt-zhong_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,37.46,0,37.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531009,"每0315-tt-xcztz-wyt-zhong_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,3.99,0,3.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531010,"每0315-tt-xcztz-wyt-zhong_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1.51,0,1.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531011,"每0315-tt-xcztz-wyt-zhong_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,11.20,0,11.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21213,531012,"n96-ios-黎明血战-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,39.96,1,39.9600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531013,"n96-ios-黎明血战-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,63.84,2,31.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531014,"n96-ios-黎明血战-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,1659.45,33,50.2863636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531015,"n96-ios-黎明血战-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,53.39,1,53.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531016,"n96-ios-黎明血战-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,1534.32,11,139.4836364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531038,"bdna-非海马云-幸存者挑战-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2701.47,91,29.6864835,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531041,"bdna-非海马云-幸存者挑战-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2345.34,50,46.9068000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531187,"au-黎明血战--周一-3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,679.51,11,61.7736364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531189,"th-黎明血战--周一-3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,179.24,0,179.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531190,"au-黎明血战--周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,75.27,0,75.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531194,"th-黎明血战--周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,2508.92,43,58.3469767,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531188,"roi系数-0315-BY-幸存_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,305.27,10,30.5270000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20541,531195,"RICH-研究所-财神到我家-通投_混SSR2791_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,2183.63,7,311.9471429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531197,"RICH-研究所-财神到我家-通投_混SSR2797_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,88.80,0,88.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531199,"RICH-研究所-财神到我家-通投_混SSR2749_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,1143.12,1,1143.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531200,"RICH-研究所-财神到我家-通投_混SSR2789_0315_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,813.08,0,813.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531201,"RICH-研究所-财神到我家-通投_混SSR2783_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,1568.03,2,784.0150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531203,"05-幸存者-MCFC0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,433.57,7,61.9385714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531202,"05-幸存者-MCFC0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2803.74,51,54.9752941,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531205,"05-幸存者-MCFC0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2803.74,39,71.8907692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20541,531208,"RICH-研究所-财神到我家-通投_创意组8_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,1653.34,1,1653.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531211,"th-黎明血战--周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,49.77,0,49.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531212,"th-黎明血战--周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,390.00,1,390.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531207,"roi系数-0315-BY-幸存_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,559.87,8,69.9837500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531209,"roi系数-0315-BY-幸存_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,267.01,2,133.5050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20755,531215,"th-黎明血战--周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,386.82,2,193.4100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531214,"roi系数-0315-BY-幸存_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,508.28,2,254.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531213,"roi系数-0315-BY-幸存_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,50.73,0,50.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[21245,531210,"roi系数-0315-BY-幸存_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,784.04,5,156.8080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531206,"roi系数-0315-BY-幸存_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,529.12,1,529.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531216,"05-幸存者-MCFC0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,8.14,0,8.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,531217,"05-幸存者-MCFC0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,36.43,0,36.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20755,531227,"th-黎明血战--周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,106.53,2,53.2650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531229,"au-黎明血战--周一-3.15_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,17.58,0,17.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531230,"th-黎明血战--周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,637.41,3,212.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531231,"au-黎明血战--周一-3.15_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,418.74,0,418.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531232,"au-黎明血战--周一-3.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,476.02,1,476.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531233,"au-黎明血战--周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,69.94,0,69.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531234,"roi系数-0315-BY-幸存_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,304.39,4,76.0975000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20755,531252,"au-黎明血战--周一-3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,48.58,0,48.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531257,"th-黎明血战--周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,415.40,2,207.7000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531259,"au-黎明血战--周一-3.15_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,3,159.31,0,159.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531256,"05-幸存者-MCFC0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1073.08,20,53.6540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531258,"05-幸存者-MCFC0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,926.95,18,51.4972222,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531254,"05-幸存者-MCFC0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2.74,1,2.7400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531290,"0315幸存者393z每次付费排除3K_2797_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1202.22,3,400.7400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531289,"0315幸存者393z每次付费排除3K_2742_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1781.97,7,254.5671429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531291,"0315幸存者393z每次付费排除3K_2791_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1612.04,15,107.4693333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531292,"0315幸存者393z每次付费排除3K_2798_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1320.65,14,94.3321429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531287,"0315幸存者393z每次付费排除3K_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,5530.46,74,74.7359459,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531293,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1155.31,10,115.5310000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531294,"cxq-tt338幸存者-穿山甲每次付费-0315_30002797","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,93.94,0,93.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531298,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,395.33,7,56.4757143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531297,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2731","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,180.30,0,180.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531296,"cxq-tt338幸存者-穿山甲每次付费-0315_30002798","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,98.57,1,98.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531295,"cxq-tt338幸存者-穿山甲每次付费-0315_30002796","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1798.86,18,99.9366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20546,531300,"0315wtd末日研究所_2332_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,3440.76,28,122.8842857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531303,"0315wtd末日研究所_2730_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,890.32,1,890.3200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531302,"0315wtd末日研究所_2762_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,1949.39,11,177.2172727,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531304,"0315wtd末日研究所_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,2917.04,14,208.3600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531306,"263-幸存者-ROI-0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,763.58,22,34.7081818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531314,"263-幸存者-ROI-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2728.54,80,34.1067500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531307,"263-幸存者-ROI-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2910.18,58,50.1755172,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531315,"263-幸存者-ROI-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,236.45,2,118.2250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20546,531576,"0315ph648末日研究所_2800_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,236.37,1,236.3700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531578,"0315ph648末日研究所_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,7286.80,81,89.9604938,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531579,"0315ph648末日研究所_2783_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,93.76,0,93.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531580,"0315ph648末日研究所_2519_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,3,29.25,0,29.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531583,"263-幸存者-MCFC0315_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,330.96,0,330.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,531582,"263-幸存者-MCFC0315_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,470.23,2,235.1150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531585,"263-幸存者-MCFC0315_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1102.03,5,220.4060000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531581,"263-幸存者-MCFC0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,68.49,1,68.4900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531584,"263-幸存者-MCFC0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,2672.80,33,80.9939394,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531706,"0315-508幸存危机-每次付费_2800_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,3,742.06,1,742.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,531771,"r-自动ROI-幸存者挑战晚上七点-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,16.65,0,16.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21212,531772,"r-自动ROI-幸存者挑战晚上七点-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,4590.59,90,51.0065556,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531773,"r-自动ROI-幸存者挑战晚上七点-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,1305.40,6,217.5666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531774,"r-自动ROI-幸存者挑战晚上七点-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,6.50,0,6.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21212,531775,"r-自动ROI-幸存者挑战晚上七点-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,3,644.52,3,214.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[20540,529214,"RICH-末日-穿山甲-付费_通投_混SSR2778_0313_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,7,386.41,9,42.9344444,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529215,"RICH-末日-穿山甲-付费_春节DMP_混SSR2762_0313_7","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,7,3431.44,80,42.8930000,0,0.000000,0.00,0E-7,0.0946,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529216,"RICH-末日-穿山甲-付费_春节DMP_混SSR2746_0313_9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,7,6253.95,138,45.3184783,3,2084.650000,280.00,0.0447717,0.0860,0.02174,3,2,null,null,"DEEP_BID_DEFAULT"],[20540,529217,"RICH-末日-穿山甲-付费_通投_混SSR2762_0313_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,7,2767.96,68,40.7052941,0,0.000000,0.00,0E-7,0.1020,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529220,"RICH-末日-穿山甲-付费_通投_混SSR2746_0313_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,7,6139.13,139,44.1664029,1,6139.130000,30.00,0.0048867,0.0606,0.00719,1,2,null,null,"DEEP_BID_DEFAULT"],[20540,529221,"RICH-末日-穿山甲-付费_春节DMP_混SSR2778_0313_8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,7,767.90,15,51.1933333,0,0.000000,0.00,0E-7,0.1818,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529222,"RICH-末日-穿山甲-付费_游戏_混SSR2746_0313_6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,7,51.60,1,51.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21212,520745,"f-xy-末日研究所充值是大哥-03.08_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001294,1,7,2803.74,36,77.8816667,0,0.000000,0.00,0E-7,0.0750,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21212,523084,"f-jhff-末日研究所也想做大哥-03.09_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001294,1,7,1410.67,7,201.5242857,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21212,523086,"f-jhff-末日研究所也想做大哥-03.09_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001294,1,7,2800.42,54,51.8596296,0,0.000000,0.00,0E-7,0.0182,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21212,523088,"f-jhff-末日研究所也想做大哥-03.09_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001294,1,7,4660.83,111,41.9894595,1,4660.830000,98.00,0.0210263,0.0242,0.00901,0,2,null,null,"DEEP_BID_DEFAULT"],[21212,523089,"f-jhff-末日研究所也想做大哥-03.09_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001294,1,7,2803.74,120,23.3645000,0,0.000000,0.00,0E-7,0.0317,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21212,523092,"f-jhff-末日研究所也想做大哥-03.09_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001294,1,7,2803.74,115,24.3803478,0,0.000000,0.00,0E-7,0.0261,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,527902,"J9-付费-MRYJS-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001294,1,7,827.07,7,118.1528571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,527903,"J9-付费-MRYJS-0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001294,1,7,930.34,4,232.5850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,527904,"J9-付费-MRYJS-0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001294,1,7,2585.06,20,129.2530000,1,2585.060000,12.00,0.0046421,0.1429,0.05000,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,527905,"J9-付费-MRYJS-0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001294,1,7,2789.00,21,132.8095238,0,0.000000,0.00,0E-7,0.0476,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,508714,"7tt自建_自动ROI_幸__0227_844166028","2021-03-16 00:00:00","2021-02-27 00:00:00",10,1001379,1,7,983.90,19,51.7842105,1,983.900000,68.00,0.0691127,0.0833,0.05263,0,2,null,null,"ROI_PACING"],[6770,509135,"7tt自建_自动ROI_幸__0228_48331367","2021-03-16 00:00:00","2021-02-28 00:00:00",10,1001379,1,7,1138.94,29,39.2737931,0,0.000000,0.00,0E-7,0.0313,0.00000,0,2,null,null,"ROI_PACING"],[6770,509920,"7tt自建_自动ROI_幸__0301_12425141","2021-03-16 00:00:00","2021-03-01 00:00:00",10,1001379,1,7,89.23,1,89.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,522136,"7tt自建_自动ROI_幸__0309_36742130","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,100.99,1,100.9900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,524724,"7tt_y_幸_每次付费__0310_60236469","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,593.91,2,296.9550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524725,"7tt_y_幸_每次付费__0310_60236457","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,3244.80,56,57.9428571,3,1081.600000,410.00,0.1263560,0.1452,0.05357,0,2,null,null,"BID_PER_ACTION"],[6770,524726,"7tt_y_幸_每次付费__0310_60236445","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,463.24,3,154.4133333,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524727,"7tt_y_幸_每次付费__0310_60236458","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1863.29,22,84.6950000,1,1863.290000,30.00,0.0161006,0.0435,0.04545,0,2,null,null,"BID_PER_ACTION"],[6770,524728,"7tt_y_幸_每次付费__0310_60236444","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,3245.41,105,30.9086667,0,0.000000,0.00,0E-7,0.0727,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524729,"7tt_y_幸_每次付费__0310_60236456","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,696.55,6,116.0916667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524730,"7tt_y_幸_每次付费__0310_60236432","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,6876.31,255,26.9659216,2,3438.155000,72.00,0.0104707,0.0430,0.00784,0,2,null,null,"BID_PER_ACTION"],[6770,524731,"7tt_y_幸_每次付费__0310_60236443","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,3314.42,169,19.6119527,1,3314.420000,490.00,0.1478388,0.0351,0.00592,0,2,null,null,"BID_PER_ACTION"],[6770,524732,"7tt_y_幸_每次付费__0310_602364610","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,785.58,4,196.3950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524733,"7tt_y_幸_每次付费__0310_602364713","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,3031.08,52,58.2900000,2,1515.540000,60.00,0.0197949,0.3770,0.03846,1,2,null,null,"BID_PER_ACTION"],[6770,524734,"7tt_y_幸_每次付费__0310_60236410","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,7816.09,139,56.2308633,2,3908.045000,530.00,0.0678088,0.0890,0.01439,0,2,null,null,"BID_PER_ACTION"],[6770,524735,"7tt_y_幸_每次付费__0310_602364714","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,460.28,4,115.0700000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524738,"7tt_y_幸_每次付费__0310_60236431","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,517.49,7,73.9271429,1,517.490000,12.00,0.0231889,0.1250,0.14286,0,2,null,null,"BID_PER_ACTION"],[6770,524739,"7tt_y_幸_每次付费__0310_602364611","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,204.96,1,204.9600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524740,"7tt_y_幸_每次付费__0310_602364712","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,955.84,7,136.5485714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,525568,"7tt_529_幸存者挑战_自动ROI_3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,239.72,9,26.6355556,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525569,"7tt_529_幸存者挑战_自动ROI_3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,474.12,21,22.5771429,0,0.000000,0.00,0E-7,0.0476,0.00000,0,2,null,null,"ROI_PACING"],[6770,525570,"7tt_529_幸存者挑战_自动ROI_3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,621.32,23,27.0139130,0,0.000000,0.00,0E-7,0.0870,0.00000,0,2,null,null,"ROI_PACING"],[6770,525571,"7tt_529_幸存者挑战_自动ROI_3.10_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,208.75,3,69.5833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525572,"7tt_529_幸存者挑战_自动ROI_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,11684.25,486,24.0416667,5,2336.850000,400.00,0.0342341,0.0682,0.01029,4,2,null,null,"ROI_PACING"],[6770,525573,"7tt_529_幸存者挑战_自动ROI_3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,17.26,1,17.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525576,"7tt_529_幸存者挑战_自动ROI_3.10_9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,174.28,1,174.2800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525577,"7tt_498_幸存者挑战_自动ROI_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,13725.15,420,32.6789286,9,1525.016667,734.00,0.0534785,0.0936,0.02143,5,2,null,null,"ROI_PACING"],[6770,525582,"7tt_529_幸存者挑战_自动ROI_3.10_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1013.29,12,84.4408333,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"ROI_PACING"],[6770,525584,"7tt_498_幸存者挑战_自动ROI_3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,41.23,1,41.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,525585,"7tt_498_幸存者挑战_自动ROI_3.10_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,118.43,1,118.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,526366,"7tt_y_幸_每次付费__0311_64275001","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,634.76,6,105.7933333,1,634.760000,30.00,0.0472620,0.1250,0.16667,0,2,null,null,"BID_PER_ACTION"],[6770,526367,"7tt_y_幸_每次付费__0311_64275024","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3297.26,135,24.4241481,1,3297.260000,12.00,0.0036394,0.0584,0.00741,0,2,null,null,"BID_PER_ACTION"],[6770,526368,"7tt_y_幸_每次付费__0311_64275025","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3150.03,85,37.0591765,1,3150.030000,30.00,0.0095237,0.0349,0.01176,0,2,null,null,"BID_PER_ACTION"],[6770,526369,"7tt_y_幸_每次付费__0311_64275013","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,973.56,26,37.4446154,0,0.000000,0.00,0E-7,0.0714,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526370,"7tt_y_幸_每次付费__0311_64275049","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,275.12,6,45.8533333,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526373,"7tt_y_幸_每次付费__0311_64275038","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,636.19,6,106.0316667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526375,"7tt_y_幸_每次付费__0311_642750510","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,510.79,4,127.6975000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526379,"7tt_y_幸_每次付费__0311_642750714","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3276.35,108,30.3365741,0,0.000000,0.00,0E-7,0.0442,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526380,"7tt_y_幸_每次付费__0311_64273300","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,261.17,2,130.5850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527390,"7tt_529_幸存者挑战_每次付费3.12_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,599.06,2,299.5300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527391,"7tt_498_幸存者挑战_每次付费3.12_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,626.77,1,626.7700000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527392,"7tt_529_幸存者挑战_每次付费3.12_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3264.61,29,112.5727586,0,0.000000,0.00,0E-7,0.1034,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527395,"7tt_529_幸存者挑战_每次付费3.12_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,855.37,9,95.0411111,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527396,"7tt_498_幸存者挑战_每次付费3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,439.23,1,439.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527397,"7tt_529_幸存者挑战_每次付费3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3307.04,42,78.7390476,1,3307.040000,188.00,0.0568484,0.0476,0.02381,0,2,null,null,"BID_PER_ACTION"],[6770,527398,"7tt_529_幸存者挑战_每次付费3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,500.81,4,125.2025000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527399,"7tt_529_幸存者挑战_每次付费3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,886.33,11,80.5754545,0,0.000000,0.00,0E-7,0.1818,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527400,"7tt_529_幸存者挑战_每次付费3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3303.14,42,78.6461905,0,0.000000,0.00,0E-7,0.0698,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527401,"7tt_529_幸存者挑战_每次付费3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1219.59,12,101.6325000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527402,"7tt_529_幸存者挑战_每次付费3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,328.82,3,109.6066667,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527404,"7tt_498_幸存者挑战_每次付费3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,573.68,1,573.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527405,"7tt_498_幸存者挑战_每次付费3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3327.00,23,144.6521739,0,0.000000,0.00,0E-7,0.1739,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527406,"7tt_498_幸存者挑战_每次付费3.12_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,628.98,3,209.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527407,"7tt_498_幸存者挑战_每次付费3.12_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,891.23,3,297.0766667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527408,"7tt_498_幸存者挑战_每次付费3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,433.02,3,144.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527409,"7tt_498_幸存者挑战_每次付费3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3327.00,40,83.1750000,2,1663.500000,90.00,0.0270514,0.0732,0.05000,0,2,null,null,"BID_PER_ACTION"],[6770,527938,"7tt_y_幸_每次付费__0312_16294735","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,202.63,2,101.3150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527940,"7tt_y_幸_每次付费__0312_16294724","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,408.38,3,136.1266667,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527952,"7tt_y_幸_每次付费__0312_162947717","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,634.88,2,317.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527954,"7tt_y_幸_每次付费__0312_162947819","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1642.17,39,42.1069231,3,547.390000,164.00,0.0998679,0.1190,0.07692,1,2,null,null,"BID_PER_ACTION"],[6770,527955,"7tt_y_幸_每次付费__0312_162947920","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3276.98,125,26.2158400,0,0.000000,0.00,0E-7,0.0155,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527956,"7tt_y_幸_每次付费__0312_162948021","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3253.03,130,25.0233077,2,1626.515000,42.00,0.0129110,0.0821,0.01538,0,2,null,null,"BID_PER_ACTION"],[6770,527957,"7tt_y_幸_每次付费__0312_162948122","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,485.11,1,485.1100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527958,"7tt_y_幸_每次付费__0312_162948223","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,91.18,2,45.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527959,"7tt_y_幸_每次付费__0312_162948224","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3190.20,38,83.9526316,2,1595.100000,60.00,0.0188076,0.1750,0.05263,0,2,null,null,"BID_PER_ACTION"],[6770,527960,"7tt_y_幸_每次付费__0312_162948325","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,6121.37,87,70.3605747,4,1530.342500,378.00,0.0617509,0.1596,0.04598,0,2,null,null,"BID_PER_ACTION"],[6770,527961,"7tt_y_幸_每次付费__0312_162948426","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,241.53,5,48.3060000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527962,"7tt_y_幸_每次付费__0312_162948527","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,17.16,1,17.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527963,"7tt_y_幸_每次付费__0312_162948628","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,156.81,1,156.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527964,"7tt_y_幸_每次付费__0312_162948629","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,29.52,1,29.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528066,"7tt_幸_自动ROI__0312_88256792","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,601.46,11,54.6781818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528068,"7tt_幸_自动ROI__0312_882568211","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,52.85,2,26.4250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528077,"7tt_幸_自动ROI__0312_882568417","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,50.26,1,50.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528079,"7tt_幸_自动ROI__0312_882568520","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,119.15,3,39.7166667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[6770,528774,"7tt_穿穿_每次付费__0312_02259785","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,49.74,3,16.5800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528778,"7tt_穿穿_每次付费__0312_02259786","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3844.38,17,226.1400000,0,0.000000,0.00,0E-7,0.2222,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528780,"7tt_穿穿_每次付费__0312_02259760","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2977.51,6,496.2516667,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528783,"7tt_穿穿_每次付费__0312_022597911","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,556.97,4,139.2425000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528785,"7tt_穿穿_每次付费__0312_02259798","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,153.98,3,51.3266667,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528788,"7tt_穿穿_每次付费__0312_022598014","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2395.66,6,399.2766667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528791,"7tt_穿穿_每次付费__0312_022598017","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,6805.07,244,27.8896311,6,1134.178333,370.00,0.0543712,0.0839,0.02459,3,2,null,null,"BID_PER_ACTION"],[6770,528792,"7tt_穿穿_每次付费__0312_022598118","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,16.03,0,16.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528795,"7tt_穿穿_每次付费__0312_022598121","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3109.05,5,621.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528797,"7tt_穿穿_每次付费__0312_022598223","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3194.87,6,532.4783333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528800,"7tt_穿穿_每次付费__0312_022598226","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3014.51,3,1004.8366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528859,"7tt_498_穿哥_每次付费_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2072.06,59,35.1196610,0,0.000000,0.00,0E-7,0.1579,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528860,"7tt_498_穿哥_每次付费_3.12_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,13.29,0,13.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528861,"7tt_498_穿哥_每次付费_3.12_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,552.32,13,42.4861538,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528862,"7tt_498_穿哥_每次付费_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,6394.47,168,38.0623214,2,3197.235000,322.00,0.0503560,0.1489,0.01190,2,2,null,null,"BID_PER_ACTION"],[6770,528863,"7tt_498_穿哥_每次付费_3.12_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,209.21,11,19.0190909,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528864,"7tt_498_穿哥_每次付费_3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,308.50,9,34.2777778,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528865,"7tt_498_穿哥_每次付费_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,26.24,2,13.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528868,"7tt_498_穿哥_每次付费_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,316.21,12,26.3508333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529171,"7tt_穿穿_每次付费__0313_84191313","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,277.75,3,92.5833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529187,"7tt_穿穿_每次付费__0313_84191301","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2452.32,1,2452.3200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529192,"7tt_穿穿_每次付费__0313_84191318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,4.85,1,4.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529198,"7tt_穿穿_每次付费__0313_841913315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2718.67,8,339.8337500,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529201,"7tt_穿穿_每次付费__0313_841913317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1739.88,1,1739.8800000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529202,"7tt_穿穿_每次付费__0313_841913318","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,23.21,1,23.2100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529467,"7tt_穿穿_每次付费__0314_72230824","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,30.03,1,30.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529470,"7tt_穿穿_每次付费__0314_722308311","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1064.14,1,1064.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,523415,"3.9-幸存者MG540-每次付费-充值天龙八部_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,5079.14,69,73.6107246,1,5079.140000,2288.00,0.4504700,0.0417,0.01449,0,2,null,null,"BID_PER_ACTION"],[6867,523416,"3.9-幸存者MG540-每次付费-充值天龙八部_14","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,6833.48,122,56.0121311,3,2277.826667,2946.00,0.4311127,0.0894,0.02459,0,2,null,null,"BID_PER_ACTION"],[6867,523417,"3.9-幸存者MG540-每次付费-充值天龙八部_7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2851.71,25,114.0684000,0,0.000000,0.00,0E-7,0.1852,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523418,"3.9-幸存者MG540-每次付费-充值天龙八部_16","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1602.60,10,160.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523420,"3.9-幸存者MG540-每次付费-充值天龙八部_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,605.52,1,605.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523421,"3.9-幸存者MG540-每次付费-充值天龙八部_10","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1201.63,7,171.6614286,1,1201.630000,110.00,0.0915423,0.1429,0.14286,0,2,null,null,"BID_PER_ACTION"],[6867,523422,"3.9-幸存者MG540-每次付费-充值天龙八部_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,997.79,2,498.8950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523423,"3.9-幸存者MG540-每次付费-充值天龙八部_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1315.23,14,93.9450000,0,0.000000,0.00,0E-7,0.2143,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523424,"3.9-幸存者MG540-每次付费-充值天龙八部_9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1337.02,1,1337.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523425,"3.9-幸存者MG540-每次付费-充值天龙八部_12","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,982.51,8,122.8137500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[6867,523426,"3.9-幸存者MG540-每次付费-充值天龙八部_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,7468.57,69,108.2401449,2,3734.285000,60.00,0.0080337,0.0725,0.02899,0,2,null,null,"BID_PER_ACTION"],[6867,523427,"3.9-幸存者MG540-每次付费-充值天龙八部_6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,969.49,10,96.9490000,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523428,"3.9-幸存者MG540-每次付费-充值天龙八部_11","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2137.97,18,118.7761111,0,0.000000,0.00,0E-7,0.1053,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523429,"3.9-幸存者MG540-每次付费-充值天龙八部_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2044.46,11,185.8600000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523430,"3.9-幸存者MG540-每次付费-充值天龙八部_13","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,6933.58,125,55.4686400,1,6933.580000,30.00,0.0043268,0.0547,0.00800,0,2,null,null,"BID_PER_ACTION"],[6867,523431,"3.9-幸存者MG540-每次付费-充值天龙八部_15","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,798.16,2,399.0800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523432,"3.9-幸存者MG540-每次付费-充值天龙八部_17","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,914.63,5,182.9260000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,523433,"3.9-幸存者MG540-每次付费-充值天龙八部_18","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,695.72,2,347.8600000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525251,"0310-每次付费-幸存者MG550_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,970.42,16,60.6512500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525252,"0310-每次付费-幸存者MG550_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,52.40,0,52.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,525253,"0310-每次付费-幸存者MG550_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,307.98,3,102.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525254,"0310-每次付费-幸存者MG550_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2851.34,88,32.4015909,0,0.000000,0.00,0E-7,0.0111,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525255,"0310-每次付费-幸存者MG550_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,558.10,3,186.0333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525256,"0310-每次付费-幸存者MG550_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2503.75,66,37.9356061,1,2503.750000,1362.00,0.5439840,0.0423,0.01515,0,2,null,null,"BID_PER_ACTION"],[6867,525258,"0310-每次付费-幸存者MG550_10","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1123.08,7,160.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525259,"0310-每次付费-幸存者MG550_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,292.35,7,41.7642857,0,0.000000,0.00,0E-7,0.2857,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525260,"0310-每次付费-幸存者MG550_9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,38.39,1,38.3900000,1,38.390000,328.00,8.5438916,1.0000,1.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525261,"0310-每次付费-幸存者MG550_11","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2851.71,28,101.8467857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525262,"0310-每次付费-幸存者MG550_12","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,701.43,7,100.2042857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525263,"0310-每次付费-幸存者MG550_13","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2622.71,18,145.7061111,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525264,"0310-每次付费-幸存者MG550_14","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,185.80,4,46.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525265,"0310-每次付费-幸存者MG550_15","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,17.82,1,17.8200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525267,"0310-每次付费-幸存者MG550_17","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,355.60,2,177.8000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,525268,"0310-每次付费-幸存者MG550_18","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,250.50,2,125.2500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526751,"0311-tve-每次付费-幸_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,796.14,8,99.5175000,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526752,"0311-tve-每次付费-幸_20","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,13746.97,373,36.8551475,5,2749.394000,382.00,0.0277879,0.0913,0.01340,4,2,null,null,"BID_PER_ACTION"],[6867,526754,"0311-tve-每次付费-幸_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,522.25,5,104.4500000,1,522.250000,12.00,0.0229775,0.2000,0.20000,0,2,null,null,"BID_PER_ACTION"],[6867,526756,"0311-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,8519.49,120,70.9957500,3,2839.830000,102.00,0.0119725,0.0865,0.02500,0,2,null,null,"BID_PER_ACTION"],[6867,526757,"0311-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4751.26,60,79.1876667,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526758,"0311-tve-每次付费-幸_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1808.28,25,72.3312000,0,0.000000,0.00,0E-7,0.1500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526760,"0311-tve-每次付费-幸_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,94.81,1,94.8100000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526763,"0311-tve-每次付费-幸_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1128.69,6,188.1150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526766,"0311-tve-每次付费-幸_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1143.96,6,190.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526767,"0311-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,963.95,11,87.6318182,0,0.000000,0.00,0E-7,0.3636,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526768,"0311-tve-每次付费-幸_17","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1296.97,8,162.1212500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526769,"0311-tve-每次付费-幸_18","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,9242.05,144,64.1809028,4,2310.512500,2340.00,0.2531906,0.1181,0.02778,1,2,null,null,"BID_PER_ACTION"],[6867,526770,"0311-tve-每次付费-幸_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3159.81,66,47.8759091,1,3159.810000,98.00,0.0310145,0.1087,0.01515,1,2,null,null,"BID_PER_ACTION"],[6867,527472,"0312540-每次付费-幸_24","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1686.16,24,70.2566667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527473,"0312540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2101.34,44,47.7577273,1,2101.340000,188.00,0.0894667,0.0500,0.02273,1,2,null,null,"BID_PER_ACTION"],[6867,527475,"0312540-每次付费-幸_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,6670.17,73,91.3721918,1,6670.170000,42.00,0.0062967,0.0541,0.01370,0,2,null,null,"DEEP_BID_DEFAULT"],[6867,527478,"0312540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1242.21,9,138.0233333,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527479,"0312540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1526.71,10,152.6710000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527481,"0312540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1082.86,4,270.7150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527483,"0312540-每次付费-幸_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,17665.87,279,63.3185305,6,2944.311667,5578.00,0.3157501,0.0538,0.02151,3,2,null,null,"BID_PER_ACTION"],[6867,527484,"0312540-每次付费-幸_11","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,190.52,2,95.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527485,"0312540-每次付费-幸_12","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1628.16,20,81.4080000,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527487,"0312540-每次付费-幸_14","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,798.99,3,266.3300000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527488,"0312540-每次付费-幸_16","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,689.36,2,344.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527489,"0312540-每次付费-幸_17","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1077.50,5,215.5000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527491,"0312540-每次付费-幸_19","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3379.10,43,78.5837209,0,0.000000,0.00,0E-7,0.0222,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527492,"0312540-每次付费-幸_20","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1191.47,14,85.1050000,1,1191.470000,30.00,0.0251790,0.1429,0.07143,1,2,null,null,"BID_PER_ACTION"],[6867,527493,"0312540-每次付费-幸_21","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2871.31,116,24.7526724,0,0.000000,0.00,0E-7,0.0417,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527495,"0312540-每次付费-幸_23","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,393.26,2,196.6300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527496,"0312540-每次付费-幸_25","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1135.38,4,283.8450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,527497,"0312540-每次付费-幸_26","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,487.71,9,54.1900000,1,487.710000,30.00,0.0615120,0.2857,0.11111,0,2,null,null,"BID_PER_ACTION"],[6867,527498,"0312540-每次付费-幸_27","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,875.52,1,875.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,529659,"0314-tve-每次付费-幸_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4522.57,47,96.2248936,2,2261.285000,42.00,0.0092868,0.1667,0.04255,2,0,null,null,"BID_PER_ACTION"],[6867,529660,"0314-tve-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2054.95,43,47.7895349,0,0.000000,0.00,0E-7,0.0909,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529661,"0314-tve-每次付费-幸_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1641.12,17,96.5364706,0,0.000000,0.00,0E-7,0.0588,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529662,"0314-tve-每次付费-幸_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1691.61,17,99.5064706,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529663,"0314-tve-每次付费-幸_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1702.45,17,100.1441176,2,851.225000,212.00,0.1245264,0.3529,0.11765,1,0,null,null,"BID_PER_ACTION"],[6867,529664,"0314-tve-每次付费-幸_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,5517.03,70,78.8147143,1,5517.030000,30.00,0.0054377,0.1111,0.01429,0,0,null,null,"BID_PER_ACTION"],[6867,529665,"0314-tve-每次付费-幸_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,832.97,8,104.1212500,0,0.000000,0.00,0E-7,0.1250,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529666,"0314-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,187.94,4,46.9850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529668,"0314-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1035.91,13,79.6853846,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529669,"0314-tve-每次付费-幸_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,7948.23,95,83.6655789,1,7948.230000,102.00,0.0128330,0.0702,0.01053,1,0,null,null,"BID_PER_ACTION"],[6867,529670,"0314-tve-每次付费-幸_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2632.16,27,97.4874074,0,0.000000,0.00,0E-7,0.0370,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529671,"0314-tve-每次付费-幸_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2616.41,32,81.7628125,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529672,"0314-tve-每次付费-幸_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,5607.30,118,47.5194915,2,2803.650000,24.00,0.0042801,0.0429,0.01695,0,0,null,null,"BID_PER_ACTION"],[6867,529674,"0314-tve-每次付费-幸_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2762.90,24,115.1208333,1,2762.900000,60.00,0.0217163,0.0833,0.04167,1,0,null,null,"BID_PER_ACTION"],[20451,499074,"306-幸存者-ROI02.21_5","2021-03-16 00:00:00","2021-02-21 00:00:00",10,1001379,1,7,3360.27,157,21.4029936,1,3360.270000,12.00,0.0035711,0.0000,0.00637,1,2,null,null,"ROI_PACING"],[20451,521070,"FJ-幸存者-每次03.08-9","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,0.01,2,0.0050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521105,"306-幸存者-ROI系03.08-2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,9091.58,206,44.1338835,7,1298.797143,1662.00,0.1828065,0.1408,0.03398,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521106,"303-幸存者-ROI系03.08-3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,713.36,10,71.3360000,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521110,"303-幸存者-ROI系03.08-4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,592.98,9,65.8866667,1,592.980000,42.00,0.0708287,0.2000,0.11111,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521111,"303-幸存者-ROI系03.08-2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,525.49,4,131.3725000,1,525.490000,12.00,0.0228358,0.5000,0.25000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521112,"301-幸存者-ROI系03.08-2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,402.42,11,36.5836364,0,0.000000,0.00,0E-7,0.1333,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,521128,"301-幸存者-ROI系03.08-1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,302.35,4,75.5875000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,521132,"301-幸存者-ROI系03.08-6","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,294.75,2,147.3750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,521136,"306-幸存者-ROI系03.08-0","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,337.39,4,84.3475000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521890,"FJ-幸存者-每次03.09-1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,558.40,3,186.1333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521891,"FJ-幸存者-每次03.09-2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1065.38,6,177.5633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521892,"FJ-幸存者-每次03.09-4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1342.11,16,83.8818750,0,0.000000,0.00,0E-7,0.1579,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521893,"FJ-幸存者-每次03.09-6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3020.56,47,64.2672340,1,3020.560000,30.00,0.0099319,0.1852,0.02128,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521894,"FJ-幸存者-每次03.09-7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1723.42,18,95.7455556,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521895,"FJ-幸存者-每次03.09-9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1173.87,10,117.3870000,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521896,"FW-幸存者-每次03.09-1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,6041.12,91,66.3859341,1,6041.120000,12.00,0.0019864,0.0842,0.01099,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521897,"FW-幸存者-每次03.09-2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,362.59,0,362.5900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521898,"FW-幸存者-每次03.09-4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,259.17,2,129.5850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521900,"FW-幸存者-每次03.09-7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,487.69,2,243.8450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521903,"FJ-幸存者-每次03.09-3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,14450.03,185,78.1082703,7,2064.290000,764.00,0.0528719,0.1302,0.03784,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521904,"FJ-幸存者-每次03.09-0","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,170.16,0,170.1600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521907,"FW-幸存者-每次03.09-0","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,409.35,2,204.6750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521908,"FW-幸存者-每次03.09-3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,429.96,0,429.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521909,"FJ-幸存者-每次03.09-8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2327.12,35,66.4891429,1,2327.120000,98.00,0.0421121,0.1707,0.02857,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521910,"FW-幸存者-每次03.09-8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3020.56,69,43.7762319,1,3020.560000,12.00,0.0039728,0.1053,0.01449,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,522251,"303搜-幸存者-ROI系03.09-3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3360.27,90,37.3363333,0,0.000000,0.00,0E-7,0.0426,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,522253,"303搜-幸存者-ROI系03.09-5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3360.27,70,48.0038571,0,0.000000,0.00,0E-7,0.0482,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,522254,"303搜-幸存者-ROI系03.09-7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,442.63,5,88.5260000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,522255,"303搜-幸存者-ROI系03.09-2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1035.81,12,86.3175000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,522256,"303搜-幸存者-ROI系03.09-6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,16882.58,404,41.7885644,7,2411.797143,660.00,0.0390936,0.1318,0.01733,1,2,null,null,"ROI_COEFFICIENT"],[20451,522258,"303搜-幸存者-ROI系03.09-4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,508.19,6,84.6983333,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,522259,"303搜-幸存者-ROI系03.09-0","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,15564.90,172,90.4936047,7,2223.557143,1562.00,0.1003540,0.1854,0.04070,3,2,null,null,"ROI_COEFFICIENT"],[20451,522263,"303搜-幸存者-ROI系03.09-1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,20926.03,366,57.1749454,7,2989.432857,13754.00,0.6572675,0.1058,0.01913,0,2,null,null,"ROI_COEFFICIENT"],[20451,522265,"303搜-幸存者-ROI系03.09-9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,454.70,5,90.9400000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,522266,"303搜-幸存者-ROI系03.09-8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,524.84,5,104.9680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,522473,"306搜-幸存者-ROI系03.09-5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,222.29,2,111.1450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,522474,"306搜-幸存者-ROI系03.09-4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,346.81,3,115.6033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,522478,"306搜-幸存者-ROI系03.09-2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,954.51,27,35.3522222,0,0.000000,0.00,0E-7,0.1613,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,522479,"306搜-幸存者-ROI系03.09-3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,519.60,10,51.9600000,2,259.800000,60.00,0.1154734,0.2857,0.20000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,522480,"306搜-幸存者-ROI系03.09-8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,42.30,0,42.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,524921,"306搜-幸存者-ROI系03.10-6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,214.42,6,35.7366667,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,524922,"306搜-幸存者-ROI系03.10-7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1964.53,36,54.5702778,0,0.000000,0.00,0E-7,0.2340,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,524923,"306搜-幸存者-ROI系03.10-1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1290.33,19,67.9121053,0,0.000000,0.00,0E-7,0.0870,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,524925,"306搜-幸存者-ROI系03.10-3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1067.45,23,46.4108696,1,1067.450000,30.00,0.0281044,0.1481,0.04348,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,524926,"306搜-幸存者-ROI系03.10-4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,408.42,4,102.1050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,524927,"306搜-幸存者-ROI系03.10-0","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,954.88,32,29.8400000,1,954.880000,184.00,0.1926944,0.1563,0.03125,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,524928,"306搜-幸存者-ROI系03.10-9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1900.35,52,36.5451923,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,524929,"306搜-幸存者-ROI系03.10-8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,359.80,3,119.9333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,524930,"306搜-幸存者-ROI系03.10-2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,354.54,4,88.6350000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,526857,"306-幸存者-ROI系03.11-1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,504.73,4,126.1825000,1,504.730000,158.00,0.3130387,0.2500,0.25000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526858,"301-幸存者-ROI系03.11-1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,387.60,4,96.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526859,"303-幸存者-ROI系03.11-1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,342.82,4,85.7050000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526866,"306-幸存者-ROI系03.11-5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4357.27,164,26.5687195,3,1452.423333,218.00,0.0500313,0.0888,0.01829,0,2,null,null,"ROI_COEFFICIENT"],[20451,526867,"306-幸存者-ROI系03.11-3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1076.47,31,34.7248387,0,0.000000,0.00,0E-7,0.1212,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526868,"306-幸存者-ROI系03.11-2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,65768.66,1611,40.8247424,37,1777.531351,9674.00,0.1470913,0.0669,0.02297,10,2,null,null,"ROI_COEFFICIENT"],[20451,526869,"306-幸存者-ROI系03.11-4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3342.76,147,22.7398639,0,0.000000,0.00,0E-7,0.0596,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526870,"306-幸存者-ROI系03.11-6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,401.32,6,66.8866667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526871,"303-幸存者-ROI系03.11-2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2400.43,55,43.6441818,0,0.000000,0.00,0E-7,0.0339,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526872,"303-幸存者-ROI系03.11-3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3900.09,71,54.9308451,1,3900.090000,196.00,0.0502553,0.1053,0.01408,0,2,null,null,"ROI_COEFFICIENT"],[20451,526873,"303-幸存者-ROI系03.11-4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3360.27,136,24.7078676,0,0.000000,0.00,0E-7,0.0616,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526874,"303-幸存者-ROI系03.11-5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3187.79,132,24.1499242,0,0.000000,0.00,0E-7,0.0903,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526875,"303-幸存者-ROI系03.11-6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,299.74,3,99.9133333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526876,"301-幸存者-ROI系03.11-2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,9825.51,237,41.4578481,3,3275.170000,388.00,0.0394890,0.0615,0.01266,0,2,null,null,"ROI_COEFFICIENT"],[20451,526877,"301-幸存者-ROI系03.11-3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2981.39,17,175.3758824,1,2981.390000,60.00,0.0201248,0.0588,0.05882,0,2,null,null,"ROI_COEFFICIENT"],[20451,526878,"301-幸存者-ROI系03.11-4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,31669.07,1597,19.8303507,17,1862.886471,1500.00,0.0473648,0.0683,0.01064,10,2,null,null,"ROI_COEFFICIENT"],[20451,526879,"301-幸存者-ROI系03.11-5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,469.09,5,93.8180000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,526880,"301-幸存者-ROI系03.11-6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,298.36,3,99.4533333,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,527348,"EZ-财神到-每次03.12-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,988.34,9,109.8155556,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527349,"EZ-财神到-每次03.12-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2437.14,11,221.5581818,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527351,"SR-财神到-每次03.12-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,455.93,3,151.9766667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527353,"PB-财神到-每次03.12-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,195.13,1,195.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527354,"PB-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,313.27,4,78.3175000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527355,"EZ-财神到-每次03.12-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3303.74,24,137.6558333,1,3303.740000,48.00,0.0145290,0.1923,0.04167,0,2,null,null,"BID_PER_ACTION"],[20451,527356,"EZ-财神到-每次03.12-9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,7511.38,147,51.0978231,2,3755.690000,298.00,0.0396731,0.0733,0.01361,0,2,null,null,"BID_PER_ACTION"],[20451,527357,"SR-财神到-每次03.12-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,528.44,6,88.0733333,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527358,"EZ-财神到-每次03.12-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,8093.03,76,106.4872368,4,2023.257500,502.00,0.0620287,0.0921,0.05263,0,2,null,null,"BID_PER_ACTION"],[20451,527359,"EZ-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4443.46,65,68.3609231,1,4443.460000,72.00,0.0162036,0.0149,0.01538,1,2,null,null,"BID_PER_ACTION"],[20451,527360,"EZ-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,192.09,3,64.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527361,"SR-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3290.64,31,106.1496774,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527362,"EZ-财神到-每次03.12-8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2018.72,24,84.1133333,0,0.000000,0.00,0E-7,0.0800,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527363,"EZ-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3303.74,26,127.0669231,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527364,"EZ-财神到-每次03.12-0","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3303.74,62,53.2861290,0,0.000000,0.00,0E-7,0.0161,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527365,"SR-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,22224.59,348,63.8637644,9,2469.398889,16610.00,0.7473704,0.1174,0.02586,5,2,null,null,"BID_PER_ACTION"],[20451,527366,"SR-财神到-每次03.12-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,13392.17,198,67.6372222,6,2232.028333,1328.00,0.0991624,0.0606,0.03030,6,2,null,null,"BID_PER_ACTION"],[20451,527367,"SR-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,15042.81,159,94.6088679,6,2507.135000,12076.00,0.8027755,0.1594,0.03774,6,2,null,null,"BID_PER_ACTION"],[20451,527369,"SR-财神到-每次03.12-9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,696.53,4,174.1325000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527370,"SR-财神到-每次03.12-0","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3303.74,43,76.8311628,2,1651.870000,24.00,0.0072645,0.0455,0.04651,0,2,null,null,"BID_PER_ACTION"],[20451,527371,"PB-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,13293.67,119,111.7115126,2,6646.835000,5378.00,0.4045534,0.0632,0.01681,0,2,null,null,"BID_PER_ACTION"],[20451,527372,"PB-财神到-每次03.12-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,392.61,4,98.1525000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527373,"PB-财神到-每次03.12-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,591.21,2,295.6050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527374,"PB-财神到-每次03.12-7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3665.89,48,76.3727083,1,3665.890000,340.00,0.0927469,0.0882,0.02083,1,2,null,null,"BID_PER_ACTION"],[20451,527375,"PB-财神到-每次03.12-8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,393.37,2,196.6850000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,527376,"PB-财神到-每次03.12-9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,33964.27,559,60.7589803,8,4245.533750,10500.00,0.3091484,0.0733,0.01431,6,2,null,null,"BID_PER_ACTION"],[20451,527377,"PB-财神到-每次03.12-0","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,25042.22,670,37.3764478,8,3130.277500,750.00,0.0299494,0.0687,0.01194,8,2,null,null,"BID_PER_ACTION"],[20451,528504,"FW-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3303.74,74,44.6451351,2,1651.870000,208.00,0.0629589,0.1169,0.02703,0,2,null,null,"BID_PER_ACTION"],[20451,528505,"FJ-财神到-每次03.12-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,9844.75,111,88.6914414,5,1968.950000,504.00,0.0511948,0.1441,0.04505,3,2,null,null,"BID_PER_ACTION"],[20451,528506,"FJ-财神到-每次03.12-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3303.75,41,80.5792683,0,0.000000,0.00,0E-7,0.0244,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528507,"FW-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,451.85,2,225.9250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528508,"FJ-财神到-每次03.12-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,10039.93,132,76.0600758,4,2509.982500,396.00,0.0394425,0.0909,0.03030,1,2,null,null,"BID_PER_ACTION"],[20451,528509,"FW-财神到-每次03.12-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3338.25,54,61.8194444,1,3338.250000,60.00,0.0179735,0.0351,0.01852,1,2,null,null,"BID_PER_ACTION"],[20451,528510,"FW-财神到-每次03.12-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,538.84,5,107.7680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528511,"FW-财神到-每次03.12-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3351.51,58,57.7846552,0,0.000000,0.00,0E-7,0.0862,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528512,"FJ-财神到-每次03.12-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,431.34,2,215.6700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528513,"FJ-财神到-每次03.12-4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,496.30,3,165.4333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528912,"FW-财神到-每次03.13-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,623.15,6,103.8583333,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528913,"FJ-财神到-每次03.13-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,26540.03,398,66.6834925,10,2654.003000,1828.00,0.0688771,0.1057,0.02513,9,2,null,null,"BID_PER_ACTION"],[20451,528914,"FJ-财神到-每次03.13-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,10005.61,161,62.1466460,4,2501.402500,182.00,0.0181898,0.0962,0.02484,3,2,null,null,"BID_PER_ACTION"],[20451,528915,"FW-财神到-每次03.13-1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3303.74,57,57.9603509,0,0.000000,0.00,0E-7,0.0333,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528917,"FW-财神到-每次03.13-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4622.29,77,60.0297403,2,2311.145000,3838.00,0.8303244,0.0909,0.02597,1,2,null,null,"BID_PER_ACTION"],[20451,528918,"FJ-财神到-每次03.13-6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,7231.92,92,78.6078261,3,2410.640000,120.00,0.0165931,0.1667,0.03261,4,2,null,null,"BID_PER_ACTION"],[20451,528920,"FJ-财神到-每次03.13-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,473.87,1,473.8700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528921,"FW-财神到-每次03.13-3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,727.99,6,121.3316667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,528923,"FW-财神到-每次03.13-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,794.58,8,99.3225000,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,452869,"HSX-幸存者-自动ROI-通投-头抖_混SSR1909_0128_20","2021-03-16 00:00:00","2021-01-28 00:00:00",10,1001379,1,7,2933.74,134,21.8935821,0,0.000000,0.00,0E-7,0.0633,0.00000,0,2,null,null,"ROI_PACING"],[20540,489063,"RICH-幸存者_春节-沉浸付费_安卓通投_69686_0215_07317475","2021-03-16 00:00:00","2021-02-15 00:00:00",10,1001379,1,7,11859.31,256,46.3254297,1,11859.310000,68.00,0.0057339,0.0738,0.00391,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,507233,"RICH-财神保佑-幸存者-每次付费_春节游戏DMP_竖SSR2285_0226_5","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001379,1,7,2381.33,76,31.3332895,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,517614,"RICH-幸存者-我也爆R-每次付费_安卓游戏_竖SSR2402_0305_3","2021-03-16 00:00:00","2021-03-05 00:00:00",10,1001379,1,7,0.77,0,0.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,521061,"RICH-财神到我家-幸存者-每次付费_竖SSR2285_XTSC1614654964-5_0308_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,4223.87,90,46.9318889,0,0.000000,0.00,0E-7,0.0316,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,521062,"RICH-财神到我家-幸存者-每次付费_横SSR2185_XTSC1614654964-5_0308_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,128.55,2,64.2750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,521063,"RICH-财神到我家-幸存者-每次付费_横SSR2363_XTSC1614654964-5_0308_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1527.20,15,101.8133333,1,1527.200000,12.00,0.0078575,0.0556,0.06667,0,2,null,null,"BID_PER_ACTION"],[20540,521065,"RICH-财神到我家-幸存者-每次付费_竖SSR2332_XTSC1614654964-5_0308_14","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,823.92,7,117.7028571,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,521086,"RICH-财神到我家-幸存者-每次付费_竖SSR2728_XTSC1614654964-5_0308_15","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,5462.32,71,76.9340845,0,0.000000,0.00,0E-7,0.0694,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,521118,"RICH-财神到我家-幸存者-自动ROI-_TT排除_竖SSR2332_0308_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,94.62,4,23.6550000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"ROI_PACING"],[20540,521119,"RICH-财神到我家-幸存者-自动ROI-_TT排除_横SSR2248_0308_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,138.17,5,27.6340000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"ROI_PACING"],[20540,521121,"RICH-财神到我家-幸存者-自动ROI-_TT排除_SSR2363_0308_12","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,566.91,13,43.6084615,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"ROI_PACING"],[20540,521129,"RICH-财神到我家-幸存者-自动ROI-_TT排除_竖SSR2504_0308_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,61.13,1,61.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20540,521134,"RICH-财神到我家-幸存者-自动ROI-_TT排除_横SSR2444_0308_11","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,66.91,0,66.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526528,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2363_0311_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6600.53,41,160.9885366,2,3300.265000,848.00,0.1284745,0.1111,0.04878,1,2,null,null,"BID_PER_ACTION"],[20540,526529,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2271_0311_25","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,275.50,9,30.6111111,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526531,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2332_0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,7682.16,109,70.4785321,1,7682.160000,102.00,0.0132775,0.0917,0.00917,0,2,null,null,"BID_PER_ACTION"],[20540,526533,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2484_0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,19.56,1,19.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526534,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2285_0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2925.97,45,65.0215556,0,0.000000,0.00,0E-7,0.0222,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526535,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2735_0311_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,236.63,1,236.6300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,526536,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2622_0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4291.41,66,65.0213636,1,4291.410000,48.00,0.0111851,0.0758,0.01515,0,2,null,null,"BID_PER_ACTION"],[20540,526862,"RICH-财神到我家-幸存者-自动roi-_春节DMP_横SSR2736_0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,7.25,1,7.2500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20540,526865,"RICH-财神到我家-幸存者-自动roi-_游戏_横SSR1909_0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.66,1,0.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20540,527690,"RICH-财神到我家_幸存者-每次付费_排3000_71833_0312_95325541","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2351.51,28,83.9825000,0,0.000000,0.00,0E-7,0.0370,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,527691,"RICH-财神到我家_幸存者-每次付费_排3000_73011_0312_95325530","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,8779.27,78,112.5547436,4,2194.817500,2464.00,0.2806611,0.0667,0.05128,4,2,null,null,"BID_PER_ACTION"],[20540,527692,"RICH-财神到我家_幸存者-每次付费_排3000_70989_0312_95325565","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4648.24,91,51.0795604,1,4648.240000,42.00,0.0090357,0.0330,0.01099,0,2,null,null,"BID_PER_ACTION"],[20540,527698,"RICH-财神到我家_幸存者-每次付费_排3000_76461_0312_95325564","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1225.64,8,153.2050000,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,527795,"RICH-财神到我家_幸存者-每次付费_排3000_78846_0312_95325567","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1112.80,11,101.1636364,0,0.000000,0.00,0E-7,0.1538,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529036,"RICH-财神到我家_幸存者-每次付费_排3000_70019_0313_08286060","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3838.44,45,85.2986667,2,1919.220000,766.00,0.1995602,0.2188,0.04444,1,2,null,null,"BID_PER_ACTION"],[20540,529038,"RICH-财神到我家_幸存者-每次付费_排3000_77023_0313_08286082","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3535.06,33,107.1230303,2,1767.530000,2502.00,0.7077673,0.1053,0.06061,2,2,null,null,"BID_PER_ACTION"],[20540,529204,"RICH-财神到我家-幸存者-每次付费_春节DMP-混SSR2762-313_6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1072.18,9,119.1311111,0,0.000000,0.00,0E-7,0.2222,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529205,"RICH-财神到我家-幸存者-每次付费_游戏-混SSR2762-313_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2650.25,42,63.1011905,1,2650.250000,170.00,0.0641449,0.2000,0.02381,1,2,null,null,"BID_PER_ACTION"],[20540,529206,"RICH-财神到我家-幸存者-每次付费_游戏-混SSR2762-313_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1164.24,8,145.5300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529207,"RICH-财神到我家-幸存者-每次付费_通投-混SSR2762-313_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1345.35,10,134.5350000,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529208,"RICH-财神到我家-幸存者-每次付费_游戏-混SSR2762-313_7","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1220.10,11,110.9181818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529209,"RICH-财神到我家-幸存者-每次付费_通投-混SSR2762-313_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,754.22,4,188.5550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529210,"RICH-财神到我家-幸存者-每次付费_春节DMP-混SSR2762-313_9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2721.20,39,69.7743590,0,0.000000,0.00,0E-7,0.0476,0.00000,0,2,null,null,"BID_PER_ACTION"],[20540,529211,"RICH-财神到我家-幸存者-每次付费_通投-混SSR2762-313_8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,18349.82,201,91.2926368,4,4587.455000,2774.00,0.1511731,0.0408,0.01990,3,2,null,null,"BID_PER_ACTION"],[20540,529212,"RICH-财神到我家-幸存者-每次付费_春节DMP-混SSR2762-313_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2706.90,36,75.1916667,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,520536,"0308幸存者393z每次付费莱卡_2444_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,387.69,5,77.5380000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20545,520546,"0308幸存者393z每次付费莱卡_2558_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1178.93,2,589.4650000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,520735,"0308oyo393自动roi莱卡_1806_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,6.64,12,0.5533333,0,0.000000,0.00,0E-7,0.0714,0.00000,0,2,null,null,"ROI_PACING"],[20545,520736,"0308oyo393自动roi莱卡_2285_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,406.67,20,20.3335000,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"ROI_PACING"],[20545,520737,"0308oyo393自动roi莱卡_2363_4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1463.40,34,43.0411765,0,0.000000,0.00,0E-7,0.0571,0.00000,0,2,null,null,"ROI_PACING"],[20545,520739,"0308oyo393自动roi莱卡_2724_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,6657.84,146,45.6016438,3,2219.280000,482.00,0.0723959,0.0596,0.02055,0,2,null,null,"ROI_PACING"],[20545,520740,"0308oyo393自动roi莱卡_2723_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1837.44,31,59.2722581,2,918.720000,78.00,0.0424504,0.0645,0.06452,0,2,null,null,"ROI_PACING"],[20545,523085,"0309幸存者393z每次付费莱卡_2285_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,5602.43,126,44.4637302,1,5602.430000,12.00,0.0021419,0.0465,0.00794,0,2,null,null,"BID_PER_ACTION"],[20545,523087,"0309幸存者393z每次付费莱卡_2053_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2714.30,28,96.9392857,0,0.000000,0.00,0E-7,0.0333,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,523090,"0309幸存者393z每次付费莱卡_2732_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1877.05,12,156.4208333,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,523091,"0309幸存者393z每次付费莱卡_2738_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2155.89,19,113.4678947,0,0.000000,0.00,0E-7,0.0476,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,523093,"0309幸存者393z每次付费莱卡_2737_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2653.36,28,94.7628571,2,1326.680000,212.00,0.0798987,0.1034,0.07143,0,2,null,null,"BID_PER_ACTION"],[20545,523130,"0309oyo393自动roi莱卡_2738_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,4618.44,181,25.5162431,3,1539.480000,2412.00,0.5222543,0.0328,0.01657,0,2,null,null,"ROI_PACING"],[20545,523131,"0309oyo393自动roi莱卡_2728_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,10.99,1,10.9900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,524369,"0310-幸存者XLY全民533每次付费_1909","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2851.71,79,36.0975949,3,950.570000,174.00,0.0610160,0.0795,0.03797,0,2,null,null,"BID_PER_ACTION"],[20545,524370,"0310-幸存者XLY全民533每次付费_2484","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2851.71,39,73.1207692,1,2851.710000,42.00,0.0147280,0.0250,0.02564,0,2,null,null,"BID_PER_ACTION"],[20545,524371,"0310-幸存者XLY全民533每次付费_2332","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2851.71,60,47.5285000,5,570.342000,784.00,0.2749228,0.1846,0.08333,0,2,null,null,"BID_PER_ACTION"],[20545,524372,"0310-幸存者XLY全民533每次付费_2731","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2851.71,41,69.5539024,0,0.000000,0.00,0E-7,0.0698,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,524373,"0310-幸存者XLY全民533每次付费_2738","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1884.05,18,104.6694444,1,1884.050000,182.00,0.0966004,0.0556,0.05556,0,2,null,null,"BID_PER_ACTION"],[20545,525269,"0310oyo393自动roi莱卡_2332_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1488.40,62,24.0064516,1,1488.400000,42.00,0.0282182,0.0806,0.01613,0,2,null,null,"ROI_PACING"],[20545,525274,"0310oyo393自动roi莱卡_2746_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,82.37,4,20.5925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,525302,"0310全民534自动roi基定排除2k_2724_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,530.02,9,58.8911111,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,525303,"0310全民534自动roi基定排除2k_2745_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,4870.99,98,49.7039796,0,0.000000,0.00,0E-7,0.1122,0.00000,0,2,null,null,"ROI_PACING"],[20545,525304,"0310全民534自动roi基定排除2k_2742_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,12182.75,184,66.2105978,6,2030.458333,352.00,0.0288933,0.1413,0.03261,2,2,null,null,"ROI_PACING"],[20545,525305,"0310全民534自动roi基定排除2k_2746_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,5436.66,178,30.5430337,3,1812.220000,208.00,0.0382588,0.0749,0.01685,0,2,null,null,"ROI_PACING"],[20545,525306,"0310全民534自动roi基定排除2k_2733_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,162.02,2,81.0100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,525315,"0310幸存者393z每次付费莱卡_2332_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1604.15,16,100.2593750,0,0.000000,0.00,0E-7,0.0625,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,525316,"0310幸存者393z每次付费莱卡_2746_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1339.55,9,148.8388889,1,1339.550000,346.00,0.2582957,0.2222,0.11111,0,2,null,null,"BID_PER_ACTION"],[20545,525317,"0310幸存者393z每次付费莱卡_2745_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1727.15,14,123.3678571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526881,"0311幸存者393z每次付费3k_2731_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4128.32,76,54.3200000,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526882,"0311幸存者393z每次付费3k_2332_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,5579.10,133,41.9481203,3,1859.700000,448.00,0.0802997,0.0959,0.02256,0,2,null,null,"BID_PER_ACTION"],[20545,526883,"0311幸存者393z每次付费3k_2735_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4710.21,89,52.9237079,4,1177.552500,516.00,0.1095493,0.1429,0.04494,0,2,null,null,"BID_PER_ACTION"],[20545,526884,"0311幸存者393z每次付费3k_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6072.73,114,53.2695614,4,1518.182500,644.00,0.1060479,0.0826,0.03509,0,2,null,null,"BID_PER_ACTION"],[20545,526885,"0311幸存者393z每次付费3k_2724_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3485.74,53,65.7686792,0,0.000000,0.00,0E-7,0.0943,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526886,"0311全民534自动roi排除3k_2332_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2961.06,63,47.0009524,3,987.020000,496.00,0.1675076,0.1549,0.04762,0,2,null,null,"ROI_PACING"],[20545,526887,"0311全民534自动roi排除3k_2731_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2604.12,70,37.2017143,3,868.040000,102.00,0.0391687,0.1571,0.04286,1,2,null,null,"ROI_PACING"],[20545,526888,"0311全民534自动roi排除3k_2754_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,5012.36,84,59.6709524,4,1253.090000,562.00,0.1121228,0.1605,0.04762,2,2,null,null,"ROI_PACING"],[20545,526889,"0311全民534自动roi排除3k_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2913.27,83,35.0996386,0,0.000000,0.00,0E-7,0.0602,0.00000,0,2,null,null,"ROI_PACING"],[20545,526890,"0311全民534自动roi排除3k_2738_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4335.33,165,26.2747273,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"ROI_PACING"],[20545,527857,"0312oyo393自动roi莱卡_2738_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3085.16,110,28.0469091,0,0.000000,0.00,0E-7,0.0631,0.00000,0,2,null,null,"ROI_PACING"],[20545,527858,"0312oyo393自动roi莱卡_2332_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2435.63,30,81.1876667,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"ROI_PACING"],[20545,527861,"0312oyo393自动roi莱卡_2742_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,7018.74,111,63.2318919,4,1754.685000,182.00,0.0259306,0.1321,0.03604,2,2,null,null,"ROI_PACING"],[20545,527862,"0312oyo393自动roi莱卡_2735_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,8042.97,244,32.9629918,2,4021.485000,630.00,0.0783293,0.0615,0.00820,3,2,null,null,"ROI_PACING"],[20545,527864,"0312oyo393自动roi莱卡_2746_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3112.36,109,28.5537615,2,1556.180000,184.00,0.0591191,0.1197,0.01835,0,2,null,null,"ROI_PACING"],[20545,527893,"0312幸存者393z每次付费3k_2738_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2851.71,43,66.3188372,1,2851.710000,12.00,0.0042080,0.1395,0.02326,0,2,null,null,"BID_PER_ACTION"],[20545,527894,"0312幸存者393z每次付费3k_2332_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,5703.42,139,41.0317986,3,1901.140000,212.00,0.0371707,0.0946,0.02158,0,2,null,null,"BID_PER_ACTION"],[20545,527895,"0312幸存者393z每次付费3k_2735_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,714.82,19,37.6221053,0,0.000000,0.00,0E-7,0.0526,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,527896,"0312幸存者393z每次付费3k_2731_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1231.23,23,53.5317391,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,527897,"0312幸存者393z每次付费3k_2746_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1089.60,26,41.9076923,0,0.000000,0.00,0E-7,0.0385,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528099,"0312全民534自动roi排除3k_2738_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3320.16,88,37.7290909,2,1660.080000,102.00,0.0307214,0.0707,0.02273,0,2,null,null,"ROI_PACING"],[20545,528104,"0312全民534自动roi排除3k_2762_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3194.78,108,29.5812963,1,3194.780000,354.00,0.1108058,0.0550,0.00926,0,2,null,null,"ROI_PACING"],[20545,528105,"0312全民534自动roi排除3k_2766_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,5575.87,142,39.2666901,2,2787.935000,590.00,0.1058131,0.0704,0.01408,1,2,null,null,"ROI_PACING"],[20545,528106,"0312全民534自动roi排除3k_2764_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1781.45,19,93.7605263,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20545,528107,"0312全民534自动roi排除3k_2768_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,7239.28,209,34.6377033,3,2413.093333,54.00,0.0074593,0.0622,0.01435,3,2,null,null,"ROI_PACING"],[20545,528839,"0312全民534自动roi穿山甲_2622_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2486.90,80,31.0862500,2,1243.450000,220.00,0.0884635,0.0875,0.02500,2,2,null,null,"BID_PER_ACTION"],[20545,528840,"0312全民534自动roi穿山甲_2332_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,82.17,2,41.0850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528841,"0312全民534自动roi穿山甲_2762_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,9509.33,310,30.6752581,5,1901.866000,668.00,0.0702468,0.0742,0.01613,4,2,null,null,"BID_PER_ACTION"],[20545,528844,"0312全民534自动roi穿山甲_2779_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3203.07,125,25.6245600,1,3203.070000,98.00,0.0305956,0.0391,0.00800,1,2,null,null,"BID_PER_ACTION"],[20545,528846,"0312全民534自动roi穿山甲_2778_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,786.57,15,52.4380000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,523047,"每0309-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,627.01,4,156.7525000,1,627.010000,12.00,0.0191385,0.2500,0.25000,0,2,null,null,"BID_PER_ACTION"],[20635,523049,"每0309-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,362.36,2,181.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,523050,"每0309-tt-xcztz-wyt-rk_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,325.03,0,325.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,523051,"每0309-tt-xcztz-wyt-rk_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3999.82,15,266.6546667,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,524185,"每0310-tt-xcztz-wyt-rk_10","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,7691.47,71,108.3305634,1,7691.470000,196.00,0.0254828,0.0274,0.01408,0,2,null,null,"BID_PER_ACTION"],[20635,524188,"每0310-tt-xcztz-wyt-rk_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,3359.40,18,186.6333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,524191,"每0310-tt-xcztz-wyt-rk_9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,308.91,0,308.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,524192,"每0310-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,415.98,2,207.9900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,524193,"每0310-tt-xcztz-wyt-rk_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,184.71,1,184.7100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,524194,"每0310-tt-xcztz-wyt-rk_11","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,500.62,3,166.8733333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,524195,"每0310-tt-xcztz-wyt-rk_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,372.61,1,372.6100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,524196,"每0310-tt-xcztz-wyt-rk_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,390.72,1,390.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,524198,"每0310-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,283.85,1,283.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,524201,"每0310-tt-xcztz-wyt-rk_14","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,897.68,1,897.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,524205,"每0310-tt-xcztz-wyt-rk_19","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,287.63,1,287.6300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,525067,"每0310-tt-xcztz-wyt-rk新_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,5607.48,244,22.9814754,0,0.000000,0.00,0E-7,0.0325,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,525068,"每0310-tt-xcztz-wyt-rk新_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,511.88,2,255.9400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,525069,"每0310-tt-xcztz-wyt-rk新_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1111.29,1,1111.2900000,1,1111.290000,4872.00,4.3840942,1.0000,1.00000,0,2,null,null,"BID_PER_ACTION"],[20635,525070,"每0310-tt-xcztz-wyt-rk新_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,290.45,1,290.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,526426,"每0311-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1014.86,5,202.9720000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,526427,"每0311-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2798.43,14,199.8878571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,526429,"每0311-tt-xcztz-wyt-rk_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3659.23,10,365.9230000,1,3659.230000,366.00,0.1000210,0.1000,0.10000,0,2,null,null,"BID_PER_ACTION"],[20635,526430,"每0311-tt-xcztz-wyt-rk_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,436.78,2,218.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,526431,"每0311-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,670.51,3,223.5033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,526432,"每0311-tt-xcztz-wyt-rk_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,291.24,1,291.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,526434,"每0311-tt-xcztz-wyt-rk_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,475.98,2,237.9900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,526435,"每0311-tt-xcztz-wyt-rk_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,496.12,3,165.3733333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527500,"每0312-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,311.35,2,155.6750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,527501,"每0312-tt-xcztz-wyt-rk_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,768.90,3,256.3000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,527503,"每0312-tt-xcztz-wyt-rk_16","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,279.64,2,139.8200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527507,"每0312-tt-xcztz-wyt-rk_23","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,356.67,2,178.3350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527510,"每0312-tt-xcztz-wyt-rk_26","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,418.48,1,418.4800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527515,"每0312-tt-xcztz-wyt-rk_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3697.37,23,160.7552174,1,3697.370000,12.00,0.0032456,0.0870,0.04348,0,2,null,null,"BID_PER_ACTION"],[20635,527517,"每0312-tt-xcztz-wyt-rk_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,83.48,1,83.4800000,1,83.480000,68.00,0.8145664,1.0000,1.00000,1,0,null,null,"BID_PER_ACTION"],[20635,527521,"每0312-tt-xcztz-wyt-rk_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,467.67,1,467.6700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527525,"每0312-tt-xcztz-wyt-rk_15","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,48.30,1,48.3000000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527526,"每0312-tt-xcztz-wyt-rk_17","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3172.36,22,144.1981818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527529,"每0312-tt-xcztz-wyt-rk_21","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,63.23,1,63.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527531,"每0312-tt-xcztz-wyt-rk_24","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,11752.89,104,113.0085577,5,2350.578000,2962.00,0.2520231,0.0769,0.04808,2,2,null,null,"BID_PER_ACTION"],[20635,527532,"每0312-tt-xcztz-wyt-rk_27","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,179.93,4,44.9825000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,527533,"每0312-tt-xcztz-wyt-rk_28","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,144.97,3,48.3233333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20635,529473,"每0314-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,810.03,3,270.0100000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529480,"每0314-tt-xcztz-wyt-rk新_19","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,802.69,5,160.5380000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529494,"每0314-tt-xcztz-wyt-rk新_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,484.59,4,121.1475000,1,484.590000,266.00,0.5489176,0.2500,0.25000,1,0,null,null,"BID_PER_ACTION"],[20635,529495,"每0314-tt-xcztz-wyt-rk新_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,466.24,5,93.2480000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529499,"每0314-tt-xcztz-wyt-rk新_20","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,452.98,4,113.2450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529500,"每0314-tt-xcztz-wyt-rk新_21","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,25,112.1496000,1,2803.740000,30.00,0.0107000,0.1200,0.04000,0,0,null,null,"BID_PER_ACTION"],[20635,529503,"每0314-tt-xcztz-wyt-rk新_26","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,788.60,3,262.8666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529504,"每0314-tt-xcztz-wyt-rk新_27","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,253.04,5,50.6080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,529508,"每0314-tt-xcztz-wyt-rk新_33","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,411.79,1,411.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,521883,"05-幸存者-MCFC0309_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3143.87,48,65.4972917,1,3143.870000,372.00,0.1183255,0.1000,0.02083,0,2,null,null,"BID_PER_ACTION"],[20637,521886,"05-幸存者-MCFC0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,32723.52,240,136.3480000,9,3635.946667,10758.00,0.3287544,0.0678,0.03750,0,2,null,null,"BID_PER_ACTION"],[20637,521887,"05-幸存者-MCFC0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2803.74,59,47.5210169,1,2803.740000,12.00,0.0042800,0.0500,0.01695,0,2,null,null,"BID_PER_ACTION"],[20637,521888,"05-幸存者-MCFC0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,746.00,1,746.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,521889,"05-幸存者-MCFC0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2986.86,27,110.6244444,2,1493.430000,5324.00,1.7824739,0.1481,0.07407,0,2,null,null,"BID_PER_ACTION"],[20637,522925,"05-幸存者-MCFC0309_6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,501.43,3,167.1433333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,522927,"05-幸存者-MCFC0309_7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,93.58,1,93.5800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,522928,"05-幸存者-MCFC0309_10","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,261.45,2,130.7250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,522929,"05-幸存者-MCFC0309_9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3419.22,47,72.7493617,2,1709.610000,938.00,0.2743316,0.1277,0.04255,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,522930,"05-幸存者-MCFC0309_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,508.50,3,169.5000000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,524540,"05-幸存者-MCFC0310_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1364.54,19,71.8178947,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,524542,"05-幸存者-MCFC0310_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,802.57,5,160.5140000,1,802.570000,66.00,0.0822358,0.4000,0.20000,0,2,null,null,"BID_PER_ACTION"],[20637,524545,"05-幸存者-MCFC0310_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,442.75,1,442.7500000,1,442.750000,278.00,0.6278938,1.0000,1.00000,0,2,null,null,"BID_PER_ACTION"],[20637,524550,"05-幸存者-MCFC0310_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,693.22,2,346.6100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,524552,"05-幸存者-MCFC0310_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,946.07,2,473.0350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526140,"05-幸存者-MCFC0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,112.83,1,112.8300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526141,"05-幸存者-MCFC0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,392.90,1,392.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,526142,"05-幸存者-MCFC0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,942.01,12,78.5008333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526143,"05-幸存者-MCFC0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,102.02,0,102.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,526144,"05-幸存者-MCFC0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1555.65,15,103.7100000,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526145,"05-幸存者-MCFC0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,55.65,2,27.8250000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526401,"263-幸存者-ROI-0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6659.20,111,59.9927928,1,6659.200000,1406.00,0.2111365,0.1304,0.00901,0,2,null,null,"ROI_COEFFICIENT"],[20637,526403,"263-幸存者-ROI-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2859.24,63,45.3847619,3,953.080000,670.00,0.2343280,0.0758,0.04762,0,2,null,null,"ROI_COEFFICIENT"],[20637,526404,"263-幸存者-ROI-0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,225.77,4,56.4425000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20637,526405,"263-幸存者-ROI-0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,608.87,19,32.0457895,0,0.000000,0.00,0E-7,0.2105,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20637,526407,"263-幸存者-ROI-0311_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,230.14,4,57.5350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,526408,"263-幸存者-ROI-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,9652.99,209,46.1865550,3,3217.663333,370.00,0.0383301,0.1000,0.01435,3,2,null,null,"ROI_COEFFICIENT"],[20637,526409,"263-幸存者-ROI-0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2786.44,51,54.6360784,0,0.000000,0.00,0E-7,0.0727,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20637,526410,"263-幸存者-ROI-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4808.79,116,41.4550862,2,2404.395000,336.00,0.0698720,0.0690,0.01724,2,2,null,null,"DEEP_BID_DEFAULT"],[20637,526411,"263-幸存者-ROI-0311_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,171.49,0,171.4900000,1,171.490000,224.00,1.3061986,1.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[20637,526842,"263-幸存者-MCFC0311_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,414.83,2,207.4150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526843,"263-幸存者-MCFC0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2784.07,21,132.5747619,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526844,"263-幸存者-MCFC0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,555.92,0,555.9200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,526845,"263-幸存者-MCFC0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,17680.47,346,51.0996243,11,1607.315455,25340.00,1.4332198,0.0932,0.03179,8,2,null,null,"BID_PER_ACTION"],[20637,526846,"263-幸存者-MCFC0311_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,847.60,4,211.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526847,"263-幸存者-MCFC0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,24423.87,354,68.9939831,6,4070.645000,1220.00,0.0499511,0.0593,0.01695,2,2,null,null,"BID_PER_ACTION"],[20637,526848,"263-幸存者-MCFC0311_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2426.63,3,808.8766667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526849,"263-幸存者-MCFC0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,5756.73,67,85.9213433,1,5756.730000,716.00,0.1243762,0.0423,0.01493,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,526850,"263-幸存者-MCFC0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1629.05,14,116.3607143,0,0.000000,0.00,0E-7,0.0714,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526851,"263-幸存者-MCFC0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1438.33,2,719.1650000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526933,"05-幸存者-MCFC0311_7_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,7741.91,111,69.7469369,2,3870.955000,290.00,0.0374585,0.0708,0.01802,0,2,null,null,"BID_PER_ACTION"],[20637,526936,"05-幸存者-MCFC0311_7_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,523.89,1,523.8900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526940,"05-幸存者-MCFC0311_7_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1603.29,20,80.1645000,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,526941,"05-幸存者-MCFC0311_7_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1402.51,3,467.5033333,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,526943,"05-幸存者-MCFC0311_7_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,50199.20,861,58.3033682,19,2642.063158,16370.00,0.3261008,0.0735,0.02207,10,2,null,null,"BID_PER_ACTION"],[20637,526945,"05-幸存者-MCFC0311_7_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,541.79,1,541.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528659,"990-幸存者-MCFC-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,32.76,1,32.7600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528893,"05-幸存者-MCFC0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2368.19,21,112.7709524,0,0.000000,0.00,0E-7,0.1364,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528894,"05-幸存者-MCFC0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,307.80,4,76.9500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528896,"05-幸存者-MCFC0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,867.15,3,289.0500000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528898,"05-幸存者-MCFC0312_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,398.99,5,79.7980000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528900,"05-幸存者-MCFC0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,265.65,1,265.6500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528901,"05-幸存者-MCFC0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,170.95,2,85.4750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528902,"05-幸存者-MCFC0312_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,98.13,1,98.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528924,"263-幸存者-MCFC0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,7529.98,79,95.3162025,1,7529.980000,30.00,0.0039841,0.0886,0.01266,0,2,null,null,"BID_PER_ACTION"],[20637,528925,"263-幸存者-MCFC0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,917.08,9,101.8977778,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528926,"263-幸存者-MCFC0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,428.56,3,142.8533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20637,528927,"263-幸存者-MCFC0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,240.84,3,80.2800000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20637,528928,"263-幸存者-MCFC0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,866.25,8,108.2812500,1,866.250000,60.00,0.0692641,0.2500,0.12500,1,2,null,null,"BID_PER_ACTION"],[20637,528929,"263-幸存者-MCFC0312_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,5405.74,42,128.7080952,1,5405.740000,98.00,0.0181289,0.0698,0.02381,0,2,null,null,"BID_PER_ACTION"],[20637,528930,"263-幸存者-MCFC0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3608.70,25,144.3480000,1,3608.700000,2434.00,0.6744811,0.0800,0.04000,0,2,null,null,"BID_PER_ACTION"],[20637,528963,"263-幸存者-MCFC0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3568.92,25,142.7568000,2,1784.460000,128.00,0.0358652,0.1250,0.08000,2,2,null,null,"BID_PER_ACTION"],[20637,529561,"05-幸存者-MCFC0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,7757.01,103,75.3107767,2,3878.505000,392.00,0.0505349,0.0606,0.01942,2,0,null,null,"BID_PER_ACTION"],[20637,529562,"05-幸存者-MCFC0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,26,107.8361538,0,0.000000,0.00,0E-7,0.0769,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529563,"05-幸存者-MCFC0314_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,17360.96,128,135.6325000,9,1928.995556,6320.00,0.3640352,0.0826,0.07031,9,0,null,null,"BID_PER_ACTION"],[20637,529564,"05-幸存者-MCFC0314_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1937.23,24,80.7179167,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529565,"05-幸存者-MCFC0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,334.05,2,167.0250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529566,"05-幸存者-MCFC0314_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,25,112.1496000,0,0.000000,0.00,0E-7,0.0400,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529567,"05-幸存者-MCFC0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.63,16,175.2268750,0,0.000000,0.00,0E-7,0.1250,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529568,"05-幸存者-MCFC0314_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,5514.36,42,131.2942857,1,5514.360000,98.00,0.0177718,0.1667,0.02381,1,0,null,null,"BID_PER_ACTION"],[20637,529569,"05-幸存者-MCFC0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3729.36,50,74.5872000,2,1864.680000,392.00,0.1051119,0.0698,0.04000,1,0,null,null,"BID_PER_ACTION"],[20637,529570,"05-幸存者-MCFC0314_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2606.03,22,118.4559091,0,0.000000,0.00,0E-7,0.0909,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529571,"05-幸存者-MCFC0314_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,7757.01,91,85.2418681,1,7757.010000,2170.00,0.2797470,0.0159,0.01099,1,0,null,null,"BID_PER_ACTION"],[20637,529572,"05-幸存者-MCFC0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,34,82.4629412,0,0.000000,0.00,0E-7,0.0882,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,516719,"cxq-tt549幸存者-roi系数-0304_优SSR23323000排除","2021-03-16 00:00:00","2021-03-04 00:00:00",10,1001379,1,7,605.09,9,67.2322222,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20646,519332,"cxq-tt747幸存者-抖自动roi0306_516-6SSR2263","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,1256.46,50,25.1292000,2,628.230000,78.00,0.0620792,0.0769,0.04000,0,2,null,null,"ROI_PACING"],[20646,519333,"cxq-tt747幸存者-抖自动roi0306_516-6SSR2387","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,889.05,43,20.6755814,0,0.000000,0.00,0E-7,0.0213,0.00000,0,2,null,null,"ROI_PACING"],[20646,519334,"cxq-tt338幸存者-抖自动roi0306_606-1SSR1414","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,231.24,14,16.5171429,0,0.000000,0.00,0E-7,0.0714,0.00000,0,2,null,null,"ROI_PACING"],[20646,520923,"cxq-tt338幸存者游戏-自动ROI-0308_包12363","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,75.64,0,75.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520937,"cxq-tt549幸存者游戏-自动ROI-0308_包2小2728","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,0.87,0,0.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520952,"cxq-tt747幸存者游戏-自动ROI-0308_包22363","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,610.75,9,67.8611111,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"ROI_PACING"],[20646,523103,"cxq-tt552幸存者-0309-每次付费_088-11车SSR2558","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,485.99,3,161.9966667,1,485.990000,3574.00,7.3540608,0.6667,0.33333,0,2,null,null,"BID_PER_ACTION"],[20646,523104,"cxq-tt982幸存者-0309-每次付费_078-5拉环SSR2332","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,396.40,2,198.2000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,523106,"cxq-tt549幸存者-0309-每次付费_078-5拉环SSR2332","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,327.71,2,163.8550000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,523107,"cxq-tt549幸存者-0309-每次付费_078-5录2733","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,296.62,1,296.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,523108,"cxq-tt552幸存者-0309-每次付费_843-15车SSR2668","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,343.77,0,343.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523109,"cxq-tt552幸存者-0309-每次付费_088-11车SSR2668","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,492.54,3,164.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,523111,"cxq-tt747幸存者-0309-每次付费_拉环SSR2532","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2670.93,25,106.8372000,0,0.000000,0.00,0E-7,0.0741,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,523112,"cxq-tt747幸存者-0309-每次付费_拉环SSR2332","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1425.36,8,178.1700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,523117,"cxq-tt530幸存者-0309-843-15每次付费_录2733","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,832.94,2,416.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,523118,"cxq-tt530幸存者-0309-843-15每次付费_录1306","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,448.85,2,224.4250000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,523122,"cxq-tt982幸存者-0309-每次付费_843-2车SSR2558","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,735.10,2,367.5500000,1,735.100000,30.00,0.0408108,0.5000,0.50000,0,2,null,null,"BID_PER_ACTION"],[20646,523137,"cxq-tt338幸存者-自动ROI-0309_排除SSR2332","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,237.78,7,33.9685714,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"ROI_PACING"],[20646,523139,"cxq-tt530幸存者-自动ROI-0309_莱卡SSR2332","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,172.77,1,172.7700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20646,523164,"cxq-tt338幸存者-自动ROI-0309_排除2738","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,5.16,0,5.1600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523166,"cxq-tt747幸存者-自动ROI-0309_品牌2738","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,23.58,0,23.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523167,"cxq-tt747幸存者-自动ROI-0309_品牌录SSR2556","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,24.75,1,24.7500000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"ROI_PACING"],[20646,523168,"cxq-tt530幸存者-自动ROI-0309_莱卡录SSR2556","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,6.39,1,6.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20646,523169,"cxq-tt338幸存者-自动ROI-0309_排除录SSR2556","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,67.90,4,16.9750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20646,523170,"cxq-tt747幸存者-自动ROI-0309_品牌SSR2532","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,5.87,1,5.8700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[20646,523171,"cxq-tt747幸存者-自动ROI-0309_品牌SSR2332","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,155.55,0,155.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,525773,"cxq-tt552幸存者-穿山甲-付费-0311_排除2736","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,74.35,1,74.3500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,526460,"cxq-tt幸存者-每次付费-排除3000-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1303.31,12,108.6091667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526461,"cxq-tt幸存者-每次付费-排除3000-0311_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,447.13,1,447.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526462,"cxq-tt幸存者-每次付费-排除3000-0311_17","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,297.33,2,148.6650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526463,"cxq-tt幸存者-每次付费-排除3000-0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,262.13,4,65.5325000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526464,"cxq-tt幸存者-每次付费-排除3000-0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,472.76,4,118.1900000,1,472.760000,30.00,0.0634571,0.0000,0.25000,0,2,null,null,"BID_PER_ACTION"],[20646,526465,"cxq-tt幸存者-每次付费-排除3000-0311_21","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,342.40,3,114.1333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526467,"cxq-tt幸存者-每次付费-排除3000-0311_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,464.91,5,92.9820000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526468,"cxq-tt幸存者-每次付费-排除3000-0311_22","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,329.70,1,329.7000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526469,"cxq-tt幸存者-每次付费-排除3000-0311_24","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1385.62,13,106.5861538,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526470,"cxq-tt幸存者-每次付费-排除3000-0311_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,258.36,1,258.3600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526471,"cxq-tt幸存者-每次付费-排除3000-0311_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,306.15,1,306.1500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526472,"cxq-tt幸存者-每次付费-排除3000-0311_18","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,96.89,1,96.8900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526473,"cxq-tt幸存者-每次付费-排除3000-0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,411.39,5,82.2780000,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526474,"cxq-tt幸存者-每次付费-排除3000-0311_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,115.35,2,57.6750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526475,"cxq-tt幸存者-每次付费-排除3000-0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,310.93,3,103.6433333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526476,"cxq-tt幸存者-每次付费-排除3000-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2474.54,25,98.9816000,1,2474.540000,206.00,0.0832478,0.0400,0.04000,0,2,null,null,"BID_PER_ACTION"],[20646,526478,"cxq-tt幸存者-每次付费-排除3000-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,78.64,0,78.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526479,"cxq-tt幸存者-每次付费-排除3000-0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6782.45,135,50.2403704,0,0.000000,0.00,0E-7,0.0593,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526481,"cxq-tt幸存者-每次付费-排除3000-0311_20","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,605.89,13,46.6069231,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526482,"cxq-tt幸存者-每次付费-排除3000-0311_23","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2829.89,29,97.5824138,1,2829.890000,874.00,0.3088459,0.0345,0.03448,0,2,null,null,"BID_PER_ACTION"],[20646,526483,"cxq-tt幸存者-每次付费-排除3000-0311_25","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,455.85,5,91.1700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526484,"cxq-tt幸存者-每次付费-排除3000-0311_26","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,672.63,5,134.5260000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526485,"cxq-tt幸存者-每次付费-排除3000-0311_27","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,543.10,6,90.5166667,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526538,"cxq-tt幸存者-每次付费-排除负向0311_21","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,287.66,1,287.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526539,"cxq-tt幸存者-每次付费-排除负向0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,218.02,1,218.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526546,"cxq-tt幸存者-每次付费-排除负向0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6298.27,114,55.2479825,2,3149.135000,534.00,0.0847852,0.0789,0.01754,1,2,null,null,"BID_PER_ACTION"],[20646,526547,"cxq-tt幸存者-每次付费-排除负向0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,296.76,0,296.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526549,"cxq-tt幸存者-每次付费-排除负向0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,36.08,1,36.0800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526551,"cxq-tt幸存者-每次付费-排除负向0311_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,465.60,7,66.5142857,0,0.000000,0.00,0E-7,0.2857,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526552,"cxq-tt幸存者-每次付费-排除负向0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,28.79,1,28.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526553,"cxq-tt幸存者-每次付费-排除负向0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4.21,1,4.2100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526555,"cxq-tt幸存者-每次付费-排除负向0311_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,42.77,1,42.7700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526557,"cxq-tt幸存者-每次付费-排除负向0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,210.33,1,210.3300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,526572,"cxq-tt幸存者-每次付费-排除负向0311_20","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2446.92,33,74.1490909,2,1223.460000,110.00,0.0449545,0.0588,0.06061,0,2,null,null,"BID_PER_ACTION"],[20646,526574,"cxq-tt幸存者-每次付费-排除负向0311_23","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,217.03,3,72.3433333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527535,"cxq-tt338幸存者-每次付费穿山甲0312_排30002736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,536.63,7,76.6614286,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527536,"cxq-tt338幸存者-每次付费穿山甲0312_排30002248","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,368.11,5,73.6220000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527537,"cxq-tt338幸存者-每次付费穿山甲0312_排负2746小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2795.73,54,51.7727778,0,0.000000,0.00,0E-7,0.1481,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,527538,"cxq-tt552幸存者-每次付费穿山甲0312_排30002751小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3931.89,109,36.0723853,4,982.972500,308.00,0.0783338,0.1081,0.03670,2,2,null,null,"BID_PER_ACTION"],[20646,527539,"cxq-tt552幸存者-每次付费穿山甲0312_排30002762小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,14792.83,368,40.1979076,6,2465.471667,864.00,0.0584067,0.0726,0.01630,5,2,null,null,"BID_PER_ACTION"],[20646,527540,"cxq-tt338幸存者-每次付费穿山甲0312_排负2762小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1730.42,58,29.8348276,0,0.000000,0.00,0E-7,0.0877,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527541,"cxq-tt552幸存者-每次付费穿山甲0312_排30002746小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,6523.53,195,33.4540000,1,6523.530000,12.00,0.0018395,0.0667,0.00513,1,2,null,null,"BID_PER_ACTION"],[20646,527542,"cxq-tt552幸存者-每次付费穿山甲0312_排负2248","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,604.98,7,86.4257143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527543,"cxq-tt338幸存者-每次付费穿山甲0312_排负2751小","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,845.92,11,76.9018182,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527544,"cxq-tt338幸存者-每次付费穿山甲0312_排30002765","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,250.45,2,125.2250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527546,"cxq-tt552幸存者-每次付费穿山甲0312_排负2765","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,221.21,6,36.8683333,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527679,"cxq-tt747幸存者-每次付费-0312_排负+通全选小2735","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,582.76,4,145.6900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527680,"cxq-tt549幸存者-每次付费-0312_排负+男优2729","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1208.83,15,80.5886667,1,1208.830000,98.00,0.0810701,0.1333,0.06667,1,2,null,null,"BID_PER_ACTION"],[20646,527681,"cxq-tt747幸存者-每次付费-0312_排负+通全选小2746","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,298.01,1,298.0100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527682,"cxq-tt549幸存者-每次付费-0312_排负+男优2730","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,292.09,1,292.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527683,"cxq-tt747幸存者-每次付费-0312_排负+通全选小2762","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,13826.59,205,67.4467805,6,2304.431667,1528.00,0.1105117,0.0927,0.02927,4,2,null,null,"DEEP_BID_DEFAULT"],[20646,527684,"cxq-tt747幸存者-每次付费-0312_排负+通全选小2751","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,325.58,2,162.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527685,"cxq-tt549幸存者-每次付费-0312_排负+男优2736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,491.51,1,491.5100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,527686,"cxq-tt549幸存者-每次付费-0312_排负+男优2765","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,340.81,1,340.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,528869,"cxq-tt982幸存者-每次付费-优选-0312_820-1小2737","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,806.51,7,115.2157143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,528870,"cxq-tt982幸存者-每次付费-优选-0312_2736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,303.69,2,151.8450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,528871,"cxq-tt530幸存者-每次付费-优选-0312_小2779843-19","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,427.53,4,106.8825000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,528872,"cxq-tt530幸存者-每次付费-优选-0312_车2248088-11","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,469.58,4,117.3950000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,528873,"cxq-tt982幸存者-每次付费-优选-0312_820-1小2762","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1095.48,7,156.4971429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,528874,"cxq-tt530幸存者-每次付费-优选-0312_2736088-11","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,339.44,1,339.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,528875,"cxq-tt982幸存者-每次付费-优选-0312_车2248206-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,462.93,2,231.4650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,528876,"cxq-tt530幸存者-每次付费-优选-0312_小2746843-19","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,8112.87,125,64.9029600,5,1622.574000,8888.00,1.0955433,0.0720,0.04000,4,2,null,null,"DEEP_BID_DEFAULT"],[20646,529223,"cxq-tt338幸存者-每次付费-穿山甲0313_27622000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,9633.67,228,42.2529386,4,2408.417500,230.00,0.0238746,0.1327,0.01754,3,2,null,null,"BID_PER_ACTION"],[20646,529225,"cxq-tt338幸存者-每次付费-穿山甲0313_27462000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3813.67,113,33.7492920,2,1906.835000,132.00,0.0346123,0.1250,0.01770,2,2,null,null,"BID_PER_ACTION"],[20646,529226,"cxq-tt552幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,508.14,14,36.2957143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529229,"cxq-tt552幸存者-每次付费-穿山甲0313_27462000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,5148.18,107,48.1138318,2,2574.090000,132.00,0.0256401,0.0943,0.01869,2,2,null,null,"BID_PER_ACTION"],[20646,529230,"cxq-tt552幸存者-每次付费-穿山甲0313_27622000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3769.27,69,54.6271014,0,0.000000,0.00,0E-7,0.1014,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529233,"cxq-tt338幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1776.97,50,35.5394000,1,1776.970000,12.00,0.0067531,0.2000,0.02000,1,2,null,null,"BID_PER_ACTION"],[20646,529234,"cxq-tt552幸存者-每次付费-穿山甲0313_27662000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3609.21,95,37.9916842,1,3609.210000,12.00,0.0033248,0.0667,0.01053,0,2,null,null,"BID_PER_ACTION"],[20646,529252,"cxq-tt549幸存者-每次付费-0313_122-1SSR2285","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3197.10,219,14.5986301,1,3197.100000,212.00,0.0663101,0.0320,0.00457,1,2,null,null,"BID_PER_ACTION"],[20646,529253,"cxq-tt747幸存者-每次付费-0313_843-18SSR2762","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1398.83,11,127.1663636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529254,"cxq-tt982幸存者-每次付费-0313_包1SSR2285","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3330.96,80,41.6370000,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529255,"cxq-tt747幸存者-每次付费-0313_843-18SSR2285","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3275.23,71,46.1300000,0,0.000000,0.00,0E-7,0.0563,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529256,"cxq-tt982幸存者-每次付费-0313_包1SSR2762","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,521.93,1,521.9300000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529257,"cxq-tt747幸存者-每次付费-0313_843-18SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2225.61,11,202.3281818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529258,"cxq-tt982幸存者-每次付费-0313_包1SSR1414","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,831.45,4,207.8625000,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529259,"cxq-tt982幸存者-每次付费-0313_包1SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,4673.02,68,68.7208824,2,2336.510000,308.00,0.0659103,0.0652,0.02941,1,2,null,null,"BID_PER_ACTION"],[20646,529260,"cxq-tt747幸存者-每次付费-0313_843-18SSR1414","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,842.08,14,60.1485714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529261,"cxq-tt549幸存者-每次付费-0313_122-1SSR1414","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3539.65,100,35.3965000,2,1769.825000,24.00,0.0067803,0.0400,0.02000,2,2,null,null,"BID_PER_ACTION"],[20646,529262,"cxq-tt549幸存者-每次付费-0313_122-1SSR2762","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1150.25,13,88.4807692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529263,"cxq-tt549幸存者-每次付费-0313_122-1SSR2755","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,5698.00,195,29.2205128,2,2849.000000,110.00,0.0193050,0.0667,0.01026,2,2,null,null,"BID_PER_ACTION"],[20754,511954,"zhan-每次付费-幸存-3.2_4","2021-03-16 00:00:00","2021-03-02 00:00:00",10,1001379,1,7,6542.05,212,30.8587264,1,6542.050000,294.00,0.0449400,0.0617,0.00472,0,2,null,null,"BID_PER_ACTION"],[20754,519396,"hix-每次付费-ssr-3.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,15774.98,334,47.2304790,3,5258.326667,2490.00,0.1578449,0.0749,0.00898,0,2,null,null,"BID_PER_ACTION"],[20754,519397,"zhan-每次付费-ssr-3.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,6542.06,157,41.6691720,0,0.000000,0.00,0E-7,0.0683,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519399,"hix-每次付费-ssr-3.6_6","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,2153.38,8,269.1725000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519401,"zhan-每次付费-ssr-3.6_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,19163.31,221,86.7118100,5,3832.662000,772.00,0.0402853,0.0995,0.02262,3,2,null,null,"BID_PER_ACTION"],[20754,519405,"hix-每次付费-ssr-3.6_8","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,3277.79,25,131.1116000,2,1638.895000,248.00,0.0756607,0.2000,0.08000,0,2,null,null,"BID_PER_ACTION"],[20754,519406,"hix-每次付费-ssr-3.6_2","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,2626.65,18,145.9250000,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519409,"zhan-每次付费-ssr-3.6_6","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,662.59,1,662.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,519411,"zhan-每次付费-ssr-3.6_8","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,16355.15,192,85.1830729,4,4088.787500,2408.00,0.1472319,0.0515,0.02083,1,2,null,null,"BID_PER_ACTION"],[20754,520091,"yo-roi系数roi-想个好名字-2.6_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,11601.51,193,60.1114508,5,2320.302000,11180.00,0.9636677,0.1257,0.02591,2,2,null,null,"ROI_COEFFICIENT"],[20754,520097,"yo-roi系数roi-想个好名字-2.6_3","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,5121.07,171,29.9477778,1,5121.070000,60.00,0.0117163,0.0893,0.00585,0,2,null,null,"ROI_COEFFICIENT"],[20754,520100,"ka-roi系数-想个好名字-2.6_11","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,3613.33,137,26.3746715,2,1806.665000,254.00,0.0702953,0.0216,0.01460,0,2,null,null,"ROI_COEFFICIENT"],[20754,520169,"zhan-每次付费-周天-3.7_1_8","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,7,5327.10,91,58.5395604,2,2663.550000,174.00,0.0326632,0.0625,0.02198,0,2,null,null,"BID_PER_ACTION"],[20754,520171,"zhan-每次付费-周天-3.7_1_5","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,7,3270.26,41,79.7624390,0,0.000000,0.00,0E-7,0.1136,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,520174,"zhan-每次付费-周天-3.7_1_6","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,7,5319.66,107,49.7164486,2,2659.830000,60.00,0.0112789,0.0459,0.01869,0,2,null,null,"BID_PER_ACTION"],[20754,520177,"ka-自动roi-周天-2.7_6","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,7,1134.61,50,22.6922000,1,1134.610000,108.00,0.0951869,0.0545,0.02000,0,2,null,null,"ROI_COEFFICIENT"],[20754,520178,"ka-自动roi-周天-2.7_5","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,7,7491.46,455,16.4647473,1,7491.460000,12.00,0.0016018,0.0566,0.00220,1,2,null,null,"DEEP_BID_DEFAULT"],[20754,520180,"yo-自动roi-周天-2.7_3","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001379,1,7,9.68,0,9.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[20754,523277,"hix-每次付费--3.9_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,967.30,6,161.2166667,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,523278,"hix-每次付费--3.9_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,817.63,6,136.2716667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,523279,"hix-每次付费--3.9_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,520.29,4,130.0725000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,523280,"zhan-每次付费--3.9_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,21525.20,314,68.5515924,5,4305.040000,2910.00,0.1351904,0.0332,0.01592,0,2,null,null,"BID_PER_ACTION"],[20754,523281,"zhan-每次付费--3.9_6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,4305.44,111,38.7877477,1,4305.440000,128.00,0.0297298,0.0619,0.00901,0,2,null,null,"BID_PER_ACTION"],[20754,523282,"zhan-每次付费--3.9_7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,441.98,3,147.3266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,523294,"hix-每次付费--3.9_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,430.93,3,143.6433333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,523295,"zhan-每次付费--3.9_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,12038.34,111,108.4535135,5,2407.668000,1504.00,0.1249342,0.0577,0.04505,2,2,null,null,"BID_PER_ACTION"],[20754,524815,"zhan-每次付费--3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,917.56,9,101.9511111,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524816,"zhan-每次付费--3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,439.38,2,219.6900000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524817,"zhan-每次付费--3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,20090.34,281,71.4958719,10,2009.034000,2570.00,0.1279222,0.1028,0.03559,3,2,null,null,"DEEP_BID_DEFAULT"],[20754,524818,"zhan-每次付费--3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,384.60,2,192.3000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524819,"zhan-每次付费--3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,68.07,1,68.0700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524821,"hix-每次付费--3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,447.93,3,149.3100000,0,0.000000,0.00,0E-7,0.6667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524822,"zhan-每次付费--3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,7714.16,94,82.0655319,2,3857.080000,102.00,0.0132224,0.0490,0.02128,0,2,null,null,"BID_PER_ACTION"],[20754,524823,"hix-每次付费--3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,883.26,6,147.2100000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524824,"hix-每次付费--3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,299.28,0,299.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,524825,"hix-每次付费--3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,3077.79,15,205.1860000,1,3077.790000,98.00,0.0318410,0.0714,0.06667,0,2,null,null,"BID_PER_ACTION"],[20754,524830,"zhan-每次付费--3.10_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,386.68,2,193.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524831,"jap-每次付费--3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,7703.98,91,84.6591209,4,1925.995000,224.00,0.0290759,0.0879,0.04396,1,2,null,null,"BID_PER_ACTION"],[20754,524832,"jap-每次付费--3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,3204.46,17,188.4976471,1,3204.460000,300.00,0.0936195,0.0625,0.05882,0,2,null,null,"BID_PER_ACTION"],[20754,524833,"hix-每次付费--3.10_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,215.00,0,215.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,524834,"hix-每次付费--3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,4394.17,44,99.8675000,1,4394.170000,30.00,0.0068272,0.1136,0.02273,0,2,null,null,"BID_PER_ACTION"],[20754,524836,"tw-每次付费--3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1151.06,24,47.9608333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524837,"tw-每次付费--3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,629.62,22,28.6190909,0,0.000000,0.00,0E-7,0.0870,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524838,"tw-每次付费--3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2711.59,24,112.9829167,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524839,"tw-每次付费--3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1243.27,18,69.0705556,0,0.000000,0.00,0E-7,0.1053,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524841,"jap-每次付费--3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,78.47,1,78.4700000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,524842,"jap-每次付费--3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1104.28,14,78.8771429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,525043,"yo-roi系数-爆好-3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,324.43,5,64.8860000,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,525044,"ka-roi系数-爆好-3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,6492.68,169,38.4182249,3,2164.226667,502.00,0.0773178,0.0893,0.01775,0,2,null,null,"ROI_COEFFICIENT"],[20754,525045,"ka-roi系数-爆好-3.10_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1123.93,16,70.2456250,0,0.000000,0.00,0E-7,0.1579,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,525046,"yo-roi系数-爆好-3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2557.25,73,35.0308219,0,0.000000,0.00,0E-7,0.0267,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,525047,"ka-roi系数-爆好-3.10_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1317.63,38,34.6744737,2,658.815000,24.00,0.0182145,0.0513,0.05263,0,2,null,null,"ROI_COEFFICIENT"],[20754,525048,"ka-roi系数-爆好-3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2589.73,39,66.4033333,0,0.000000,0.00,0E-7,0.0652,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,525049,"yo-roi系数-爆好-3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1198.79,23,52.1213043,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,525050,"yo-roi系数-爆好-3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,302.90,16,18.9312500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,527171,"jap-每次付费--3.11_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1138.14,21,54.1971429,0,0.000000,0.00,0E-7,0.0455,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527172,"hix-每次付费--3.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,125.47,1,125.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527173,"jap-每次付费--3.11_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,883.31,4,220.8275000,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527174,"hix-每次付费--3.11_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,198.86,1,198.8600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527175,"tw-每次付费--3.11_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1016.49,6,169.4150000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527176,"hix-每次付费--3.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6009.16,62,96.9219355,2,3004.580000,464.00,0.0772155,0.1091,0.03226,1,2,null,null,"BID_PER_ACTION"],[20754,527177,"jap-每次付费--3.11_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,7903.79,97,81.4823711,1,7903.790000,876.00,0.1108329,0.0612,0.01031,0,2,null,null,"BID_PER_ACTION"],[20754,527178,"tw-每次付费--3.11_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6140.08,136,45.1476471,3,2046.693333,8828.00,1.4377663,0.0809,0.02206,2,2,null,null,"DEEP_BID_DEFAULT"],[20754,527179,"hix-每次付费--3.11_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,462.90,4,115.7250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527181,"tw-每次付费--3.11_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,417.38,3,139.1266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527182,"tw-每次付费--3.11_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1571.40,17,92.4352941,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527183,"zhan-每次付费--3.11_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4553.02,69,65.9857971,1,4553.020000,4766.00,1.0467777,0.1129,0.01449,0,2,null,null,"BID_PER_ACTION"],[20754,527184,"zhan-每次付费--3.11_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2234.94,14,159.6385714,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527185,"zhan-每次付费--3.11_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1328.40,11,120.7636364,0,0.000000,0.00,0E-7,0.1818,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,527186,"zhan-每次付费--3.11_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1295.66,13,99.6661538,0,0.000000,0.00,0E-7,0.1538,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528134,"jap-每次付费--3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,209.34,2,104.6700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528135,"hix-每次付费--3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,453.68,5,90.7360000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528136,"hix-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,283.11,3,94.3700000,1,283.110000,196.00,0.6923104,1.0000,0.33333,0,2,null,null,"BID_PER_ACTION"],[20754,528138,"hix-每次付费--3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,411.94,7,58.8485714,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528139,"hix-每次付费--3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,42.88,2,21.4400000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528140,"tw-每次付费--3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,48.31,1,48.3100000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528141,"jap-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,450.19,5,90.0380000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528143,"tw-每次付费--3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,27.64,1,27.6400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528145,"zhan-每次付费--3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,218.35,1,218.3500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528146,"zhan-每次付费--3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,186.26,2,93.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528147,"zhan-每次付费--3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,480.27,3,160.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528148,"zhan-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4560.40,63,72.3873016,2,2280.200000,358.00,0.0785019,0.1091,0.03175,1,2,null,null,"BID_PER_ACTION"],[20754,528149,"zhan-每次付费--3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,49.19,1,49.1900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528151,"jap-每次付费--3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4755.14,56,84.9132143,1,4755.140000,110.00,0.0231329,0.0984,0.01786,0,2,null,null,"BID_PER_ACTION"],[20754,528153,"hix-每次付费--3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,7738.55,88,87.9380682,2,3869.275000,104.00,0.0134392,0.0568,0.02273,2,2,null,null,"BID_PER_ACTION"],[20754,528154,"hix-每次付费--3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,413.38,2,206.6900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528157,"tw-每次付费--3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1497.20,36,41.5888889,1,1497.200000,42.00,0.0280524,0.1667,0.02778,0,2,null,null,"BID_PER_ACTION"],[20754,528158,"tw-每次付费--3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,9.08,0,9.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528159,"tw-每次付费--3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2699.07,51,52.9229412,0,0.000000,0.00,0E-7,0.0566,0.00000,0,2,null,null,"BID_PER_ACTION"],[20754,528345,"yo-roi系数-周五-3.12_4_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4499.56,111,40.5365766,1,4499.560000,12.00,0.0026669,0.0769,0.00901,0,2,null,null,"ROI_COEFFICIENT"],[20754,528346,"yo-roi系数-周五-3.12_4_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.74,43,65.2032558,0,0.000000,0.00,0E-7,0.1489,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,528347,"ka-roi系数-周五-3.12_4_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.74,68,41.2314706,0,0.000000,0.00,0E-7,0.0946,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,528348,"ka-roi系数-周五-3.12_4_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.74,91,30.8103297,0,0.000000,0.00,0E-7,0.0532,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,528349,"ka-roi系数-周五-3.12_4_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,6074.77,114,53.2874561,1,6074.770000,356.00,0.0586030,0.0508,0.00877,0,2,null,null,"ROI_COEFFICIENT"],[20754,528350,"yo-roi系数-周五-3.12_4_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.74,64,43.8084375,1,2803.740000,12.00,0.0042800,0.0290,0.01563,0,2,null,null,"ROI_COEFFICIENT"],[20754,528351,"yo-roi系数-周五-3.12_4_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,7813.46,254,30.7616535,4,1953.365000,222.00,0.0284125,0.1111,0.01575,4,2,null,null,"DEEP_BID_DEFAULT"],[20754,528352,"ka-roi系数-周五-3.12_4_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2725.63,40,68.1407500,0,0.000000,0.00,0E-7,0.1190,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,528353,"ka-roi系数-周五-3.12_4_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,7767.26,175,44.3843429,3,2589.086667,262.00,0.0337313,0.0920,0.01714,2,2,null,null,"ROI_COEFFICIENT"],[20754,528354,"yo-roi系数-周五-3.12_4_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,9698.12,268,36.1870149,6,1616.353333,822.00,0.0847587,0.0597,0.02239,3,2,null,null,"DEEP_BID_DEFAULT"],[20754,528355,"ka-roi系数-周五-3.12_4_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.74,103,27.2207767,0,0.000000,0.00,0E-7,0.0385,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,528356,"ka-roi系数-周五-3.12_4_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,11831.39,191,61.9444503,7,1690.198571,3806.00,0.3216866,0.1030,0.03665,6,2,null,null,"ROI_COEFFICIENT"],[20754,528357,"yo-roi系数-周五-3.12_4_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.08,55,50.9650909,0,0.000000,0.00,0E-7,0.0492,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,528358,"yo-roi系数-周五-3.12_4_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,113.98,0,113.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[20754,529154,"ka-自动roi-周六-3.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,4298.62,131,32.8138931,2,2149.310000,60.00,0.0139580,0.0687,0.01527,2,2,null,null,"ROI_PACING"],[20754,529155,"ka-自动roi-周六-3.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2264.44,64,35.3818750,0,0.000000,0.00,0E-7,0.0323,0.00000,0,2,null,null,"ROI_PACING"],[20754,529156,"yo-roi系数-周六-3.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2803.74,54,51.9211111,1,2803.740000,12.00,0.0042800,0.0714,0.01852,0,2,null,null,"ROI_COEFFICIENT"],[20754,529157,"ka-自动roi-周六-3.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2043.72,53,38.5607547,0,0.000000,0.00,0E-7,0.0476,0.00000,0,2,null,null,"ROI_PACING"],[20754,529158,"ka-自动roi-周六-3.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,5631.22,167,33.7198802,3,1877.073333,236.00,0.0419092,0.0802,0.01796,3,2,null,null,"ROI_PACING"],[20754,529159,"yo-roi系数-周六-3.13_44","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,12865.06,349,36.8626361,6,2144.176667,1378.00,0.1071118,0.0575,0.01719,6,2,null,null,"ROI_COEFFICIENT"],[20754,529160,"yo-roi系数-周六-3.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2803.74,68,41.2314706,0,0.000000,0.00,0E-7,0.0137,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,529161,"yo-roi系数-周六-3.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,14223.76,289,49.2171626,7,2031.965714,504.00,0.0354337,0.0664,0.02422,7,2,null,null,"ROI_COEFFICIENT"],[20754,529162,"yo-roi系数-周六-3.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2803.74,47,59.6540426,0,0.000000,0.00,0E-7,0.0392,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20754,529587,"hix-每次付费--3.13_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,504.15,9,56.0166667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529589,"jap-每次付费--3.13_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,581.66,4,145.4150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529590,"hix-每次付费--3.13_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,36,77.8816667,0,0.000000,0.00,0E-7,0.0556,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529591,"jap-每次付费--3.13_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1627.32,32,50.8537500,0,0.000000,0.00,0E-7,0.0645,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529592,"jap-每次付费--3.13_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,37,75.7767568,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529593,"tw-每次付费--3.13_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1682.15,41,41.0280488,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529594,"hix-每次付费--3.13_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,456.30,2,228.1500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529595,"hix-每次付费--3.13_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,44.73,2,22.3650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529596,"tw-每次付费--3.13_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4857.47,177,27.4433333,1,4857.470000,494.00,0.1016990,0.0395,0.00565,1,0,null,null,"BID_PER_ACTION"],[20754,529597,"tw-每次付费--3.13_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,42,66.7557143,0,0.000000,0.00,0E-7,0.0476,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529598,"zhan-每次付费--3.13_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,92.43,3,30.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529599,"tw-每次付费--3.13_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4859.81,74,65.6731081,1,4859.810000,102.00,0.0209885,0.0270,0.01351,1,0,null,null,"BID_PER_ACTION"],[20754,529600,"zhan-每次付费--3.13_17","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4492.21,77,58.3403896,3,1497.403333,304.00,0.0676727,0.0857,0.03896,2,0,null,null,"BID_PER_ACTION"],[20754,529601,"zhan-每次付费--3.13_18","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,463.50,2,231.7500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,529602,"zhan-每次付费--3.13_19","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,32.47,1,32.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,520454,"295-blcb-每次付费爆R爆量_3.8_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,3541.20,61,58.0524590,0,0.000000,0.00,0E-7,0.0794,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,520455,"295-blcb-每次付费爆R爆量_3.8_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,51.22,1,51.2200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,520458,"295-blcb-每次付费爆R爆量_3.8_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,32.28,1,32.2800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,520459,"295-blcb-每次付费爆R爆量_3.8_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,12.19,1,12.1900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,520823,"470-tv-每次付费爆R爆量_3.8_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1150.47,5,230.0940000,1,1150.470000,508.00,0.4415587,0.2857,0.20000,0,2,null,null,"BID_PER_ACTION"],[21208,520824,"470-tv-每次付费爆R爆量_3.8_4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1700.33,20,85.0165000,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,520828,"470-tv-每次付费爆R爆量_3.8_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,930.73,15,62.0486667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,520829,"470-tv-每次付费爆R爆量_3.8_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2803.74,34,82.4629412,1,2803.740000,140.00,0.0499333,0.0571,0.02941,0,2,null,null,"BID_PER_ACTION"],[21208,521729,"295-blcb-每次付费爆R爆量_3.9_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2920.33,20,146.0165000,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,521738,"295-blcb-每次付费爆R爆量_3.9_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,121.76,1,121.7600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,521773,"470-tv-每次付费爆R爆量_3.9_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,921.79,11,83.7990909,0,0.000000,0.00,0E-7,0.1818,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,521774,"470-tv-每次付费爆R爆量_3.9_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2769.99,17,162.9405882,2,1384.995000,42.00,0.0151625,0.0588,0.11765,0,2,null,null,"BID_PER_ACTION"],[21208,521775,"470-tv-每次付费爆R爆量_3.9_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1379.31,10,137.9310000,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,521776,"470-tv-每次付费爆R爆量_3.9_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3998.11,41,97.5148780,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,521777,"470-tv-每次付费爆R爆量_3.9_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,577.48,12,48.1233333,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,524322,"295-blcb-ROI爆R爆量_3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,360.02,10,36.0020000,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,524325,"295-blcb-ROI爆R爆量_3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,79.55,3,26.5166667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,524328,"295-blcb-ROI爆R爆量_3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,56.28,1,56.2800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,524334,"295-blcb-ROI爆R爆量_3.10_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,4800.42,158,30.3824051,1,4800.420000,60.00,0.0124989,0.0247,0.00633,0,2,null,null,"ROI_COEFFICIENT"],[21208,524352,"470-tv-单付费爆R爆量_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,554.07,20,27.7035000,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,524354,"470-tv-单付费爆R爆量_3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2962.96,173,17.1269364,1,2962.960000,618.00,0.2085752,0.0652,0.00578,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,524355,"470-tv-单付费爆R爆量_3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,40.04,3,13.3466667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,524356,"470-tv-单付费爆R爆量_3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,575.74,19,30.3021053,1,575.740000,140.00,0.2431653,0.0526,0.05263,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,524357,"470-tv-单付费爆R爆量_3.10_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1410.41,51,27.6550980,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,524358,"470-tv-单付费爆R爆量_3.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,37.57,3,12.5233333,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,524360,"470-tv-单付费爆R爆量_3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,26.61,0,26.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,525337,"295-blcb-ROI爆R爆量_3.10_10","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,101.43,1,101.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,525338,"295-blcb-ROI爆R爆量_3.10_9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1428.99,34,42.0291176,1,1428.990000,30.00,0.0209938,0.1714,0.02941,0,2,null,null,"ROI_COEFFICIENT"],[21208,525339,"295-blcb-ROI爆R爆量_3.10_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1536.75,36,42.6875000,0,0.000000,0.00,0E-7,0.1463,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,525385,"470-tv-单付费爆R爆量_3.10_9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,145.67,1,145.6700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,525388,"470-tv-单付费爆R爆量_3.10_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,197.40,4,49.3500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,525399,"295-blcb-ROI爆R爆量_3.10_13","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,490.41,6,81.7350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,525400,"470-tv-单付费爆R爆量_3.10_12","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2803.32,119,23.5573109,0,0.000000,0.00,0E-7,0.0630,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,525401,"295-blcb-ROI爆R爆量_3.10_12","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2651.20,70,37.8742857,0,0.000000,0.00,0E-7,0.0946,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,525402,"470-tv-单付费爆R爆量_3.10_13","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2276.46,90,25.2940000,0,0.000000,0.00,0E-7,0.0800,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,525744,"295-blcb-ROI爆R爆量_3.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2803.74,72,38.9408333,0,0.000000,0.00,0E-7,0.0519,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,525745,"295-blcb-ROI爆R爆量_3.11_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2728.26,57,47.8642105,0,0.000000,0.00,0E-7,0.0462,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,525746,"295-blcb-ROI爆R爆量_3.11_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2827.63,105,26.9298095,2,1413.815000,2906.00,1.0277158,0.0667,0.01905,0,2,null,null,"ROI_COEFFICIENT"],[21208,525747,"295-blcb-ROI爆R爆量_3.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2803.74,80,35.0467500,0,0.000000,0.00,0E-7,0.0250,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,525748,"295-blcb-ROI爆R爆量_3.11_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2803.74,37,75.7767568,0,0.000000,0.00,0E-7,0.0789,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[21208,526382,"470-tv-单付费爆R爆量_3.11_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2648.12,9,294.2355556,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,526383,"470-tv-单付费爆R爆量_3.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2820.45,100,28.2045000,0,0.000000,0.00,0E-7,0.1262,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,526385,"470-tv-单付费爆R爆量_3.11_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2797.65,98,28.5474490,0,0.000000,0.00,0E-7,0.1000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,526386,"470-tv-单付费爆R爆量_3.11_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2695.35,92,29.2972826,0,0.000000,0.00,0E-7,0.1579,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,526387,"470-tv-单付费爆R爆量_3.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4668.85,103,45.3286408,1,4668.850000,42.00,0.0089958,0.0435,0.00971,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,526852,"295-blcb-ROI爆R爆量_3.11_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2268.22,43,52.7493023,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21208,526854,"295-blcb-ROI爆R爆量_3.11_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,810.01,16,50.6256250,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21208,526856,"295-blcb-ROI爆R爆量_3.11_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,208.52,2,104.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21208,527304,"295-blcb-每次付费爆R爆量_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3758.07,89,42.2255056,2,1879.035000,244.00,0.0649269,0.0308,0.02247,2,2,null,null,"BID_PER_ACTION"],[21208,527307,"295-blcb-每次付费爆R爆量_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,438.64,9,48.7377778,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527310,"295-blcb-每次付费爆R爆量_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1899.76,14,135.6971429,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527311,"295-blcb-每次付费爆R爆量_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2141.75,37,57.8851351,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527314,"295-blcb-每次付费爆R爆量_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,89.90,5,17.9800000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527315,"295-blcb-每次付费爆R爆量_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,162.96,2,81.4800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527343,"470-tv-每次付费爆R爆量_3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,23.05,2,11.5250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527344,"470-tv-每次付费爆R爆量_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,115.06,1,115.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527345,"470-tv-每次付费爆R爆量_3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,264.79,1,264.7900000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527346,"470-tv-每次付费爆R爆量_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1775.40,10,177.5400000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,448415,"r-xy-xcztz-01.25_3","2021-03-16 00:00:00","2021-01-25 00:00:00",10,1001379,1,7,2799.88,116,24.1368966,1,2799.880000,12.00,0.0042859,0.0420,0.00862,0,2,null,null,"DEEP_BID_DEFAULT"],[21212,490373,"g-自动ROI-xy-xcztz-02.16_6","2021-03-16 00:00:00","2021-02-16 00:00:00",10,1001379,1,7,1283.95,125,10.2716000,1,1283.950000,188.00,0.1464231,0.0441,0.00800,0,2,null,null,"ROI_PACING"],[21212,510184,"k-自动ROI-挑战词随机出价_20","2021-03-16 00:00:00","2021-03-01 00:00:00",10,1001379,1,7,785.32,30,26.1773333,0,0.000000,0.00,0E-7,0.1333,0.00000,0,2,null,null,"ROI_PACING"],[21212,510189,"g-自动ROI-挑战包随机出价_23","2021-03-16 00:00:00","2021-03-01 00:00:00",10,1001379,1,7,694.35,44,15.7806818,0,0.000000,0.00,0E-7,0.0682,0.00000,0,2,null,null,"ROI_PACING"],[21212,512526,"幸存者自动出价-xy-03.02_3","2021-03-16 00:00:00","2021-03-02 00:00:00",10,1001379,1,7,2366.93,112,21.1333036,1,2366.930000,60.00,0.0253493,0.0682,0.00893,1,2,null,null,"DEEP_BID_DEFAULT"],[21212,514472,"r-自动ROI-幸存者一定行-03.03_3","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001379,1,7,1502.00,92,16.3260870,2,751.000000,42.00,0.0279627,0.1042,0.02174,0,2,null,null,"ROI_PACING"],[21212,517623,"g-自动ROI-幸存者充值是大哥-03.05_24","2021-03-16 00:00:00","2021-03-05 00:00:00",10,1001379,1,7,4672.90,307,15.2211726,2,2336.450000,318.00,0.0680520,0.0464,0.00651,0,2,null,null,"ROI_PACING"],[21212,517632,"r-自动ROI-幸存者充值是大哥-包排2k03.05_12","2021-03-16 00:00:00","2021-03-05 00:00:00",10,1001379,1,7,1718.13,20,85.9065000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,519418,"r3-每次付费-幸存者小游戏专场-03.06_4","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,789.14,7,112.7342857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,519425,"r3-每次付费-幸存者小游戏专场-03.06_5","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,447.06,2,223.5300000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,520587,"r-自动ROI-幸存者充值是大哥大-03.08_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2886.79,105,27.4932381,1,2886.790000,66.00,0.0228628,0.0496,0.00952,1,2,null,null,"ROI_PACING"],[21212,520588,"r-自动ROI-幸存者充值是大哥大-03.08_6","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2793.69,41,68.1387805,1,2793.690000,406.00,0.1453275,0.0769,0.02439,0,2,null,null,"ROI_PACING"],[21212,520590,"r-自动ROI-幸存者充值是大哥大-03.08_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2811.85,119,23.6289916,0,0.000000,0.00,0E-7,0.0299,0.00000,0,2,null,null,"ROI_PACING"],[21212,520601,"r-自动ROI-幸存者充值是大哥大-03.08_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1026.95,62,16.5637097,0,0.000000,0.00,0E-7,0.0563,0.00000,0,2,null,null,"ROI_PACING"],[21212,520602,"r-自动ROI-幸存者充值是大哥大-03.08_4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,3078.13,143,21.5253846,0,0.000000,0.00,0E-7,0.0596,0.00000,0,2,null,null,"ROI_PACING"],[21212,521272,"g-自动ROI-幸存者加批计划跑晚上-03.08_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,0.03,3,0.0100000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"ROI_PACING"],[21212,521273,"g-自动ROI-幸存者加批计划跑晚上-03.08_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,6304.91,244,25.8397951,2,3152.455000,668.00,0.1059492,0.0925,0.00820,0,2,null,null,"ROI_PACING"],[21212,522088,"r-自动ROI-幸存者yyds-03.09_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1871.45,52,35.9894231,0,0.000000,0.00,0E-7,0.0702,0.00000,0,2,null,null,"ROI_PACING"],[21212,522089,"r-自动ROI-幸存者yyds-03.09_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1339.71,43,31.1560465,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"ROI_PACING"],[21212,522090,"r-自动ROI-幸存者yyds-03.09_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,812.83,14,58.0592857,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"ROI_PACING"],[21212,522091,"r-自动ROI-幸存者yyds-03.09_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,485.56,12,40.4633333,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"ROI_PACING"],[21212,522092,"r-自动ROI-幸存者yyds-03.09_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,6904.83,337,20.4891098,4,1726.207500,9270.00,1.3425385,0.1132,0.01187,2,2,null,null,"ROI_PACING"],[21212,522093,"r-自动ROI-幸存者yyds-03.09_6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,496.85,12,41.4041667,0,0.000000,0.00,0E-7,0.1538,0.00000,0,2,null,null,"ROI_PACING"],[21212,522906,"每次付费-幸存者请珍惜机会-03.09_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2803.74,67,41.8468657,0,0.000000,0.00,0E-7,0.0588,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,522909,"每次付费-幸存者请珍惜机会-03.09_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2803.74,55,50.9770909,0,0.000000,0.00,0E-7,0.0179,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,522910,"每次付费-幸存者请珍惜机会-03.09_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2325.54,15,155.0360000,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,522913,"每次付费-幸存者请珍惜机会-03.09_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1168.50,27,43.2777778,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,522917,"每次付费-幸存者请珍惜机会-03.09_6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1811.33,20,90.5665000,0,0.000000,0.00,0E-7,0.1739,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,524760,"r-自动ROI-幸存者挑战排除-03.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,891.45,62,14.3782258,0,0.000000,0.00,0E-7,0.0323,0.00000,0,2,null,null,"ROI_PACING"],[21212,524761,"r-自动ROI-幸存者挑战排除-03.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,454.12,10,45.4120000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"ROI_PACING"],[21212,524762,"r-自动ROI-幸存者挑战排除-03.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,152.01,4,38.0025000,0,0.000000,0.00,0E-7,0.7500,0.00000,0,2,null,null,"ROI_PACING"],[21212,524764,"r-自动ROI-幸存者挑战排除-03.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,132.28,5,26.4560000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,524765,"r-自动ROI-幸存者挑战排除-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,721.43,37,19.4981081,1,721.430000,12.00,0.0166336,0.0811,0.02703,0,2,null,null,"ROI_PACING"],[21212,525384,"i-每次付费-幸存者挑战-03.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,868.50,17,51.0882353,0,0.000000,0.00,0E-7,0.0455,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,525387,"i-每次付费-幸存者挑战-03.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,799.64,5,159.9280000,0,0.000000,0.00,0E-7,0.4000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,525390,"i-每次付费-幸存者挑战-03.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,551.75,4,137.9375000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,525391,"i-每次付费-幸存者挑战-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1772.99,12,147.7491667,1,1772.990000,30.00,0.0169206,0.1538,0.08333,0,2,null,null,"BID_PER_ACTION"],[21212,526438,"r-每次付费-幸存者挑战-03.11_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2788.32,63,44.2590476,0,0.000000,0.00,0E-7,0.0294,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526439,"r-每次付费-幸存者挑战-03.11_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2800.68,60,46.6780000,1,2800.680000,12.00,0.0042847,0.0492,0.01667,0,2,null,null,"BID_PER_ACTION"],[21212,526440,"r3-每次付费-幸存者挑战-03.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,8293.48,138,60.0976812,2,4146.740000,1116.00,0.1345635,0.0533,0.01449,0,2,null,null,"BID_PER_ACTION"],[21212,526443,"r3-每次付费-幸存者挑战-03.11_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,664.90,2,332.4500000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526444,"r3-每次付费-幸存者挑战-03.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3813.91,36,105.9419444,1,3813.910000,214.00,0.0561104,0.2857,0.02778,0,2,null,null,"BID_PER_ACTION"],[21212,526445,"r3-每次付费-幸存者挑战-03.11_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2472.90,23,107.5173913,2,1236.450000,42.00,0.0169841,0.0435,0.08696,0,2,null,null,"BID_PER_ACTION"],[21212,526447,"r-每次付费-幸存者挑战-03.11_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2803.74,39,71.8907692,0,0.000000,0.00,0E-7,0.0732,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526451,"r-每次付费-幸存者挑战-03.11_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,547.56,4,136.8900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526453,"r3-每次付费-幸存者挑战-03.11_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,10631.07,65,163.5549231,3,3543.690000,1544.00,0.1452347,0.0909,0.04615,0,2,null,null,"BID_PER_ACTION"],[21212,526456,"r-每次付费-幸存者挑战-03.11_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,573.06,3,191.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526457,"r-每次付费-幸存者挑战-03.11_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2371.08,20,118.5540000,0,0.000000,0.00,0E-7,0.0952,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,527581,"r3-每次付费-幸存者挑战-03.12_11","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,594.84,5,118.9680000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,527582,"r3-每次付费-幸存者挑战-03.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1528.82,15,101.9213333,1,1528.820000,12.00,0.0078492,0.1667,0.06667,0,2,null,null,"BID_PER_ACTION"],[21212,527584,"i-每次付费-幸存者挑战-03.12_17","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.74,53,52.9007547,0,0.000000,0.00,0E-7,0.0556,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,527585,"i-每次付费-幸存者挑战-03.12_13","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,637.17,4,159.2925000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,527586,"r3-每次付费-幸存者挑战-03.12_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1102.66,9,122.5177778,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,527587,"i-每次付费-幸存者挑战-03.12_16","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,238.60,1,238.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,527590,"r3-每次付费-幸存者挑战-03.12_12","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2298.03,18,127.6683333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,527591,"i-每次付费-幸存者挑战-03.12_14","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1426.02,14,101.8585714,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,527592,"i-每次付费-幸存者挑战-03.12_18","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2761.47,39,70.8069231,0,0.000000,0.00,0E-7,0.0488,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,528035,"r-自动ROI-幸存者挑战-03.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,57.17,2,28.5850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,528056,"r-自动ROI-幸存者挑战-03.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,438.65,29,15.1258621,0,0.000000,0.00,0E-7,0.1724,0.00000,0,2,null,null,"ROI_PACING"],[21212,528057,"r-自动ROI-幸存者挑战-03.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3174.79,120,26.4565833,0,0.000000,0.00,0E-7,0.0227,0.00000,0,2,null,null,"ROI_PACING"],[21212,528058,"r-自动ROI-幸存者挑战-03.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,208.98,6,34.8300000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"ROI_PACING"],[21212,528059,"r-自动ROI-幸存者挑战-03.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,227.89,2,113.9450000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"ROI_PACING"],[21212,528558,"g-自动ROI-幸存者挑战-03.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3838.48,379,10.1279156,2,1919.240000,250.00,0.0651299,0.0431,0.00528,2,2,null,null,"ROI_PACING"],[21212,528559,"g-自动ROI-幸存者挑战-03.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,77.23,1,77.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,528560,"g-自动ROI-幸存者挑战-03.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,338.60,6,56.4333333,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"ROI_PACING"],[21212,528561,"g-自动ROI-幸存者挑战-03.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2276.48,116,19.6248276,0,0.000000,0.00,0E-7,0.0583,0.00000,0,2,null,null,"ROI_PACING"],[21212,528562,"g-自动ROI-幸存者挑战-03.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,5018.34,129,38.9018605,2,2509.170000,806.00,0.1606109,0.0609,0.01550,2,2,null,null,"ROI_PACING"],[21212,529098,"r3-每次付费-幸存者挑战周六-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,4906.48,40,122.6620000,2,2453.240000,340.00,0.0692961,0.1579,0.05000,2,2,null,null,"BID_PER_ACTION"],[21212,529099,"r3-每次付费-幸存者挑战周六-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,5316.36,39,136.3169231,3,1772.120000,2726.00,0.5127568,0.1538,0.07692,2,2,null,null,"BID_PER_ACTION"],[21212,529100,"r3-每次付费-幸存者挑战周六-03.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2336.45,48,48.6760417,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529102,"r3-每次付费-幸存者挑战周六-03.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,274.95,1,274.9500000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529103,"r3-每次付费-幸存者挑战周六-03.13_6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2336.45,30,77.8816667,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529109,"r-自动ROI-幸存者挑战带穿山甲-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1713.43,81,21.1534568,1,1713.430000,12.00,0.0070035,0.0370,0.01235,1,2,null,null,"ROI_PACING"],[21212,529110,"r-自动ROI-幸存者挑战带穿山甲-03.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,50.34,1,50.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,529111,"r-自动ROI-幸存者挑战带穿山甲-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,768.57,35,21.9591429,0,0.000000,0.00,0E-7,0.1143,0.00000,0,2,null,null,"ROI_PACING"],[21212,529112,"r-自动ROI-幸存者挑战带穿山甲-03.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,172.43,4,43.1075000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21212,529113,"r-自动ROI-幸存者挑战带穿山甲-03.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,753.28,38,19.8231579,0,0.000000,0.00,0E-7,0.0526,0.00000,0,2,null,null,"ROI_PACING"],[21212,529538,"i-每次付费-幸存者挑战周日-03.14_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1325.42,10,132.5420000,0,0.000000,0.00,0E-7,0.3000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529539,"i-每次付费-幸存者挑战周日-03.14_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,42,66.7557143,0,0.000000,0.00,0E-7,0.0238,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529540,"r3-每次付费-幸存者挑战周日-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4871.34,46,105.8986957,0,0.000000,0.00,0E-7,0.0526,0.00000,1,0,null,null,"BID_PER_ACTION"],[21212,529541,"r3-每次付费-幸存者挑战周日-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3518.33,29,121.3217241,3,1172.776667,1008.00,0.2864996,0.3333,0.10345,3,0,null,null,"BID_PER_ACTION"],[21212,529542,"i-每次付费-幸存者挑战周日-03.14_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,544.15,5,108.8300000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529543,"r3-每次付费-幸存者挑战周日-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,52,53.9180769,0,0.000000,0.00,0E-7,0.0577,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529544,"i-每次付费-幸存者挑战周日-03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,984.42,11,89.4927273,1,984.420000,434.00,0.4408687,0.1667,0.09091,1,0,null,null,"BID_PER_ACTION"],[21212,529545,"r3-每次付费-幸存者挑战周日-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,6139.32,105,58.4697143,2,3069.660000,422.00,0.0687373,0.0769,0.01905,2,0,null,null,"BID_PER_ACTION"],[21212,529546,"i-每次付费-幸存者挑战周日-03.14_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,5878.93,111,52.9633333,2,2939.465000,550.00,0.0935544,0.0580,0.01802,2,0,null,null,"BID_PER_ACTION"],[21212,529547,"r3-每次付费-幸存者挑战周日-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,494.71,2,247.3550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,529548,"g-自动ROI-幸存者挑战带穿山甲-03.14_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3226.82,209,15.4393301,0,0.000000,0.00,0E-7,0.0478,0.00000,0,0,null,null,"ROI_PACING"],[21212,529549,"g-自动ROI-幸存者挑战带穿山甲-03.14_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1128.39,55,20.5161818,0,0.000000,0.00,0E-7,0.0364,0.00000,0,0,null,null,"ROI_PACING"],[21212,529550,"g-自动ROI-幸存者挑战带穿山甲-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4918.00,228,21.5701754,1,4918.000000,188.00,0.0382269,0.1000,0.00439,1,0,null,null,"ROI_PACING"],[21212,529551,"r-自动ROI-幸存者挑战带穿山甲-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2723.69,180,15.1316111,1,2723.690000,250.00,0.0917872,0.0281,0.00556,1,0,null,null,"ROI_PACING"],[21212,529552,"r-自动ROI-幸存者挑战带穿山甲-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3128.51,123,25.4350407,1,3128.510000,30.00,0.0095892,0.0813,0.00813,1,0,null,null,"ROI_PACING"],[21212,529553,"r-自动ROI-幸存者挑战带穿山甲-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2893.99,204,14.1862255,1,2893.990000,12.00,0.0041465,0.0343,0.00490,1,0,null,null,"ROI_PACING"],[21212,529554,"r-自动ROI-幸存者挑战带穿山甲-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,291.06,10,29.1060000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,529555,"g-自动ROI-幸存者挑战带穿山甲-03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,940.88,32,29.4025000,0,0.000000,0.00,0E-7,0.0645,0.00000,0,0,null,null,"ROI_PACING"],[21212,529557,"r-每次付费-幸存者挑战-穿山甲专场-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1855.91,11,168.7190909,1,1855.910000,60.00,0.0323292,0.2500,0.09091,1,0,null,null,"BID_PER_ACTION"],[21245,518930,"每次付费-dm-xcztz-0306_2","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,2907.55,88,33.0403409,0,0.000000,0.00,0E-7,0.0421,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,519366,"BY每次付费-拉环小游戏-xcztz-周六-0306_4","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,1939.36,49,39.5787755,0,0.000000,0.00,0E-7,0.0612,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,519378,"I2每次付费-拉环小游戏-xcztz-周六-0306_9","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,9956.45,242,41.1423554,4,2489.112500,2236.00,0.2245780,0.0472,0.01653,0,2,null,null,"BID_PER_ACTION"],[21245,519381,"BY每次付费-拉环小游戏-xcztz-周六-0306_1","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001379,1,7,2803.74,97,28.9045361,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520835,"dm每次付费-xcztz-0308_17","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,445.79,20,22.2895000,0,0.000000,0.00,0E-7,0.0455,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520836,"i2每次付费-xcztz-0308_12","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,541.80,16,33.8625000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520837,"by每次付费-xcztz-0308_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,25178.61,705,35.7143404,8,3147.326250,11712.00,0.4651567,0.0521,0.01135,1,2,null,null,"BID_PER_ACTION"],[21245,520839,"i2每次付费-xcztz-0308_11","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,163.92,5,32.7840000,1,163.920000,12.00,0.0732064,0.3333,0.20000,0,2,null,null,"BID_PER_ACTION"],[21245,520840,"by每次付费-xcztz-0308_6","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2803.74,94,29.8270213,1,2803.740000,68.00,0.0242533,0.0421,0.01064,0,2,null,null,"BID_PER_ACTION"],[21245,520841,"by每次付费-xcztz-0308_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2803.74,29,96.6806897,1,2803.740000,12.00,0.0042800,0.0333,0.03448,0,2,null,null,"BID_PER_ACTION"],[21245,520845,"by每次付费-xcztz-0308_4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2108.98,28,75.3207143,1,2108.980000,42.00,0.0199148,0.0690,0.03571,0,2,null,null,"BID_PER_ACTION"],[21245,520846,"by每次付费-xcztz-0308_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,618.37,2,309.1850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520849,"i2每次付费-xcztz-0308_7","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,121.01,5,24.2020000,0,0.000000,0.00,0E-7,0.3750,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520850,"by每次付费-xcztz-0308_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,511.29,1,511.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520852,"i2每次付费-xcztz-0308_10","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,91.14,1,91.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520853,"dm每次付费-xcztz-0308_13","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2803.74,35,80.1068571,0,0.000000,0.00,0E-7,0.0286,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520854,"dm每次付费-xcztz-0308_15","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,3280.64,31,105.8270968,1,3280.640000,258.00,0.0786432,0.1250,0.03226,0,2,null,null,"BID_PER_ACTION"],[21245,520855,"dm每次付费-xcztz-0308_16","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,450.93,4,112.7325000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520858,"dm每次付费-xcztz-0308_14","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,493.39,4,123.3475000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,520859,"i2每次付费-xcztz-0308_8","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1179.82,3,393.2733333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,521876,"自动ROI-JK-排3K包-拉环-_0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3068.04,131,23.4201527,1,3068.040000,72.00,0.0234678,0.0780,0.00763,0,2,null,null,"ROI_PACING"],[21245,521877,"自动ROI-JK-排3K包-拉环-_0309_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3023.33,120,25.1944167,0,0.000000,0.00,0E-7,0.0242,0.00000,0,2,null,null,"ROI_PACING"],[21245,521878,"自动ROI-JK-排3K包-拉环-_0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,26298.03,1286,20.4494790,15,1753.202000,1198.00,0.0455547,0.0592,0.01166,0,2,null,null,"ROI_PACING"],[21245,521879,"自动ROI-JK-排3K包-拉环-_0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,8035.49,358,22.4455028,2,4017.745000,102.00,0.0126937,0.0399,0.00559,0,2,null,null,"ROI_PACING"],[21245,521885,"自动ROI-JK-排3K包-拉环-_0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2874.09,65,44.2167692,0,0.000000,0.00,0E-7,0.0676,0.00000,0,2,null,null,"ROI_PACING"],[21245,521917,"每次付费-JK-排3K段-拉环-_0309_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,604.71,10,60.4710000,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"ROI_PACING"],[21245,521918,"每次付费-JK-排3K段-拉环-_0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2837.32,77,36.8483117,0,0.000000,0.00,0E-7,0.0759,0.00000,0,2,null,null,"ROI_PACING"],[21245,521919,"每次付费-JK-排3K段-拉环-_0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,514.45,12,42.8708333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,524565,"自动ROI-排3K包-0310_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2803.79,155,18.0889677,0,0.000000,0.00,0E-7,0.0314,0.00000,0,2,null,null,"ROI_PACING"],[21245,524566,"自动ROI-排3K包-0310_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2809.83,109,25.7782569,0,0.000000,0.00,0E-7,0.0796,0.00000,0,2,null,null,"ROI_PACING"],[21245,524567,"自动ROI-排3K包-0310_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,10506.96,364,28.8652747,5,2101.392000,3270.00,0.3112223,0.0894,0.01374,0,2,null,null,"ROI_PACING"],[21245,524575,"自动ROI-排3K包-0310_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,7151.11,228,31.3645175,6,1191.851667,1106.00,0.1546613,0.0752,0.02632,0,2,null,null,"ROI_PACING"],[21245,524576,"自动ROI-排3K包-0310_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,3037.55,85,35.7358824,0,0.000000,0.00,0E-7,0.0588,0.00000,0,2,null,null,"ROI_PACING"],[21245,524577,"自动ROI-排3K包-0310_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2806.97,70,40.0995714,0,0.000000,0.00,0E-7,0.0143,0.00000,0,2,null,null,"ROI_PACING"],[21245,524581,"自动ROI-排3K包-0310_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,3027.64,62,48.8329032,1,3027.640000,660.00,0.2179916,0.1613,0.01613,0,2,null,null,"ROI_PACING"],[21245,524582,"自动ROI-排3K包-0310_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2811.94,51,55.1360784,1,2811.940000,1238.00,0.4402654,0.1186,0.01961,0,2,null,null,"ROI_PACING"],[21245,524999,"BY每次付费-xcztz-拉环卡车-0310-_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,4320.73,93,46.4594624,0,0.000000,0.00,0E-7,0.1075,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525000,"BY每次付费-xcztz-拉环卡车-0310-_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1976.47,56,35.2941071,1,1976.470000,128.00,0.0647619,0.1071,0.01786,1,2,null,null,"BID_PER_ACTION"],[21245,525001,"I2每次付费-xcztz-拉环卡车-0310-_14","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,6617.99,216,30.6388426,3,2205.996667,506.00,0.0764583,0.0636,0.01389,0,2,null,null,"BID_PER_ACTION"],[21245,525002,"I2每次付费-xcztz-拉环卡车-0310-_15","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2811.28,60,46.8546667,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525003,"BY每次付费-xcztz-拉环卡车-0310-_7","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,6207.49,146,42.5170548,3,2069.163333,54.00,0.0086992,0.0753,0.02055,1,2,null,null,"BID_PER_ACTION"],[21245,525004,"I2每次付费-xcztz-拉环卡车-0310-_16","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2700.86,19,142.1505263,1,2700.860000,122.00,0.0451708,0.1053,0.05263,0,2,null,null,"BID_PER_ACTION"],[21245,525005,"DM每次付费-xcztz-拉环卡车-0310-_23","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,8917.62,230,38.7722609,4,2229.405000,1258.00,0.1410690,0.0562,0.01739,1,2,null,null,"BID_PER_ACTION"],[21245,525006,"DM每次付费-xcztz-拉环卡车-0310-_22","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1424.33,17,83.7841176,0,0.000000,0.00,0E-7,0.2941,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525008,"I2每次付费-xcztz-拉环卡车-0310-_9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2701.90,30,90.0633333,0,0.000000,0.00,0E-7,0.0333,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525009,"BY每次付费-xcztz-拉环卡车-0310-_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1723.92,29,59.4455172,1,1723.920000,30.00,0.0174022,0.1724,0.03448,1,2,null,null,"BID_PER_ACTION"],[21245,525010,"I2每次付费-xcztz-拉环卡车-0310-_11","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,736.94,5,147.3880000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525011,"BY每次付费-xcztz-拉环卡车-0310-_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2044.24,9,227.1377778,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525013,"I2每次付费-xcztz-拉环卡车-0310-_13","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,502.35,1,502.3500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525014,"BY每次付费-xcztz-拉环卡车-0310-_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,828.14,15,55.2093333,0,0.000000,0.00,0E-7,0.2667,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525015,"BY每次付费-xcztz-拉环卡车-0310-_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2442.09,18,135.6716667,1,2442.090000,12.00,0.0049138,0.1111,0.05556,0,2,null,null,"BID_PER_ACTION"],[21245,525017,"I2每次付费-xcztz-拉环卡车-0310-_12","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,462.59,1,462.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525018,"DM每次付费-xcztz-拉环卡车-0310-_17","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2803.74,22,127.4427273,0,0.000000,0.00,0E-7,0.0455,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,525019,"DM每次付费-xcztz-拉环卡车-0310-_18","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,2046.01,14,146.1435714,2,1023.005000,152.00,0.0742909,0.1429,0.14286,0,2,null,null,"BID_PER_ACTION"],[21245,526394,"jk自动ROI-排3K-xcztz-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,897.49,4,224.3725000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526395,"jk自动ROI-排3K-xcztz-0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,137.60,8,17.2000000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"ROI_PACING"],[21245,526396,"jk自动ROI-排3K-xcztz-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,544.79,39,13.9689744,0,0.000000,0.00,0E-7,0.1282,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,526397,"jk自动ROI-排3K-xcztz-0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,793.07,111,7.1447748,0,0.000000,0.00,0E-7,0.0631,0.00000,0,2,null,null,"ROI_PACING"],[21245,526398,"jk自动ROI-排3K-xcztz-0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,96.00,2,48.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526399,"jk自动ROI-排3K-xcztz-0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2810.31,69,40.7291304,0,0.000000,0.00,0E-7,0.1806,0.00000,0,2,null,null,"ROI_PACING"],[21245,526400,"jk自动ROI-排3K-xcztz-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,258.19,2,129.0950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526576,"DM自动ROI-xcztz-排3K-0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1623.09,201,8.0750746,1,1623.090000,30.00,0.0184833,0.0448,0.00498,0,2,null,null,"ROI_PACING"],[21245,526578,"DM自动ROI-xcztz-排3K-0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,768.07,38,20.2123684,0,0.000000,0.00,0E-7,0.1842,0.00000,0,2,null,null,"ROI_PACING"],[21245,526579,"DM自动ROI-xcztz-排3K-0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,124.50,6,20.7500000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"ROI_PACING"],[21245,526580,"DM自动ROI-xcztz-排3K-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,889.50,5,177.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,526582,"DM自动ROI-xcztz-排3K-0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,64.91,7,9.2728571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_PACING"],[21245,527263,"每次-2762-冲冲冲_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2459.01,25,98.3604000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,527264,"每次-2762-冲冲冲_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.74,33,84.9618182,0,0.000000,0.00,0E-7,0.0541,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,527265,"每次-2762-冲冲冲_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.74,33,84.9618182,0,0.000000,0.00,0E-7,0.0303,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,527266,"每次-2762-冲冲冲_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,12565.10,116,108.3198276,3,4188.366667,3028.00,0.2409850,0.0704,0.02586,0,2,null,null,"BID_PER_ACTION"],[21245,527267,"每次-2762-冲冲冲_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,10907.45,116,94.0297414,3,3635.816667,2734.00,0.2506544,0.0847,0.02586,1,2,null,null,"BID_PER_ACTION"],[21245,527268,"每次-2762-冲冲冲_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2803.74,35,80.1068571,1,2803.740000,12.00,0.0042800,0.0278,0.02857,0,2,null,null,"BID_PER_ACTION"],[21245,528100,"每次-2762-xcztz-j9-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,243.57,1,243.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21245,528103,"每次-2762-xcztz-j9-0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4018.50,43,93.4534884,1,4018.500000,128.00,0.0318527,0.0870,0.02326,0,2,null,null,"BID_PER_ACTION"],[21245,529374,"JK-自动ROI排3K周日发力-0313_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,524.14,48,10.9195833,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21245,529375,"JK-自动ROI排3K周日发力-0313_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,666.41,45,14.8091111,1,666.410000,60.00,0.0900347,0.0667,0.02222,1,0,null,null,"ROI_PACING"],[21245,529376,"JK-自动ROI排3K周日发力-0313_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3473.89,103,33.7270874,1,3473.890000,208.00,0.0598752,0.0854,0.00971,1,0,null,null,"DEEP_BID_DEFAULT"],[21245,529378,"JK-自动ROI排3K周日发力-0313_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,102.88,3,34.2933333,0,0.000000,0.00,0E-7,0.5000,0.00000,0,0,null,null,"ROI_PACING"],[21245,529399,"每次-周末冲刺-0314_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2802.70,31,90.4096774,1,2802.700000,12.00,0.0042816,0.1613,0.03226,1,0,null,null,"BID_PER_ACTION"],[21245,529400,"每次-周末冲刺-0314_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,26,107.8361538,0,0.000000,0.00,0E-7,0.1154,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529401,"每次-周末冲刺-0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,38,73.7826316,0,0.000000,0.00,0E-7,0.0526,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529402,"每次-周末冲刺-0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,62,45.2216129,0,0.000000,0.00,0E-7,0.0323,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529403,"每次-周末冲刺-0314_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1781.02,20,89.0510000,0,0.000000,0.00,0E-7,0.0500,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529404,"每次-周末冲刺-0314_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,173,16.2065896,2,1401.870000,422.00,0.1505132,0.0578,0.01156,2,0,null,null,"BID_PER_ACTION"],[21245,529405,"每次-周末冲刺-0314_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,25,112.1496000,0,0.000000,0.00,0E-7,0.0400,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529406,"每次-周末冲刺-0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,21,133.5114286,1,2803.740000,12.00,0.0042800,0.1429,0.04762,0,0,null,null,"BID_PER_ACTION"],[21245,529407,"每次-周末冲刺-0314_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,74,37.8883784,1,2803.740000,42.00,0.0149800,0.0405,0.01351,1,0,null,null,"BID_PER_ACTION"],[21245,529408,"每次-周末冲刺-0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,10140.91,84,120.7251190,3,3380.303333,3212.00,0.3167369,0.0769,0.03571,3,0,null,null,"BID_PER_ACTION"],[21245,529409,"每次-周末冲刺-0314_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1051.60,3,350.5333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529410,"每次-周末冲刺-0314_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,41,68.3839024,0,0.000000,0.00,0E-7,0.1220,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529411,"每次-周末冲刺-0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,457.52,1,457.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529413,"每次-周末冲刺-0314_17","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,472.52,1,472.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529415,"每次-周末冲刺-0314_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2803.74,18,155.7633333,2,1401.870000,200.00,0.0713333,0.0556,0.11111,2,0,null,null,"BID_PER_ACTION"],[6867,494510,"0219-自动ROI-MG540迷岛_31304077","2021-03-16 00:00:00","2021-02-19 00:00:00",10,1001400,1,7,8766.47,92,95.2877174,1,8766.470000,1918.00,0.2187882,0.0652,0.01087,0,2,null,null,"ROI_PACING"],[20541,506968,"RICH-财神到 -研究-通投_横SSR2138_0226_16","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,7,28759.15,336,85.5927083,7,4108.450000,1052.00,0.0365797,0.0944,0.02083,3,2,null,null,"DEEP_BID_DEFAULT"],[20541,507537,"RICH-财神到-研究所_IOS通投_横SSR2363_0226_3","2021-03-16 00:00:00","2021-02-26 00:00:00",10,1001441,2,7,64430.56,803,80.2373101,25,2577.222400,5956.00,0.0924406,0.0844,0.03113,16,2,null,null,"DEEP_BID_DEFAULT"],[20541,508920,"RICH-夜请财神-研究所-通投-_横SSR2402_0227_23","2021-03-16 00:00:00","2021-02-27 00:00:00",10,1001441,2,7,705.83,6,117.6383333,0,0.000000,0.00,0E-7,0.1429,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,508922,"RICH-夜请财神-研究所-通投-_横SSR2499_0227_22","2021-03-16 00:00:00","2021-02-27 00:00:00",10,1001441,2,7,1.49,0,1.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,513880,"RICH-研究所-招R体-通投_横SSR2248_0303_12","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,7,33458.08,428,78.1730841,6,5576.346667,738.00,0.0220575,0.1297,0.01402,4,2,null,null,"DEEP_BID_DEFAULT"],[20541,513881,"RICH-研究所-招R体-通投_横SSR2185_0303_1","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,7,279.21,2,139.6050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,513883,"RICH-研究所-招R体-通投_横SSR2666_0303_10","2021-03-16 00:00:00","2021-03-03 00:00:00",10,1001441,2,7,5628.34,56,100.5060714,1,5628.340000,12.00,0.0021321,0.1724,0.01786,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,520811,"RICH-研究所-财神到我家-通投_横SSR2723_0308_22","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,1710.45,15,114.0300000,0,0.000000,0.00,0E-7,0.0667,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,520813,"RICH-研究所-财神到我家-通投_横SSR2726_0308_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,769.39,8,96.1737500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,520814,"RICH-研究所-财神到我家-通投_横SSR2332_0308_24","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,310.05,5,62.0100000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,520818,"RICH-研究所-财神到我家-通投_横SSR2724_0308_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,4377.33,14,312.6664286,1,4377.330000,188.00,0.0429486,0.0714,0.07143,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,523052,"RICH-研究所-财神到我家-通投_横SSR2248_0309_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,3674.39,50,73.4878000,1,3674.390000,12.00,0.0032658,0.0769,0.02000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,523053,"RICH-研究所-财神到我家-通投_横SSR2363_0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,16042.29,233,68.8510300,4,4010.572500,520.00,0.0324143,0.0802,0.01717,2,2,null,null,"DEEP_BID_DEFAULT"],[20541,523054,"RICH-研究所-财神到我家-通投_横SSR2138_0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,20969.78,184,113.9661957,6,3494.963333,4062.00,0.1937073,0.0987,0.03261,1,2,null,null,"DEEP_BID_DEFAULT"],[20541,523055,"RICH-研究所-财神到我家-通投_横SSR1806_0309_17","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,3538.52,54,65.5281481,1,3538.520000,12.00,0.0033912,0.0741,0.01852,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,523056,"RICH-研究所-财神到我家-通投_横SSR2536_0309_19","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,9661.16,114,84.7470175,4,2415.290000,268.00,0.0277399,0.1651,0.03509,1,2,null,null,"DEEP_BID_DEFAULT"],[20541,523057,"RICH-研究所-财神到我家-通投_横SSR2495_0309_16","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,913.93,10,91.3930000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,523058,"RICH-研究所-财神到我家-通投_横SSR2519_0309_20","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,2098.87,8,262.3587500,1,2098.870000,42.00,0.0200108,0.2000,0.12500,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,523059,"RICH-研究所-财神到我家-通投_横SSR2710_0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,3835.24,35,109.5782857,2,1917.620000,90.00,0.0234666,0.1579,0.05714,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,523060,"RICH-研究所-财神到我家-通投_横SSR2285_0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,3483.84,48,72.5800000,0,0.000000,0.00,0E-7,0.0204,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,523061,"RICH-研究所-财神到我家-通投_横SSR1909_0309_18","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001441,2,7,9187.43,139,66.0966187,2,4593.715000,182.00,0.0198097,0.0365,0.01439,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526237,"RICH-研究所-财神到我家_通投-男女通投-_SSR2248_0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,3281.97,28,117.2132143,1,3281.970000,12.00,0.0036563,0.0714,0.03571,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526238,"RICH-研究所-财神到我家_通投-网络-_SSR2248_0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,4367.22,22,198.5100000,2,2183.610000,42.00,0.0096171,0.1364,0.09091,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526239,"RICH-研究所-财神到我家_系统推荐-_SSR2248_0311_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,1417.69,9,157.5211111,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526240,"RICH-研究所-财神到我家_通投-年龄-_SSR2248_0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,1441.36,5,288.2720000,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526241,"RICH-研究所-财神到我家_通投-包过滤-_SSR2248_0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,788.88,8,98.6100000,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526242,"RICH-研究所-财神到我家_通投-新用户-_SSR2248_0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,1564.17,6,260.6950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526243,"RICH-研究所-财神到我家_通投-不过滤-_SSR2248_0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,136.61,1,136.6100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526244,"RICH-研究所-财神到我家_通投-年龄-_SSR2248_0311_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,1519.14,11,138.1036364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526245,"RICH-研究所-财神到我家_通投-手机价格-_SSR2248_0311_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,86.71,1,86.7100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526246,"RICH-研究所-财神到我家_通投-手机价格-_SSR2248_0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,536.42,4,134.1050000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526247,"RICH-研究所-财神到我家_通投-网络-_SSR2248_0311_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,1501.44,9,166.8266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526248,"RICH-研究所-财神到我家_通投-新用户-_SSR2248_0311_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,4837.61,37,130.7462162,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526250,"RICH-研究所-财神到我家_通投-男女通投-_SSR2248_0311_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,517.95,4,129.4875000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526251,"RICH-研究所-财神到我家_通投-包过滤-_SSR2248_0311_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,4703.50,40,117.5875000,0,0.000000,0.00,0E-7,0.1081,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528016,"RICH-研究所-财神到我家_通投_横SSR2285_0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,2939.18,35,83.9765714,0,0.000000,0.00,0E-7,0.0800,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528017,"RICH-研究所-财神到我家_通投_横SSR2495_0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,1159.05,7,165.5785714,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528019,"RICH-研究所-财神到我家_通投_横SSR2710_0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,5999.32,53,113.1947170,2,2999.660000,54.00,0.0090010,0.1250,0.03774,2,2,null,null,"DEEP_BID_DEFAULT"],[20541,528020,"RICH-研究所-财神到我家_通投_横SSR2138_0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,3195.16,24,133.1316667,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528021,"RICH-研究所-财神到我家_通投_横SSR1909_0312_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,744.25,18,41.3472222,1,744.250000,30.00,0.0403090,0.1333,0.05556,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528025,"RICH-研究所-财神到我家_通投_横SSR2138_0312_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,3554.25,21,169.2500000,1,3554.250000,98.00,0.0275726,0.5000,0.04762,1,2,null,null,"DEEP_BID_DEFAULT"],[20546,519646,"0306wtd末日研究所_2720_4","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001441,2,7,343.35,4,85.8375000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,520856,"0308wtd末日研究所_2723_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,209.28,6,34.8800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,520857,"0308wtd末日研究所_2332_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,5842.79,49,119.2406122,0,0.000000,0.00,0E-7,0.0385,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,520860,"0308wtd末日研究所_2558_4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,6121.92,87,70.3668966,3,2040.640000,576.00,0.0940881,0.1087,0.03448,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,520861,"0308wtd末日研究所_2726_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,1.88,2,0.9400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,520863,"0308wtd末日研究所_2725_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,449.55,3,149.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526922,"0311wtd末日研究所_2754_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,2899.50,18,161.0833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526923,"0311wtd末日研究所_2760_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,3147.48,11,286.1345455,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526925,"0311wtd末日研究所_2736_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,3575.59,15,238.3726667,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526926,"0311wtd末日研究所_2248_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,1834.15,12,152.8458333,1,1834.150000,110.00,0.0599733,0.0769,0.08333,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526931,"0311ph648末日研究所_2697_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,3629.38,18,201.6322222,0,0.000000,0.00,0E-7,0.0526,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526932,"0311ph648末日研究所_2724_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,1421.90,9,157.9888889,1,1421.900000,66.00,0.0464168,0.3000,0.11111,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526934,"0311ph648末日研究所_1909_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,3738.32,79,47.3205063,0,0.000000,0.00,0E-7,0.0444,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526935,"0311ph648末日研究所_2716_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,4437.43,52,85.3351923,1,4437.430000,30.00,0.0067607,0.0909,0.01923,0,2,null,null,"DEEP_BID_DEFAULT"],[20546,526937,"0311ph648末日研究所_2757_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,4181.65,55,76.0300000,0,0.000000,0.00,0E-7,0.0536,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520892,"0308XM-MGTTN1计划_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,77.02,1,77.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520894,"0308XM-MGTTN1计划_8","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,14.27,2,7.1350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520895,"0308XM-MGTTN1计划_6","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,10369.96,99,104.7470707,5,2073.992000,340.00,0.0327870,0.0865,0.05051,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520897,"0308XM-MGTTN1计划_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,1306.16,8,163.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520898,"0308XM-MGTTN1计划_4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,738.67,12,61.5558333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520900,"0308XM-MGTTN1计划_10","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,374.41,5,74.8820000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520901,"0308XM-MGTTN1计划_11","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,109.23,1,109.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520902,"0308XM-MGTTN1计划_12","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,16.82,0,16.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520904,"0308XM-MGTTN1计划_14","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,13.91,1,13.9100000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520905,"0308XM-MGTTN1计划_15","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,25.21,3,8.4033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520906,"0308XM-MGTTN1计划_16","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,3.07,2,1.5350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520907,"0308XM-MGTTN1计划_18","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,32.95,1,32.9500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520909,"0308XM-MGTTN1计划_20","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,120.55,1,120.5500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520910,"0308XM-MGTTN1计划_21","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,4.22,0,4.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520913,"0308XM-MGTTN1计划_24","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,2.77,0,2.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520915,"0308XM-MGTTN1计划_26","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,39.02,1,39.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520918,"0308XM-MGTTN1计划_30","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,5.46,2,2.7300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522583,"0309-XM-N1计划-来点充值吧哥哥_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,7298.12,121,60.3150413,3,2432.706667,328.00,0.0449431,0.0840,0.02479,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522584,"0309-XM-N1计划-来点充值吧哥哥_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,1319.37,14,94.2407143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522585,"0309-XM-N1计划-来点充值吧哥哥_12","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,5574.21,64,87.0970313,1,5574.210000,196.00,0.0351619,0.0896,0.01563,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522586,"0309-XM-N1计划-来点充值吧哥哥_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,1079.63,7,154.2328571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522587,"0309-XM-N1计划-来点充值吧哥哥_14","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,3652.79,40,91.3197500,0,0.000000,0.00,0E-7,0.0930,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522588,"0309-XM-N1计划-来点充值吧哥哥_13","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,1000.97,3,333.6566667,1,1000.970000,12.00,0.0119884,0.3333,0.33333,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522631,"0309-XM-N1计划-来点充值吧哥哥_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,100.86,1,100.8600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522636,"0309-XM-N1计划-来点充值吧哥哥_6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,1023.26,17,60.1917647,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522638,"0309-XM-N1计划-来点充值吧哥哥_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,63.15,1,63.1500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522639,"0309-XM-N1计划-来点充值吧哥哥_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,488.97,2,244.4850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522640,"0309-XM-N1计划-来点充值吧哥哥_9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,2239.62,33,67.8672727,0,0.000000,0.00,0E-7,0.1714,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522641,"0309-XM-N1计划-来点充值吧哥哥_10","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,6.87,1,6.8700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522642,"0309-XM-N1计划-来点充值吧哥哥_11","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,668.31,0,668.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522643,"0309-XM-N1计划-来点充值吧哥哥_15","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,784.75,2,392.3750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522644,"0309-XM-N1计划-来点充值吧哥哥_16","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,154.24,1,154.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522645,"0309-XM-N1计划-来点充值吧哥哥_17","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,983.84,7,140.5485714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522649,"0309-XM-N1计划-来点充值吧哥哥_21","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,235.01,2,117.5050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522650,"0309-XM-N1计划-来点充值吧哥哥_22","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,209.34,0,209.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528805,"0312XM-MGTTN1计划_3_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,579.92,5,115.9840000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528806,"0312XM-MGTTN1计划_3_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,895.17,10,89.5170000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528807,"0312XM-MGTTN1计划_3_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,325.78,4,81.4450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528808,"0312XM-MGTTN1计划_3_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,173.45,4,43.3625000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528811,"0312XM-MGTTN1计划_3_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,79.67,3,26.5566667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528812,"0312XM-MGTTN1计划_3_8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,1612.12,39,41.3364103,1,1612.120000,268.00,0.1662407,0.0303,0.02564,1,2,null,null,"DEEP_BID_DEFAULT"],[20143,528813,"0312XM-MGTTN1计划_3_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,951.01,5,190.2020000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,519147,"cxq-tt548N1计划-0306_474-794车2558","2021-03-16 00:00:00","2021-03-06 00:00:00",10,1001447,2,7,0.06,2,0.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520218,"cxq-ttN1计划-0307_16","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001447,2,7,180.18,5,36.0360000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520219,"cxq-ttN1计划-0307_19","2021-03-16 00:00:00","2021-03-07 00:00:00",10,1001447,2,7,287.57,6,47.9283333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520795,"cxq-tt894N1计划-选0308_040-5SSR2332","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,96.07,1,96.0700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520796,"cxq-tt894N1计划-选0308_474-794SSR2332","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,45.02,1,45.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520799,"cxq-tt548N1计划-选0308_474-794SSR2248","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,2786.42,58,48.0417241,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520801,"cxq-tt548N1计划-选0308_040-5SSR2248","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,3985.29,65,61.3121538,0,0.000000,0.00,0E-7,0.0714,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520802,"cxq-tt894N1计划-选0308_736-3SSR2332","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,1110.99,20,55.5495000,0,0.000000,0.00,0E-7,0.0417,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520804,"cxq-tt548N1计划-选0308_474-794SSR2659","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,19.90,0,19.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520805,"cxq-tt548N1计划-选0308_040-5SSR2659","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,28.65,1,28.6500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521489,"cxq-tt548N1计划-车-0308包2车2558","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,3327.00,84,39.6071429,0,0.000000,0.00,0E-7,0.0208,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521490,"cxq-tt894N1计划-小0308_包22332","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,1368.62,18,76.0344444,0,0.000000,0.00,0E-7,0.0500,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521491,"cxq-tt548N1计划-车-0308包2车2248","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,309.25,11,28.1136364,0,0.000000,0.00,0E-7,0.0833,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521493,"cxq-tt548N1计划-车-0308包1车2558","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,1958.78,55,35.6141818,0,0.000000,0.00,0E-7,0.0294,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521494,"cxq-tt548N1计划-车-0308包1车2248","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,0.01,9,0.0011111,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521495,"cxq-tt894N1计划-小0308_包32332","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,152.07,1,152.0700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521496,"cxq-tt894N1计划-小0308_包12332","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,283.18,5,56.6360000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521497,"cxq-tt548N1计划-车-0308包3车2558","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,3311.21,68,48.6942647,1,3311.210000,42.00,0.0126842,0.0571,0.01471,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521498,"cxq-tt894N1计划-小0308_包22728","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,33.27,1,33.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521499,"cxq-tt548N1计划-车-0308包2车2729","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,462.37,3,154.1233333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521500,"cxq-tt548N1计划-车-0308包1车2729","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,429.60,4,107.4000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521501,"cxq-tt894N1计划-小0308_包12728","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,70.42,0,70.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521503,"cxq-tt548N1计划-车-0308包3车2729","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,38.28,1,38.2800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522447,"cxq-tt548N1计划-0309_排3000拉环SSR2332","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,316.62,3,105.5400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525141,"cxq-tt548N1计划-0310_春节包通投小SSR2735","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,651.40,5,130.2800000,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525142,"cxq-tt548N1计划-0310_春节包通投小SSR2731","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,848.39,11,77.1263636,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525144,"cxq-tt548N1计划-0310_包12742","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,10143.11,118,85.9585593,3,3381.036667,788.00,0.0776882,0.0932,0.02542,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525146,"cxq-tt548N1计划-0310_包12741","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,1201.99,8,150.2487500,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525147,"cxq-tt548N1计划-0310_包12743","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,177.01,1,177.0100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525149,"cxq-tt894N1计划-0310_春节包通投2743","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,1142.33,9,126.9255556,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525151,"cxq-tt894N1计划-0310_春节包通投2741","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,1327.82,11,120.7109091,2,663.910000,24.00,0.0180747,0.0909,0.18182,2,2,null,null,"DEEP_BID_DEFAULT"],[20647,525152,"cxq-tt894N1计划-0310_春节包通投2742","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,188.47,2,94.2350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526681,"cxq-tt548N1计划-0311__3小2755","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,2189.54,22,99.5245455,0,0.000000,0.00,0E-7,0.0455,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526683,"cxq-tt548N1计划-0311_12742","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,8044.04,62,129.7425806,2,4022.020000,128.00,0.0159124,0.0000,0.03226,1,2,null,null,"DEEP_BID_DEFAULT"],[20647,526684,"cxq-tt894N1计划-0311_22558","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,5940.35,60,99.0058333,1,5940.350000,12.00,0.0020201,0.1000,0.01667,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526685,"cxq-tt894N1计划-0311_4小2753","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,775.98,4,193.9950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526687,"cxq-tt894N1计划-0311_3小2755","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,696.73,4,174.1825000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526688,"cxq-tt894N1计划-0311_6小2723","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,2369.93,9,263.3255556,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526694,"cxq-tt548N1计划-0311__52750","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,457.29,6,76.2150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526696,"cxq-tt894N1计划-0311_12742","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,67.50,3,22.5000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,522024,"Ca499-N1-付费拉环-天选之子-0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,3802.28,63,60.3536508,2,1901.140000,54.00,0.0142020,0.0597,0.03175,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,522043,"Ca499-N1-付费拉环-天选之子-0309_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,3802.28,81,46.9417284,0,0.000000,0.00,0E-7,0.0235,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,522044,"Ca499-N1-付费拉环-天选之子-0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,3763.39,45,83.6308889,0,0.000000,0.00,0E-7,0.0612,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,523642,"Ca499-n1计划-付费拉环-天选之子-0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,513.54,6,85.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,523643,"Ca499-n1计划-付费拉环-天选之子-0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,3154.27,63,50.0677778,0,0.000000,0.00,0E-7,0.0139,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,523648,"Ca499-n1计划-付费拉环-天选之子-0309_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,3091.65,54,57.2527778,1,3091.650000,12.00,0.0038814,0.0441,0.01852,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,526695,"N1穿山甲-iOS-499-0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,1426.05,12,118.8375000,0,0.000000,0.00,0E-7,0.0625,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,526701,"N1穿山甲-iOS-499-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,4974.20,42,118.4333333,0,0.000000,0.00,0E-7,0.0889,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,526703,"N1穿山甲-iOS-499-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,13291.56,98,135.6281633,5,2658.312000,3120.00,0.2347354,0.1059,0.05102,3,2,null,null,"DEEP_BID_DEFAULT"],[21246,526704,"N1穿山甲-iOS-499-0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,1951.79,21,92.9423810,0,0.000000,0.00,0E-7,0.1250,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528854,"付费-穿山甲-n1-499-0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,380.80,5,76.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528855,"付费-穿山甲-n1-499-0312_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,178.98,2,89.4900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528857,"付费-穿山甲-n1-499-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,1071.59,15,71.4393333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,529419,"穿-n1-周末付费冲-0314_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,7,3039.72,18,168.8733333,2,1519.860000,90.00,0.0296080,0.1667,0.11111,1,0,null,null,"DEEP_BID_DEFAULT"],[21246,529421,"穿-n1-周末付费冲-0314_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,7,129.76,1,129.7600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529422,"穿-n1-周末付费冲-0314_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,7,267.40,1,267.4000000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[6770,522166,"7tt_n_每次付费_幸存危机_0309_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,2961.29,36,82.2580556,1,2961.290000,158.00,0.0533551,0.1463,0.02778,1,2,null,null,"BID_PER_ACTION"],[6770,522167,"7tt_n_每次付费_幸存危机_0309_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,507.80,6,84.6333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522168,"7tt_n_每次付费_幸存危机_0309_6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,181.51,5,36.3020000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522169,"7tt_n_每次付费_幸存危机_0309_9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,2436.30,41,59.4219512,1,2436.300000,30.00,0.0123138,0.2051,0.02439,0,2,null,null,"BID_PER_ACTION"],[6770,522170,"7tt_n_每次付费_幸存危机_0309_7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,1388.10,18,77.1166667,1,1388.100000,140.00,0.1008573,0.0556,0.05556,0,2,null,null,"BID_PER_ACTION"],[6770,522171,"7tt_n_每次付费_幸存危机_0309_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,782.75,5,156.5500000,0,0.000000,0.00,0E-7,0.2000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522172,"7tt_n_每次付费_幸存危机_0309_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,1872.09,40,46.8022500,0,0.000000,0.00,0E-7,0.0750,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522173,"7tt_n_每次付费_幸存危机_0309_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,197.05,1,197.0500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,522174,"7tt_n_每次付费_幸存危机_0309_10","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,2916.86,30,97.2286667,1,2916.860000,60.00,0.0205701,0.1333,0.03333,0,2,null,null,"BID_PER_ACTION"],[6770,522175,"7tt_n_每次付费_幸存危机_0309_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,12956.13,376,34.4577926,4,3239.032500,7976.00,0.6156159,0.1138,0.01064,1,2,null,null,"BID_PER_ACTION"],[6770,524789,"7tt_n_幸存危机_每次付费__0310_72275170","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,820.12,9,91.1244444,0,0.000000,0.00,0E-7,0.2222,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524791,"7tt_n_幸存危机_每次付费__0310_72275202","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,228.54,1,228.5400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524792,"7tt_n_幸存危机_每次付费__0310_72275203","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,522.75,4,130.6875000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524793,"7tt_n_幸存危机_每次付费__0310_72275191","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,192.55,0,192.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,524794,"7tt_n_幸存危机_每次付费__0310_72275226","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,504.54,2,252.2700000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524795,"7tt_n_幸存危机_每次付费__0310_72275227","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,362.88,4,90.7200000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524796,"7tt_n_幸存危机_每次付费__0310_72275204","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,198.02,1,198.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524801,"7tt_n_幸存危机_每次付费__0310_72275228","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,530.58,4,132.6450000,0,0.000000,0.00,0E-7,0.2500,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524802,"7tt_n_幸存危机_每次付费__0310_72275239","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,815.96,7,116.5657143,1,815.960000,384.00,0.4706113,0.1429,0.14286,0,2,null,null,"BID_PER_ACTION"],[6770,524805,"7tt_n_幸存危机_每次付费__0310_722752412","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,1066.63,20,53.3315000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,524806,"7tt_n_幸存危机_每次付费__0310_722752413","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,1893.46,20,94.6730000,1,1893.460000,358.00,0.1890719,0.0417,0.05000,0,2,null,null,"BID_PER_ACTION"],[6770,524807,"7tt_n_幸存危机_每次付费__0310_722752514","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,16.75,0,16.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526324,"7tt_n_幸存危机_每次付费__0311_52245330","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,422.50,5,84.5000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526325,"7tt_n_幸存危机_每次付费__0311_52245342","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,1476.65,19,77.7184211,0,0.000000,0.00,0E-7,0.1579,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526326,"7tt_n_幸存危机_每次付费__0311_52245343","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,167.07,5,33.4140000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526329,"7tt_n_幸存危机_每次付费__0311_52245356","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,303.05,7,43.2928571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526331,"7tt_n_幸存危机_每次付费__0311_52245358","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,149.17,2,74.5850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526332,"7tt_n_幸存危机_每次付费__0311_52245369","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,617.64,6,102.9400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,526336,"7tt_n_幸存危机_每次付费__0311_522453713","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,135.52,1,135.5200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527935,"7tt_n_幸存危机_每次付费__0312_16295372","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,916.52,7,130.9314286,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527967,"7tt_n_幸存危机_每次付费__0312_16295373","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,612.00,14,43.7142857,0,0.000000,0.00,0E-7,0.3077,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527969,"7tt_n_幸存危机_每次付费__0312_16295385","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,122.29,3,40.7633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527970,"7tt_n_幸存危机_每次付费__0312_16295386","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,547.64,5,109.5280000,1,547.640000,286.00,0.5222409,0.4000,0.20000,1,2,null,null,"BID_PER_ACTION"],[6770,527973,"7tt_n_幸存危机_每次付费__0312_16295389","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,1674.40,31,54.0129032,2,837.200000,208.00,0.1242236,0.1333,0.06452,0,2,null,null,"BID_PER_ACTION"],[6770,527974,"7tt_n_幸存危机_每次付费__0312_162953910","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,450.73,2,225.3650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527975,"7tt_n_幸存危机_每次付费__0312_162953911","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,175.91,2,87.9550000,0,0.000000,0.00,0E-7,0.5000,0.00000,1,2,null,null,"BID_PER_ACTION"],[6770,527976,"7tt_n_幸存危机_每次付费__0312_162953912","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,570.65,5,114.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,527977,"7tt_n_幸存危机_每次付费__0312_162953913","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,609.08,0,609.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529172,"7tt_n_幸存危机_每次付费__0313_84190332","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,228.24,1,228.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529173,"7tt_n_幸存危机_每次付费__0313_84190333","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,726.15,4,181.5375000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529175,"7tt_n_幸存危机_每次付费__0313_84190349","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,1663.14,17,97.8317647,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529176,"7tt_n_幸存危机_每次付费__0313_84190336","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,1766.09,21,84.0995238,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529177,"7tt_n_幸存危机_每次付费__0313_84190347","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,286.56,2,143.2800000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529179,"7tt_n_幸存危机_每次付费__0313_84190334","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,425.15,2,212.5750000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529180,"7tt_n_幸存危机_每次付费__0313_84190335","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,4136.65,26,159.1019231,2,2068.325000,24.00,0.0058018,0.1538,0.07692,1,2,null,null,"BID_PER_ACTION"],[6770,529182,"7tt_n_幸存危机_每次付费__0313_841903411","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,776.36,4,194.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529183,"7tt_n_幸存危机_每次付费__0313_841903412","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,861.68,9,95.7422222,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529184,"7tt_n_幸存危机_每次付费__0313_841903513","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,595.81,5,119.1620000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20451,529020,"CJ-财神到-ROI系03.13-0","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,1330.09,33,40.3057576,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529021,"VR-财神到-ROI系03.13-1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,1076.89,10,107.6890000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529022,"VR-财神到-ROI系03.13-3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,3303.74,73,45.2567123,2,1651.870000,80.00,0.0242150,0.1096,0.02740,2,2,null,null,"ROI_COEFFICIENT"],[20451,529023,"CJ-财神到-ROI系03.13-8","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,3300.27,28,117.8667857,0,0.000000,0.00,0E-7,0.0690,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529024,"CJ-财神到-ROI系03.13-9","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,1612.81,31,52.0261290,1,1612.810000,184.00,0.1140866,0.1613,0.03226,1,2,null,null,"ROI_COEFFICIENT"],[20451,529025,"CJ-财神到-ROI系03.13-7","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,2549.25,51,49.9852941,0,0.000000,0.00,0E-7,0.2807,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529027,"VR-财神到-ROI系03.13-2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,424.82,4,106.2050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529028,"VR-财神到-ROI系03.13-5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,415.98,4,103.9950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529029,"CJ-财神到-ROI系03.13-6","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,2161.62,29,74.5386207,0,0.000000,0.00,0E-7,0.0690,0.00000,0,2,null,null,"ROI_COEFFICIENT"],[20451,529364,"VR-危机-每次03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,547.21,2,273.6050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529365,"CJ-危机-每次03.14_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,726.81,2,363.4050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529366,"VR-危机-每次03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,156.87,1,156.8700000,0,0.000000,0.00,0E-7,1.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529367,"CJ-危机-每次03.14_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,1938.18,20,96.9090000,1,1938.180000,110.00,0.0567543,0.2000,0.05000,1,0,null,null,"BID_PER_ACTION"],[20451,529370,"CJ-危机-每次03.14_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,749.56,4,187.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529371,"CJ-危机-每次03.14_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,720.60,1,720.6000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529372,"VR-危机-每次03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,1100.06,7,157.1514286,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,529373,"VR-危机-每次03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,679.15,3,226.3833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,520611,"0308-507幸存危机每次付费-排除3k_2484_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,1830.32,33,55.4642424,0,0.000000,0.00,0E-7,0.1176,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,520612,"0308-507幸存危机每次付费-排除3k_2723_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,85.14,1,85.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,520613,"0308-507幸存危机每次付费-排除3k_2722_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,1255.43,12,104.6191667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,520614,"0308-507幸存危机每次付费-排除3k_2724_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,1020.18,18,56.6766667,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,520615,"0308-507幸存危机每次付费-排除3k_2700_4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,2286.65,53,43.1443396,1,2286.650000,230.00,0.1005838,0.0909,0.01887,0,2,null,null,"BID_PER_ACTION"],[20545,520869,"0308-508幸存危机-每次付费_2332_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,3458.82,32,108.0881250,2,1729.410000,2730.00,0.7892865,0.1563,0.06250,0,2,null,null,"BID_PER_ACTION"],[20545,520870,"0308-508幸存危机-每次付费_2724_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,475.71,4,118.9275000,1,475.710000,98.00,0.2060079,0.2500,0.25000,0,2,null,null,"BID_PER_ACTION"],[20545,520871,"0308-508幸存危机-每次付费_2728_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,146.07,2,73.0350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,522460,"0309-508幸存危机-每次付费_1806_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,2851.71,47,60.6746809,1,2851.710000,12.00,0.0042080,0.0625,0.02128,0,2,null,null,"BID_PER_ACTION"],[20545,522461,"0309-508幸存危机-每次付费_2285_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,941.42,12,78.4516667,0,0.000000,0.00,0E-7,0.1667,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,522465,"0309-508幸存危机-每次付费_2271_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,3333.46,58,57.4734483,5,666.692000,1216.00,0.3647861,0.2182,0.08621,0,2,null,null,"BID_PER_ACTION"],[20545,522919,"0309-507幸存危机每次付费-排除3k_2285_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,2851.71,59,48.3340678,2,1425.855000,120.00,0.0420800,0.1167,0.03390,0,2,null,null,"BID_PER_ACTION"],[20545,522921,"0309-507幸存危机每次付费-排除3k_2735_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,883.02,5,176.6040000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,522923,"0309-507幸存危机每次付费-排除3k_2737_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,1651.06,31,53.2600000,0,0.000000,0.00,0E-7,0.0645,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,524344,"0310-508幸存危机-每次付费竖版沉浸_2332_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,876.37,12,73.0308333,1,876.370000,316.00,0.3605783,0.0833,0.08333,0,2,null,null,"BID_PER_ACTION"],[20545,524348,"0310-508幸存危机-每次付费竖版沉浸_1909_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,2851.71,90,31.6856667,1,2851.710000,432.00,0.1514881,0.0526,0.01111,0,2,null,null,"BID_PER_ACTION"],[20545,524349,"0310-508幸存危机-每次付费竖版沉浸_2622_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,2179.41,22,99.0640909,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,524351,"0310-508幸存危机-每次付费竖版沉浸_2738_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,5346.93,75,71.2924000,0,0.000000,0.00,0E-7,0.0395,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,524960,"0310-507幸存危机每次付费_2332_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,2178.36,18,121.0200000,0,0.000000,0.00,0E-7,0.0526,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,524961,"0310-507幸存危机每次付费_1909_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,2851.71,60,47.5285000,1,2851.710000,12.00,0.0042080,0.0333,0.01667,0,2,null,null,"BID_PER_ACTION"],[20545,524962,"0310-507幸存危机每次付费_2484_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,1149.02,11,104.4563636,1,1149.020000,42.00,0.0365529,0.0909,0.09091,0,2,null,null,"BID_PER_ACTION"],[20545,526911,"0311-508幸存危机-每次付费竖版沉浸_2332_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,296.61,2,148.3050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,526912,"0311-508幸存危机-每次付费竖版沉浸_2622_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,1201.79,11,109.2536364,0,0.000000,0.00,0E-7,0.0909,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528126,"0312-507幸存危机每次付费_2138_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,1786.32,7,255.1885714,1,1786.320000,452.00,0.2530342,0.1429,0.14286,1,2,null,null,"BID_PER_ACTION"],[20545,528127,"0312-507幸存危机每次付费_2332_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,1790.62,14,127.9014286,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528129,"0312-507幸存危机每次付费_2766_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,1648.36,10,164.8360000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528130,"0312-507幸存危机每次付费_2762_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,1183.56,5,236.7120000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528133,"0312-507幸存危机每次付费_2768_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,6934.74,70,99.0677143,2,3467.370000,96.00,0.0138433,0.1268,0.02857,2,2,null,null,"BID_PER_ACTION"],[20545,528547,"0312-508幸存危机-每次付费竖版沉浸_2332_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,468.15,9,52.0166667,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528550,"0312-508幸存危机-每次付费竖版沉浸_2777_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,194.23,1,194.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528551,"0312-508幸存危机-每次付费竖版沉浸_2762_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,1476.81,12,123.0675000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20545,528553,"0312-508幸存危机-每次付费竖版沉浸_2737_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,2071.09,17,121.8288235,1,2071.090000,30.00,0.0144851,0.1765,0.05882,0,2,null,null,"BID_PER_ACTION"],[21208,521837,"481IC-幸存危机-648大集合-3.9_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,130.68,1,130.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,521840,"481IC-幸存危机-648大集合-3.9_1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,8292.17,48,172.7535417,4,2073.042500,268.00,0.0323196,0.0625,0.08333,0,2,null,null,"BID_PER_ACTION"],[21208,523123,"481IC-幸存危机-648大集合-3.9_8","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,87.37,1,87.3700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,523125,"481IC-幸存危机-648大集合-3.9_7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,4038.04,16,252.3775000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527869,"481IC-幸存危机-648大集合-3.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,551.97,1,551.9700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527870,"481IC-幸存危机-648大集合-3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,1500.64,2,750.3200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527871,"481IC-幸存危机-648大集合-3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,6488.82,23,282.1226087,1,6488.820000,182.00,0.0280482,0.1304,0.04348,1,2,null,null,"BID_PER_ACTION"],[21208,527872,"481IC-幸存危机-648大集合-3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,528.04,3,176.0133333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21208,527873,"481IC-幸存危机-648大集合-3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,4614.92,17,271.4658824,1,4614.920000,48.00,0.0104010,0.3077,0.05882,2,2,null,null,"BID_PER_ACTION"],[21212,524365,"k-每次付费-危机窄定向-03.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,363.94,8,45.4925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,524366,"k-每次付费-危机窄定向-03.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,1401.83,21,66.7538095,0,0.000000,0.00,0E-7,0.0870,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,524367,"k-每次付费-危机窄定向-03.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,7122.45,52,136.9701923,2,3561.225000,1222.00,0.1715702,0.0741,0.03846,1,2,null,null,"BID_PER_ACTION"],[21212,524515,"g-自动ROI-危机-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,3266.98,112,29.1694643,0,0.000000,0.00,0E-7,0.0531,0.00000,0,2,null,null,"ROI_PACING"],[21212,524516,"g-自动ROI-危机-03.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,3261.42,38,85.8268421,1,3261.420000,42.00,0.0128778,0.1750,0.02632,0,2,null,null,"ROI_PACING"],[21212,524517,"g-自动ROI-危机-03.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,3271.03,40,81.7757500,0,0.000000,0.00,0E-7,0.0714,0.00000,0,2,null,null,"ROI_PACING"],[21212,524518,"g-自动ROI-危机-03.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,3262.74,130,25.0980000,0,0.000000,0.00,0E-7,0.0296,0.00000,0,2,null,null,"ROI_PACING"],[21212,524519,"g-自动ROI-危机-03.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,482.63,2,241.3150000,0,0.000000,0.00,0E-7,0.3333,0.00000,0,2,null,null,"ROI_PACING"],[21212,525586,"k-每次付费-幸存危机加量-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,3496.06,21,166.4790476,3,1165.353333,1534.00,0.4387797,0.2222,0.14286,2,2,null,null,"BID_PER_ACTION"],[21212,525587,"k-每次付费-幸存危机加量-03.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,939.76,9,104.4177778,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526341,"i-每次付费-危机无惧315-03.11_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,536.85,2,268.4250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526343,"i-每次付费-危机无惧315-03.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,494.89,4,123.7225000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526344,"i-每次付费-危机无惧315-03.11_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,481.87,4,120.4675000,0,0.000000,0.00,0E-7,0.5000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526346,"i-每次付费-危机无惧315-03.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,451.08,2,225.5400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526348,"k-每次付费-危机无惧315-03.11_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,512.49,5,102.4980000,0,0.000000,0.00,0E-7,0.4000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526350,"k-每次付费-危机无惧315-03.11_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,9.84,2,4.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526352,"k-每次付费-危机无惧315-03.11_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,157.75,0,157.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526353,"k-每次付费-危机无惧315-03.11_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,474.56,2,237.2800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,526354,"k-每次付费-危机无惧315-03.11_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,137.29,1,137.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529114,"k-每次付费-幸存危机-03.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,510.17,2,255.0850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529115,"k-每次付费-幸存危机-03.13_1","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,483.99,1,483.9900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[21212,529122,"k-每次付费-幸存危机-03.13_3","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,669.59,5,133.9180000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20755,526938,"th-黎明血战-3.11_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,5700.24,67,85.0782090,3,1900.080000,414.00,0.0726285,0.0800,0.04478,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526939,"au-黎明血战-3.11_12","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,2336.45,24,97.3520833,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526942,"th-黎明血战-3.11_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,449.79,1,449.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526946,"th-黎明血战-3.11_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,561.51,2,280.7550000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526947,"th-黎明血战-3.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,763.92,4,190.9800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526949,"au-黎明血战-3.11_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,72.45,1,72.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526951,"au-黎明血战-3.11_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,472.17,4,118.0425000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526952,"au-黎明血战-3.11_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,2336.45,22,106.2022727,0,0.000000,0.00,0E-7,0.0417,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526953,"au-黎明血战-3.11_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,445.12,1,445.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526954,"au-黎明血战-3.11_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,2319.28,18,128.8488889,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526955,"th-黎明血战-3.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,620.01,3,206.6700000,0,0.000000,0.00,0E-7,0.6667,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526689,"n-ios-xy-黎明血战-03.11_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,3983.90,38,104.8394737,1,3983.900000,176.00,0.0441778,0.0263,0.02632,1,2,null,null,"DEEP_BID_DEFAULT"],[21213,526690,"n-ios-xy-黎明血战-03.11_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,740.54,4,185.1350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526691,"n-ios-xy-黎明血战-03.11_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,4746.81,42,113.0192857,1,4746.810000,48.00,0.0101121,0.0952,0.02381,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526692,"n-ios-xy-黎明血战-03.11_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,2803.74,25,112.1496000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526698,"n96-ios-xy-黎明血战-03.11_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,594.50,3,198.1666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526702,"n96-ios-xy-黎明血战-03.11_17","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,4752.85,68,69.8948529,1,4752.850000,72.00,0.0151488,0.0676,0.01471,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526705,"n96-ios-xy-黎明血战-03.11_20","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,2650.83,15,176.7220000,1,2650.830000,12.00,0.0045269,0.0000,0.06667,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526706,"n96-ios-xy-黎明血战-03.11_18","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,936.57,8,117.0712500,0,0.000000,0.00,0E-7,0.1111,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526709,"n96-ios-xy-黎明血战-03.11_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,1121.54,6,186.9233333,1,1121.540000,12.00,0.0106996,0.1667,0.16667,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,526710,"n-ios-xy-黎明血战-03.11_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,621.20,3,207.0666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528877,"n96-ios-黎明血战-03.12_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,7,1361.26,11,123.7509091,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528878,"n96-ios-黎明血战-03.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,7,811.47,7,115.9242857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528879,"n96-ios-黎明血战-03.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,7,714.28,12,59.5233333,0,0.000000,0.00,0E-7,0.0769,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528880,"n96-ios-黎明血战-03.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,7,2376.43,30,79.2143333,0,0.000000,0.00,0E-7,0.0556,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528882,"n96-ios-黎明血战-03.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,7,100.92,1,100.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,528883,"n96-ios-黎明血战-03.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001610,2,7,93.10,1,93.1000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"DEEP_BID_DEFAULT"],[21213,529573,"n96-iOS-黎明血战周日-03.14_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,7,4752.88,123,38.6413008,1,4752.880000,80.00,0.0168319,0.0625,0.00813,1,0,null,null,"DEEP_BID_DEFAULT"],[21213,529574,"n96-iOS-黎明血战周日-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,7,1174.32,13,90.3323077,0,0.000000,0.00,0E-7,0.0909,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,529575,"n96-iOS-黎明血战周日-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001610,2,7,1038.24,22,47.1927273,0,0.000000,0.00,0E-7,0.1364,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[6867,529646,"0314-540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4310.70,79,54.5658228,1,4310.700000,98.00,0.0227341,0.0000,0.01266,1,0,null,null,"BID_PER_ACTION"],[6867,529652,"0314-540-每次付费-幸_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,12389.74,515,24.0577476,2,6194.870000,1322.00,0.1067012,0.0000,0.00388,2,0,null,null,"BID_PER_ACTION"],[6867,529655,"0314-540-每次付费-幸_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,5653.59,139,40.6733094,1,5653.590000,110.00,0.0194567,0.0000,0.00719,0,0,null,null,"BID_PER_ACTION"],[6867,529656,"0314-540-每次付费-幸_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4422.75,52,85.0528846,1,4422.750000,12.00,0.0027132,0.0000,0.01923,0,0,null,null,"BID_PER_ACTION"],[6867,529658,"0314-540-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,18874.63,268,70.4277239,4,4718.657500,418.00,0.0221461,0.0000,0.01493,4,0,null,null,"BID_PER_ACTION"],[6867,529919,"0315-204-穿山甲-每次付费-幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2010.23,62,32.4230645,2,1005.115000,518.00,0.2576820,0.0000,0.03226,1,0,null,null,"BID_PER_ACTION"],[6867,529921,"0315-204-穿山甲-每次付费-幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,439.52,21,20.9295238,1,439.520000,12.00,0.0273025,0.0000,0.04762,0,0,null,null,"BID_PER_ACTION"],[6867,530808,"0315FP-ROI系数-MG幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3766.77,14,269.0550000,1,3766.770000,2624.00,0.6966181,0.0000,0.07143,1,0,null,null,"ROI_COEFFICIENT"],[20451,530717,"306-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,7688.27,237,32.4399578,1,7688.270000,30.00,0.0039020,0.0000,0.00422,1,0,null,null,"ROI_COEFFICIENT"],[20451,530719,"303-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3356.80,89,37.7168539,1,3356.800000,256.00,0.0762631,0.0000,0.01124,1,0,null,null,"ROI_COEFFICIENT"],[20451,530936,"FJ-幸存者-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3302.07,47,70.2568085,1,3302.070000,42.00,0.0127193,0.0000,0.02128,0,0,null,null,"BID_PER_ACTION"],[20540,530798,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2285315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2534.93,64,39.6082813,1,2534.930000,12.00,0.0047339,0.0000,0.01563,1,0,null,null,"BID_PER_ACTION"],[20545,530526,"0315oyo393-roi系数-排除3k_2138_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1656.50,19,87.1842105,1,1656.500000,12.00,0.0072442,0.0000,0.05263,0,0,null,null,"ROI_COEFFICIENT"],[20545,530714,"0315-幸存者393z每次付费排除3k_2789_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,124.21,1,124.2100000,1,124.210000,298.00,2.3991627,0.0000,1.00000,1,0,null,null,"BID_PER_ACTION"],[20545,530715,"0315-幸存者393z每次付费排除3k_2792_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,4240.95,69,61.4630435,1,4240.950000,30.00,0.0070739,0.0000,0.01449,1,0,null,null,"BID_PER_ACTION"],[20635,530152,"每0315-tt-xcztz-wyt-rk_68","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,8937.37,78,114.5816667,2,4468.685000,250.00,0.0279724,0.0000,0.02564,2,0,null,null,"BID_PER_ACTION"],[20637,529851,"263-幸存者-MCFC0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,7889.73,135,58.4424444,2,3944.865000,168.00,0.0212935,0.0000,0.01481,1,0,null,null,"BID_PER_ACTION"],[20637,529866,"263-幸存者-MCFC0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2851.71,50,57.0342000,1,2851.710000,128.00,0.0448853,0.0000,0.02000,0,0,null,null,"BID_PER_ACTION"],[20637,529974,"990-幸存者-MCFC-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1068.89,18,59.3827778,1,1068.890000,128.00,0.1197504,0.0000,0.05556,1,0,null,null,"BID_PER_ACTION"],[20637,531204,"05-幸存者-MCFC0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,167.78,3,55.9266667,1,167.780000,392.00,2.3363929,0.0000,0.33333,1,0,null,null,"BID_PER_ACTION"],[20637,531251,"05-幸存者-MCFC0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1168.82,16,73.0512500,1,1168.820000,226.00,0.1933574,0.0000,0.06250,1,0,null,null,"BID_PER_ACTION"],[20646,530670,"cxq-tt982幸存者-ROI系数0315_2762","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1648.92,48,34.3525000,1,1648.920000,188.00,0.1140140,0.0000,0.02083,1,0,null,null,"ROI_COEFFICIENT"],[20754,530906,"jap-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1142.78,6,190.4633333,1,1142.780000,200.00,0.1750118,0.0000,0.16667,1,0,null,null,"BID_PER_ACTION"],[20754,530924,"tw-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1087.12,17,63.9482353,2,543.560000,3096.00,2.8478917,0.0000,0.11765,2,0,null,null,"BID_PER_ACTION"],[20754,530959,"ka-roi系数-周一-3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,4853.92,117,41.4864957,4,1213.480000,182.00,0.0374955,0.0000,0.03419,1,0,null,null,"ROI_COEFFICIENT"],[20754,530961,"ka-roi系数-周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,5097.48,91,56.0162637,2,2548.740000,652.00,0.1279063,0.0000,0.02198,1,0,null,null,"ROI_COEFFICIENT"],[20754,530962,"ka-roi系数-周一-3.15_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2803.74,72,38.9408333,1,2803.740000,12.00,0.0042800,0.0000,0.01389,0,0,null,null,"ROI_COEFFICIENT"],[20754,530966,"yo-roi系数-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2659.43,26,102.2857692,1,2659.430000,42.00,0.0157929,0.0000,0.03846,1,0,null,null,"ROI_COEFFICIENT"],[20754,530968,"ka-roi系数-周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2115.48,31,68.2412903,2,1057.740000,238.00,0.1125040,0.0000,0.06452,2,0,null,null,"ROI_COEFFICIENT"],[21208,531039,"bdna-非海马云-幸存者挑战-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2803.74,66,42.4809091,1,2803.740000,12.00,0.0042800,0.0000,0.01515,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531040,"bdna-非海马云-幸存者挑战-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2664.20,7,380.6000000,1,2664.200000,12.00,0.0045042,0.0000,0.14286,1,0,null,null,"DEEP_BID_DEFAULT"],[21208,531042,"bdna-非海马云-幸存者挑战-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2393.44,25,95.7376000,1,2393.440000,74.00,0.0309178,0.0000,0.04000,1,0,null,null,"DEEP_BID_DEFAULT"],[21212,530169,"r3-每次付费-幸存者挑战优选位-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2708.30,13,208.3307692,1,2708.300000,12.00,0.0044308,0.0000,0.07692,0,0,null,null,"BID_PER_ACTION"],[21212,530178,"i-每次付费-幸存者挑战只限男优先位-03.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,6533.02,84,77.7740476,1,6533.020000,678.00,0.1037805,0.0000,0.01190,1,0,null,null,"BID_PER_ACTION"],[21212,530212,"r3-每次付费-幸存者挑战优选位-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2035.04,22,92.5018182,1,2035.040000,30.00,0.0147417,0.0000,0.04545,1,0,null,null,"BID_PER_ACTION"],[21245,531193,"roi系数-0315-BY-幸存_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2820.12,162,17.4081481,1,2820.120000,110.00,0.0390054,0.0000,0.00617,1,0,null,null,"ROI_COEFFICIENT"],[21245,531228,"roi系数-0315-BY-幸存_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2948.60,41,71.9170732,1,2948.600000,30.00,0.0101743,0.0000,0.02439,0,0,null,null,"ROI_COEFFICIENT"],[20541,531196,"RICH-研究所-财神到我家-通投_混SSR2796_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,3588.63,19,188.8752632,3,1196.210000,122.00,0.0339963,0.0000,0.15789,3,0,null,null,"DEEP_BID_DEFAULT"],[20541,531198,"RICH-研究所-财神到我家-通投_混SSR2792_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,2750.84,18,152.8244444,1,2750.840000,98.00,0.0356255,0.0000,0.05556,1,0,null,null,"DEEP_BID_DEFAULT"],[20451,530040,"CJ-幸存危机-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,2256.19,17,132.7170588,1,2256.190000,30.00,0.0132968,0.0000,0.05882,1,0,null,null,"BID_PER_ACTION"],[21208,529911,"481IC-幸存危机-2756爆量爆R-3.5_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,4546.33,62,73.3279032,1,4546.330000,12.00,0.0026395,0.0000,0.01613,1,0,null,null,"BID_PER_ACTION"],[21208,529912,"481IC-幸存危机-2756爆量爆R-3.5_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,1254.35,10,125.4350000,1,1254.350000,72.00,0.0574002,0.0000,0.10000,1,0,null,null,"BID_PER_ACTION"],[21208,529915,"481IC-幸存危机-2756爆量爆R-3.5_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,2643.53,27,97.9085185,1,2643.530000,384.00,0.1452603,0.0000,0.03704,1,0,null,null,"BID_PER_ACTION"],[21208,520453,"295-blcb-每次付费爆R爆量_3.8_4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1.18,0,1.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,520539,"0308幸存者393z每次付费莱卡_2725_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,12.74,0,12.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20647,520798,"cxq-tt548N1计划-选0308_474-794SSR2519","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,2.08,0,2.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,520800,"cxq-tt548N1计划-选0308_040-5SSR2519","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,8.38,0,8.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,520816,"RICH-研究所-财神到我家-通投_横SSR2722_0308_1","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001441,2,7,3.55,0,3.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,520825,"470-tv-每次付费爆R爆量_3.8_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,50.48,0,50.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,520872,"0308-508幸存危机-每次付费_2723_4","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,19.65,0,19.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,520873,"0308-508幸存危机-每次付费_2659_5","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001540,1,7,197.43,0,197.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20143,520889,"0308XM-MGTTN1计划_7","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,18.80,0,18.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520893,"0308XM-MGTTN1计划_3","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,1.09,0,1.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520896,"0308XM-MGTTN1计划_9","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,3.00,0,3.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520899,"0308XM-MGTTN1计划_2","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,68.09,0,68.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520908,"0308XM-MGTTN1计划_19","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,3.12,0,3.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,520911,"0308XM-MGTTN1计划_22","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,11.55,0,11.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,520919,"cxq-tt549幸存者游戏-自动ROI-0308_包12363","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1.77,0,1.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520920,"cxq-tt549幸存者游戏-自动ROI-0308_包1SSR2665","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520922,"cxq-tt549幸存者游戏-自动ROI-0308_包2SSR2665","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,0.08,0,0.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520926,"cxq-tt549幸存者游戏-自动ROI-0308_包2小2726","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,8.86,0,8.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520924,"cxq-tt549幸存者游戏-自动ROI-0308_包1小2726","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,4.02,0,4.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520929,"cxq-tt549幸存者游戏-自动ROI-0308_包1小2728","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,6.71,0,6.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520932,"cxq-tt549幸存者游戏-自动ROI-0308_包1SSR2265","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1.24,0,1.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520938,"cxq-tt338幸存者游戏-自动ROI-0308_包2小2727","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1.13,0,1.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520942,"cxq-tt338幸存者游戏-自动ROI-0308_包2SSR2265","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,3.81,0,3.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520949,"cxq-tt747幸存者游戏-自动ROI-0308_包2小2727","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520950,"cxq-tt747幸存者游戏-自动ROI-0308_包2小2726","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2.05,0,2.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520951,"cxq-tt747幸存者游戏-自动ROI-0308_包2小2728","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,1.07,0,1.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,520953,"cxq-tt747幸存者游戏-自动ROI-0308_包2SSR2665","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,521120,"RICH-财神到我家-幸存者-自动ROI-_TT排除_横SSR2566_0308_10","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001379,1,7,2.56,0,2.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20647,521502,"cxq-tt894N1计划-小0308_包32728","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,90.24,0,90.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521504,"cxq-tt894N1计划-小0308_包32727","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,4.55,0,4.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521505,"cxq-tt894N1计划-小0308_包12727","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,7.26,0,7.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,521506,"cxq-tt894N1计划-小0308_包22727","2021-03-16 00:00:00","2021-03-08 00:00:00",10,1001447,2,7,4.68,0,4.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21208,521732,"295-blcb-每次付费爆R爆量_3.9_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,22.69,0,22.6900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,521739,"295-blcb-每次付费爆R爆量_3.9_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,16.34,0,16.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,521750,"295-blcb-每次付费爆R爆量_3.9_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,34.30,0,34.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,521838,"481IC-幸存危机-648大集合-3.9_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,206.96,0,206.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,521839,"481IC-幸存危机-648大集合-3.9_3","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,123.86,0,123.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,521836,"481IC-幸存危机-648大集合-3.9_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,60.74,0,60.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,521899,"FW-幸存者-每次03.09-6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,396.74,0,396.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521901,"FW-幸存者-每次03.09-9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,394.81,0,394.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521905,"FJ-幸存者-每次03.09-5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,294.88,0,294.8800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,521906,"FW-幸存者-每次03.09-5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,135.79,0,135.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,522138,"7tt自建_自动ROI_幸__0309_36742245","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,26.55,0,26.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522137,"7tt自建_自动ROI_幸__0309_36742234","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522140,"7tt自建_自动ROI_幸__0309_36742258","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522144,"7tt自建_自动ROI_幸__0309_36742246","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522141,"7tt自建_自动ROI_幸__0309_367422611","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2.60,0,2.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522139,"7tt自建_自动ROI_幸__0309_36742247","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3.09,0,3.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522142,"7tt自建_自动ROI_幸__0309_36742181","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1.53,0,1.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522143,"7tt自建_自动ROI_幸__0309_367422612","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.68,0,0.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522146,"7tt自建_自动ROI_幸__0309_367422510","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,45.38,0,45.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522147,"7tt自建_自动ROI_幸__0309_36742233","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.67,0,0.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522149,"7tt自建_自动ROI_幸__0309_367422613","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,6.06,0,6.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522150,"7tt自建_自动ROI_幸__0309_367422714","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.48,0,0.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522151,"7tt自建_自动ROI_幸__0309_367422715","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.67,0,0.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522152,"7tt自建_自动ROI_幸__0309_367422716","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,5.65,0,5.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522153,"7tt自建_自动ROI_幸__0309_367422817","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1.25,0,1.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522155,"7tt自建_自动ROI_幸__0309_367422919","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2.44,0,2.4400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522156,"7tt自建_自动ROI_幸__0309_367422920","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1.91,0,1.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522157,"7tt自建_自动ROI_幸__0309_367422921","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.80,0,0.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522158,"7tt自建_自动ROI_幸__0309_367422922","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1.74,0,1.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522160,"7tt自建_自动ROI_幸__0309_367423024","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522162,"7tt自建_自动ROI_幸__0309_367423126","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3.34,0,3.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522161,"7tt自建_自动ROI_幸__0309_367423025","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.46,0,0.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522163,"7tt自建_自动ROI_幸__0309_367423127","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.01,0,0.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522164,"7tt自建_自动ROI_幸__0309_367423228","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.92,0,0.9200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,522165,"7tt自建_自动ROI_幸__0309_367423229","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,25.91,0,25.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20647,522439,"cxq-tt548N1计划-0309_排2000车SSR2710","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,17.27,0,17.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522440,"cxq-tt894N1计划-0309_排3000SSR2710","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,2.41,0,2.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522441,"cxq-tt894N1计划-0309_排3000SSR2185","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,1.87,0,1.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522442,"cxq-tt894N1计划-0309_排3000拉环SSR2332","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,46.51,0,46.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522443,"cxq-tt548N1计划-0309_排2000车SSR2185","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,15.82,0,15.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522453,"cxq-tt548N1计划-0309_排1500录2459","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,42.74,0,42.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522455,"cxq-tt548N1计划-0309_排1500录1306","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,5.40,0,5.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522454,"cxq-tt548N1计划-0309_排3000拉环2735","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,13.70,0,13.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522456,"cxq-tt894N1计划-0309_排3000录2459","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,13.43,0,13.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522457,"cxq-tt894N1计划-0309_排3000拉环2735","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,5.32,0,5.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,522458,"cxq-tt894N1计划-0309_排3000录1306","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,0.66,0,0.6600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20545,522459,"0309-508幸存危机-每次付费_2387_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,45.98,0,45.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,522466,"0309-508幸存危机-每次付费_2735_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,34.63,0,34.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,522475,"306搜-幸存者-ROI系03.09-0","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3.00,0,3.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,522476,"306搜-幸存者-ROI系03.09-9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,20.85,0,20.8500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,522477,"306搜-幸存者-ROI系03.09-6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,5.29,0,5.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,522481,"306搜-幸存者-ROI系03.09-7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,4.20,0,4.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20451,522482,"306搜-幸存者-ROI系03.09-1","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,5.55,0,5.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522633,"0309-XM-N1计划-来点充值吧哥哥_7","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,5.29,0,5.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522646,"0309-XM-N1计划-来点充值吧哥哥_18","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,7.15,0,7.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522647,"0309-XM-N1计划-来点充值吧哥哥_19","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,8.76,0,8.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,522648,"0309-XM-N1计划-来点充值吧哥哥_20","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001447,2,7,54.55,0,54.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21212,522920,"每次付费-幸存者请珍惜机会-03.09_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,956.33,0,956.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,522922,"0309-507幸存危机每次付费-排除3k_2736_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,242.02,0,242.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,522924,"0309-507幸存危机每次付费-排除3k_2532_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,115.66,0,115.6600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,523048,"每0309-tt-xcztz-wyt-rk_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,401.95,0,401.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523105,"cxq-tt552幸存者-0309-每次付费_843-15车SSR2558","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,308.03,0,308.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523110,"cxq-tt549幸存者-0309-每次付费_078-5录1306","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,38.14,0,38.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523113,"cxq-tt982幸存者-0309-每次付费_088-11录2733","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,407.56,0,407.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523114,"cxq-tt549幸存者-0309-每次付费_078-5拉环SSR2532","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,20.81,0,20.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523116,"cxq-tt982幸存者-0309-每次付费_078-5拉环SSR2532","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,5.41,0,5.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523115,"cxq-tt982幸存者-0309-每次付费_088-11录1306","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,347.76,0,347.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523121,"cxq-tt338幸存者-0309-每次付费_车SSR2558","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.25,0,0.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523120,"cxq-tt982幸存者-0309-每次付费_843-2车SSR2668","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,160.61,0,160.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,523119,"cxq-tt338幸存者-0309-每次付费_车SSR2668","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.41,0,0.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,523124,"481IC-幸存危机-648大集合-3.9_9","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,239.82,0,239.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,523126,"481IC-幸存危机-648大集合-3.9_6","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001540,1,7,461.63,0,461.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,523127,"0309oyo393自动roi莱卡_2723_5","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1.27,0,1.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,523128,"0309oyo393自动roi莱卡_2724_4","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,61.00,0,61.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,523129,"0309oyo393自动roi莱卡_2733_2","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,4.05,0,4.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523134,"cxq-tt338幸存者-自动ROI-0309_排除SSR2532","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.98,0,0.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523140,"cxq-tt530幸存者-自动ROI-0309_莱卡2739","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,87.85,0,87.8500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523141,"cxq-tt530幸存者-自动ROI-0309_莱卡SSR2532","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,0.52,0,0.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523133,"cxq-tt747幸存者-自动ROI-0309_品牌2739","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,3.18,0,3.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523173,"cxq-tt338幸存者-自动ROI-0309_排除车SSR2648","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,15.73,0,15.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523165,"cxq-tt747幸存者-自动ROI-0309_品牌车SSR2648","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,7.54,0,7.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523172,"cxq-tt530幸存者-自动ROI-0309_莱卡2738","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,2.40,0,2.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20646,523174,"cxq-tt530幸存者-自动ROI-0309_莱卡车SSR2648","2021-03-16 00:00:00","2021-03-09 00:00:00",10,1001379,1,7,1.58,0,1.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21208,524145,"293bd-末日海马云测试_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,7,2568.77,23,111.6856522,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524146,"293bd-末日海马云测试_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,7,2590.05,0,2590.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524147,"293bd-末日海马云测试_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,7,2696.61,1,2696.6100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524148,"293bd-末日海马云测试_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,7,2467.83,0,2467.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,524149,"293bd-末日海马云测试_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001430,1,7,2546.98,0,2546.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20635,524184,"每0310-tt-xcztz-wyt-rk_15","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,111.52,0,111.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,524186,"每0310-tt-xcztz-wyt-rk_20","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,283.01,0,283.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,524190,"每0310-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,304.64,0,304.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,524197,"每0310-tt-xcztz-wyt-rk_12","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,29.50,0,29.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,524199,"每0310-tt-xcztz-wyt-rk_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,57.48,0,57.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,524200,"每0310-tt-xcztz-wyt-rk_13","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,416.86,0,416.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,524202,"每0310-tt-xcztz-wyt-rk_16","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1069.60,0,1069.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,524203,"每0310-tt-xcztz-wyt-rk_17","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,381.15,0,381.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,524204,"每0310-tt-xcztz-wyt-rk_18","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,145.79,0,145.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,524332,"295-blcb-ROI爆R爆量_3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,26.38,0,26.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[21208,524326,"295-blcb-ROI爆R爆量_3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,57.72,0,57.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[21208,524324,"295-blcb-ROI爆R爆量_3.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,45.48,0,45.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[21212,524368,"k-每次付费-危机窄定向-03.10_2","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,26.82,0,26.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,524541,"05-幸存者-MCFC0310_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,394.42,0,394.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,524763,"r-自动ROI-幸存者挑战排除-03.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,65.73,0,65.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,524797,"7tt_n_幸存危机_每次付费__0310_72275215","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,61.35,0,61.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,524803,"7tt_n_幸存危机_每次付费__0310_722752310","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,24.09,0,24.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,524804,"7tt_n_幸存危机_每次付费__0310_722752311","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,206.40,0,206.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,524820,"hix-每次付费--3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,178.36,0,178.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,524829,"jap-每次付费--3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,7.14,0,7.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,524835,"tw-每次付费--3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,88.52,0,88.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,524924,"306搜-幸存者-ROI系03.10-5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,480.66,0,480.6600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20545,524963,"0310-507幸存危机每次付费_2742_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,202.26,0,202.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,524964,"0310-507幸存危机每次付费_2743_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,239.71,0,239.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525007,"DM每次付费-xcztz-拉环卡车-0310-_24","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,171.41,0,171.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525012,"BY每次付费-xcztz-拉环卡车-0310-_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,105.86,0,105.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525016,"I2每次付费-xcztz-拉环卡车-0310-_10","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,131.94,0,131.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525020,"DM每次付费-xcztz-拉环卡车-0310-_19","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,1.96,0,1.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525021,"DM每次付费-xcztz-拉环卡车-0310-_20","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,21.35,0,21.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,525022,"DM每次付费-xcztz-拉环卡车-0310-_21","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,379.11,0,379.1100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,525071,"每0310-tt-xcztz-wyt-rk新_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,367.75,0,367.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,525066,"每0310-tt-xcztz-wyt-rk新_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,299.45,0,299.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20647,525143,"cxq-tt894N1计划-0310_包1小SSR2731","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,47.78,0,47.7800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,525148,"cxq-tt894N1计划-0310_包1小SSR2735","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001447,2,7,21.14,0,21.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6867,525257,"0310-每次付费-幸存者MG550_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,37.63,0,37.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,525266,"0310-每次付费-幸存者MG550_16","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,46.44,0,46.4400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,525270,"0310oyo393自动roi莱卡_2363_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,26.05,0,26.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,525273,"0310oyo393自动roi莱卡_2743_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,20.52,0,20.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,525319,"0310幸存者393z每次付费莱卡_2743_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,521.49,0,521.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,525386,"470-tv-单付费爆R爆量_3.10_11","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,8.55,0,8.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,525574,"7tt_498_幸存者挑战_自动ROI_3.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,5.18,0,5.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525578,"7tt_498_幸存者挑战_自动ROI_3.10_5","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,12.10,0,12.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525580,"7tt_498_幸存者挑战_自动ROI_3.10_8","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,45.18,0,45.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525575,"7tt_529_幸存者挑战_自动ROI_3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,158.79,0,158.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525579,"7tt_498_幸存者挑战_自动ROI_3.10_1","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,25.13,0,25.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525581,"7tt_498_幸存者挑战_自动ROI_3.10_9","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,24.07,0,24.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,525583,"7tt_498_幸存者挑战_自动ROI_3.10_6","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001379,1,7,13.94,0,13.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21212,525588,"k-每次付费-幸存危机加量-03.10_3","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,123.83,0,123.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,525589,"k-每次付费-幸存危机加量-03.10_4","2021-03-16 00:00:00","2021-03-10 00:00:00",10,1001540,1,7,272.17,0,272.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,525768,"cxq-tt338幸存者-穿山甲-付费-0311_排除SSR2722","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.68,0,0.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,525770,"cxq-tt552幸存者-穿山甲-付费-0311_排除2724","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,13.80,0,13.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,525769,"cxq-tt338幸存者-穿山甲-付费-0311_排除2752","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,7.55,0,7.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,525771,"cxq-tt552幸存者-穿山甲-付费-0311_包22752","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.15,0,0.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,525774,"cxq-tt552幸存者-穿山甲-付费-0311_包2SSR2722","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.04,0,0.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,525772,"cxq-tt552幸存者-穿山甲-付费-0311_包2SSR2723","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.25,0,0.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,525775,"cxq-tt552幸存者-穿山甲-付费-0311_排除2737","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1.07,0,1.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,525776,"cxq-tt338幸存者-穿山甲-付费-0311_排除SSR2723","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4.47,0,4.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,525779,"cxq-tt338幸存者-穿山甲-付费-0311_包22737","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,525781,"cxq-tt338幸存者-穿山甲-付费-0311_包22724","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526249,"RICH-研究所-财神到我家_通投-不过滤-_SSR2248_0311_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,59.95,0,59.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,526252,"RICH-研究所-财神到我家_系统推荐-_SSR2248_0311_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,78.38,0,78.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,526311,"7tt_n_幸存危机_每次付费__0311_52245341","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,26.16,0,26.1600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526327,"7tt_n_幸存危机_每次付费__0311_52245354","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,190.41,0,190.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526328,"7tt_n_幸存危机_每次付费__0311_52245355","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,158.26,0,158.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526330,"7tt_n_幸存危机_每次付费__0311_52245357","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,157.79,0,157.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526334,"7tt_n_幸存危机_每次付费__0311_522453611","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,102.34,0,102.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526333,"7tt_n_幸存危机_每次付费__0311_522453610","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,28.45,0,28.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526337,"7tt_n_幸存危机_每次付费__0311_522453714","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,520.66,0,520.6600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526342,"k-每次付费-危机无惧315-03.11_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,42.15,0,42.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526340,"i-每次付费-危机无惧315-03.11_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,5.38,0,5.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526345,"i-每次付费-危机无惧315-03.11_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,14.07,0,14.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526347,"i-每次付费-危机无惧315-03.11_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,34.77,0,34.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526351,"i-每次付费-危机无惧315-03.11_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,66.01,0,66.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526349,"k-每次付费-危机无惧315-03.11_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,74.08,0,74.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526371,"7tt_y_幸_每次付费__0311_64275012","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,55.79,0,55.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526372,"7tt_y_幸_每次付费__0311_64275026","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,315.43,0,315.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526374,"7tt_y_幸_每次付费__0311_64275037","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,97.19,0,97.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526376,"7tt_y_幸_每次付费__0311_642750511","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,9.66,0,9.6600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526377,"7tt_y_幸_每次付费__0311_642750612","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,9.87,0,9.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,526378,"7tt_y_幸_每次付费__0311_642750613","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,24.33,0,24.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,526406,"263-幸存者-ROI-0311_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,48.41,0,48.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[20635,526428,"每0311-tt-xcztz-wyt-rk_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,562.27,0,562.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,526433,"每0311-tt-xcztz-wyt-rk_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,31.82,0,31.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526441,"r3-每次付费-幸存者挑战-03.11_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,221.57,0,221.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526442,"r3-每次付费-幸存者挑战-03.11_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,585.58,0,585.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526450,"r3-每次付费-幸存者挑战-03.11_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,36.61,0,36.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,526455,"r-每次付费-幸存者挑战-03.11_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,93.48,0,93.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526459,"cxq-tt幸存者-每次付费-排除3000-0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,284.79,0,284.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526466,"cxq-tt幸存者-每次付费-排除3000-0311_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,339.70,0,339.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526477,"cxq-tt幸存者-每次付费-排除3000-0311_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,14.77,0,14.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526480,"cxq-tt幸存者-每次付费-排除3000-0311_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,307.31,0,307.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526532,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2484_0311_27","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,24.66,0,24.6600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,526530,"RICH-财神到我家-幸存者-每次付费_通投-排3K-_横SSR2723_0311_26","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,174.34,0,174.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526537,"cxq-tt幸存者-每次付费-排除负向0311_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,23.07,0,23.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526548,"cxq-tt幸存者-每次付费-排除负向0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,33.50,0,33.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526554,"cxq-tt幸存者-每次付费-排除负向0311_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2.98,0,2.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526550,"cxq-tt幸存者-每次付费-排除负向0311_9","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,40.09,0,40.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526556,"cxq-tt幸存者-每次付费-排除负向0311_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.89,0,0.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526558,"cxq-tt幸存者-每次付费-排除负向0311_15","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,53.01,0,53.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526559,"cxq-tt幸存者-每次付费-排除负向0311_16","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,17.18,0,17.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526561,"cxq-tt幸存者-每次付费-排除负向0311_18","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6.90,0,6.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526560,"cxq-tt幸存者-每次付费-排除负向0311_17","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,288.13,0,288.1300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526562,"cxq-tt幸存者-每次付费-排除负向0311_24","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,23.54,0,23.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526563,"cxq-tt幸存者-每次付费-排除负向0311_25","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,73.47,0,73.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526564,"cxq-tt幸存者-每次付费-排除负向0311_26","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,2.17,0,2.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526571,"cxq-tt幸存者-每次付费-排除负向0311_27","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6.93,0,6.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526573,"cxq-tt幸存者-每次付费-排除负向0311_22","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.81,0,0.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,526570,"cxq-tt幸存者-每次付费-排除负向0311_19","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1.64,0,1.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21245,526577,"DM自动ROI-xcztz-排3K-0311_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,28.86,0,28.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21245,526581,"DM自动ROI-xcztz-排3K-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,191.57,0,191.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20647,526678,"cxq-tt548N1计划-0311__22558","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,27.61,0,27.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526682,"cxq-tt548N1计划-0311__4小2753","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,1.44,0,1.4400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20647,526686,"cxq-tt548N1计划-0311__6小2723","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001447,2,7,0.95,0,0.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,526731,"J9-穿山甲-xcztz-0311_2","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3.17,0,3.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,526732,"J9-穿山甲-xcztz-0311_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1.52,0,1.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,526734,"J9-穿山甲-xcztz-0311_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,6.77,0,6.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,526739,"J9-穿山甲-xcztz-0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.38,0,0.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21245,526738,"J9-穿山甲-xcztz-0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4.63,0,4.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6867,526753,"0311-tve-每次付费-幸_11","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,762.71,11,69.3372727,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526755,"0311-tve-每次付费-幸_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,27.56,0,27.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526759,"0311-tve-每次付费-幸_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,149.99,1,149.9900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6867,526761,"0311-tve-每次付费-幸_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,172.12,0,172.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526762,"0311-tve-每次付费-幸_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,63.57,0,63.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526764,"0311-tve-每次付费-幸_13","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,107.81,0,107.8100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,526765,"0311-tve-每次付费-幸_14","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,17.88,0,17.8800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,526853,"295-blcb-ROI爆R爆量_3.11_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,7.15,0,7.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[21208,526855,"295-blcb-ROI爆R爆量_3.11_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4.45,0,4.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526860,"RICH-财神到我家-幸存者-自动roi-_通投_横SSR2736_0311_10","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,3.22,0,3.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526861,"RICH-财神到我家-幸存者-自动roi-_春节DMP_横SSR1909_0311_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,0.54,0,0.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526864,"RICH-财神到我家-幸存者-自动roi-_游戏_横SSR2736_0311_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,4.12,0,4.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,526863,"RICH-财神到我家-幸存者-自动roi-_通投_横SSR1909_0311_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,1.58,0,1.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20545,526913,"0311-508幸存危机-每次付费竖版沉浸_2736_3","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,1.39,0,1.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,526915,"0311-508幸存危机-每次付费竖版沉浸_2697_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,31.17,0,31.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20545,526914,"0311-508幸存危机-每次付费竖版沉浸_2695_4","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001540,1,7,6.27,0,6.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20546,526924,"0311wtd末日研究所_2733_5","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001441,2,7,828.26,0,828.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526944,"th-黎明血战-3.11_1","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,17.05,0,17.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526948,"au-黎明血战-3.11_8","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,34.94,0,34.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20755,526950,"th-黎明血战-3.11_6","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001610,2,7,380.52,0,380.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20754,527180,"jap-每次付费--3.11_7","2021-03-16 00:00:00","2021-03-11 00:00:00",10,1001379,1,7,32.80,0,32.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527342,"470-tv-每次付费爆R爆量_3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,32.21,0,32.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21208,527347,"470-tv-每次付费爆R爆量_3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,136.61,0,136.6100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,527350,"SR-财神到-每次03.12-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,20.32,0,20.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,527352,"PB-财神到-每次03.12-2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,294.01,0,294.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,527368,"SR-财神到-每次03.12-8","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,64.06,0,64.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527403,"7tt_498_幸存者挑战_每次付费3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,424.50,0,424.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527470,"0312540-每次付费-幸_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,30.66,0,30.6600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527471,"0312540-每次付费-幸_15","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,291.79,0,291.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6867,527477,"0312540-每次付费-幸_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,102.11,0,102.1100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527506,"每0312-tt-xcztz-wyt-rk_18","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,296.72,0,296.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527513,"每0312-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,287.17,0,287.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,527527,"每0312-tt-xcztz-wyt-rk_19","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,120.47,0,120.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527528,"每0312-tt-xcztz-wyt-rk_20","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,46.12,0,46.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20635,527530,"每0312-tt-xcztz-wyt-rk_22","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,92.32,0,92.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,527534,"cxq-tt552幸存者-每次付费穿山甲0312_排负2736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,378.23,0,378.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,527589,"r3-每次付费-幸存者挑战-03.12_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,11.45,0,11.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,527583,"i-每次付费-幸存者挑战-03.12_15","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,47.95,0,47.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,527588,"r3-每次付费-幸存者挑战-03.12_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,45.01,0,45.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527695,"RICH-财神到我家_幸存者-每次付费_排3000_78397_0312_95325553","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,67.24,0,67.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527697,"RICH-财神到我家_幸存者-每次付费_排3000_78431_0312_95325566","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,15.27,0,15.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527794,"RICH-财神到我家_幸存者-每次付费_排3000_75017_0312_95325552","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,41.91,0,41.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,527859,"RICH-财神到我家_幸存者-自动ROI_排3000_78449_0312_01169240","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.38,0,0.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527863,"RICH-财神到我家_幸存者-自动ROI_排3000_67560_0312_01169274","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,204.57,0,204.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527865,"RICH-财神到我家_幸存者-自动ROI_排3000_63730_0312_01169262","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.92,0,0.9200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527866,"RICH-财神到我家_幸存者-自动ROI_排3000_77952_0312_01169251","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4.27,0,4.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527867,"RICH-财神到我家_幸存者-自动ROI_排3000_78397_0312_01169275","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.47,0,0.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[20540,527868,"RICH-财神到我家_幸存者-自动ROI_排3000_73141_0312_01169273","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.49,0,0.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,527934,"7tt_y_幸_每次付费__0312_16294722","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,2.26,0,2.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527942,"7tt_y_幸_每次付费__0312_16294736","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.19,0,0.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527939,"7tt_y_幸_每次付费__0312_16294700","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.97,0,0.9700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527941,"7tt_y_幸_每次付费__0312_16294749","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,38.50,0,38.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527936,"7tt_y_幸_每次付费__0312_16294711","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4.43,0,4.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527943,"7tt_y_幸_每次付费__0312_16294723","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1.30,0,1.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527944,"7tt_y_幸_每次付费__0312_16294748","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.83,0,0.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527945,"7tt_y_幸_每次付费__0312_162947410","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.54,0,0.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527937,"7tt_y_幸_每次付费__0312_16294737","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.72,0,0.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527946,"7tt_y_幸_每次付费__0312_162947511","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,5.67,0,5.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527947,"7tt_y_幸_每次付费__0312_162947512","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,39.00,0,39.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527948,"7tt_y_幸_每次付费__0312_162947513","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,46.75,0,46.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527949,"7tt_y_幸_每次付费__0312_162947614","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,11.35,0,11.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527951,"7tt_y_幸_每次付费__0312_162947616","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1.95,0,1.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527953,"7tt_y_幸_每次付费__0312_162947718","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,72.04,0,72.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527965,"7tt_n_幸存危机_每次付费__0312_16295360","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,87.12,0,87.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527968,"7tt_n_幸存危机_每次付费__0312_16295384","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,223.14,0,223.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527971,"7tt_n_幸存危机_每次付费__0312_16295387","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,80.33,0,80.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527972,"7tt_n_幸存危机_每次付费__0312_16295388","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,101.14,0,101.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,527978,"7tt_n_幸存危机_每次付费__0312_162953914","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001540,1,7,124.40,0,124.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,528001,"RICH-财神到我家_幸存者-每次付费_排3000_69832_0312_22237942","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,252.23,1,252.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20541,528018,"RICH-研究所-财神到我家_通投_横SSR2285_0312_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,64.23,0,64.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528022,"RICH-研究所-财神到我家_通投_横SSR2495_0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528023,"RICH-研究所-财神到我家_通投_横SSR1909_0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,25.84,0,25.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20541,528024,"RICH-研究所-财神到我家_通投_横SSR2710_0312_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001441,2,7,3.57,0,3.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,528062,"7tt_幸_自动ROI__0312_88256793","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,11.03,0,11.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528071,"7tt_幸_自动ROI__0312_88256829","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,10.46,0,10.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528074,"7tt_幸_自动ROI__0312_882568314","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3.43,1,3.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[6770,528075,"7tt_幸_自动ROI__0312_882568415","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,5.17,0,5.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528085,"7tt_幸_自动ROI__0312_882568926","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1.84,0,1.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528086,"7tt_幸_自动ROI__0312_882569027","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,6.40,0,6.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,528088,"7tt_幸_自动ROI__0312_882569129","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,31.56,0,31.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21245,528102,"每次-2762-xcztz-j9-0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,568.63,0,568.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528137,"jap-每次付费--3.12_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,25.50,0,25.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528142,"jap-每次付费--3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,14.88,0,14.8800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528144,"tw-每次付费--3.12_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,29.48,0,29.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528152,"jap-每次付费--3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,13.20,0,13.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528156,"jap-每次付费--3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,13.30,0,13.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528150,"tw-每次付费--3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3.63,0,3.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528155,"hix-每次付费--3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,118.89,0,118.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528160,"zhan-每次付费--3.12_6","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,102.68,0,102.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,528161,"zhan-每次付费--3.12_7","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,84.64,0,84.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528657,"990-幸存者-MCFC-0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,11.99,0,11.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528655,"990-幸存者-MCFC-0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,13.95,0,13.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528656,"990-幸存者-MCFC-0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,40.33,0,40.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528658,"990-幸存者-MCFC-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3.97,0,3.9700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528784,"7tt_穿穿_每次付费__0312_02259772","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,54.58,0,54.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528777,"7tt_穿穿_每次付费__0312_02259799","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,93.54,1,93.5400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528786,"7tt_穿穿_每次付费__0312_022598012","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,3062.35,3,1020.7833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528789,"7tt_穿穿_每次付费__0312_022598015","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,9.07,0,9.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528790,"7tt_穿穿_每次付费__0312_022598016","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.06,0,0.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528793,"7tt_穿穿_每次付费__0312_022598119","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.26,0,0.2600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528794,"7tt_穿穿_每次付费__0312_022598120","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,1.17,0,1.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528796,"7tt_穿穿_每次付费__0312_022598122","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.14,0,0.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528798,"7tt_穿穿_每次付费__0312_022598224","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,4.37,0,4.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528801,"7tt_穿穿_每次付费__0312_022598227","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,6.40,1,6.4000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,528802,"7tt_穿穿_每次付费__0312_022598328","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,0.01,0,0.0100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528803,"7tt_穿穿_每次付费__0312_022598329","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,83.96,0,83.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20143,528804,"0312XM-MGTTN1计划_3_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,24.91,0,24.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528809,"0312XM-MGTTN1计划_3_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,137.94,0,137.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20143,528810,"0312XM-MGTTN1计划_3_10","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,55.30,0,55.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528852,"付费-穿山甲-n1-499-0312_3","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,54.14,0,54.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528856,"付费-穿山甲-n1-499-0312_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,5.03,0,5.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[21246,528858,"付费-穿山甲-n1-499-0312_1","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001447,2,7,57.77,0,57.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[6770,528866,"7tt_498_穿哥_每次付费_3.12_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,107.74,0,107.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,528867,"7tt_498_穿哥_每次付费_3.12_5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,26.11,0,26.1100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528895,"05-幸存者-MCFC0312_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,135.80,0,135.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528897,"05-幸存者-MCFC0312_4","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,100.70,0,100.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528899,"05-幸存者-MCFC0312_2","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,117.74,0,117.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,528919,"FJ-财神到-每次03.13-5","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,11.88,0,11.8800000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20637,528964,"263-幸存者-MCFC0312_9","2021-03-16 00:00:00","2021-03-12 00:00:00",10,1001379,1,7,130.91,0,130.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,529026,"VR-财神到-ROI系03.13-4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,91.94,0,91.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_COEFFICIENT"],[20540,529037,"RICH-财神到我家_幸存者-每次付费_排3000_73012_0313_08286071","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,5.40,0,5.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,529101,"r3-每次付费-幸存者挑战周六-03.13_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,257.39,0,257.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,529121,"k-每次付费-幸存危机-03.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,406.65,0,406.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[21212,529123,"k-每次付费-幸存危机-03.13_2","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,58.37,0,58.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20754,529163,"ka-自动roi-周六-3.13_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,25.70,0,25.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"ROI_PACING"],[6770,529170,"7tt_n_幸存危机_每次付费__0313_84190331","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,66.15,0,66.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529169,"7tt_穿穿_每次付费__0313_84191302","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,6.60,0,6.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529178,"7tt_n_幸存危机_每次付费__0313_84190348","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,61.70,0,61.7000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529181,"7tt_n_幸存危机_每次付费__0313_841903410","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,231.74,0,231.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529174,"7tt_n_幸存危机_每次付费__0313_84190320","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,198.09,0,198.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529185,"7tt_n_幸存危机_每次付费__0313_841903514","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001540,1,7,228.91,0,228.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529186,"7tt_穿穿_每次付费__0313_84191290","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3614.51,14,258.1792857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529189,"7tt_穿穿_每次付费__0313_84191315","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,14.42,0,14.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529190,"7tt_穿穿_每次付费__0313_84191316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,17.70,2,8.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529191,"7tt_穿穿_每次付费__0313_84191317","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3331.83,1,3331.8300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529193,"7tt_穿穿_每次付费__0313_84191329","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2188.97,1,2188.9700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529194,"7tt_穿穿_每次付费__0313_841913210","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2.47,0,2.4700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529196,"7tt_穿穿_每次付费__0313_841913212","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,44.24,1,44.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529197,"7tt_穿穿_每次付费__0313_841913213","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,3780.34,0,3780.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529199,"7tt_穿穿_每次付费__0313_841913214","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,30.13,1,30.1300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[6770,529200,"7tt_穿穿_每次付费__0313_841913316","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,13.36,0,13.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[6770,529203,"7tt_穿穿_每次付费__0313_841913319","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,2.83,0,2.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20540,529218,"RICH-末日-穿山甲-付费_游戏_混SSR2778_0313_5","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,7,9.84,0,9.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20540,529219,"RICH-末日-穿山甲-付费_游戏_混SSR2762_0313_4","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001294,1,7,3.93,0,3.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"DEEP_BID_DEFAULT"],[20646,529228,"cxq-tt552幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,616.77,19,32.4615789,0,0.000000,0.00,0E-7,0.0000,0.00000,0,2,null,null,"BID_PER_ACTION"],[20646,529227,"cxq-tt338幸存者-每次付费-穿山甲0313_27623000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,24.76,0,24.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529224,"cxq-tt338幸存者-每次付费-穿山甲0313_27463000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,61.07,0,61.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529232,"cxq-tt552幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,1.55,0,1.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20646,529231,"cxq-tt338幸存者-每次付费-穿山甲0313_27663000","2021-03-16 00:00:00","2021-03-13 00:00:00",10,1001379,1,7,7.40,0,7.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,2,null,null,"BID_PER_ACTION"],[20451,529368,"CJ-危机-每次03.14_9","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,7.65,0,7.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,529369,"VR-危机-每次03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001540,1,7,67.25,2,33.6250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21245,529377,"JK-自动ROI排3K周日发力-0313_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,101.30,0,101.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21245,529412,"每次-周末冲刺-0314_18","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,33.57,0,33.5700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21245,529414,"每次-周末冲刺-0314_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,170.04,0,170.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21245,529416,"每次-周末冲刺-0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,205.53,0,205.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21246,529417,"穿-n1-周末付费冲-0314_1","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,7,62.60,0,62.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529418,"穿-n1-周末付费冲-0314_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,7,48.83,0,48.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21246,529420,"穿-n1-周末付费冲-0314_6","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001447,2,7,49.93,0,49.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[6770,529462,"7tt_穿穿_每次付费__0314_72230811","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,27.15,0,27.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529461,"7tt_穿穿_每次付费__0314_72230800","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,60.08,0,60.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529464,"7tt_穿穿_每次付费__0314_72230827","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3382.48,10,338.2480000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529466,"7tt_穿穿_每次付费__0314_72230829","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2541.11,3,847.0366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529463,"7tt_穿穿_每次付费__0314_72230825","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,32.16,1,32.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529465,"7tt_穿穿_每次付费__0314_72230826","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,41.00,0,41.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529468,"7tt_穿穿_每次付费__0314_72230813","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1275.59,2,637.7950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529469,"7tt_穿穿_每次付费__0314_72230812","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,164.99,2,82.4950000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6770,529471,"7tt_穿穿_每次付费__0314_72230828","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,13.34,0,13.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6770,529472,"7tt_穿穿_每次付费__0314_722308210","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,0.02,0,0.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529474,"每0314-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,33.83,0,33.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529482,"每0314-tt-xcztz-wyt-rk新_24","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,244.84,0,244.8400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529475,"每0314-tt-xcztz-wyt-rk新_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,41.80,0,41.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529483,"每0314-tt-xcztz-wyt-rk新_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,296.09,0,296.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529481,"每0314-tt-xcztz-wyt-rk新_25","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1.34,0,1.3400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529484,"每0314-tt-xcztz-wyt-rk新_30","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,12.79,0,12.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529485,"每0314-tt-xcztz-wyt-rk新_31","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,0.32,0,0.3200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529486,"每0314-tt-xcztz-wyt-rk新_36","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,321.38,0,321.3800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529496,"每0314-tt-xcztz-wyt-rk新_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,166.99,0,166.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529502,"每0314-tt-xcztz-wyt-rk新_23","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1.51,0,1.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529506,"每0314-tt-xcztz-wyt-rk新_29","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3.64,0,3.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529507,"每0314-tt-xcztz-wyt-rk新_32","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,280.60,0,280.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,529510,"每0314-tt-xcztz-wyt-rk新_35","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3.50,0,3.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529556,"r-每次付费-幸存者挑战-穿山甲专场-03.14_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,7.95,0,7.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529558,"r-每次付费-幸存者挑战-穿山甲专场-03.14_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,9.04,0,9.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529559,"r-每次付费-幸存者挑战-穿山甲专场-03.14_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,15.14,0,15.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21212,529560,"r-每次付费-幸存者挑战-穿山甲专场-03.14_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4.65,0,4.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529588,"jap-每次付费--3.13_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,84.21,0,84.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529606,"tw-每次付费--3.13_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,50.42,0,50.4200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529605,"hix-每次付费--3.13_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,5.80,0,5.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529608,"jap-每次付费--3.13_10","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4.31,0,4.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,529609,"zhan-每次付费--3.13_20","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3.99,0,3.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529645,"0314-540-每次付费-幸_15","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3042.01,142,21.4226056,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529648,"0314-540-每次付费-幸_5","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,601.80,2,300.9000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529643,"0314-540-每次付费-幸_12","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3374.57,60,56.2428333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529644,"0314-540-每次付费-幸_4","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3434.77,33,104.0839394,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529647,"0314-540-每次付费-幸_7","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,4294.94,81,53.0239506,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529650,"0314-540-每次付费-幸_13","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,1410.25,12,117.5208333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529649,"0314-540-每次付费-幸_14","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3594.66,158,22.7510127,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529651,"0314-540-每次付费-幸_2","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,2926.56,53,55.2181132,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529653,"0314-540-每次付费-幸_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,3451.77,59,58.5045763,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529654,"0314-540-每次付费-幸_3","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,945.63,0,945.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529657,"0314-540-每次付费-幸_11","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,641.13,2,320.5650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529667,"0314-tve-每次付费-幸_8","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,25.86,0,25.8600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529673,"0314-tve-每次付费-幸_16","2021-03-16 00:00:00","2021-03-14 00:00:00",10,1001379,1,7,247.68,1,247.6800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529719,"470-tv-每次付费拉环爆R爆量_3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,13.85,1,13.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529715,"470-tv-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,345.64,14,24.6885714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529716,"470-tv-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,851.44,16,53.2150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529718,"470-tv-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,688.95,25,27.5580000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529720,"470-tv-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,118.75,5,23.7500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529722,"470-tv-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,100.57,3,33.5233333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529721,"470-tv-每次付费拉环爆R爆量_3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2803.74,31,90.4432258,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529723,"470-tv-每次付费拉环爆R爆量_3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,137.01,5,27.4020000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529724,"470-tv-每次付费拉环爆R爆量_3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,38.90,0,38.9000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529725,"470-tv-每次付费拉环爆R爆量_3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,589.15,0,589.1500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529747,"295-blcb-每次付费拉环爆R爆量_3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,114.76,3,38.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529749,"295-blcb-每次付费拉环爆R爆量_3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,176.98,8,22.1225000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529748,"295-blcb-每次付费拉环爆R爆量_3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2803.74,18,155.7633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529754,"295-blcb-每次付费拉环爆R爆量_3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,133.70,2,66.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529753,"295-blcb-每次付费拉环爆R爆量_3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1149.38,4,287.3450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529755,"295-blcb-每次付费拉环爆R爆量_3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,213.23,8,26.6537500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529752,"295-blcb-每次付费拉环爆R爆量_3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,8.18,1,8.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529847,"263-幸存者-MCFC0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2619.13,33,79.3675758,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529845,"263-幸存者-MCFC0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,422.61,8,52.8262500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529863,"263-幸存者-MCFC0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,171.92,1,171.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529864,"263-幸存者-MCFC0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,415.87,0,415.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529865,"263-幸存者-MCFC0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1586.14,27,58.7459259,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529910,"0315-204-穿山甲-每次付费-幸_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,17.04,0,17.0400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529909,"0315-204-穿山甲-每次付费-幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,63.17,0,63.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21208,529918,"481IC-幸存危机-2756爆量爆R-3.5_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,287.28,0,287.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529914,"0315-204-穿山甲-每次付费-幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,95.97,4,23.9925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529913,"0315-204-穿山甲-每次付费-幸_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2991.81,59,50.7086441,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21208,529916,"481IC-幸存危机-2756爆量爆R-3.5_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,5607.48,81,69.2281481,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529917,"0315-204-穿山甲-每次付费-幸_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,89.67,7,12.8100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529920,"0315-204-穿山甲-每次付费-幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,806.49,32,25.2028125,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529922,"0315-204-穿山甲-每次付费-幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,14.31,0,14.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529923,"0315-204-穿山甲-每次付费-幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,8.55,0,8.5500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529924,"0315-204-穿山甲-每次付费-幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,193.22,4,48.3050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,529925,"0315-204-穿山甲-每次付费-幸_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,42.37,0,42.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529926,"0315-204-穿山甲-每次付费-幸_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,24.58,0,24.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[6867,529927,"0315-204-穿山甲-每次付费-幸_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,5.17,0,5.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529975,"990-幸存者-MCFC-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,9.48,0,9.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529977,"990-幸存者-MCFC-0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,288.92,5,57.7840000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529978,"990-幸存者-MCFC-0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.29,0,1.2900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529981,"990-幸存者-MCFC-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,5.06,0,5.0600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529979,"990-幸存者-MCFC-0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.67,0,0.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529980,"990-幸存者-MCFC-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3.95,0,3.9500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,529976,"990-幸存者-MCFC-0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,792.00,15,52.8000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529983,"990-幸存者-MCFC-0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,616.76,26,23.7215385,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,529982,"990-幸存者-MCFC-0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,546.02,6,91.0033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530038,"CJ-幸存危机-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,28.17,0,28.1700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530041,"CJ-幸存危机-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,3303.74,60,55.0623333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530039,"CJ-幸存危机-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,614.50,11,55.8636364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530050,"CJ-幸存危机-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,16.87,0,16.8700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530102,"每0315-tt-xcztz-wyt-rk_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,18.10,0,18.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530101,"每0315-tt-xcztz-wyt-rk_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,15.23,0,15.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530104,"每0315-tt-xcztz-wyt-rk_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3.78,0,3.7800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530109,"每0315-tt-xcztz-wyt-rk_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,5.76,0,5.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530100,"每0315-tt-xcztz-wyt-rk_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.08,0,0.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530111,"每0315-tt-xcztz-wyt-rk_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,7.22,0,7.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530112,"每0315-tt-xcztz-wyt-rk_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,13.48,0,13.4800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530113,"每0315-tt-xcztz-wyt-rk_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,49.71,0,49.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530114,"每0315-tt-xcztz-wyt-rk_21","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,100.41,0,100.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530115,"每0315-tt-xcztz-wyt-rk_22","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.05,0,1.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530117,"每0315-tt-xcztz-wyt-rk_24","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,83.54,0,83.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530122,"每0315-tt-xcztz-wyt-rk_31","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,6.31,0,6.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530125,"每0315-tt-xcztz-wyt-rk_34","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,54.63,0,54.6300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530126,"每0315-tt-xcztz-wyt-rk_37","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,8.21,0,8.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530127,"每0315-tt-xcztz-wyt-rk_38","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.24,0,0.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530128,"每0315-tt-xcztz-wyt-rk_39","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.36,0,1.3600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530129,"每0315-tt-xcztz-wyt-rk_40","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,11.37,0,11.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530130,"每0315-tt-xcztz-wyt-rk_41","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,92.33,0,92.3300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530131,"每0315-tt-xcztz-wyt-rk_42","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.49,0,1.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530132,"每0315-tt-xcztz-wyt-rk_45","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2.00,0,2.0000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530133,"每0315-tt-xcztz-wyt-rk_46","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,469.30,4,117.3250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530134,"每0315-tt-xcztz-wyt-rk_47","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,13.72,0,13.7200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530135,"每0315-tt-xcztz-wyt-rk_48","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,302.18,1,302.1800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530139,"每0315-tt-xcztz-wyt-rk_52","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.23,0,0.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530140,"每0315-tt-xcztz-wyt-rk_53","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.49,0,0.4900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530141,"每0315-tt-xcztz-wyt-rk_54","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.41,0,0.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530145,"每0315-tt-xcztz-wyt-rk_58","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,21.41,0,21.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530146,"每0315-tt-xcztz-wyt-rk_59","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.03,0,0.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530147,"每0315-tt-xcztz-wyt-rk_60","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530148,"每0315-tt-xcztz-wyt-rk_62","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.22,0,0.2200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530149,"每0315-tt-xcztz-wyt-rk_61","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3.46,0,3.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530150,"每0315-tt-xcztz-wyt-rk_64","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,9.99,0,9.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530151,"每0315-tt-xcztz-wyt-rk_63","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,29.08,0,29.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530155,"每0315-tt-xcztz-wyt-rk_70","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,7.02,0,7.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530154,"每0315-tt-xcztz-wyt-rk_65","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,408.44,1,408.4400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530153,"每0315-tt-xcztz-wyt-rk_67","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,951.02,11,86.4563636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530157,"每0315-tt-xcztz-wyt-rk_66","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,265.58,3,88.5266667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530156,"每0315-tt-xcztz-wyt-rk_69","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,8.07,0,8.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530158,"每0315-tt-xcztz-wyt-rk_71","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.10,0,0.1000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530159,"每0315-tt-xcztz-wyt-rk_72","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,275.30,2,137.6500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530168,"r3-每次付费-幸存者挑战优选位-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2336.53,65,35.9466154,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530173,"r3-每次付费-幸存者挑战优选位-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1658.99,14,118.4992857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530175,"i-每次付费-幸存者挑战只限男优先位-03.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,783.36,20,39.1680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530174,"r3-每次付费-幸存者挑战优选位-03.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1913.11,24,79.7129167,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530176,"i-每次付费-幸存者挑战只限男优先位-03.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1174.93,9,130.5477778,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530167,"r3-每次付费-幸存者挑战优选位-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,485.07,3,161.6900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530177,"i-每次付费-幸存者挑战只限男优先位-03.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,309.76,3,103.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530210,"i-每次付费-幸存者挑战只限男优先位-03.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,430.16,1,430.1600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530211,"i-每次付费-幸存者挑战只限男优先位-03.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,577.04,4,144.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,530385,"ROI系数-幸存者挑战-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3107.77,112,27.7479464,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530386,"ROI系数-幸存者挑战-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3070.08,51,60.1976471,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530387,"ROI系数-幸存者挑战-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3015.56,96,31.4120833,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530388,"ROI系数-幸存者挑战-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1111.52,34,32.6917647,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21212,530389,"ROI系数-幸存者挑战-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2983.30,81,36.8308642,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530514,"0315幸存者393z每次付费3k_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1844.31,1,1844.3100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530516,"0315幸存者393z每次付费3k_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1863.95,5,372.7900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530515,"0315幸存者393z每次付费3k_2731_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2252.57,2,1126.2850000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530517,"0315幸存者393z每次付费3k_2746_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3237.84,38,85.2063158,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530518,"0315幸存者393z每次付费3k_2735_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1199.35,0,1199.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530524,"0315oyo393-roi系数-排除3k_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2885.28,52,55.4861538,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530531,"0315oyo393-roi系数-排除3k_2731_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1394.48,4,348.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530529,"0315oyo393-roi系数-排除3k_2522_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1046.06,12,87.1716667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530530,"0315oyo393-roi系数-排除3k_2762_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1331.56,12,110.9633333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20635,530537,"每0315-tt-xcztz-wyt-yan_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,173.89,4,43.4725000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530540,"每0315-tt-xcztz-wyt-yan_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,314.88,5,62.9760000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530541,"每0315-tt-xcztz-wyt-yan_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,294.41,3,98.1366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530542,"每0315-tt-xcztz-wyt-yan_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,95.29,1,95.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530543,"每0315-tt-xcztz-wyt-yan_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,302.67,3,100.8900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530544,"每0315-tt-xcztz-wyt-yan_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,496.49,10,49.6490000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530545,"每0315-tt-xcztz-wyt-yan_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2486.77,40,62.1692500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530546,"每0315-tt-xcztz-wyt-yan_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.28,0,1.2800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530547,"每0315-tt-xcztz-wyt-yan_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.94,0,0.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530548,"每0315-tt-xcztz-wyt-yan_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,4.45,0,4.4500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530549,"每0315-tt-xcztz-wyt-yan_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.09,0,0.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530550,"每0315-tt-xcztz-wyt-yan_19","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2.71,0,2.7100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530551,"每0315-tt-xcztz-wyt-yan_20","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2.11,1,2.1100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,530552,"每0315-tt-xcztz-wyt-yan_21","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,4.98,0,4.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,530671,"cxq-tt747幸存者-ROI系数0315_2738","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1968.98,62,31.7577419,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530672,"cxq-tt549幸存者-ROI系数0315_2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2855.54,85,33.5945882,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530673,"cxq-tt747幸存者-ROI系数0315_2332","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,739.36,4,184.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530674,"cxq-tt982幸存者-ROI系数0315_2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,150.86,2,75.4300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530675,"cxq-tt530幸存者-ROI系数0315_录2789","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,314.35,0,314.3500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530676,"cxq-tt530幸存者-ROI系数0315_录2793","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,261.52,0,261.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530678,"cxq-tt530幸存者-ROI系数0315_录2764","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,260.82,0,260.8200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20646,530677,"cxq-tt549幸存者-ROI系数0315_2731","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,361.83,5,72.3660000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530679,"cxq-tt982幸存者-ROI系数0315_2746","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,337.08,4,84.2700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20646,530680,"cxq-tt549幸存者-ROI系数0315_2781","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,437.21,4,109.3025000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530693,"0315-508幸存危机-每次付费竖版沉浸_2762_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,135.38,1,135.3800000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530692,"0315-508幸存危机-每次付费竖版沉浸_2332_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,23.62,0,23.6200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20545,530698,"0315-508幸存危机-每次付费竖版沉浸_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,681.82,7,97.4028571,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530696,"0315-508幸存危机-每次付费竖版沉浸_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,161.82,1,161.8200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530721,"303-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,318.02,1,318.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530722,"306-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,61.94,3,20.6466667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530718,"0315-幸存者393z每次付费排除3k_2784_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,315.11,4,78.7775000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,530713,"0315-幸存者393z每次付费排除3k_2783_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1101.04,4,275.2600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530720,"306-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,559.71,7,79.9585714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20545,530716,"0315-幸存者393z每次付费排除3k_2786_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,49.52,0,49.5200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530724,"301-幸存者-ROI系03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,342.01,3,114.0033333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530725,"301-幸存者-ROI系03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,316.00,3,105.3333333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530723,"303-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1383.69,36,38.4358333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530726,"301-幸存者-ROI系03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,6.18,0,6.1800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530728,"306-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,95.06,1,95.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530729,"306-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,13.65,0,13.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530731,"306-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,8.09,0,8.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530732,"303-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2.37,0,2.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530733,"306-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,27.41,0,27.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530734,"303-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,303.88,3,101.2933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530736,"306-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2153.20,55,39.1490909,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530735,"303-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.21,0,1.2100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530730,"306-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,5.67,0,5.6700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530737,"306-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,367.57,8,45.9462500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530738,"303-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.83,0,1.8300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530739,"303-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,29.18,2,14.5900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530740,"303-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.41,0,1.4100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530741,"301-幸存者-ROI系03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,44.92,2,22.4600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530742,"301-幸存者-ROI系03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,438.56,6,73.0933333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530743,"301-幸存者-ROI系03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,722.79,20,36.1395000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530744,"301-幸存者-ROI系03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,304.04,1,304.0400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530745,"301-幸存者-ROI系03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,4.43,0,4.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530746,"301-幸存者-ROI系03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2.79,0,2.7900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20451,530747,"301-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,79.66,1,79.6600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20451,530759,"303-幸存者-ROI系03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.02,0,1.0200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530779,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2332_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,648.74,19,34.1442105,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530781,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2731_0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,12.56,0,12.5600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530780,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,34.53,0,34.5300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530782,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2751_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2.03,0,2.0300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530783,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2794_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,119.91,0,119.9100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530784,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2271_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,361.69,15,24.1126667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530786,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2484_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,234.18,17,13.7752941,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530785,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1692.29,57,29.6892982,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20540,530787,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2762_0315_17","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,155.94,0,155.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530788,"RICH-财神到我家-幸存者-自动roi-排除-_混SSR2789_0315_18","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,4.05,0,4.0500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20540,530799,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2622315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,956.08,8,119.5100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20540,530802,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2789315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,99.25,0,99.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530803,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2761315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,75.60,0,75.6000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20540,530800,"RICH-财神到我家-幸存者-每次付费-排除-_混SSR2746315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,697.88,6,116.3133333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[6867,530806,"0315FP-ROI系数-MG幸_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3269.14,10,326.9140000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530805,"0315FP-ROI系数-MG幸_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1537.46,20,76.8730000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530807,"0315FP-ROI系数-MG幸_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2333.97,7,333.4242857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530810,"0315FP-ROI系数-MG幸_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1061.06,2,530.5300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530809,"0315FP-ROI系数-MG幸_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1255.06,1,1255.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530844,"0315FP-ROI系数-MG幸_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1171.66,4,292.9150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530843,"0315FP-ROI系数-MG幸_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1840.60,6,306.7666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530846,"0315FP-ROI系数-MG幸_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2177.36,20,108.8680000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[6867,530845,"0315FP-ROI系数-MG幸_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1762.77,5,352.5540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530908,"tw-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,970.04,5,194.0080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530913,"tw-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,107.09,0,107.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530914,"hix-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1105.87,8,138.2337500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530915,"zhan-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,389.75,0,389.7500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530912,"jap-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,31.46,0,31.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530910,"hix-每次付费-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,418.61,2,209.3050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530911,"hix-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,388.89,2,194.4450000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530907,"tw-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2713.20,32,84.7875000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530916,"zhan-每次付费-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,230.98,0,230.9800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530917,"zhan-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,518.88,4,129.7200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530909,"jap-每次付费-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2588.93,30,86.2976667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530920,"jap-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.39,0,1.3900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530918,"hix-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,8.43,0,8.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530919,"jap-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,285.03,1,285.0300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530921,"zhan-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,701.53,6,116.9216667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530925,"hix-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,29.89,0,29.8900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530926,"hix-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,38.19,0,38.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530923,"tw-每次付费-周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,156.00,1,156.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530922,"jap-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,13.37,0,13.3700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20754,530927,"zhan-每次付费-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,285.16,4,71.2900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530928,"zhan-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,995.17,7,142.1671429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530929,"tw-每次付费-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,54.24,1,54.2400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530931,"FJ-幸存者-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,753.31,7,107.6157143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530933,"FW-幸存者-每次03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,26.64,0,26.6400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530930,"FJ-幸存者-每次03.15-8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,258.76,3,86.2533333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530932,"FW-幸存者-每次03.15-1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,719.29,5,143.8580000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530934,"FW-幸存者-每次03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,528.73,6,88.1216667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530935,"FW-幸存者-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,956.90,8,119.6125000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530937,"FJ-幸存者-每次03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,42.93,0,42.9300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530940,"FJ-幸存者-每次03.15-2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,35.09,0,35.0900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530939,"FJ-幸存者-每次03.15-0","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,5.68,0,5.6800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530938,"FW-幸存者-每次03.15-6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,374.85,1,374.8500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530943,"FW-幸存者-每次03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1606.62,15,107.1080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530948,"FJ-幸存者-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,164.54,0,164.5400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530947,"FJ-幸存者-每次03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,602.25,5,120.4500000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530946,"FW-幸存者-每次03.15-5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,413.23,0,413.2300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530949,"FJ-幸存者-每次03.15-7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,647.84,4,161.9600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530944,"FJ-幸存者-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,400.58,0,400.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20451,530945,"FW-幸存者-每次03.15-3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1025.53,11,93.2300000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530942,"FW-幸存者-每次03.15-9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3303.74,39,84.7112821,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20451,530941,"FW-幸存者-每次03.15-4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,509.93,5,101.9860000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20754,530964,"yo-roi系数-周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2803.74,76,36.8913158,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530963,"ka-roi系数-周一-3.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1648.62,24,68.6925000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530958,"yo-roi系数-周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,110.73,0,110.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[20754,530960,"yo-roi系数-周一-3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2803.74,40,70.0935000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530965,"yo-roi系数-周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,736.83,17,43.3429412,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530967,"yo-roi系数-周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,365.61,6,60.9350000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530969,"yo-roi系数-周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,834.29,14,59.5921429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530973,"ka-roi系数-周一-3.15_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,630.42,10,63.0420000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530970,"ka-roi系数-周一-3.15_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,791.80,17,46.5764706,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530972,"ka-roi系数-周一-3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,407.12,2,203.5600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20754,530971,"yo-roi系数-周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,440.39,1,440.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20635,530999,"每0315-tt-xcztz-wyt-zhong_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,4.12,0,4.1200000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,530998,"每0315-tt-xcztz-wyt-zhong_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1302.96,14,93.0685714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531000,"每0315-tt-xcztz-wyt-zhong_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.07,0,0.0700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531001,"每0315-tt-xcztz-wyt-zhong_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,620.65,8,77.5812500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531002,"每0315-tt-xcztz-wyt-zhong_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3.62,1,3.6200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531006,"每0315-tt-xcztz-wyt-zhong_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,71.09,1,71.0900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531004,"每0315-tt-xcztz-wyt-zhong_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,672.76,13,51.7507692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531003,"每0315-tt-xcztz-wyt-zhong_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,921.25,3,307.0833333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20635,531005,"每0315-tt-xcztz-wyt-zhong_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,0.19,0,0.1900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531007,"每0315-tt-xcztz-wyt-zhong_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,22.40,0,22.4000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531008,"每0315-tt-xcztz-wyt-zhong_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,37.46,0,37.4600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531009,"每0315-tt-xcztz-wyt-zhong_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,3.99,0,3.9900000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531010,"每0315-tt-xcztz-wyt-zhong_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1.51,0,1.5100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20635,531011,"每0315-tt-xcztz-wyt-zhong_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,11.20,0,11.2000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[21213,531012,"n96-ios-黎明血战-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,39.96,1,39.9600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531013,"n96-ios-黎明血战-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,63.84,2,31.9200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531014,"n96-ios-黎明血战-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,1659.45,33,50.2863636,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531015,"n96-ios-黎明血战-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,53.39,1,53.3900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21213,531016,"n96-ios-黎明血战-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,1534.32,11,139.4836364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531038,"bdna-非海马云-幸存者挑战-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2701.47,91,29.6864835,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21208,531041,"bdna-非海马云-幸存者挑战-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2345.34,50,46.9068000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531187,"au-黎明血战--周一-3.15_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,679.51,11,61.7736364,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531189,"th-黎明血战--周一-3.15_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,179.24,0,179.2400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531190,"au-黎明血战--周一-3.15_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,75.27,0,75.2700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531194,"th-黎明血战--周一-3.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,2508.92,43,58.3469767,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531188,"roi系数-0315-BY-幸存_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,305.27,10,30.5270000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20541,531195,"RICH-研究所-财神到我家-通投_混SSR2791_0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,2183.63,7,311.9471429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531197,"RICH-研究所-财神到我家-通投_混SSR2797_0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,88.80,0,88.8000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531199,"RICH-研究所-财神到我家-通投_混SSR2749_0315_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,1143.12,1,1143.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531200,"RICH-研究所-财神到我家-通投_混SSR2789_0315_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,813.08,0,813.0800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20541,531201,"RICH-研究所-财神到我家-通投_混SSR2783_0315_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,1568.03,2,784.0150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531203,"05-幸存者-MCFC0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,433.57,7,61.9385714,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531202,"05-幸存者-MCFC0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2803.74,51,54.9752941,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531205,"05-幸存者-MCFC0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2803.74,39,71.8907692,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20541,531208,"RICH-研究所-财神到我家-通投_创意组8_0315_16","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,1653.34,1,1653.3400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531211,"th-黎明血战--周一-3.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,49.77,0,49.7700000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531212,"th-黎明血战--周一-3.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,390.00,1,390.0000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531207,"roi系数-0315-BY-幸存_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,559.87,8,69.9837500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531209,"roi系数-0315-BY-幸存_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,267.01,2,133.5050000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20755,531215,"th-黎明血战--周一-3.15_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,386.82,2,193.4100000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531214,"roi系数-0315-BY-幸存_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,508.28,2,254.1400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531213,"roi系数-0315-BY-幸存_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,50.73,0,50.7300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_COEFFICIENT"],[21245,531210,"roi系数-0315-BY-幸存_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,784.04,5,156.8080000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[21245,531206,"roi系数-0315-BY-幸存_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,529.12,1,529.1200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531216,"05-幸存者-MCFC0315_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,8.14,0,8.1400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,531217,"05-幸存者-MCFC0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,36.43,0,36.4300000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20755,531227,"th-黎明血战--周一-3.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,106.53,2,53.2650000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531229,"au-黎明血战--周一-3.15_14","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,17.58,0,17.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531230,"th-黎明血战--周一-3.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,637.41,3,212.4700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531231,"au-黎明血战--周一-3.15_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,418.74,0,418.7400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531232,"au-黎明血战--周一-3.15_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,476.02,1,476.0200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531233,"au-黎明血战--周一-3.15_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,69.94,0,69.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[21245,531234,"roi系数-0315-BY-幸存_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,304.39,4,76.0975000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20755,531252,"au-黎明血战--周一-3.15_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,48.58,0,48.5800000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531257,"th-黎明血战--周一-3.15_7","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,415.40,2,207.7000000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20755,531259,"au-黎明血战--周一-3.15_15","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001610,2,7,159.31,0,159.3100000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531256,"05-幸存者-MCFC0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1073.08,20,53.6540000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531258,"05-幸存者-MCFC0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,926.95,18,51.4972222,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531254,"05-幸存者-MCFC0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2.74,1,2.7400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531290,"0315幸存者393z每次付费排除3K_2797_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1202.22,3,400.7400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531289,"0315幸存者393z每次付费排除3K_2742_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1781.97,7,254.5671429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531291,"0315幸存者393z每次付费排除3K_2791_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1612.04,15,107.4693333,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531288,"0315幸存者393z每次付费排除3K_2796_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,10905.36,259,42.1056371,3,3635.120000,3214.00,0.2947175,0.0000,0.01158,2,0,null,null,"BID_PER_ACTION"],[20637,531308,"263-幸存者-ROI-0315_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,16963.69,458,37.0386245,4,4240.922500,602.00,0.0354876,0.0000,0.00873,4,0,null,null,"ROI_COEFFICIENT"],[20637,531312,"263-幸存者-ROI-0315_6","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1496.10,14,106.8642857,1,1496.100000,226.00,0.1510594,0.0000,0.07143,1,0,null,null,"ROI_COEFFICIENT"],[20637,531586,"263-幸存者-MCFC0315_10","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,7135.46,49,145.6216327,1,7135.460000,42.00,0.0058861,0.0000,0.02041,1,0,null,null,"BID_PER_ACTION"],[20546,531301,"0315wtd末日研究所_2138_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,7969.71,71,112.2494366,1,7969.710000,60.00,0.0075285,0.0000,0.01408,1,0,null,null,"DEEP_BID_DEFAULT"],[20546,531577,"0315ph648末日研究所_2138_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,3836.34,34,112.8335294,1,3836.340000,886.00,0.2309493,0.0000,0.02941,1,0,null,null,"DEEP_BID_DEFAULT"],[20545,531701,"0315-508幸存危机-每次付费_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,2764.52,47,58.8195745,2,1382.260000,1354.00,0.4897776,0.0000,0.04255,1,0,null,null,"BID_PER_ACTION"],[20545,531702,"0315-508幸存危机-每次付费_2053_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,4513.16,74,60.9886486,1,4513.160000,30.00,0.0066472,0.0000,0.01351,1,0,null,null,"BID_PER_ACTION"],[20545,531704,"0315-508幸存危机-每次付费_2332_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,2778.84,28,99.2442857,1,2778.840000,12.00,0.0043183,0.0000,0.03571,0,0,null,null,"BID_PER_ACTION"],[20545,531292,"0315幸存者393z每次付费排除3K_2798_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1320.65,14,94.3321429,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531287,"0315幸存者393z每次付费排除3K_2790_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,5530.46,74,74.7359459,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531293,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2484","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1155.31,10,115.5310000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531294,"cxq-tt338幸存者-穿山甲每次付费-0315_30002797","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,93.94,0,93.9400000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531298,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2761","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,395.33,7,56.4757143,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531297,"cxq-tt338幸存者-穿山甲每次付费-0315_排除+高2731","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,180.30,0,180.3000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20646,531296,"cxq-tt338幸存者-穿山甲每次付费-0315_30002798","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,98.57,1,98.5700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20646,531295,"cxq-tt338幸存者-穿山甲每次付费-0315_30002796","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1798.86,18,99.9366667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20546,531300,"0315wtd末日研究所_2332_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,3440.76,28,122.8842857,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531303,"0315wtd末日研究所_2730_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,890.32,1,890.3200000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531302,"0315wtd末日研究所_2762_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,1949.39,11,177.2172727,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531304,"0315wtd末日研究所_2731_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,2917.04,14,208.3600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531306,"263-幸存者-ROI-0315_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,763.58,22,34.7081818,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531314,"263-幸存者-ROI-0315_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2728.54,80,34.1067500,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531307,"263-幸存者-ROI-0315_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2910.18,58,50.1755172,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20637,531315,"263-幸存者-ROI-0315_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,236.45,2,118.2250000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_COEFFICIENT"],[20546,531576,"0315ph648末日研究所_2800_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,236.37,1,236.3700000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531578,"0315ph648末日研究所_2762_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,7286.80,81,89.9604938,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531579,"0315ph648末日研究所_2783_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,93.76,0,93.7600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20546,531580,"0315ph648末日研究所_2519_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001441,2,7,29.25,0,29.2500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"DEEP_BID_DEFAULT"],[20637,531583,"263-幸存者-MCFC0315_12","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,330.96,0,330.9600000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"BID_PER_ACTION"],[20637,531582,"263-幸存者-MCFC0315_13","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,470.23,2,235.1150000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531585,"263-幸存者-MCFC0315_11","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1102.03,5,220.4060000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531581,"263-幸存者-MCFC0315_8","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,68.49,1,68.4900000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20637,531584,"263-幸存者-MCFC0315_9","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,2672.80,33,80.9939394,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[20545,531706,"0315-508幸存危机-每次付费_2800_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001540,1,7,742.06,1,742.0600000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"BID_PER_ACTION"],[21212,531771,"r-自动ROI-幸存者挑战晚上七点-03.15_5","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,16.65,0,16.6500000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21212,531772,"r-自动ROI-幸存者挑战晚上七点-03.15_3","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,4590.59,90,51.0065556,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531773,"r-自动ROI-幸存者挑战晚上七点-03.15_1","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,1305.40,6,217.5666667,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"],[21212,531774,"r-自动ROI-幸存者挑战晚上七点-03.15_2","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,6.50,0,6.5000000,0,0.000000,0.00,0E-7,0.0000,0,0,0,null,null,"ROI_PACING"],[21212,531775,"r-自动ROI-幸存者挑战晚上七点-03.15_4","2021-03-16 00:00:00","2021-03-15 00:00:00",10,1001379,1,7,644.52,3,214.8400000,0,0.000000,0.00,0E-7,0.0000,0.00000,0,0,null,null,"ROI_PACING"]]}
